### Scraping the link

In [ ]:
import undetected_chromedriver as uc
import random

def random_user_agent():
    # This can be expanded. Use a library or list to randomly select a user-agent.
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
        # ... add more user-agents
    ]
    return random.choice(user_agents)

def random_delay(minimum=5, maximum=15):
    return time.sleep(random.uniform(minimum, maximum))

options = uc.ChromeOptions()
options.add_argument(f"user-agent={random_user_agent()}")

# Add other options like proxies here, if you have them

browser = uc.Chrome(options=options)

base_url = "https://www.realtor.com/apartments/Las-Vegas_NV/pg-10"


property_links = []

browser.get(url)

    # Introduce random delay after loading each page
random_delay()

    # Scroll through the page to load more content
scroll_page()

    # Wait until the presence of the property links
wait = WebDriverWait(browser, 60)  # Increased the wait time to 60 seconds
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card-anchor")))

    # Extract the property links from the page
links = browser.find_elements(By.CSS_SELECTOR, "a.card-anchor")
for link in links:
    property_links.append(link.get_attribute("href"))

    # Introduce random delay before moving to the next page
random_delay()

# Close the browser
browser.quit()

# Save links to CSV
with open('property_links.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Link'])
    for link in property_links:
        writer.writerow([link])

print("Scraping completed and saved to property_links.csv!")


### Scraping each links information

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

# List of links you want to scrape
links = [
'https://www.realtor.com/rentals/details/3501-S-Maryland-Pkwy_Las-Vegas_NV_89169_M14058-58461',
'https://www.realtor.com/rentals/details/4000-E-Bonanza-Rd_Las-Vegas_NV_89110_M27725-71219',
'https://www.realtor.com/rentals/details/2120-Van-Patten-Pl_Las-Vegas_NV_89104_M19992-31808',
'https://www.realtor.com/rentals/details/9490-Bermuda-Rd_Las-Vegas_NV_89123_M14284-51408',
'https://www.realtor.com/rentals/details/3896-University-Center-Dr_Las-Vegas_NV_89119_M95695-89840',
'https://www.realtor.com/rentals/details/2255-Spruce-Goose-St_Las-Vegas_NV_89135_M97751-35398',
'https://www.realtor.com/rentals/details/9950-W-Tropicana-Ave_Las-Vegas_NV_89147_M97868-55464',
'https://www.realtor.com/rentals/details/5091-Moose-Falls-Dr_Las-Vegas_NV_89141_M29539-47285',
'https://www.realtor.com/rentals/details/6978-Walden-Park-St_Las-Vegas_NV_89166_M17335-05214',
'https://www.realtor.com/rentals/details/7565-Norman-Rockwell-Ln-Ste-110_Las-Vegas_NV_89143_M95950-01113',
'https://www.realtor.com/rentals/details/11055-Las-Vegas-Blvd-S_Las-Vegas_NV_89183_M90939-36612',
'https://www.realtor.com/rentals/details/7659-Glorietta-Bay-Ct_Las-Vegas_NV_89139_M19296-60665',
'https://www.realtor.com/rentals/details/3400-Cabana-Dr-15_Las-Vegas_NV_89122_M94543-50448',
'https://www.realtor.com/rentals/details/3024-Reef-View-St_Las-Vegas_NV_89117_M25843-96157',
'https://www.realtor.com/rentals/details/9156-Conquest-Ct_Las-Vegas_NV_89149_M22396-41923',
'https://www.realtor.com/rentals/details/5164-Mineral-Lake-Dr_Las-Vegas_NV_89122_M21178-23409',
'https://www.realtor.com/rentals/details/3750-Corpolo-Ave_Las-Vegas_NV_89141_M23958-51519',
'https://www.realtor.com/rentals/details/7209-Beachwood-Crest-St_Las-Vegas_NV_89166_M25908-79728',
'https://www.realtor.com/rentals/details/9428-Fort-Connah-Ct_Las-Vegas_NV_89178_M28705-83371',
'https://www.realtor.com/rentals/details/10617-Capitol-Peak-Ave_Las-Vegas_NV_89166_M10535-47525',
'https://www.realtor.com/rentals/details/8262-Vanity-Fair-Ln_Las-Vegas_NV_89113_M22681-13345',
'https://www.realtor.com/rentals/details/4715-Longshot-Dr_Las-Vegas_NV_89122_M17389-80283',
'https://www.realtor.com/rentals/details/7941-Violet-Dawn-St_Las-Vegas_NV_89149_M26442-87048',
'https://www.realtor.com/rentals/details/7105-Powderhorn-Cir_Las-Vegas_NV_89128_M28344-00770',
'https://www.realtor.com/rentals/details/6460-Hamilton-Grove-Ave_Las-Vegas_NV_89122_M24350-90418',
'https://www.realtor.com/rentals/details/5033-Whistling-Acres-Ave_Las-Vegas_NV_89131_M11631-14748',
'https://www.realtor.com/rentals/details/7912-Point-Oaks-Ct_Las-Vegas_NV_89149_M24793-02712',
'https://www.realtor.com/rentals/details/4792-W-Shelbourne-Ave_Las-Vegas_NV_89139_M13639-06795',
'https://www.realtor.com/rentals/details/7112-Mercurio-Ave_Las-Vegas_NV_89131_M25641-73407',
'https://www.realtor.com/rentals/details/7315-Larix-Rd_Las-Vegas_NV_89113_M29691-85907',
'https://www.realtor.com/rentals/details/8261-Oliver-Twist-Ln_Las-Vegas_NV_89113_M10510-50707',
'https://www.realtor.com/rentals/details/10525-Autumn-Pine-Ave-Apt-205_Las-Vegas_NV_89144_M13423-92454',
'https://www.realtor.com/rentals/details/8819-W-Oquendo-Rd_Las-Vegas_NV_89148_M10494-27730',
'https://www.realtor.com/rentals/details/595-S-Royal-Crest-Cir-Unit-7_Las-Vegas_NV_89169_M17649-29455',
'https://www.realtor.com/rentals/details/9594-Vega-Carpio-Ave_Las-Vegas_NV_89178_M28291-64246',
'https://www.realtor.com/rentals/details/5894-Grechetto-Ct_Las-Vegas_NV_89141_M26091-82381',
'https://www.realtor.com/rentals/details/8920-Picket-Fence-Ave_Las-Vegas_NV_89143_M29292-10403',
'https://www.realtor.com/rentals/details/4973-Blue-Marlin-Ave_Las-Vegas_NV_89115_M25339-22847',
'https://www.realtor.com/rentals/details/5725-Smoke-Ranch-Rd-Unit-C_Las-Vegas_NV_89108_M16486-35746',
'https://www.realtor.com/rentals/details/7224-Cottonsparrow-St_Las-Vegas_NV_89131_M15670-54301',
'https://www.realtor.com/rentals/details/725-S-Hualapai-Way_Las-Vegas_NV_89145_M10327-15621',
'https://www.realtor.com/rentals/details/3240-Las-Vegas-Blvd-N_Las-Vegas_NV_89115_M15790-49379',
'https://www.realtor.com/rentals/details/2700-N-Rainbow-Blvd_Las-Vegas_NV_89108_M12346-64196',
'https://www.realtor.com/rentals/details/9625-W-Russell-Rd_Las-Vegas_NV_89148_M23144-53806',
'https://www.realtor.com/rentals/details/818-E-Flamingo-Rd_Las-Vegas_NV_89119_M16729-90948',
'https://www.realtor.com/rentals/details/2375-Spruce-Goose-St_Las-Vegas_NV_89135_M23574-93064',
'https://www.realtor.com/rentals/details/6614-Blue-Diamond-Rd_Las-Vegas_NV_89139_M98789-01931',
'https://www.realtor.com/rentals/details/10136-Serenity-Star-Way_Las-Vegas_NV_89147_M94416-26958',
'https://www.realtor.com/rentals/details/9501-W-Sahara-Ave_Las-Vegas_NV_89117_M10102-07234',
'https://www.realtor.com/rentals/details/4450-Karen-Ave_Las-Vegas_NV_89121_M29403-05227',

]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data_to_save = []

for link in links:
    response = requests.get(link, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {link}")
        continue

    # Parse page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract apartment name
    name_tag = soup.find('h1', {'class': lambda x: x and 'base__StyledType-rui' in x and 'name' in x})
    apartment_name = name_tag.text if name_tag else None
    
    # Extract apartment address
    address_tag = soup.find('p', {'class': lambda x: x and 'base__StyledType-rui' in x and 'address' in x})
    apartment_address = address_tag.text.strip() if address_tag else None

    # Extract ZIP code
    zip_code = None
    if apartment_address:
        zip_code = apartment_address.split()[-1]
    
    # Extract price
    price_tag = soup.find('div', class_=re.compile('Pricestyles__StyledPrice-rui__'))
    price = price_tag.text.split('-')[0].strip().replace('$', '') if price_tag else None

    # Extract square feet
    sqft_tag = soup.find('li', class_=re.compile('PropertySqftMetastyles__StyledPropertySqftMeta-rui__'))
    sqft = sqft_tag.find('span', class_='meta-value').text.split('-')[0].strip().replace('sqft', '').strip() if sqft_tag and sqft_tag.find('span', class_='meta-value') else None

    # Extract beds
    beds_tag = soup.find('li', attrs={"data-testid": "property-meta-beds"})
    beds_value = beds_tag.find('span', attrs={"data-testid": "meta-value"}).text if beds_tag else None

    if beds_value:
        if 'Studio' in beds_value:
            beds = 'Studio'
        else:
            beds = beds_value.split('-')[0].strip()

    # Extract baths
    baths_tag = soup.find('li', attrs={"data-testid": "property-meta-baths"})
    baths_value = baths_tag.find('span', attrs={"data-testid": "meta-value"}).text if baths_tag else None
    baths = baths_value.split('-')[0].strip() if baths_value else None
    
    # Extract Property Type
    property_type = None
    type_tag = soup.find('div', string='Property type')
    if type_tag and type_tag.find_next_sibling():
        property_type = type_tag.find_next_sibling().text

    # Extract Year Built
    year_built = None
    year_built_tag = soup.find('div', string='Year built')
    if year_built_tag and year_built_tag.find_next_sibling():
        year_built = year_built_tag.find_next_sibling().text
    
    # Append data to the list
    data_to_save.append([link, apartment_name, apartment_address, zip_code, price, sqft, beds, baths, property_type, year_built])


# Write data to CSV
with open('apartment_data56.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Added 'URL' to the header
    writer.writerow(['URL', 'Apartment Name', 'Apartment Address', 'ZIP Code', 'Price', 'Sqft', 'Beds', 'Baths', 'Property Type', 'Year Built'])  
    writer.writerows(data_to_save)

print("Data saved to 'apartment_data.csv'")


Scrape Links



In [8]:
import csv
import requests
from bs4 import BeautifulSoup

def is_listing_link(tag):
    """Check if a tag is an anchor tag with a listing link."""
    return (tag.name == 'a' and 
            tag.has_attr('title') and 
            tag.has_attr('href') and
            'NV' in tag['title'])

def get_links(url):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    listing_links = soup.find_all(is_listing_link)
    
    # Prepend the base URL to each link
    base_url = 'https://www.redfin.com'
    links = [f'{base_url}{link["href"]}' for link in listing_links]
    return links

def save_to_csv(data, filename):
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for link in data:
            writer.writerow([link])

def main(urls, filename):
    for url in urls:
        print(f'Processing {url}...')
        links = get_links(url)
        save_to_csv(links, filename)
        print(f'Collected {len(links)} links from {url}')

# Example usage:
urls = [
 'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.19015:36.06439:-115.00042:-115.24573,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.18656:36.0608:-114.9266:-115.1719,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.09999:35.97408:-115.1551:-115.40041,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.07823:35.95229:-115.05734:-115.30265,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.10364:35.97774:-114.95321:-115.19851,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.14648:36.02066:-114.84162:-115.08693,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.02769:35.90167:-114.99184:-115.23715,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.045:35.98203:-114.90853:-115.03119,no-outline/page-9',

]
filename = 'Scraped_Links_sales-3.csv'

# Make sure to clear the old CSV file if it exists
with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Link"])  # header

if __name__ == '__main__':
    main(urls, filename)


Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline...
Collected 40 links from https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline
Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-2...
Collected 40 links from https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-2
Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+townhouse,max-price=950k,viewport=36.1841:36.05833:-115.09865:-115.34396,no-outline/page-3...
Collected 40 links from https://www.redfin.com/city/10201/NV/Las-Vegas/filter/property-type=house+t

Scraping Features


In [9]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import re

def find_text(parent, text):
    element = parent.find(text=re.compile(text))
    return element.find_next('span').text if element else None

def extract_scores_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the GET request failed
    return extract_scores(response.content)

def extract_scores(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Function to extract score from div with given class_name
    def get_score(class_name):
        div = soup.find('div', class_=class_name)
        if div:
            percentage = div.find('span', {'class': 'value'})
            if percentage:
                return percentage.text
        return None  # Return None if score isn't found
    
    walk_score = get_score('walkscore')
    transit_score = get_score('transitscore')
    bike_score = get_score('bikescore')
    
    return walk_score, transit_score, bike_score

def scrape_school_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    school_data = []
    
    # Find all the school entries
    school_entries = soup.find_all("div", class_="ListItem col-10 clickable")
    
    for entry in school_entries:
        school_name_element = entry.find("h3", class_="ListItem__heading font-body-base-bold color-text-primary")
        school_score_element = entry.find("span", class_="rating-num font-size-base font-weight-bold")
        school_details_element = entry.find("p", class_="ListItem__description font-body-small-compact color-text-secondary")

        # If any element is not found, skip this entry
        if not school_name_element or not school_score_element or not school_details_element:
            continue

        school_name = school_name_element.text
        school_score = school_score_element.text
        school_details = school_details_element.text

        # Identify school type by checking the grades it serves
        if "PreK-5" in school_details:
            school_type = "Elementary School"
        elif "6-8" in school_details:
            school_type = "Middle School"
        elif "9-12" in school_details:
            school_type = "High School"
        else:
            school_type = "Unknown"
        
        school_data.append({
            "Name": school_name,
            "Type": school_type,
            "Score": school_score
        })

    return school_data
def extract_additional_details(soup):
    details = soup.find('div', class_='keyDetailsList')
    if not details:
        return None, None, None, None, None, None

    extracted = {}
    for row in details.find_all('div', class_='table-row'):
        label_elem = row.find('span', class_='table-label')
        value_elem = row.find('div', class_='table-value')

        if label_elem and value_elem:
            label = label_elem.text.strip()
            value = value_elem.text.strip()
            extracted[label] = value

    time_on_redfin = extracted.get('Time on Redfin', None)
    year_built = extracted.get('Year Built', None)
    property_type = extracted.get('Property Type', None)
    style = extracted.get('Style', None)
    hoa_dues = extracted.get('HOA Dues', None)
    lot_size = extracted.get('Lot Size', None)
    
    return time_on_redfin, year_built, property_type, style, hoa_dues, lot_size

def extract_additional_features(soup):
    
    def find_text(parent, text):
        element = parent.find(text=re.compile(text))
        return element.find_next('span').text if element else None
    
    
    exterior_features_div = soup.find('div', {'class': 'amenity-group'}, text=re.compile('Exterior Features'))

    garage_spaces = find_text(soup, '# of Garage Spaces:') or find_text(soup, '# of Carport Spaces:')
    floor_type = find_text(soup, 'Flooring:')
    heat_type = find_text(soup, 'Heating:')
    cool_type = find_text(soup, 'Cooling:')
    tax_amount = find_text(soup, 'Annual Amount:')
    
    
    patio = find_text(soup, 'Patio And Porch Features:')
    # Locate the 'li' element containing 'Pool Features:'
    pool_private = find_text(soup, 'Pool Features:')
    
        
    return garage_spaces, pool_private, floor_type, heat_type, cool_type, tax_amount, patio  # updated 'pool' to 'pool_private'
   



def extract_real_estate_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    top_stats = soup.find('div', class_='top-stats')
    if not top_stats:
        return None, None, None, None, None, None
    
    address_div = top_stats.find('header', class_='address')
    price_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-price'})
    beds_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-beds'})
    baths_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-baths'})
    sqft_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-sqFt'})
    est_payment_div = price_div.find('span', class_='est-monthly-payment') if price_div else None


    address = address_div.get_text(strip=True) if address_div else None
    price = price_div.div.text if price_div and price_div.div else None
    beds = beds_div.div.text if beds_div and beds_div.div else None
    baths = baths_div.div.text if baths_div and baths_div.div else None
    sqft = sqft_div.span.text if sqft_div and sqft_div.span else None
    est_payment = est_payment_div.text if est_payment_div else None
    time_on_redfin, year_built, property_type, style, hoa_dues, lot_size = extract_additional_details(soup)
    
    return address, price, beds, baths, sqft, est_payment, time_on_redfin, year_built, property_type, style, hoa_dues, lot_size



def combine_data_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the GET request failed
    soup = BeautifulSoup(response.content, 'html.parser')
    walk, transit, bike = extract_scores(response.content)
    school_data = scrape_school_data(url)
    address, price, beds, baths, sqft, est_payment, time_on_redfin, year_built, property_type, style, hoa_dues, lot_size = extract_real_estate_info(response.content)
    garage_spaces, pool_private, floor_type, heat_type, cool_type, tax_amount, patio = extract_additional_features(soup)


    elementary_school_name, elementary_school_score = None, None
    middle_school_name, middle_school_score = None, None
    high_school_name, high_school_score = None, None

    for school in school_data:
        if school["Type"] == "Elementary School" and not elementary_school_name:
            elementary_school_name = school["Name"]
            elementary_school_score = school["Score"]
        elif school["Type"] == "Middle School" and not middle_school_name:
            middle_school_name = school["Name"]
            middle_school_score = school["Score"]
        elif school["Type"] == "High School" and not high_school_name:
            high_school_name = school["Name"]
            high_school_score = school["Score"]
    
    return {
        "URL": url,
        "Walk Score (out of 100)": walk,
        "Transit Score (out of 100)": transit,
        "Bike Score (out of 100)": bike,
        "Elementary School Name": elementary_school_name,
        "Elementary School Score (out of 10)": elementary_school_score,
        "Middle School Name": middle_school_name,
        "Middle School Score (out of 10)": middle_school_score,
        "High School Name": high_school_name,
        "High School Score (out of 10)": high_school_score,
        "Address": address,
        "Price": price,
        "Beds": beds,
        "Baths": baths,
        "Sq Ft": sqft,
        "Est. Monthly Payment": est_payment,
        "Time on Redfin": time_on_redfin,
        "Year Built": year_built,
        "Property Type": property_type,
        "Style": style,
        "HOA Dues": hoa_dues,
        "Lot Size": lot_size,
        "Garage Spaces": garage_spaces,
        "Has Pool": pool_private,
        "Floor Type": floor_type,
        "Heat Type": heat_type,
        "Cool Type": cool_type,
        "Tax Amount": tax_amount,
        "Has Patio": patio,
       
    }


# Extract combined data for all URLs
urls = [


]  # Add more URLs as required
combined_data = [combine_data_from_url(url) for url in urls]

# Export data to JSON
with open('data_list.json', 'w') as f:
    json.dump(combined_data, f, indent=4)

# Update csv_columns to include new fields
csv_columns = [
    "URL", 
    "Walk Score (out of 100)", 
    "Transit Score (out of 100)", 
    "Bike Score (out of 100)",
    "Elementary School Name", 
    "Elementary School Score (out of 10)",
    "Middle School Name", 
    "Middle School Score (out of 10)",
    "High School Name", 
    "High School Score (out of 10)",
    "Address",
    "Price",
    "Beds",
    "Baths",
    "Sq Ft",
    "Est. Monthly Payment",
    "Time on Redfin",
    "Year Built",
    "Property Type",
    "Style",
    "HOA Dues",
    "Lot Size",
    "Garage Spaces",
    "Has Pool",
    "Floor Type",
    "Heat Type",
    "Cool Type",
    "Tax Amount",
    "Has Patio",
   
]


csv_file = "data_list.csv"
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in combined_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

C:\Users\binhd\AppData\Local\Temp\ipykernel_39964\2611699579.py:104: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  exterior_features_div = soup.find('div', {'class': 'amenity-group'}, text=re.compile('Exterior Features'))
C:\Users\binhd\AppData\Local\Temp\ipykernel_39964\2611699579.py:100: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  element = parent.find(text=re.compile(text))


Testing script


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_factors(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    factors = {}

    headings = soup.find_all('h3', class_='ListItem__heading font-body-base-bold color-text-primary')
    for heading in headings:
        text_content = heading.text
        if ' - ' in text_content:
            factor, value = text_content.split(' - ', 1)
            factors[factor] = value

    return factors

def save_to_csv(factors_list):
    df = pd.DataFrame(factors_list)
    df.to_csv('factors.csv', index=False)

def main():
    urls = [
        'https://www.redfin.com/NV/Las-Vegas/1758-Shady-Elm-St-89135/home/29897110',
'https://www.redfin.com/NV/Las-Vegas/9642-Saint-Claude-Ave-89148/home/29584469',
'https://www.redfin.com/NV/Las-Vegas/361-Borgato-Ct-89145/home/169229691',
'https://www.redfin.com/NV/Las-Vegas/136-Countrywood-Cir-89107/home/29233924',
'https://www.redfin.com/NV/Las-Vegas/3980-Palos-Verdes-St-89119/home/29499707',
'https://www.redfin.com/NV/Las-Vegas/1820-Hassett-Ave-89104/home/29453957',
'https://www.redfin.com/NV/Las-Vegas/616-Heartline-Dr-89145/home/29214849',
'https://www.redfin.com/NV/Las-Vegas/10324-Juniper-Creek-Ln-89145/home/29858044',
'https://www.redfin.com/NV/Las-Vegas/7503-W-Diablo-Dr-89113/home/29581674',
'https://www.redfin.com/NV/Las-Vegas/219-Earl-St-89101/home/29348763',
'https://www.redfin.com/NV/Las-Vegas/10009-Mahogany-Grove-Ln-89117/home/29527906',
'https://www.redfin.com/NV/Las-Vegas/9830-Hanover-Grove-Ave-89148/home/29032968',
'https://www.redfin.com/NV/Las-Vegas/205-S-Mallard-St-89107/home/29232105',
'https://www.redfin.com/NV/Las-Vegas/813-Antelope-Way-89145/home/29225830',
'https://www.redfin.com/NV/Las-Vegas/3455-Golden-Lily-St-89117/home/109628827',
'https://www.redfin.com/NV/Las-Vegas/301-Courtney-Ln-89107/home/29227110',
'https://www.redfin.com/NV/Las-Vegas/11297-Colinward-Ave-89135/home/29320421',
'https://www.redfin.com/NV/Las-Vegas/709-Rivenbark-Ct-89145/home/29214767',
'https://www.redfin.com/NV/Las-Vegas/137-N-20th-St-89101/home/29349706',
'https://www.redfin.com/NV/Las-Vegas/1509-S-15th-St-89104/home/29451492',
'https://www.redfin.com/NV/Las-Vegas/3217-Harbor-Vista-St-89117/home/29532493',
'https://www.redfin.com/NV/Las-Vegas/6773-Firewood-Dr-89148/home/29011548',
'https://www.redfin.com/NV/Las-Vegas/8461-Brillancy-Ave-89147/home/29560242',
'https://www.redfin.com/NV/Las-Vegas/1833-Glenwillow-Dr-89117/home/29523563',
'https://www.redfin.com/NV/Las-Vegas/3009-Periscope-Ct-89117/home/29529997',
'https://www.redfin.com/NV/Las-Vegas/3546-Blackstone-St-89121/home/29474281',
'https://www.redfin.com/NV/Las-Vegas/6766-Regency-Valley-St-89148/home/167755549',
'https://www.redfin.com/NV/Las-Vegas/1325-Griffith-Ave-89104/home/29452827',
'https://www.redfin.com/NV/Las-Vegas/11250-Cactus-Tower-Ave-89135/unit-103/home/166299125',
'https://www.redfin.com/NV/Las-Vegas/7363-Mission-Hills-Dr-89113/home/29580589',
'https://www.redfin.com/NV/Las-Vegas/6921-Emerald-Springs-Ln-89113/home/29582433',
'https://www.redfin.com/NV/Las-Vegas/3179-Batavia-Dr-89102/home/29462894',
'https://www.redfin.com/NV/Las-Vegas/5012-Iowa-Ave-89107/home/29190146',
'https://www.redfin.com/NV/Las-Vegas/7624-Genzer-Dr-89145/home/29219632',
'https://www.redfin.com/NV/Las-Vegas/8186-Merlewood-Ave-89117/home/29923315',
'https://www.redfin.com/NV/Las-Vegas/8217-Eginton-Ave-89145/home/29215575',
'https://www.redfin.com/NV/Las-Vegas/2800-Albrook-Cir-89117/home/29529679',
'https://www.redfin.com/NV/Las-Vegas/200-Lindy-Dr-89107/home/29227848',
'https://www.redfin.com/NV/Las-Vegas/9700-Falling-Star-Ave-89117/home/29528056',
'https://www.redfin.com/NV/Las-Vegas/1904-Canosa-Ave-89104/home/29454350',
'https://www.redfin.com/NV/Las-Vegas/1017-Nye-St-89106/home/29340288',
'https://www.redfin.com/NV/Las-Vegas/613-Stanford-St-89107/home/29190098',
'https://www.redfin.com/NV/Las-Vegas/11337-Corsica-Mist-Ave-89135/home/29033536',
'https://www.redfin.com/NV/Las-Vegas/982-Bel-Air-Cir-89109/home/29466854',
'https://www.redfin.com/NV/Las-Vegas/7321-Sequoia-Springs-Dr-89147/home/29547563',
'https://www.redfin.com/NV/Las-Vegas/3529-Erva-St-89147/home/29552186',
'https://www.redfin.com/NV/Las-Vegas/5508-Morendo-Dr-89107/home/29188242',
'https://www.redfin.com/NV/Las-Vegas/3017-Piedmont-Ave-89102/home/29465035',
'https://www.redfin.com/NV/Las-Vegas/1704-Pacific-St-89104/home/29451057',
'https://www.redfin.com/NV/Las-Vegas/10608-Tyne-Pl-89144/home/29111912',
'https://www.redfin.com/NV/Las-Vegas/4355-Tall-Tree-St-89147/home/29558971',
'https://www.redfin.com/NV/Las-Vegas/3900-Blackford-Pl-89102/home/29464371',
'https://www.redfin.com/NV/Las-Vegas/1845-Lindell-Rd-89146/home/29516533',
'https://www.redfin.com/NV/Las-Vegas/5457-Olivebrook-Ct-89120/home/29511584',
'https://www.redfin.com/NV/Las-Vegas/8960-Antioch-Way-89117/home/29531082',
'https://www.redfin.com/NV/Las-Vegas/1016-Bracken-Ave-89104/home/29456289',
'https://www.redfin.com/NV/Las-Vegas/9224-Canalino-Dr-89134/home/29211829',
'https://www.redfin.com/NV/Las-Vegas/1590-Rawhide-St-89119/home/29513434',
'https://www.redfin.com/NV/Las-Vegas/746-Biltmore-Dr-89101/home/29337001',
'https://www.redfin.com/NV/Las-Vegas/7835-Harbour-Towne-Ave-89113/home/29583267',
'https://www.redfin.com/NV/Las-Vegas/9124-Worsley-Park-Pl-89145/home/29003259',
'https://www.redfin.com/NV/Las-Vegas/3117-Arville-St-89102/home/29463031',
'https://www.redfin.com/NV/Las-Vegas/3676-Starship-Ln-89147/home/29553197',
'https://www.redfin.com/NV/Las-Vegas/3147-Pradera-Cir-89121/home/29474381',
'https://www.redfin.com/NV/Las-Vegas/10236-Ritornia-Ave-89135/home/28972932',
'https://www.redfin.com/NV/Las-Vegas/10260-Riva-de-Destino-Ave-89135/home/29898177',
'https://www.redfin.com/NV/Las-Vegas/412-La-Rue-Ct-89145/home/40321831',
'https://www.redfin.com/NV/Las-Vegas/4409-Cinderella-Ln-89102/home/29463194',
'https://www.redfin.com/NV/Las-Vegas/3280-Cherum-St-89135/home/29589298',
'https://www.redfin.com/NV/Las-Vegas/5426-Spencer-St-89119/home/29514545',
'https://www.redfin.com/NV/Las-Vegas/3255-Westwind-Rd-89146/home/29539271',
'https://www.redfin.com/NV/Las-Vegas/3994-Starfield-Ln-89147/home/29557463',
'https://www.redfin.com/NV/Las-Vegas/4410-Campus-Cir-89121/home/29507803',
'https://www.redfin.com/NV/Las-Vegas/5580-Judith-Resnik-Ct-89103/home/29540835',
'https://www.redfin.com/NV/Las-Vegas/6657-Autumn-Breeze-St-89148/home/165932077',
'https://www.redfin.com/NV/Las-Vegas/3863-Chase-Glen-Cir-89121/home/28976333',
'https://www.redfin.com/NV/Las-Vegas/9482-Joe-Robby-Ct-89148/home/29869669',
'https://www.redfin.com/NV/Las-Vegas/8412-Eddy-Stone-Ave-89117/home/29521076',
'https://www.redfin.com/NV/Las-Vegas/5034-Celebrity-Cir-89119/home/29513563',
'https://www.redfin.com/NV/Las-Vegas/3735-Barrelwood-Dr-89147/home/29553107',
'https://www.redfin.com/NV/Las-Vegas/4478-Eureka-Cir-89103/home/29572117',
'https://www.redfin.com/NV/Las-Vegas/8114-Surrey-Field-Ct-89113/home/108522766',
'https://www.redfin.com/NV/Las-Vegas/544-Oakbrook-Ln-89169/home/29495732',
'https://www.redfin.com/NV/Las-Vegas/7009-Rawlins-Ct-89128/home/29201390',
'https://www.redfin.com/NV/Las-Vegas/8478-Prince-of-Tides-Ct-89113/home/29584183',
'https://www.redfin.com/NV/Las-Vegas/5369-White-Butterfly-St-89113/home/175525463',
'https://www.redfin.com/NV/Las-Vegas/6400-Penrose-Ln-89107/home/29194224',
'https://www.redfin.com/NV/Las-Vegas/229-N-Bruce-St-89101/home/29348235',
'https://www.redfin.com/NV/Las-Vegas/10071-Tanner-Rapids-Ct-89148/home/29868644',
'https://www.redfin.com/NV/Las-Vegas/4012-Calle-Mirador-89103/home/29502377',
'https://www.redfin.com/NV/Las-Vegas/3154-Downing-Pl-89121/home/29509354',
'https://www.redfin.com/NV/Las-Vegas/1310-S-13th-St-89104/home/29451519',
'https://www.redfin.com/NV/Las-Vegas/5445-Indian-Cedar-Dr-89135/home/29065524',
'https://www.redfin.com/NV/Las-Vegas/6746-Graceda-St-89148/home/28989837',
'https://www.redfin.com/NV/Las-Vegas/6117-Iris-Cir-89107/home/29196018',
'https://www.redfin.com/NV/Las-Vegas/8146-Barodo-Way-89147/home/29564092',
'https://www.redfin.com/NV/Las-Vegas/9923-W-Cherokee-Ave-89147/home/29555586',
'https://www.redfin.com/NV/Las-Vegas/10595-Acacia-Park-Pl-89135/home/52627872',
'https://www.redfin.com/NV/Las-Vegas/218-Earl-St-89101/home/29348783',
'https://www.redfin.com/NV/Las-Vegas/6234-Brea-Blvd-89118/home/29580226',
'https://www.redfin.com/NV/Las-Vegas/9753-Villa-Lorena-Ave-89147/home/29030483',
'https://www.redfin.com/NV/Las-Vegas/1357-Elizabeth-Ave-89119/unit-2/home/29505830',
'https://www.redfin.com/NV/Las-Vegas/5155-Briar-Patch-Way-89118/home/29578626',
'https://www.redfin.com/NV/Las-Vegas/7292-Lavender-Rose-Ave-89117/home/65558559',
'https://www.redfin.com/NV/Las-Vegas/8804-Borla-Dr-89117/home/29523699',
'https://www.redfin.com/NV/Las-Vegas/3927-Saddlewood-Ct-89121/home/29474253',
'https://www.redfin.com/NV/Las-Vegas/1329-Desert-Hills-Dr-89117/home/29527284',
'https://www.redfin.com/NV/Las-Vegas/3634-Surfboard-Ct-89147/home/29030322',
'https://www.redfin.com/NV/Las-Vegas/4535-Cape-Elizabeth-St-89147/home/44585202',
'https://www.redfin.com/NV/Las-Vegas/8241-Turtle-Creek-Cir-89113/home/29583101',
'https://www.redfin.com/NV/Las-Vegas/9948-Canyon-Peak-Dr-89147/home/29830025',
'https://www.redfin.com/NV/Las-Vegas/3172-Richford-Pl-89102/home/29464858',
'https://www.redfin.com/NV/Las-Vegas/3127-Pampas-Pl-89146/home/29540052',
'https://www.redfin.com/NV/Las-Vegas/4281-Rollingstone-Dr-89103/home/29573529',
'https://www.redfin.com/NV/Las-Vegas/512-Bedford-Rd-89107/home/29343607',
'https://www.redfin.com/NV/Las-Vegas/11359-Corsica-Mist-Ave-89135/home/29286184',
'https://www.redfin.com/NV/Las-Vegas/6022-Devers-Ct-89118/home/29767766',
'https://www.redfin.com/NV/Las-Vegas/6344-Brittany-Way-89107/home/29231288',
'https://www.redfin.com/NV/Las-Vegas/6928-Cobblestone-Ave-89145/home/29224550',
'https://www.redfin.com/NV/Las-Vegas/8786-Sand-Verbena-Ave-89117/home/173631446',
'https://www.redfin.com/NV/Las-Vegas/10189-Chestnut-Wood-Ave-89148/home/29320746',
'https://www.redfin.com/NV/Las-Vegas/9577-Rolling-Thunder-Ave-89148/home/29032793',
'https://www.redfin.com/NV/Las-Vegas/400-Bloomingdale-Ct-89144/home/29111549',
'https://www.redfin.com/NV/Las-Vegas/2719-Artic-St-89121/home/29471232',
'https://www.redfin.com/NV/Las-Vegas/2892-San-Lago-Ct-89121/home/29473234',
'https://www.redfin.com/NV/Las-Vegas/5323-Hollymead-Dr-89135/home/29054518',
'https://www.redfin.com/NV/Las-Vegas/2906-Bel-Air-Dr-89109/home/29467121',
'https://www.redfin.com/NV/Las-Vegas/8049-Misty-Canyon-Ave-89113/home/166317171',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89145/home/29226858',
'https://www.redfin.com/NV/Las-Vegas/4122-Adobe-Palms-St-89135/unit-324/home/187590246',
'https://www.redfin.com/NV/Las-Vegas/10956-Viaje-Ave-89135/home/167753668',
'https://www.redfin.com/NV/Las-Vegas/9709-Terrace-Green-Ave-89117/home/29529308',
'https://www.redfin.com/NV/Las-Vegas/5625-Steampunk-ST-89118/unit-26/home/187577397',
'https://www.redfin.com/NV/Las-Vegas/7246-Vireo-Dr-89147/home/29567751',
'https://www.redfin.com/NV/Las-Vegas/2212-Constantine-Ave-89101/home/29335966',
'https://www.redfin.com/NV/Las-Vegas/2287-Red-Bud-St-89135/home/28975900',
'https://www.redfin.com/NV/Las-Vegas/5820-Spectacular-Bid-St-89113/home/29585560',
'https://www.redfin.com/NV/Las-Vegas/109-Golf-Course-St-89145/home/29205798',
'https://www.redfin.com/NV/Las-Vegas/6648-Villa-Bonita-Rd-89146/home/29543608',
'https://www.redfin.com/NV/Las-Vegas/321-Yardarm-Way-89145/home/29201618',
'https://www.redfin.com/NV/Las-Vegas/10238-Donde-Ave-89135/home/28978855',
'https://www.redfin.com/NV/Las-Vegas/2851-S-Valley-View-Blvd-89102/unit-1058/home/29464720',
'https://www.redfin.com/NV/Las-Vegas/1854-Taylorville-St-89135/home/29897186',
'https://www.redfin.com/NV/Las-Vegas/6035-Journeys-End-St-89148/home/28979056',
'https://www.redfin.com/NV/Las-Vegas/3912-El-Conlon-Ave-89102/home/29463685',
'https://www.redfin.com/NV/Las-Vegas/2208-Santa-Ynez-Dr-89104/home/29455436',
'https://www.redfin.com/NV/Las-Vegas/3004-Gilmary-Ave-89102/home/29458437',
'https://www.redfin.com/NV/Las-Vegas/10250-Sun-Dusk-Ln-89144/home/29112806',
'https://www.redfin.com/NV/Las-Vegas/6233-Elton-Ave-89107/home/29228965',
'https://www.redfin.com/NV/Las-Vegas/6113-Elton-Ave-89107/home/29229057',
'https://www.redfin.com/NV/Las-Vegas/4919-Mount-Pleasant-Ln-89113/home/29582118',
'https://www.redfin.com/NV/Las-Vegas/304-Deauville-St-89106/home/29347494',
'https://www.redfin.com/NV/Las-Vegas/2851-S-Valley-View-Blvd-89102/unit-1137B/home/29464484',
'https://www.redfin.com/NV/Las-Vegas/10582-Mandarino-Ave-89135/home/29898569',
'https://www.redfin.com/NV/Las-Vegas/3116-Sonata-Dr-89121/home/29475557',
'https://www.redfin.com/NV/Las-Vegas/3369-Milenko-Dr-89121/home/29508410',
'https://www.redfin.com/NV/Las-Vegas/1020-Eaglewood-Dr-89144/home/29113367',
'https://www.redfin.com/NV/Las-Vegas/3847-S-Eastern-Ave-89169/home/29490019',
'https://www.redfin.com/NV/Las-Vegas/2516-Rainbow-Cactus-Ct-89106/home/28973538',
'https://www.redfin.com/NV/Las-Vegas/5927-Trail-Roundup-Ln-89113/home/29585366',
'https://www.redfin.com/NV/Las-Vegas/1301-Eaton-Dr-89102/home/29458730',
'https://www.redfin.com/NV/Las-Vegas/466-Las-Casitas-Way-89121/home/29411837',
'https://www.redfin.com/NV/Las-Vegas/4122-Mississippi-Ave-89103/home/29503910',
'https://www.redfin.com/NV/Las-Vegas/6887-Truffle-Gray-St-89148/home/179980384',
'https://www.redfin.com/NV/Las-Vegas/1521-Padova-Dr-89117/home/29523906',
'https://www.redfin.com/NV/Las-Vegas/5481-Birchbrook-Ct-89120/home/29511586',
'https://www.redfin.com/NV/Las-Vegas/6881-Steel-Mist-St-89148/home/169231812',
'https://www.redfin.com/NV/Las-Vegas/7814-Harbour-Towne-Ave-89113/home/29583264',
'https://www.redfin.com/NV/Las-Vegas/4134-Mississippi-Ave-89103/home/29503912',
'https://www.redfin.com/NV/Las-Vegas/2008-Summit-Pointe-Dr-89117/home/29527436',
'https://www.redfin.com/NV/Las-Vegas/1673-Caliente-Ct-89119/home/29513454',
'https://www.redfin.com/NV/Las-Vegas/10-Diamond-Cir-89106/home/29337868',
'https://www.redfin.com/NV/Las-Vegas/2784-Belcastro-St-89117/home/29537078',
'https://www.redfin.com/NV/Las-Vegas/8102-Ancient-Oaks-Ave-89113/home/113713037',
'https://www.redfin.com/NV/Las-Vegas/4774-Willow-Glen-Dr-89147/home/29562057',
'https://www.redfin.com/NV/Las-Vegas/3973-El-Camino-Rd-89103/home/29546066',
'https://www.redfin.com/NV/Las-Vegas/7137-Mission-Hills-Dr-89113/home/29582522',
'https://www.redfin.com/NV/Las-Vegas/6537-Wild-River-Dr-89108/home/29192064',
'https://www.redfin.com/NV/Las-Vegas/3627-Blackstone-St-89121/home/29474319',
'https://www.redfin.com/NV/Las-Vegas/6181-Foothill-Blvd-89118/home/29580037',
'https://www.redfin.com/NV/Las-Vegas/6213-Laredo-St-89146/home/29538542',
'https://www.redfin.com/NV/Las-Vegas/9816-Dryden-Ct-89148/home/109634253',
'https://www.redfin.com/NV/Las-Vegas/8970-Flying-Frog-Ave-89148/home/29954996',
'https://www.redfin.com/NV/Las-Vegas/3943-Tirana-Way-89103/home/29543771',
'https://www.redfin.com/NV/Las-Vegas/7123-Paradise-Bay-Dr-89119/home/29594010',
'https://www.redfin.com/NV/Las-Vegas/10982-Village-Ridge-Ln-89135/home/29913008',
'https://www.redfin.com/NV/Las-Vegas/4512-Long-Horse-Ct-89147/home/29559187',
'https://www.redfin.com/NV/Las-Vegas/1016-Monticello-Dr-89107/home/29232210',
'https://www.redfin.com/NV/Las-Vegas/160-Nene-Ct-89144/home/29113577',
'https://www.redfin.com/NV/Las-Vegas/3019-Azure-Bay-St-89117/home/29536155',
'https://www.redfin.com/NV/Las-Vegas/2332-Stockton-Ave-89104/home/29454677',
'https://www.redfin.com/NV/Las-Vegas/3013-Piedmont-Ave-89102/home/29465036',
'https://www.redfin.com/NV/Las-Vegas/4453-Alto-Verde-Dr-89119/home/29506236',
'https://www.redfin.com/NV/Las-Vegas/10366-Romantico-Dr-89135/home/28989422',
'https://www.redfin.com/NV/Las-Vegas/11321-Altura-Vista-Dr-89138/home/29003569',
'https://www.redfin.com/NV/Las-Vegas/2005-Stewart-Ave-89101/home/29349858',
'https://www.redfin.com/NV/Las-Vegas/1200-Douglas-Dr-89102/home/29458570',
'https://www.redfin.com/NV/Las-Vegas/7748-Success-Ct-89145/home/29219449',
'https://www.redfin.com/NV/Las-Vegas/5517-Morendo-Dr-89107/home/29188222',
'https://www.redfin.com/NV/Las-Vegas/6437-Brittany-Way-89107/home/29227976',
'https://www.redfin.com/NV/Las-Vegas/4404-Shortleaf-St-89119/home/29506085',
'https://www.redfin.com/NV/Las-Vegas/10517-Grazia-Ave-89135/home/28989843',
'https://www.redfin.com/NV/Las-Vegas/4766-S-Mojave-Rd-89121/home/29509601',
'https://www.redfin.com/NV/Las-Vegas/1721-Ginger-Tree-Ln-89104/home/29451224',
'https://www.redfin.com/NV/Las-Vegas/9062-Ellington-Hill-Ln-89148/home/113713627',
'https://www.redfin.com/NV/Las-Vegas/5613-Ridgeline-Ave-89107/home/29187339',
'https://www.redfin.com/NV/Las-Vegas/2006-Yaupon-Ave-89101/home/109623358',
'https://www.redfin.com/NV/Las-Vegas/5641-Steampunk-ST-89118/unit-22/home/187414212',
'https://www.redfin.com/NV/Las-Vegas/300-Wild-Plum-Ln-89107/home/28968764',
'https://www.redfin.com/NV/Las-Vegas/5631-Sartorial-ST-89118/unit-42/home/187414168',
'https://www.redfin.com/NV/Las-Vegas/3961-Argent-Star-Ct-89147/home/29553524',
'https://www.redfin.com/NV/Las-Vegas/884-Vegas-Valley-Dr-89109/home/29465825',
'https://www.redfin.com/NV/Las-Vegas/3315-Dakota-Way-89169/home/29487232',
'https://www.redfin.com/NV/Las-Vegas/2748-Heritage-Ct-89121/home/29472276',
'https://www.redfin.com/NV/Las-Vegas/4129-Mississippi-Ave-89103/home/29503904',
'https://www.redfin.com/NV/Las-Vegas/518-Crimson-View-Pl-89144/home/29112577',
'https://www.redfin.com/NV/Las-Vegas/6248-Saginaw-Dr-89108/home/29195034',
'https://www.redfin.com/NV/Las-Vegas/1100-Allure-Dr-89128/home/29204671',
'https://www.redfin.com/NV/Las-Vegas/1201-Westlund-Dr-89102/home/29460031',
'https://www.redfin.com/NV/Las-Vegas/921-Marino-Hills-Dr-89144/home/29112371',
'https://www.redfin.com/NV/Las-Vegas/8297-Spectacle-Reef-Ave-89147/home/65558721',
'https://www.redfin.com/NV/Las-Vegas/3733-Riano-Cir-89103/home/29543870',
'https://www.redfin.com/NV/Las-Vegas/916-Vegas-Valley-Dr-89109/home/29465813',
'https://www.redfin.com/NV/Las-Vegas/10717-Refectory-Ave-89135/home/29871718',
'https://www.redfin.com/NV/Las-Vegas/6902-Emerald-Springs-Ln-89113/home/29582418',
'https://www.redfin.com/NV/Las-Vegas/2451-Sabado-St-89121/home/29474088',
'https://www.redfin.com/NV/Las-Vegas/3210-N-Heritage-Way-89121/home/29471558',
'https://www.redfin.com/NV/Las-Vegas/229-Antelope-Way-89145/home/29224454',
'https://www.redfin.com/NV/Las-Vegas/1316-S-Bruce-St-89104/home/29453102',
'https://www.redfin.com/NV/Las-Vegas/2705-Cherry-Springs-Ct-89117/home/29531021',
'https://www.redfin.com/NV/Las-Vegas/3831-Tropical-Vine-St-89147/home/29828684',
'https://www.redfin.com/NV/Las-Vegas/8729-Arizona-Poppy-Ave-89117/home/29285427',
'https://www.redfin.com/NV/Las-Vegas/8016-Bonaventure-Dr-89147/home/29564534',
'https://www.redfin.com/NV/Las-Vegas/2932-Demetrius-Ave-89101/home/29332404',
'https://www.redfin.com/NV/Las-Vegas/1717-E-McWilliams-Ave-89101/home/29336153',
'https://www.redfin.com/NV/Las-Vegas/8349-Polar-Shrimp-Ct-89113/home/29870455',
'https://www.redfin.com/NV/Las-Vegas/1867-Roxbury-Ln-89119/home/29491079',
'https://www.redfin.com/NV/Las-Vegas/5709-Roseridge-Ave-89107/home/29188101',
'https://www.redfin.com/NV/Las-Vegas/5516-Madigan-Ct-89118/home/29867525',
'https://www.redfin.com/NV/Las-Vegas/3481-Villa-Knolls-South-Dr-89120/home/29441935',
'https://www.redfin.com/NV/Las-Vegas/3451-E-Reno-Ave-89120/home/29442424',
'https://www.redfin.com/NV/Las-Vegas/1041-Bayhead-Ct-89138/home/29478574',
'https://www.redfin.com/NV/Las-Vegas/1229-8th-Pl-89104/home/29455911',
'https://www.redfin.com/NV/Las-Vegas/1005-Vantage-Point-Rd-89128/home/29209003',
'https://www.redfin.com/NV/Las-Vegas/4303-Veraz-St-89135/home/166487030',
'https://www.redfin.com/NV/Las-Vegas/3768-Bacolod-Ct-89147/home/29553918',
'https://www.redfin.com/NV/Las-Vegas/3950-Voxna-St-89119/home/29500939',
'https://www.redfin.com/NV/Las-Vegas/10323-Aragon-Crown-Rd-89135/home/52452309',
'https://www.redfin.com/NV/Las-Vegas/657-Chase-Tree-St-89144/home/29114605',
'https://www.redfin.com/NV/Las-Vegas/4796-Denaro-Dr-89135/home/29873740',
'https://www.redfin.com/NV/Las-Vegas/5684-Calm-Season-St-89118/home/167745219',
'https://www.redfin.com/NV/Las-Vegas/1043-E-Hacienda-Ave-89119/home/29516338',
'https://www.redfin.com/NV/Las-Vegas/10284-Sofferto-Ave-89135/home/29003797',
'https://www.redfin.com/NV/Las-Vegas/10609-Foggy-Glen-Ave-89135/home/29935313',
'https://www.redfin.com/NV/Las-Vegas/4312-Kay-Pl-89107/home/29343593',
'https://www.redfin.com/NV/Las-Vegas/2915-E-Cedar-Ave-89101/unit-2C/home/184151554',
'https://www.redfin.com/NV/Las-Vegas/5234-Monterey-Park-Cir-89146/home/28978214',
'https://www.redfin.com/NV/Las-Vegas/2037-Ardilea-St-89135/home/29320216',
'https://www.redfin.com/NV/Las-Vegas/3072-Vegas-Valley-Dr-89121/home/29472566',
'https://www.redfin.com/NV/Las-Vegas/7224-Bellavia-Ave-89113/home/167746595',
'https://www.redfin.com/NV/Las-Vegas/400-Redstone-St-89145/home/29225571',
'https://www.redfin.com/NV/Las-Vegas/2691-Wind-Spinner-St-89135/home/29587887',
'https://www.redfin.com/NV/Las-Vegas/8012-Rosellen-Ave-89147/home/29563992',
'https://www.redfin.com/NV/Las-Vegas/2716-Grafton-Ct-89117/home/29528345',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89148/home/29488515',
'https://www.redfin.com/NV/Las-Vegas/7176-Summer-Grove-Ave-89117/home/52629111',
'https://www.redfin.com/NV/Las-Vegas/6232-Alta-Dr-89107/home/29230279',
'https://www.redfin.com/NV/Las-Vegas/1712-Hassett-Ave-89104/home/29453968',
'https://www.redfin.com/NV/Las-Vegas/5626-Deer-Creek-Falls-Ct-89118/home/29031789',
'https://www.redfin.com/NV/Las-Vegas/1617-E-Saint-Louis-Ave-89104/home/29452620',
'https://www.redfin.com/NV/Las-Vegas/3837-Amber-Flower-Ct-89147/home/29550177',
'https://www.redfin.com/NV/Las-Vegas/6118-Meadow-Vista-Ln-89103/home/29544527',
'https://www.redfin.com/NV/Las-Vegas/3641-Laguna-del-Sol-Dr-89121/home/29473890',
'https://www.redfin.com/NV/Las-Vegas/212-Bradshaw-Way-89145/home/29219647',
'https://www.redfin.com/NV/Las-Vegas/6587-Keynote-Dr-89118/home/29579741',
'https://www.redfin.com/NV/Las-Vegas/1413-Bryn-Mawr-Ave-89102/home/29457854',
'https://www.redfin.com/NV/Las-Vegas/9609-Lofty-Vista-Ave-89148/home/58656067',
'https://www.redfin.com/NV/Las-Vegas/504-Raintree-Ln-89107/home/29193249',
'https://www.redfin.com/NV/Las-Vegas/6128-Jones-Cir-89107/home/29229897',
'https://www.redfin.com/NV/Las-Vegas/10423-Ness-Wood-Ln-89135/home/29872404',
'https://www.redfin.com/NV/Las-Vegas/5373-Homeria-St-89113/home/29954896',
'https://www.redfin.com/NV/Las-Vegas/6021-Vista-Clara-St-89148/home/113713481',
'https://www.redfin.com/NV/Las-Vegas/1275-Westwind-Rd-89146/home/29516655',
'https://www.redfin.com/NV/Las-Vegas/2304-Llewellyn-Dr-89102/home/29460139',
'https://www.redfin.com/NV/Las-Vegas/5110-Vincitor-St-89135/home/28989516',
'https://www.redfin.com/NV/Las-Vegas/1071-Sweeney-Ave-89104/home/29456169',
'https://www.redfin.com/NV/Las-Vegas/4968-Stampa-Ave-89146/home/29540115',
'https://www.redfin.com/NV/Las-Vegas/10342-Kesington-Dr-89135/home/143191437',
'https://www.redfin.com/NV/Las-Vegas/5291-Harrison-Dr-89120/home/29510332',
'https://www.redfin.com/NV/Las-Vegas/2909-Bel-Air-Dr-89109/home/29465953',
'https://www.redfin.com/NV/Las-Vegas/11122-Glen-Garden-Ct-89135/home/29886139',
'https://www.redfin.com/NV/Las-Vegas/2117-Plaza-del-Fuentes-89102/home/29459197',
'https://www.redfin.com/NV/Las-Vegas/1917-Wengert-Ave-89104/home/29453082',
'https://www.redfin.com/NV/Las-Vegas/4664-Stuttgart-St-89147/home/29559607',
'https://www.redfin.com/NV/Las-Vegas/120-N-20th-St-89101/home/29350783',
'https://www.redfin.com/NV/Las-Vegas/3912-Torsby-Pl-89119/home/29500977',
'https://www.redfin.com/NV/Las-Vegas/8711-Villa-Ariel-Ln-89147/home/29559332',
'https://www.redfin.com/NV/Las-Vegas/1815-Cape-Cod-Landing-Dr-89135/home/29320678',
'https://www.redfin.com/NV/Las-Vegas/5226-Monterey-Park-Cir-89146/home/28979661',
'https://www.redfin.com/NV/Las-Vegas/6067-Dunham-Hills-Way-89113/home/29935269',
'https://www.redfin.com/NV/Las-Vegas/10495-Heritage-Park-Ave-89135/home/29897028',
'https://www.redfin.com/NV/Las-Vegas/5925-Gipsy-Ave-89107/home/29232856',
'https://www.redfin.com/NV/Las-Vegas/10518-Broadhead-Ct-89135/home/29587834',
'https://www.redfin.com/NV/Las-Vegas/9958-Regency-Square-Ave-89148/home/166319083',
'https://www.redfin.com/NV/Las-Vegas/5256-Sunny-Beach-Ln-89118/home/29577243',
'https://www.redfin.com/NV/Las-Vegas/6109-Empire-Cir-89107/home/29197325',
'https://www.redfin.com/NV/Las-Vegas/3549-Casa-Real-Way-89147/home/29551854',
'https://www.redfin.com/NV/Las-Vegas/5609-Harmony-Ave-89107/home/29232321',
'https://www.redfin.com/NV/Las-Vegas/6891-Mojave-Sage-Ct-89148/home/146417319',
'https://www.redfin.com/NV/Las-Vegas/909-E-Saint-Louis-Ave-89104/home/29456976',
'https://www.redfin.com/NV/Las-Vegas/4870-W-Montara-Cir-89121/home/29509648',
'https://www.redfin.com/NV/Las-Vegas/6325-Burgundy-Way-89107/home/29230001',
'https://www.redfin.com/NV/Las-Vegas/4142-Cascada-Piazza-Ln-89135/home/29898639',
'https://www.redfin.com/NV/Las-Vegas/7465-Sunspot-Dr-89128/home/29197511',
'https://www.redfin.com/NV/Las-Vegas/10977-Snow-Cloud-Ct-89135/home/29916977',
'https://www.redfin.com/NV/Las-Vegas/4342-Cherrystone-Ct-89121/home/29508434',
'https://www.redfin.com/NV/Las-Vegas/9800-Stellar-View-Ave-89117/home/29527938',
'https://www.redfin.com/NV/Las-Vegas/5409-Banjo-St-89107/home/29231896',
'https://www.redfin.com/NV/Las-Vegas/3146-Juanita-Dr-89102/home/29463078',
'https://www.redfin.com/NV/Las-Vegas/5568-Yellow-Ash-St-89118/home/29580667',
'https://www.redfin.com/NV/Las-Vegas/2001-Red-Rock-St-89146/home/29516523',
'https://www.redfin.com/NV/Las-Vegas/3958-Windansea-St-89147/home/29554010',
'https://www.redfin.com/NV/Las-Vegas/8143-Retriever-Ave-89147/home/29030645',
'https://www.redfin.com/NV/Las-Vegas/2205-Nieman-Way-89146/home/29520361',
'https://www.redfin.com/NV/Las-Vegas/2951-Aruba-Ct-89121/home/52629695',
'https://www.redfin.com/NV/Las-Vegas/3970-Equinox-Ct-89147/home/29553606',
'https://www.redfin.com/NV/Las-Vegas/3742-McLeod-Dr-89121/home/29475631',
'https://www.redfin.com/NV/Las-Vegas/3226-Astoria-Dr-89121/home/29472683',
'https://www.redfin.com/NV/Las-Vegas/2318-Howard-Dr-89104/home/29451242',
'https://www.redfin.com/NV/Las-Vegas/332-Wild-Plum-Ln-89107/home/29234286',
'https://www.redfin.com/NV/Las-Vegas/2708-Yardley-St-89102/home/29465029',
'https://www.redfin.com/NV/Las-Vegas/11286-Cactus-Tower-Ave-89135/unit-101/home/173454080',
'https://www.redfin.com/NV/Las-Vegas/9110-Rushing-Wind-Ave-89148/home/29950919',
'https://www.redfin.com/NV/Las-Vegas/10160-Hollis-Mountain-Ave-89148/home/29287499',
'https://www.redfin.com/NV/Las-Vegas/4488-Los-Reyes-Ct-89121/home/29507905',
'https://www.redfin.com/NV/Las-Vegas/9328-Sienna-Vista-Dr-89117/home/29523313',
'https://www.redfin.com/NV/Las-Vegas/3427-Biela-Ave-89120/home/29443450',
'https://www.redfin.com/NV/Las-Vegas/6100-Castle-Bay-Dr-89108/home/29196225',
'https://www.redfin.com/NV/Las-Vegas/1204-Smith-St-89108/home/29187464',
'https://www.redfin.com/NV/Las-Vegas/5376-Bocopa-St-89148/home/109055740',
'https://www.redfin.com/NV/Las-Vegas/6532-Moon-Roses-Ct-89108/home/29192026',
'https://www.redfin.com/NV/Las-Vegas/7778-Ebro-Valley-Ct-89113/home/109634431',
'https://www.redfin.com/NV/Las-Vegas/4434-Los-Reyes-Ct-89121/home/29507897',
'https://www.redfin.com/NV/Las-Vegas/Opal-Point/Barclay/home/184881557',
'https://www.redfin.com/NV/Las-Vegas/Opal-Point/Bedford/home/184881612',
'https://www.redfin.com/NV/Las-Vegas/Opal-Point/Griffin/home/184881690',
'https://www.redfin.com/NV/Las-Vegas/Tempo/Plan-1/home/185345820',
'https://www.redfin.com/NV/Las-Vegas/6207-Runestone-St-89113/home/183984980',
'https://www.redfin.com/NV/Las-Vegas/6264-Dolosonte-St-89113/home/183985004',
'https://www.redfin.com/NV/Las-Vegas/9545-W-Cherokee-Ave-89147/home/29555819',
'https://www.redfin.com/NV/Las-Vegas/5636-Bolton-Valley-Dr-89122/home/29001320',
'https://www.redfin.com/NV/Las-Vegas/5675-Alcott-Ave-89142/home/29406456',
'https://www.redfin.com/NV/Las-Vegas/4769-Terra-Linda-Ave-89120/home/29440466',
'https://www.redfin.com/NV/Las-Vegas/3586-Villa-Knolls-East-Dr-89120/home/29441764',
'https://www.redfin.com/NV/Las-Vegas/4164-Don-Bonito-St-89121/home/29426771',
'https://www.redfin.com/NV/Las-Vegas/5163-Woodruff-Pl-89120/home/29438267',
'https://www.redfin.com/NV/Las-Vegas/5636-Blue-Sea-St-89110/home/29395363',
'https://www.redfin.com/NV/Las-Vegas/2444-Winterwood-Blvd-89142/home/29405739',
'https://www.redfin.com/NV/Henderson/752-Panhandle-Dr-89014/home/29646550',
'https://www.redfin.com/NV/Las-Vegas/494-Mount-Gravatt-Ct-89110/home/29004288',
'https://www.redfin.com/NV/Las-Vegas/569-Greenbriar-Townhouse-Way-89121/home/29411761',
'https://www.redfin.com/NV/Las-Vegas/5035-Alp-St-89120/home/29440451',
'https://www.redfin.com/NV/Las-Vegas/4116-Park-St-89110/home/29388705',
'https://www.redfin.com/NV/Las-Vegas/4477-El-Pito-Ln-89121/home/29427249',
'https://www.redfin.com/NV/Las-Vegas/5694-Roundrock-Dr-89142/home/29416637',
'https://www.redfin.com/NV/Las-Vegas/5485-Silverheart-Ave-89142/home/29404659',
'https://www.redfin.com/NV/Las-Vegas/2747-Sunrise-Bluff-Dr-89142/home/49786771',
'https://www.redfin.com/NV/Henderson/447-Ruby-Lark-St-89011/unit-139/home/187671328',
'https://www.redfin.com/NV/Las-Vegas/911-Stonehenge-St-89110/home/29380233',
'https://www.redfin.com/NV/Las-Vegas/4607-Mountain-Vista-St-89121/home/29428479',
'https://www.redfin.com/NV/Las-Vegas/4530-Glenndavis-Dr-89121/home/29429959',
'https://www.redfin.com/NV/Las-Vegas/4287-Twilight-St-89122/home/29430430',
'https://www.redfin.com/NV/Las-Vegas/5970-Via-Capri-89122/home/29433088',
'https://www.redfin.com/NV/Las-Vegas/5475-Criollo-Dr-89122/home/28968413',
'https://www.redfin.com/NV/Las-Vegas/3548-Haverford-Ave-89121/home/29422957',
'https://www.redfin.com/NV/Las-Vegas/5629-Fairlight-Dr-89142/home/29405647',
'https://www.redfin.com/NV/Las-Vegas/4757-Vista-Flora-Way-89121/home/29421321',
'https://www.redfin.com/NV/Las-Vegas/4324-Callahan-Ave-89120/home/29446241',
'https://www.redfin.com/NV/Las-Vegas/5209-Glen-Eagles-Ln-89108/home/29189409',
'https://www.redfin.com/NV/Las-Vegas/3905-Autumn-St-89120/home/29443200',
'https://www.redfin.com/NV/Las-Vegas/5059-Long-View-Dr-89120/home/29443213',
'https://www.redfin.com/NV/Las-Vegas/980-Looking-Glass-Ln-89110/home/29376743',
'https://www.redfin.com/NV/Las-Vegas/4414-E-Viking-Rd-89121/home/29420274',
'https://www.redfin.com/NV/Las-Vegas/4137-Oxnard-Cir-89121/home/29427079',
'https://www.redfin.com/NV/Las-Vegas/4642-E-Utah-Ave-89104/home/29409223',
'https://www.redfin.com/NV/Las-Vegas/724-N-Fogg-St-89110/home/29396547',
'https://www.redfin.com/NV/Las-Vegas/3432-Wrangell-Mountain-St-89122/home/29015582',
'https://www.redfin.com/NV/Las-Vegas/445-N-Lamb-Blvd-89110/unit-A/home/29387579',
'https://www.redfin.com/NV/Las-Vegas/324-Kane-Ave-89110/home/29390886',
'https://www.redfin.com/NV/Las-Vegas/4048-Harris-Ave-89110/home/29385599',
'https://www.redfin.com/NV/Las-Vegas/3412-Townhouse-Dr-89121/home/29422108',
'https://www.redfin.com/NV/Las-Vegas/4214-E-Baltimore-Ave-89104/home/29408289',
'https://www.redfin.com/NV/Las-Vegas/4033-Russian-Rider-Dr-89122/home/29282743',
'https://www.redfin.com/NV/Las-Vegas/3730-Crellin-Cir-89120/home/29443324',
'https://www.redfin.com/NV/Las-Vegas/4905-Rita-Dr-89121/home/29430557',
'https://www.redfin.com/NV/Henderson/1549-Orchard-Falls-Ct-89014/home/144672921',
'https://www.redfin.com/NV/Las-Vegas/5987-Natures-Dr-89122/home/29431980',
'https://www.redfin.com/NV/Las-Vegas/1498-Sunburst-Dr-89110/home/29376477',
'https://www.redfin.com/NV/Las-Vegas/6171-Canter-Glen-Ave-89122/home/29482181',
'https://www.redfin.com/NV/Las-Vegas/4051-Shetland-Pony-St-89122/home/29481807',
'https://www.redfin.com/NV/Las-Vegas/6876-Bronze-Meadow-Ave-89122/home/29062461',
'https://www.redfin.com/NV/Las-Vegas/6380-Tumblegrass-Ct-89122/home/29030151',
'https://www.redfin.com/NV/Las-Vegas/3930-Ivyhill-Ave-89121/home/29426452',
'https://www.redfin.com/NV/Las-Vegas/1621-Sunrise-View-Dr-89156/home/29370740',
'https://www.redfin.com/NV/Las-Vegas/4409-Bluecrest-Rd-89121/home/29426184',
'https://www.redfin.com/NV/Las-Vegas/3780-Garden-South-Dr-89121/home/29423141',
'https://www.redfin.com/NV/Las-Vegas/6409-Dundock-Ave-89122/home/52629482',
'https://www.redfin.com/NV/Las-Vegas/5330-E-Charleston-Blvd-89142/unit-143/home/29393738',
'https://www.redfin.com/NV/Las-Vegas/4814-Plata-del-Sol-Dr-89121/home/29428846',
'https://www.redfin.com/NV/Las-Vegas/1231-Honey-Lake-St-89110/home/29378281',
'https://www.redfin.com/NV/Las-Vegas/3601-Budlong-Ave-89110/home/29386083',
'https://www.redfin.com/NV/Las-Vegas/6639-Angelina-St-89120/home/113759262',
'https://www.redfin.com/NV/Las-Vegas/2097-Audrey-Hepburn-St-89142/home/29864469',
'https://www.redfin.com/NV/Las-Vegas/6579-Isolated-Ave-89110/home/29397413',
'https://www.redfin.com/NV/Las-Vegas/1970-Citroen-St-89142/home/29404547',
'https://www.redfin.com/NV/Las-Vegas/4077-Monthill-Ave-89121/home/29426525',
'https://www.redfin.com/NV/Las-Vegas/5221-Apple-Valley-Ln-89108/home/29189511',
'https://www.redfin.com/NV/Las-Vegas/4392-Springdale-Ct-89121/home/29413841',
'https://www.redfin.com/NV/Las-Vegas/6373-Varna-Ave-89110/home/29378162',
'https://www.redfin.com/NV/Las-Vegas/1461-Arden-St-89104/home/29407588',
'https://www.redfin.com/NV/Las-Vegas/48-Prince-Ln-89110/home/29388621',
'https://www.redfin.com/NV/Las-Vegas/502-Battle-Mountain-Dr-89110/home/29393755',
'https://www.redfin.com/NV/Las-Vegas/5400-Raccoon-Valley-Ln-89122/home/29242731',
'https://www.redfin.com/NV/Las-Vegas/5376-Hillsboro-Ln-89120/home/29440604',
'https://www.redfin.com/NV/Henderson/810-Long-Branch-Dr-89014/home/29446636',
'https://www.redfin.com/NV/Las-Vegas/3797-Monument-St-89121/home/29423256',
'https://www.redfin.com/NV/Las-Vegas/1112-Pleasant-Brook-St-89142/home/29403050',
'https://www.redfin.com/NV/Las-Vegas/6596-Secluded-Ave-89110/home/29397386',
'https://www.redfin.com/NV/Henderson/1465-Grub-Stake-Cir-89014/home/29446308',
'https://www.redfin.com/NV/Las-Vegas/5947-Zawawi-Ct-89110/home/29396487',
'https://www.redfin.com/NV/Las-Vegas/2941-Vegas-Dr-89106/home/29339449',
'https://www.redfin.com/NV/Las-Vegas/5962-Crossfield-Ave-89122/home/29599913',
'https://www.redfin.com/NV/North-Las-Vegas/804-E-Owens-Ave-89030/home/29327254',
'https://www.redfin.com/NV/Las-Vegas/546-Greenbriar-Townhouse-Way-89121/home/29411752',
'https://www.redfin.com/NV/Las-Vegas/4710-Monterrey-Ave-89121/home/29428994',
'https://www.redfin.com/NV/Las-Vegas/5078-Bloodhound-St-89122/home/169230634',
'https://www.redfin.com/NV/Las-Vegas/2794-Murray-Hill-Ln-89142/home/29480757',
'https://www.redfin.com/NV/Las-Vegas/5272-Blanton-Dr-89122/home/29430197',
'https://www.redfin.com/NV/Henderson/1522-Plain-Sight-Ave-89014/home/29446750',
'https://www.redfin.com/NV/Las-Vegas/3767-Cannon-Ave-89121/home/29423469',
'https://www.redfin.com/NV/Las-Vegas/5643-Forsythe-Dr-89142/home/29406334',
'https://www.redfin.com/NV/Las-Vegas/4374-E-Cleveland-Ave-89104/home/29408322',
'https://www.redfin.com/NV/Las-Vegas/3398-El-Camino-Real-89121/home/29422080',
'https://www.redfin.com/NV/Las-Vegas/6188-Exquisite-Ave-89110/home/29933258',
'https://www.redfin.com/NV/Las-Vegas/3629-Springwood-St-89121/home/29423676',
'https://www.redfin.com/NV/Las-Vegas/6745-Keren-Marie-Ave-89110/home/29397209',
'https://www.redfin.com/NV/Las-Vegas/555-Roxella-Ln-89110/unit-C/home/29389043',
'https://www.redfin.com/NV/Las-Vegas/3964-Shetland-Pony-St-89122/home/29481790',
'https://www.redfin.com/NV/Las-Vegas/3883-Winter-Whitetail-St-89122/home/29015114',
'https://www.redfin.com/NV/Las-Vegas/3608-Tundra-Swan-St-89122/home/29029410',
'https://www.redfin.com/NV/Las-Vegas/2571-Amber-Crest-St-89142/home/29827855',
'https://www.redfin.com/NV/Las-Vegas/3729-La-Junta-Dr-89120/home/29441643',
'https://www.redfin.com/NV/Las-Vegas/4620-High-Anchor-St-89121/home/113713515',
'https://www.redfin.com/NV/Las-Vegas/4078-El-Segundo-Ave-89121/home/187007106',
'https://www.redfin.com/NV/Las-Vegas/78-Page-St-89110/home/29390343',
'https://www.redfin.com/NV/Las-Vegas/1221-Arrowhead-Ave-89106/home/29338302',
'https://www.redfin.com/NV/Las-Vegas/5019-Volcanic-Rock-Ln-89122/home/28971300',
'https://www.redfin.com/NV/Las-Vegas/6849-Wispy-Sky-Ct-89142/home/144626761',
'https://www.redfin.com/NV/Las-Vegas/5740-Auckland-Dr-89110/home/29380075',
'https://www.redfin.com/NV/Las-Vegas/4795-Straight-Flush-Dr-89122/unit-102/home/30292785',
'https://www.redfin.com/NV/Las-Vegas/3820-Placita-del-Rico-89120/home/29445162',
'https://www.redfin.com/NV/Las-Vegas/3708-Valley-Forge-Ave-89110/home/29384196',
'https://www.redfin.com/NV/Las-Vegas/4863-E-Utah-Ave-89104/home/29409244',
'https://www.redfin.com/NV/Las-Vegas/1211-Purple-Sage-Ave-89108/home/29338272',
'https://www.redfin.com/NV/Las-Vegas/1407-Fields-St-89142/home/29399755',
'https://www.redfin.com/NV/Las-Vegas/3982-Avebury-Pl-89121/home/29424645',
'https://www.redfin.com/NV/Las-Vegas/6378-Flathead-Ave-89122/home/29029690',
'https://www.redfin.com/NV/Las-Vegas/4450-Rita-Dr-89121/home/29428284',
'https://www.redfin.com/NV/Las-Vegas/5421-Chela-Dr-89120/home/29444611',
'https://www.redfin.com/NV/Las-Vegas/3795-Heritage-Ave-89121/home/29423379',
'https://www.redfin.com/NV/Henderson/862-Bronzite-St-89011/unit-21/home/187371515',
'https://www.redfin.com/NV/Henderson/854-Bronzite-St-89011/unit-23/home/187371446',
'https://www.redfin.com/NV/Las-Vegas/4603-E-Twain-Ave-89121/home/29421748',
'https://www.redfin.com/NV/Las-Vegas/4595-Califa-Dr-89122/home/29433333',
'https://www.redfin.com/NV/Henderson/902-Bronzite-St-89011/unit-11/home/187364852',
'https://www.redfin.com/NV/Las-Vegas/5456-Stacked-Chips-Rd-89122/unit-101/home/29482585',
'https://www.redfin.com/NV/Las-Vegas/5483-Walton-Heath-Ave-89142/home/29416866',
'https://www.redfin.com/NV/Las-Vegas/4224-Estaban-Ct-89110/home/29385745',
'https://www.redfin.com/NV/Las-Vegas/1747-Angel-Falls-St-89142/home/29400083',
'https://www.redfin.com/NV/Las-Vegas/1241-Blazing-Sand-St-89110/home/29378262',
'https://www.redfin.com/NV/Las-Vegas/5776-Yerington-Ave-89110/home/29394128',
'https://www.redfin.com/NV/Las-Vegas/1212-N-St-89106/home/29338702',
'https://www.redfin.com/NV/Las-Vegas/4206-Ridgecrest-Dr-89121/home/29425110',
'https://www.redfin.com/NV/Las-Vegas/6150-Fox-Creek-Ave-89122/home/29482140',
'https://www.redfin.com/NV/Las-Vegas/3751-Salt-Cedar-Ln-89121/home/29425141',
'https://www.redfin.com/NV/Las-Vegas/5118-Clatsop-St-89122/home/160680492',
'https://www.redfin.com/NV/Las-Vegas/5829-Swan-Point-Pl-89122/home/29921250',
'https://www.redfin.com/NV/Las-Vegas/4023-Welter-Ave-89104/home/183049636',
'https://www.redfin.com/NV/Las-Vegas/4017-Welter-Ave-89104/home/183049635',
'https://www.redfin.com/NV/Las-Vegas/5206-Brentmead-Dr-89120/home/29443869',
'https://www.redfin.com/NV/Las-Vegas/4005-Radbourne-Ave-89121/home/29424779',
'https://www.redfin.com/NV/Las-Vegas/2541-Madre-Grande-St-89142/home/29005139',
'https://www.redfin.com/NV/Las-Vegas/5853-Killala-Ct-89110/home/29379674',
'https://www.redfin.com/NV/Las-Vegas/358-Linn-Ln-89110/home/29394094',
'https://www.redfin.com/NV/Henderson/893-Adobe-Flat-Dr-89011/home/29447991',
'https://www.redfin.com/NV/Las-Vegas/1170-Fogg-St-89142/home/29002676',
'https://www.redfin.com/NV/Las-Vegas/4485-El-Como-Way-89121/home/29428473',
'https://www.redfin.com/NV/Henderson/916-Derringer-Ln-89014/home/29446849',
'https://www.redfin.com/NV/Las-Vegas/4125-E-Harmon-Ave-89121/home/29426254',
'https://www.redfin.com/NV/Las-Vegas/6370-Point-Isabel-Way-89122/home/145870800',
'https://www.redfin.com/NV/Las-Vegas/6475-Smithville-Ave-89142/home/169230519',
'https://www.redfin.com/NV/Henderson/1486-Orange-Jubilee-Rd-89014/home/29053081',
'https://www.redfin.com/NV/Las-Vegas/5931-Ablette-Ave-89122/home/29432747',
'https://www.redfin.com/NV/Las-Vegas/4927-Schumann-Dr-89146/home/29540283',
'https://www.redfin.com/NV/Las-Vegas/3945-Wesvan-Ct-89121/home/177328815',
'https://www.redfin.com/NV/Las-Vegas/3933-Wesvan-Ct-89121/home/177328813',
'https://www.redfin.com/NV/Las-Vegas/3927-Wesvan-Ct-89121/home/177328812',
'https://www.redfin.com/NV/Las-Vegas/3915-Wesvan-Ct-89121/home/177328810',
'https://www.redfin.com/NV/Las-Vegas/889-Lulu-Ave-89119/home/29515217',
'https://www.redfin.com/NV/Las-Vegas/2619-Burton-Ave-89102/home/29460218',
'https://www.redfin.com/NV/Las-Vegas/3669-Bufflehead-St-89122/home/29029346',
'https://www.redfin.com/NV/Las-Vegas/3464-Monte-Carlo-Dr-89121/home/29423041',
'https://www.redfin.com/NV/Las-Vegas/4300-Don-Quixote-St-89121/home/29428866',
'https://www.redfin.com/NV/Las-Vegas/3841-Budenny-Dr-89122/home/29181882',
'https://www.redfin.com/NV/Las-Vegas/6580-Clara-Bow-Ave-89122/unit-103/home/109598672',
'https://www.redfin.com/NV/Las-Vegas/3986-Beisner-St-89122/home/29282776',
'https://www.redfin.com/NV/Las-Vegas/3704-Lighthouse-Ave-89110/home/29386400',
'https://www.redfin.com/NV/Las-Vegas/4274-Swandale-Ave-89121/home/29413283',
'https://www.redfin.com/NV/Las-Vegas/7041-Montana-Ave-89110/home/29397162',
'https://www.redfin.com/NV/Las-Vegas/4037-Chalfont-Ct-89121/home/29424845',
'https://www.redfin.com/NV/Las-Vegas/656-Lincoln-Rd-89110/home/29385807',
'https://www.redfin.com/NV/Las-Vegas/6120-Fox-Creek-Ave-89122/home/29482137',
'https://www.redfin.com/NV/Las-Vegas/3553-E-Viking-Rd-89121/home/29423603',
'https://www.redfin.com/NV/Las-Vegas/4874-Longshot-Dr-89122/home/29600004',
'https://www.redfin.com/NV/Las-Vegas/4533-Charles-Ronald-Ave-89121/home/29421631',
'https://www.redfin.com/NV/Las-Vegas/4000-E-Twain-Ave-89121/home/29423908',
'https://www.redfin.com/NV/Henderson/331-Via-Della-Greca-89011/home/108522952',
'https://www.redfin.com/NV/Henderson/914-Cottonwood-Hill-Pl-89011/home/181151205',
'https://www.redfin.com/NV/Henderson/912-Cottonwood-Hill-Pl-89011/home/181151204',
'https://www.redfin.com/NV/Henderson/526-Celebratory-Pl-89011/home/181151158',
'https://www.redfin.com/NV/Henderson/524-Celebratory-Pl-89011/home/181151157',
'https://www.redfin.com/NV/Henderson/145-Rolling-Cove-Ave-89011/home/29315775',
'https://www.redfin.com/NV/Henderson/140-Cascade-Meadow-Ct-89011/home/52623522',
'https://www.redfin.com/NV/Henderson/1600-Lorna-Dr-89011/home/29643394',
'https://www.redfin.com/NV/Henderson/812-Klamath-Springs-St-89011/home/182475596',
'https://www.redfin.com/NV/Henderson/69-Cathedral-Wash-Pl-89011/home/177327917',
'https://www.redfin.com/NV/Henderson/1085-Cactus-Rock-St-89011/home/52623255',
'https://www.redfin.com/NV/Henderson/73-Cathedral-Wash-Pl-89011/home/177327916',
'https://www.redfin.com/NV/Henderson/1137-Ruby-Sparrow-St-89011/unit-468/home/187659237',
'https://www.redfin.com/NV/Henderson/861-Greenspire-St-Unknown/home/185107401',
'https://www.redfin.com/NV/Henderson/100-Emerald-Summit-Ln-89011/home/177309458',
'https://www.redfin.com/NV/Henderson/104-Emerald-Summit-Ln-89011/home/177309456',
'https://www.redfin.com/NV/Henderson/102-Emerald-Summit-Ln-89011/home/177309457',
'https://www.redfin.com/NV/Henderson/106-Emerald-Summit-Ln-89011/home/177309455',
'https://www.redfin.com/NV/Henderson/340-Heritage-Bridge-Ave-89011/home/167786581',
'https://www.redfin.com/NV/Henderson/964-Candy-Tuft-Dr-89011/home/29398456',
'https://www.redfin.com/NV/Henderson/696-Monroe-Hill-Pl-89011/home/175528259',
'https://www.redfin.com/NV/Henderson/1610-Lorna-Dr-89011/home/29643327',
'https://www.redfin.com/NV/Henderson/867-Greenspire-St-Unknown/home/185107404',
'https://www.redfin.com/NV/Henderson/90-Emory-Hills-Ln-89011/home/179983768',
'https://www.redfin.com/NV/Henderson/863-Greenspire-St-Unknown/home/185107402',
'https://www.redfin.com/NV/Henderson/694-Monroe-Hill-Pl-89011/home/175528258',
'https://www.redfin.com/NV/Henderson/928-Via-Stellato-St-89011/home/29201823',
'https://www.redfin.com/NV/Henderson/774-Willow-Oasis-St-89011/home/167786487',
'https://www.redfin.com/NV/Henderson/387-Alla-Marcia-Ln-89011/home/147876514',
'https://www.redfin.com/NV/Henderson/1146-Olivia-Pkwy-89011/home/29052955',
'https://www.redfin.com/NV/Henderson/12-Via-del-Cielo-89011/home/146418501',
'https://www.redfin.com/NV/Henderson/16-Reflection-Cove-Dr-89011/home/167696068',
'https://www.redfin.com/NV/Henderson/513-Kristin-Ln-89011/home/29643054',
'https://www.redfin.com/NV/Henderson/54-Sonata-Dawn-Ave-89011/home/185121335',
'https://www.redfin.com/NV/Henderson/17-Mirage-View-Dr-89011/home/175524631',
'https://www.redfin.com/NV/Henderson/770-Willow-Oasis-St-89011/home/167786486',
'https://www.redfin.com/NV/Henderson/865-Greenspire-St-Unknown/home/185107403',
'https://www.redfin.com/NV/Henderson/869-Via-del-Cerchi-89011/home/92908797',
'https://www.redfin.com/NV/Henderson/477-Sterling-Falls-Ave-89011/home/109871875',
'https://www.redfin.com/NV/Henderson/833-Watford-Pl-89011/home/179983620',
'https://www.redfin.com/NV/Henderson/221-Vanhoy-Ave-89011/home/187432441',
'https://www.redfin.com/NV/Henderson/732-Wild-Highlands-Ln-89015/home/187462088',
'https://www.redfin.com/NV/Henderson/843-Hemlock-Thicket-Pl-89015/home/187462068',
'https://www.redfin.com/NV/Henderson/222-Vanhoy-Ave-89011/home/187432328',
'https://www.redfin.com/NV/Henderson/220-Vanhoy-Ave-89011/home/187432807',
'https://www.redfin.com/NV/Henderson/731-Wild-Highlands-Ln-89015/home/187451526',
'https://www.redfin.com/NV/Henderson/69-Honeymoon-Dr-89011/home/177327914',
'https://www.redfin.com/NV/Henderson/88-Tatahatso-Point-St-89011/home/177327817',
'https://www.redfin.com/NV/Henderson/733-Wild-Highlands-Ln-89015/home/187448427',
'https://www.redfin.com/NV/Henderson/700-Monroe-Hill-Pl-89011/home/175528261',
'https://www.redfin.com/NV/Henderson/260-Via-Morelli-89011/home/147873730',
'https://www.redfin.com/NV/Henderson/698-Monroe-Hill-Pl-89011/home/175528260',
'https://www.redfin.com/NV/Henderson/1182-King-Lark-St-89011/unit-52/home/187431508',
'https://www.redfin.com/NV/Henderson/919-Via-del-Campo-89011/home/92907997',
'https://www.redfin.com/NV/Henderson/706-Bamboo-Bend-Pl-89011/home/175528224',
'https://www.redfin.com/NV/Henderson/337-Meadow-Brush-Pl-89011/home/169241294',
'https://www.redfin.com/NV/Henderson/751-Shadow-Reef-St-89011/home/166427167',
'https://www.redfin.com/NV/Henderson/859-Greenspire-St-Unknown/home/185107400',
'https://www.redfin.com/NV/Henderson/4-Lago-Turchino-Ct-89011/home/57056692',
'https://www.redfin.com/NV/Henderson/1524-Chestnut-St-89011/home/29643203',
'https://www.redfin.com/NV/Henderson/111-Emerald-Summit-Ln-89011/home/177309450',
'https://www.redfin.com/NV/Henderson/148-Pace-Ave-89011/home/170975665',
'https://www.redfin.com/NV/Henderson/109-Emerald-Summit-Ln-89011/home/177309449',
'https://www.redfin.com/NV/Henderson/1178-King-Lark-St-89011/unit-50/home/187343902',
'https://www.redfin.com/NV/Henderson/1016-Viale-Placenza-Pl-89011/home/28966923',
'https://www.redfin.com/NV/Henderson/918-Cottonwood-Hill-Pl-89011/home/181151208',
'https://www.redfin.com/NV/Henderson/522-Celebratory-Pl-89011/home/181151156',
'https://www.redfin.com/NV/Henderson/518-Celebratory-Pl-89011/home/181151154',
'https://www.redfin.com/NV/Henderson/514-Celebratory-Pl-89011/home/181151152',
'https://www.redfin.com/NV/Henderson/916-Cottonwood-Hill-Pl-89011/home/181151207',
'https://www.redfin.com/NV/Henderson/520-Celebratory-Pl-89011/home/181151155',
'https://www.redfin.com/NV/Henderson/516-Celebratory-Pl-89011/home/181151153',
'https://www.redfin.com/NV/Henderson/512-Celebratory-Pl-89011/home/181151151',
'https://www.redfin.com/NV/Henderson/510-Celebratory-Pl-89011/home/181151150',
'https://www.redfin.com/NV/Henderson/508-Celebratory-Pl-89011/home/181151149',
'https://www.redfin.com/NV/Henderson/530-Via-del-Corallo-Way-89011/home/29052958',
'https://www.redfin.com/NV/Henderson/1128-Strada-Cristallo-89011/home/92908709',
'https://www.redfin.com/NV/Henderson/882-Mary-Baer-Way-89011/home/185121358',
'https://www.redfin.com/NV/Henderson/Undisclosed-address-89011/home/29643016',
'https://www.redfin.com/NV/Henderson/709-Otter-Brook-St-89011/home/166427166',
'https://www.redfin.com/NV/Las-Vegas/1044-King-Richard-Ave-89119/home/29516311',
'https://www.redfin.com/NV/Henderson/1411-Bareback-Ct-89014/home/29446830',
'https://www.redfin.com/NV/Las-Vegas/4515-S-Nellis-Blvd-89121/home/29429988',
'https://www.redfin.com/NV/Las-Vegas/3974-Saguaro-Ln-89110/home/29386348',
'https://www.redfin.com/NV/Las-Vegas/3971-Foxboro-Cir-89121/home/29424677',
'https://www.redfin.com/NV/Henderson/60-Tatahatso-Point-St-89011/home/177327893',
'https://www.redfin.com/NV/Henderson/76-Tatahatso-Point-St-89011/home/177327889',
'https://www.redfin.com/NV/Henderson/701-Moonstone-Beach-Pl-89011/home/179983739',
'https://www.redfin.com/NV/Las-Vegas/255-S-17th-St-89101/home/29349114',
'https://www.redfin.com/NV/Las-Vegas/1305-Norman-Ave-89104/home/29451798',
'https://www.redfin.com/NV/Henderson/Undisclosed-address-89011/home/144626462',
'https://www.redfin.com/NV/Henderson/105-Emerald-Summit-Ln-89011/home/177309447',
'https://www.redfin.com/NV/Henderson/113-Antler-Square-Ln-89011/home/175528267',
'https://www.redfin.com/NV/Henderson/111-Antler-Square-Ln-89011/home/175528266',
'https://www.redfin.com/NV/Henderson/52-Alta-Cascata-Pl-89011/home/172505646',
'https://www.redfin.com/NV/Henderson/Lake-Las-Vegas-Reverie/Elizabeth/home/177062580',
'https://www.redfin.com/NV/Henderson/Lake-Las-Vegas-Reverie/Ethan/home/177511510',
'https://www.redfin.com/NV/Henderson/Lake-Las-Vegas-Reverie/Antoinette/home/177537147',
'https://www.redfin.com/NV/Henderson/Lake-Las-Vegas-Reverie/Pioneer/home/177537149',
'https://www.redfin.com/NV/Henderson/Everleigh-at-Cadence/Layton/home/180412874',
'https://www.redfin.com/NV/Henderson/Jasper-Point/Cambridge/home/182418827',
'https://www.redfin.com/NV/Henderson/Jasper-Point/Albany/home/182418832',
'https://www.redfin.com/NV/Henderson/Marbella-at-Lake-Las-Vegas/Sunflower/home/186050408',
'https://www.redfin.com/NV/Henderson/Marbella-at-Lake-Las-Vegas/Violet/home/186050414',
'https://www.redfin.com/NV/Henderson/Marbella-at-Lake-Las-Vegas/Tulip/home/186050415',
'https://www.redfin.com/NV/Las-Vegas/5973-Allanche-Ave-89141/home/28991918',
'https://www.redfin.com/NV/Las-Vegas/4488-Mission-Meadow-Cir-89139/home/40548372',
'https://www.redfin.com/NV/Las-Vegas/10377-Bolting-Cloud-Dr-89178/home/29067843',
'https://www.redfin.com/NV/Las-Vegas/740-Drewsberry-Ave-89178/home/143189170',
'https://www.redfin.com/NV/Las-Vegas/10764-Valencia-Hills-St-89141/home/29887871',
'https://www.redfin.com/NV/Las-Vegas/9550-Wild-Valley-Ave-89148/home/113140838',
'https://www.redfin.com/NV/Las-Vegas/7672-Eastham-Bay-Ave-89179/home/143189677',
'https://www.redfin.com/NV/Las-Vegas/3985-Painted-Lady-Ave-89141/home/178675311',
'https://www.redfin.com/NV/Las-Vegas/8724-Gateway-Glen-Dr-89178/home/28964119',
'https://www.redfin.com/NV/Las-Vegas/11215-Piazzale-St-89141/home/28993106',
'https://www.redfin.com/NV/Las-Vegas/4151-Golden-Fleece-Ave-89141/home/184024794',
'https://www.redfin.com/NV/Las-Vegas/10929-Bandol-Pl-89141/home/29873641',
'https://www.redfin.com/NV/Las-Vegas/8146-Golden-Flowers-St-89139/home/29055045',
'https://www.redfin.com/NV/Las-Vegas/10543-Bella-Camrosa-Dr-89141/home/29322250',
'https://www.redfin.com/NV/Las-Vegas/10740-Tuscan-Ct-89141/home/29885831',
'https://www.redfin.com/NV/Las-Vegas/8955-Golden-Crow-Ave-89148/home/167760197',
'https://www.redfin.com/NV/Las-Vegas/7068-Placid-Lake-Ave-89179/home/50280880',
'https://www.redfin.com/NV/Las-Vegas/3961-Painted-Lady-Ave-89141/home/178675305',
'https://www.redfin.com/NV/Las-Vegas/3977-Painted-Lady-Ave-89141/home/178675310',
'https://www.redfin.com/NV/Las-Vegas/3969-Painted-Lady-Ave-89141/home/178675308',
'https://www.redfin.com/NV/Las-Vegas/3038-San-Mamete-Ave-89141/home/40475393',
'https://www.redfin.com/NV/Las-Vegas/10466-Blue-Rock-St-89178/home/109634100',
'https://www.redfin.com/NV/Las-Vegas/3962-Painted-Lady-Ave-89141/home/178675253',
'https://www.redfin.com/NV/Las-Vegas/4233-Nopal-Serrano-Ave-89141/home/183051357',
'https://www.redfin.com/NV/Las-Vegas/7665-Glorietta-Bay-Ct-89139/home/29900654',
'https://www.redfin.com/NV/Las-Vegas/3737-Westeros-Landing-Ave-89141/home/109631814',
'https://www.redfin.com/NV/Las-Vegas/5021-Moose-Falls-Dr-89141/home/29291874',
'https://www.redfin.com/NV/Las-Vegas/8023-Esparza-Ct-89113/home/108998053',
'https://www.redfin.com/NV/Las-Vegas/8504-Alpine-Vineyards-Ct-89139/home/144164192',
'https://www.redfin.com/NV/Las-Vegas/11324-Nalleta-Springs-St-89141/home/109631703',
'https://www.redfin.com/NV/Las-Vegas/11152-Montagne-Marron-Blvd-89141/home/29874402',
'https://www.redfin.com/NV/Las-Vegas/62-E-Ford-Ave-89123/home/29626532',
'https://www.redfin.com/NV/Las-Vegas/8660-Giles-St-89123/home/29626539',
'https://www.redfin.com/NV/Las-Vegas/7193-Estonian-Pl-89113/home/29320879',
'https://www.redfin.com/NV/Las-Vegas/5983-Attavilla-Dr-89141/home/28991876',
'https://www.redfin.com/NV/Las-Vegas/6490-Parrot-Ridge-Ct-89139/home/52625615',
'https://www.redfin.com/NV/Las-Vegas/200-Fairway-Woods-Dr-89148/home/29287421',
'https://www.redfin.com/NV/Las-Vegas/20-Barbara-Ln-89183/unit-6/home/170139525',
'https://www.redfin.com/NV/Las-Vegas/7511-Whiskey-Moon-St-89139/home/29035954',
'https://www.redfin.com/NV/Las-Vegas/7601-Boyd-Ave-89179/home/52666056',
'https://www.redfin.com/NV/Las-Vegas/9110-McGinnis-Ave-89148/home/28975931',
'https://www.redfin.com/NV/Las-Vegas/7458-Alamo-Ranch-Ave-89179/home/40469711',
'https://www.redfin.com/NV/Las-Vegas/9038-Golden-Crow-Ave-89148/home/167760050',
'https://www.redfin.com/NV/Las-Vegas/4343-Vantage-Ridge-Ave-89141/home/175527359',
'https://www.redfin.com/NV/Las-Vegas/8119-Monsoon-Bay-St-89113/home/169231950',
'https://www.redfin.com/NV/Las-Vegas/5665-Heather-Breeze-Ct-89141/home/29900246',
'https://www.redfin.com/NV/Las-Vegas/9249-Black-Diamond-St-89178/home/161752131',
'https://www.redfin.com/NV/Las-Vegas/241-Lantana-Breeze-Dr-89183/home/29072031',
'https://www.redfin.com/NV/Las-Vegas/10885-Galbraith-St-89141/home/28992776',
'https://www.redfin.com/NV/Las-Vegas/715-Magrath-St-89178/home/29768262',
'https://www.redfin.com/NV/Las-Vegas/7250-Shannon-Ridge-Ct-89118/home/109871027',
'https://www.redfin.com/NV/Las-Vegas/568-Teedale-Ct-89178/home/29066759',
'https://www.redfin.com/NV/Las-Vegas/8081-Adelaide-Hills-St-89113/home/177142260',
'https://www.redfin.com/NV/Las-Vegas/9105-Hombard-Ave-89148/home/28975645',
'https://www.redfin.com/NV/Las-Vegas/6596-Polonius-Ct-89141/home/29491970',
'https://www.redfin.com/NV/Las-Vegas/9186-Rolling-Sky-Dr-89178/home/169232099',
'https://www.redfin.com/NV/Las-Vegas/7177-Laramie-Ave-89113/home/29035417',
'https://www.redfin.com/NV/Las-Vegas/7234-Shannon-Ridge-Ct-89118/home/113140086',
'https://www.redfin.com/NV/Las-Vegas/6055-Montana-Peak-Ave-89139/home/109628788',
'https://www.redfin.com/NV/Las-Vegas/8592-Garden-Valley-Ct-89178/home/29606962',
'https://www.redfin.com/NV/Las-Vegas/140-Bella-Milano-Ave-89183/home/29954297',
'https://www.redfin.com/NV/Las-Vegas/8952-College-Green-St-89148/home/29290819',
'https://www.redfin.com/NV/Las-Vegas/11139-Gold-Hitt-Ct-89179/home/44608678',
'https://www.redfin.com/NV/Las-Vegas/8476-Stanton-Heights-Ct-89178/home/29055358',
'https://www.redfin.com/NV/Las-Vegas/11076-Romola-St-89141/home/29878943',
'https://www.redfin.com/NV/Las-Vegas/8490-Classique-Ave-89178/unit-104/home/147856187',
'https://www.redfin.com/NV/Las-Vegas/8244-Selenite-St-89113/home/175526609',
'https://www.redfin.com/NV/Las-Vegas/8310-Gold-River-Ct-89113/home/175526676',
'https://www.redfin.com/NV/Las-Vegas/4607-Maxwell-Peak-Ct-89139/home/28992620',
'https://www.redfin.com/NV/Las-Vegas/7853-Limestone-Arch-Ave-89178/home/29056119',
'https://www.redfin.com/NV/Las-Vegas/208-Fox-Lake-Ave-89148/home/29899363',
'https://www.redfin.com/NV/Las-Vegas/239-Star-Ruby-Ct-89183/home/144626690',
'https://www.redfin.com/NV/Las-Vegas/5925-Banbury-Heights-Way-89139/home/29950037',
'https://www.redfin.com/NV/Las-Vegas/9057-Norwood-Creek-Rd-89139/home/29490144',
'https://www.redfin.com/NV/Las-Vegas/8768-Charm-Canyon-Ave-89178/home/29492111',
'https://www.redfin.com/NV/Las-Vegas/9338-Wilderness-Glen-Ave-89178/home/29056621',
'https://www.redfin.com/NV/Las-Vegas/8934-Cypress-Fog-Ct-89178/home/177142816',
'https://www.redfin.com/NV/Las-Vegas/7584-Eastham-Bay-Ave-89179/home/109635627',
'https://www.redfin.com/NV/Las-Vegas/10118-Cache-River-Ct-89141/home/166355328',
'https://www.redfin.com/NV/Las-Vegas/8697-Roping-Rodeo-Ave-89178/unit-102/home/29037872',
'https://www.redfin.com/NV/Las-Vegas/93-Cree-Ave-89123/home/29628594',
'https://www.redfin.com/NV/Las-Vegas/28-Living-Edens-Ct-89148/home/29873809',
'https://www.redfin.com/NV/Las-Vegas/7474-Almansor-St-89139/home/29181629',
'https://www.redfin.com/NV/Las-Vegas/304-Cascade-Mist-Ave-89123/home/29629627',
'https://www.redfin.com/NV/Las-Vegas/109-Short-Ruff-Way-89148/home/29036903',
'https://www.redfin.com/NV/Las-Vegas/7304-Drumlin-St-89139/home/29010219',
'https://www.redfin.com/NV/Las-Vegas/4966-Tioga-Pass-Ave-89139/home/29036340',
'https://www.redfin.com/NV/Las-Vegas/11587-Sweet-Nokia-St-89183/home/29043407',
'https://www.redfin.com/NV/Las-Vegas/7914-Brianna-Cheerful-Ave-89178/home/40233053',
'https://www.redfin.com/NV/Las-Vegas/8172-Deerfield-Ranch-Ct-89139/home/30500652',
'https://www.redfin.com/NV/Las-Vegas/10376-Station-Creek-Cir-89178/home/29449957',
'https://www.redfin.com/NV/Las-Vegas/9078-Marigold-Creek-St-89113/home/179959764',
'https://www.redfin.com/NV/Las-Vegas/9072-Marigold-Creek-St-89113/home/179959763',
'https://www.redfin.com/NV/Las-Vegas/9810-Fox-Estate-St-89141/home/166357316',
'https://www.redfin.com/NV/Las-Vegas/7929-Red-Rock-Ridge-Ave-89179/home/29758625',
'https://www.redfin.com/NV/Las-Vegas/4524-Pacific-Sun-Ave-89139/home/28997185',
'https://www.redfin.com/NV/Las-Vegas/9470-Castrato-Ct-89178/home/29066919',
'https://www.redfin.com/NV/Las-Vegas/95-Willow-Dove-Ave-89123/home/29628553',
'https://www.redfin.com/NV/Las-Vegas/9071-Herring-Cove-Ave-89178/home/29289096',
'https://www.redfin.com/NV/Las-Vegas/7673-Starshell-Point-Ct-89139/home/29891160',
'https://www.redfin.com/NV/Las-Vegas/11441-Roncat-Ct-89141/home/29002530',
'https://www.redfin.com/NV/Las-Vegas/8639-Horizon-Wind-Ave-89178/unit-102/home/29037716',
'https://www.redfin.com/NV/Las-Vegas/7938-Elk-Mountain-St-89113/home/28990654',
'https://www.redfin.com/NV/Las-Vegas/192-Erie-Ave-89183/home/29640241',
'https://www.redfin.com/NV/Las-Vegas/11524-Conerwood-St-89141/home/109631794',
'https://www.redfin.com/NV/Las-Vegas/9161-Dutch-Oven-Ct-89178/home/29055213',
'https://www.redfin.com/NV/Las-Vegas/7022-Solano-Canyon-Ave-89113/home/175526480',
'https://www.redfin.com/NV/Las-Vegas/4251-Marble-Bluff-Ave-89141/home/172604490',
'https://www.redfin.com/NV/Las-Vegas/9497-Diamond-Willow-Ct-89178/home/29068146',
'https://www.redfin.com/NV/Las-Vegas/126-Chateau-Whistler-Ct-89148/home/29590444',
'https://www.redfin.com/NV/Las-Vegas/63-Piropiro-Ave-89183/home/179107388',
'https://www.redfin.com/NV/Las-Vegas/85-Kokomiko-Ave-89183/home/179107366',
'https://www.redfin.com/NV/Las-Vegas/10926-Florence-Hills-St-89141/home/29609130',
'https://www.redfin.com/NV/Las-Vegas/89-Kokomiko-Ave-89183/home/179107365',
'https://www.redfin.com/NV/Las-Vegas/10143-Jeffcott-St-89178/home/29490550',
'https://www.redfin.com/NV/Las-Vegas/8605-Sparrow-Desert-Ave-89148/home/144624747',
'https://www.redfin.com/NV/Las-Vegas/365-Sapphire-Rock-Ave-89183/home/167778190',
'https://www.redfin.com/NV/Las-Vegas/8257-Aster-Meadow-Way-89113/home/144625440',
'https://www.redfin.com/NV/Las-Vegas/8795-Oak-Lawn-St-89148/home/174794260',
'https://www.redfin.com/NV/Las-Vegas/7506-Mojave-Wind-Ave-89113/home/175526103',
'https://www.redfin.com/NV/Las-Vegas/7517-Tinley-Creek-Ave-89113/home/167762204',
'https://www.redfin.com/NV/Las-Vegas/178-Twin-Towers-Ave-89123/home/29874834',
'https://www.redfin.com/NV/Las-Vegas/7115-Hickory-Post-Ave-89179/home/30182712',
'https://www.redfin.com/NV/Las-Vegas/6029-Quiet-Glow-Ave-89139/home/29054650',
'https://www.redfin.com/NV/Las-Vegas/9592-Staff-Ln-89178/home/29055832',
'https://www.redfin.com/NV/Las-Vegas/9260-Fire-Rose-St-89178/home/143191611',
'https://www.redfin.com/NV/Las-Vegas/11253-Sanbury-Brook-St-89183/home/40548678',
'https://www.redfin.com/NV/Las-Vegas/4089-Painted-Lady-Ave-89141/home/178675331',
'https://www.redfin.com/NV/Las-Vegas/4732-Lone-Grove-Dr-89139/home/29901478',
'https://www.redfin.com/NV/Las-Vegas/541-Warminster-Ave-89178/home/29066975',
'https://www.redfin.com/NV/Las-Vegas/546-Cabis-Bay-St-89178/home/29066904',
'https://www.redfin.com/NV/Las-Vegas/4635-W-Robindale-Rd-89139/home/29596668',
'https://www.redfin.com/NV/Las-Vegas/8219-Rainy-Mountain-Ct-89113/home/109871153',
'https://www.redfin.com/NV/Las-Vegas/8338-S-Pioneer-Way-89113/home/29775439',
'https://www.redfin.com/NV/Las-Vegas/6031-Lamotte-Ave-89141/home/28991902',
'https://www.redfin.com/NV/Las-Vegas/9890-Big-Window-St-89178/home/29607207',
'https://www.redfin.com/NV/Las-Vegas/4226-Via-Dana-Ave-89141/home/29878804',
'https://www.redfin.com/NV/Las-Vegas/8701-Canfield-Canyon-Ave-89178/home/29449970',
'https://www.redfin.com/NV/Las-Vegas/10089-Matter-Overlook-Dr-89178/home/171321268',
'https://www.redfin.com/NV/Las-Vegas/7156-Pine-Bridge-Ct-89148/home/109872114',
'https://www.redfin.com/NV/Las-Vegas/3748-Hillary-Elan-Ct-89139/home/29056377',
'https://www.redfin.com/NV/Las-Vegas/4100-Scarlet-Cob-Ave-89141/home/183051322',
'https://www.redfin.com/NV/Las-Vegas/4112-Scarlet-Cob-Ave-89141/home/183051324',
'https://www.redfin.com/NV/Las-Vegas/7776-Spindrift-Cove-St-89139/home/29834481',
'https://www.redfin.com/NV/Las-Vegas/212-Walkinshaw-Ave-89148/home/69350145',
'https://www.redfin.com/NV/Las-Vegas/8267-Golden-Flowers-St-89139/home/28997142',
'https://www.redfin.com/NV/Las-Vegas/10163-Kings-Amethyst-St-89141/home/177332228',
'https://www.redfin.com/NV/Las-Vegas/10866-Royal-Highlands-St-89141/home/28992843',
'https://www.redfin.com/NV/Las-Vegas/5845-Farmhouse-Ct-89141/home/29909776',
'https://www.redfin.com/NV/Las-Vegas/7425-Rainford-St-89148/home/92908550',
'https://www.redfin.com/NV/Las-Vegas/9728-Aleutian-St-89178/home/29607031',
'https://www.redfin.com/NV/Las-Vegas/7557-Spindrift-Tide-Ct-89139/home/29009436',
'https://www.redfin.com/NV/Las-Vegas/7867-Granite-Walk-Ave-89178/home/29056124',
'https://www.redfin.com/NV/Las-Vegas/7307-Woodfall-Glen-St-89148/home/113139888',
'https://www.redfin.com/NV/Las-Vegas/7037-Sunland-Hills-Ave-89113/home/173633333',
'https://www.redfin.com/NV/Las-Vegas/76-Rancho-Maria-St-89148/home/29833063',
'https://www.redfin.com/NV/Las-Vegas/5227-Tartan-Hill-Ave-89141/home/65557465',
'https://www.redfin.com/NV/Las-Vegas/6635-Hurkling-Stone-Ave-89139/home/28991347',
'https://www.redfin.com/NV/Las-Vegas/129-Opulent-Rose-Ave-89183/home/144626693',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89178/home/29037832',
'https://www.redfin.com/NV/Las-Vegas/228-Spring-River-Ave-89123/home/29835499',
'https://www.redfin.com/NV/Las-Vegas/3670-Saint-Nazaire-Ave-89141/home/29924021',
'https://www.redfin.com/NV/Las-Vegas/9822-W-Meranto-Ave-89178/home/177142645',
'https://www.redfin.com/NV/Las-Vegas/4118-Scarlet-Cob-Ave-89141/home/183051325',
'https://www.redfin.com/NV/Las-Vegas/10186-Crimson-Crown-St-89141/home/183051245',
'https://www.redfin.com/NV/Las-Vegas/8409-Cambria-Cellars-Ct-89139/home/29608752',
'https://www.redfin.com/NV/Las-Vegas/8655-Weed-Willows-Ave-89178/home/65553368',
'https://www.redfin.com/NV/Las-Vegas/388-First-on-Dr-89148/home/29034174',
'https://www.redfin.com/NV/Las-Vegas/9479-Cantata-Crest-Ct-89178/home/29066897',
'https://www.redfin.com/NV/Las-Vegas/9606-Vega-Carpio-Ave-89178/home/65556768',
'https://www.redfin.com/NV/Las-Vegas/695-Tall-Arrow-Ave-89178/home/49786959',
'https://www.redfin.com/NV/Las-Vegas/8953-Appellation-Ave-89148/home/29321179',
'https://www.redfin.com/NV/Las-Vegas/8514-Alpine-Vineyards-Ct-89139/home/145871910',
'https://www.redfin.com/NV/Las-Vegas/66-Big-Creek-Ct-89148/home/29873413',
'https://www.redfin.com/NV/Las-Vegas/8182-Ribbon-Edge-St-89139/home/29036186',
'https://www.redfin.com/NV/Las-Vegas/5692-Silent-Sky-Ave-89141/home/29900182',
'https://www.redfin.com/NV/Las-Vegas/10411-Mangakino-St-89183/home/179107393',
'https://www.redfin.com/NV/Las-Vegas/10419-Mangakino-St-89183/home/179107392',
'https://www.redfin.com/NV/Las-Vegas/5522-Yarra-Valley-Ave-89139/home/52627275',
'https://www.redfin.com/NV/Las-Vegas/4056-Villeroy-Ave-89141/home/29941456',
'https://www.redfin.com/NV/Las-Vegas/7268-Mosaic-Terrace-St-89118/home/29286451',
'https://www.redfin.com/NV/Las-Vegas/533-Sinfold-Park-St-89148/home/29900547',
'https://www.redfin.com/NV/Las-Vegas/7477-Ullom-Dr-89139/home/29596510',
'https://www.redfin.com/NV/Las-Vegas/10981-Tanner-Crest-St-89141/home/146417901',
'https://www.redfin.com/NV/Las-Vegas/210-Seven-Valleys-Ct-89183/home/175528779',
'https://www.redfin.com/NV/Las-Vegas/5350-Purple-Moon-Ave-89141/home/175526897',
'https://www.redfin.com/NV/Las-Vegas/66-Piropiro-Ave-89183/home/179107382',
'https://www.redfin.com/NV/Las-Vegas/10798-Casco-Bay-St-89179/home/161722083',
'https://www.redfin.com/NV/Las-Vegas/10089-Mallee-Point-St-89178/home/52594061',
'https://www.redfin.com/NV/Las-Vegas/4526-Herbert-River-Ave-89141/home/147876132',
'https://www.redfin.com/NV/Las-Vegas/6636-Oxendale-Ave-89139/home/28991000',
'https://www.redfin.com/NV/Las-Vegas/7456-Tinley-Creek-Ave-89113/home/170654805',
'https://www.redfin.com/NV/Las-Vegas/7771-Lots-Hills-Dr-89179/home/109628778',
'https://www.redfin.com/NV/Las-Vegas/172-Turtle-Ridge-Ave-89183/home/40234081',
'https://www.redfin.com/NV/Las-Vegas/10798-Muscari-Way-89141/home/29889129',
'https://www.redfin.com/NV/Las-Vegas/5851-Chisolm-Trl-89118/home/29577481',
'https://www.redfin.com/NV/Las-Vegas/4610-Milvio-Ave-89141/home/29877953',
'https://www.redfin.com/NV/Las-Vegas/9958-Keifer-Valley-St-89178/home/29056219',
'https://www.redfin.com/NV/Las-Vegas/11054-Fort-Bowie-St-89179/home/29794809',
'https://www.redfin.com/NV/Las-Vegas/173-Coney-Island-Ave-89123/home/29902249',
'https://www.redfin.com/NV/Las-Vegas/8960-Appellation-Ave-89148/home/29321298',
'https://www.redfin.com/NV/Las-Vegas/3370-Arcata-Point-Ave-89141/home/29006291',
'https://www.redfin.com/NV/Las-Vegas/7208-Adobe-Hills-Ave-89113/home/29938995',
'https://www.redfin.com/NV/Las-Vegas/5933-Lamotte-Ave-89141/home/28991893',
'https://www.redfin.com/NV/Las-Vegas/7736-Pista-Way-89179/home/52626986',
'https://www.redfin.com/NV/Las-Vegas/7634-Ribbon-Rock-Ct-89139/home/29900667',
'https://www.redfin.com/NV/Las-Vegas/8557-Silver-Coast-St-89139/home/166557675',
'https://www.redfin.com/NV/Las-Vegas/9804-Sunflower-Hill-St-89178/home/29292922',
'https://www.redfin.com/NV/Las-Vegas/10046-Dancing-Lasso-Ct-89178/home/29293729',
'https://www.redfin.com/NV/Las-Vegas/6353-Currant-Lake-Way-89148/home/171692637',
'https://www.redfin.com/NV/Las-Vegas/10156-Ferton-Lake-Ave-89178/home/172239591',
'https://www.redfin.com/NV/Las-Vegas/10558-Jade-Walk-St-89179/home/173454550',
'https://www.redfin.com/NV/Las-Vegas/7074-W-Camero-Ave-89113/home/173633344',
'https://www.redfin.com/NV/Las-Vegas/11108-Cataya-Ct-89141/home/29921496',
'https://www.redfin.com/NV/Las-Vegas/7874-Trail-Boss-Ct-89113/home/29584421',
'https://www.redfin.com/NV/Las-Vegas/9789-Panther-Hollow-St-89141/home/109626746',
'https://www.redfin.com/NV/Las-Vegas/10924-Royal-Highlands-St-89141/home/28992814',
'https://www.redfin.com/NV/Las-Vegas/8792-Norton-Peak-Ave-89148/home/29605600',
'https://www.redfin.com/NV/Las-Vegas/6055-Sonoma-Station-Ave-89139/home/30504762',
'https://www.redfin.com/NV/Las-Vegas/424-White-Heart-Rd-89148/home/65557739',
'https://www.redfin.com/NV/Las-Vegas/9572-Staff-Ln-89178/home/29055834',
'https://www.redfin.com/NV/Las-Vegas/6983-Bering-Strait-Ave-89179/home/87699633',
'https://www.redfin.com/NV/Las-Vegas/6950-Emerald-Springs-Ln-89113/home/29582451',
'https://www.redfin.com/NV/Las-Vegas/5442-Tartan-Hill-Ave-89141/home/65557303',
'https://www.redfin.com/NV/Las-Vegas/9622-Volk-Ave-89178/home/65556711',
'https://www.redfin.com/NV/Las-Vegas/8014-Roughcut-Ave-89113/unit-30/home/186672670',
'https://www.redfin.com/NV/Las-Vegas/7659-Belgian-Lion-St-89139/home/29834122',
'https://www.redfin.com/NV/Las-Vegas/4090-W-Maulding-Ave-89139/home/29597188',
'https://www.redfin.com/NV/Las-Vegas/5701-Buena-Martina-Way-89141/home/28979576',
'https://www.redfin.com/NV/Las-Vegas/182-Rose-Lake-St-89148/home/29899372',
'https://www.redfin.com/NV/N-Las-Vegas/10187-Glowing-Amber-St-89081/home/185795999',
'https://www.redfin.com/NV/Las-Vegas/10378-Grizzly-Forest-Dr-89178/home/29044841',
'https://www.redfin.com/NV/Las-Vegas/7220-Puffer-Lake-Ct-89118/home/101992565',
'https://www.redfin.com/NV/Las-Vegas/175-Samoset-Ct-89148/home/28973273',
'https://www.redfin.com/NV/Las-Vegas/6060-Doroca-St-89148/home/29320559',
'https://www.redfin.com/NV/Las-Vegas/7402-Mission-Hills-Dr-89113/home/29580724',
'https://www.redfin.com/NV/Las-Vegas/4182-Bottiglia-Ave-89141/home/29902633',
'https://www.redfin.com/NV/Las-Vegas/6402-Parrot-Ridge-Ct-89139/home/52625622',
'https://www.redfin.com/NV/Las-Vegas/5446-Caprock-Canyon-Ave-89139/home/29055125',
'https://www.redfin.com/NV/Las-Vegas/8549-Alberta-Falls-Ave-89113/home/65563150',
'https://www.redfin.com/NV/Las-Vegas/6956-Emerald-Springs-Ln-89113/home/29582452',
'https://www.redfin.com/NV/N-Las-Vegas/10179-Glowing-Amber-St-89081/home/185796001',
'https://www.redfin.com/NV/Las-Vegas/137-Chateau-Whistler-Ct-89148/home/29590421',
'https://www.redfin.com/NV/Las-Vegas/8407-Kettledrum-St-89139/home/29288371',
'https://www.redfin.com/NV/Las-Vegas/9764-Bedstraw-St-89178/home/29607449',
'https://www.redfin.com/NV/Las-Vegas/11168-Montagne-Marron-Blvd-89141/home/29874404',
'https://www.redfin.com/NV/Las-Vegas/7351-Salvadora-Pl-89113/home/29287091',
'https://www.redfin.com/NV/Las-Vegas/11800-Weybrook-Park-Dr-89141/home/28995384',
'https://www.redfin.com/NV/Las-Vegas/75-Piropiro-Ave-89183/home/179107385',
'https://www.redfin.com/NV/Las-Vegas/9838-Hearthfire-St-89178/home/40548922',
'https://www.redfin.com/NV/Las-Vegas/5621-Granollers-Dr-89135/home/143188740',
'https://www.redfin.com/NV/Las-Vegas/9778-Havasupai-Ave-89148/home/29912895',
'https://www.redfin.com/NV/Las-Vegas/7364-Bedford-Pine-Ct-89113/home/29066294',
'https://www.redfin.com/NV/Las-Vegas/7848-Coronado-Island-St-89139/home/29035716',
'https://www.redfin.com/NV/Las-Vegas/11264-Felice-Cohn-Ct-89179/home/109628766',
'https://www.redfin.com/NV/Las-Vegas/11809-Weybrook-Park-Dr-89141/home/28995381',
'https://www.redfin.com/NV/Las-Vegas/6224-Dolosonte-St-89113/home/183984992',
'https://www.redfin.com/NV/Las-Vegas/3954-Painted-Lady-Ave-89141/home/178675254',
'https://www.redfin.com/NV/Las-Vegas/3946-Painted-Lady-Ave-89141/home/178675255',
'https://www.redfin.com/NV/Las-Vegas/3876-Nivosus-Ave-89141/home/185796135',
'https://www.redfin.com/NV/Las-Vegas/3864-Nivosus-Ave-89141/home/185796134',
'https://www.redfin.com/NV/Las-Vegas/8933-Cherry-Barn-St-89113/home/181103653',
'https://www.redfin.com/NV/Las-Vegas/5679-Beguiling-Falls-St-89148/home/185782344',
'https://www.redfin.com/NV/Las-Vegas/5237-Meadow-Star-Ave-89141/home/175526997',
'https://www.redfin.com/NV/Las-Vegas/5692-Beguiling-Falls-St-89148/home/185782347',
'https://www.redfin.com/NV/Las-Vegas/8014-Roughcut-Ave-89113/home/183984988',
'https://www.redfin.com/NV/Las-Vegas/8006-Roughcut-Ave-89113/home/183984990',
'https://www.redfin.com/NV/Las-Vegas/8010-Roughcut-Ave-89113/home/183984989',
'https://www.redfin.com/NV/Las-Vegas/8018-Roughcut-Ave-89113/home/183984987',
'https://www.redfin.com/NV/Las-Vegas/5683-Beguiling-Falls-St-89148/home/185782345',
'https://www.redfin.com/NV/Las-Vegas/8919-Don-Valley-Ave-89148/home/144626328',
'https://www.redfin.com/NV/Las-Vegas/7244-Atascadero-Creek-Ct-89118/home/145871870',
'https://www.redfin.com/NV/Las-Vegas/7770-W-Ford-Ave-89113/home/29590895',
'https://www.redfin.com/NV/Las-Vegas/7939-Brianna-Cheerful-Ave-89178/home/40233052',
'https://www.redfin.com/NV/Las-Vegas/5638-Bartlett-Ridge-Ct-89141/home/113713469',
'https://www.redfin.com/NV/Las-Vegas/8591-Heather-Downs-Dr-89113/home/29582167',
'https://www.redfin.com/NV/Las-Vegas/6341-Brea-Blvd-89118/home/29580066',
'https://www.redfin.com/NV/Las-Vegas/5831-Honeysuckle-Ridge-St-89148/home/28964685',
'https://www.redfin.com/NV/Las-Vegas/3861-Speckled-Wood-Ave-89141/home/178675159',
'https://www.redfin.com/NV/Las-Vegas/6007-Whitmire-Ave-89139/home/177142414',
'https://www.redfin.com/NV/Las-Vegas/6322-Jackson-Spring-Rd-89118/home/144625000',
'https://www.redfin.com/NV/Las-Vegas/128-Honors-Course-Dr-89148/home/29768828',
'https://www.redfin.com/NV/Las-Vegas/8695-Weed-Willows-Ave-89178/home/65553371',
'https://www.redfin.com/NV/Las-Vegas/9792-Valmeyer-Ave-89148/home/29006926',
'https://www.redfin.com/NV/Las-Vegas/9255-Ardoa-Ave-89148/home/113713174',
'https://www.redfin.com/NV/Las-Vegas/9664-W-Diablo-Dr-89148/home/29584508',
'https://www.redfin.com/NV/Las-Vegas/11346-Notte-Calma-St-89141/home/28995668',
'https://www.redfin.com/NV/Las-Vegas/6799-Tangled-Vines-Ave-89139/home/167762458',
'https://www.redfin.com/NV/Las-Vegas/5090-Breakers-Ln-89113/home/29580662',
'https://www.redfin.com/NV/Las-Vegas/5057-Vacaville-Ave-89139/home/29488707',
'https://www.redfin.com/NV/Las-Vegas/8343-Kimberly-Diamond-St-89139/home/29039036',
'https://www.redfin.com/NV/Las-Vegas/5687-Garriga-Dr-89135/home/143188757',
'https://www.redfin.com/NV/Las-Vegas/6748-Bel-Canto-Ct-89139/home/52623675',
'https://www.redfin.com/NV/Henderson/2355-Thayer-Ave-89074/home/29656211',
'https://www.redfin.com/NV/Henderson/Undisclosed-address-89052/home/29839833',
'https://www.redfin.com/NV/Las-Vegas/937-Aspen-Valley-Ave-89123/home/29631758',
'https://www.redfin.com/NV/Henderson/1025-Jasmine-Tea-Ct-89052/home/29244179',
'https://www.redfin.com/NV/Henderson/2541-Springville-Way-89052/home/29740113',
'https://www.redfin.com/NV/Las-Vegas/908-Rainbow-Rock-St-89123/home/29631485',
'https://www.redfin.com/NV/Henderson/57-Ginger-Lily-Ter-89074/home/29675700',
'https://www.redfin.com/NV/Henderson/78-Chesney-Dr-89074/home/29620799',
'https://www.redfin.com/NV/Henderson/2874-Tori-Way-89074/home/29617689',
'https://www.redfin.com/NV/Las-Vegas/9610-Redstar-St-89123/home/29633050',
'https://www.redfin.com/NV/Las-Vegas/9419-Graceful-Gold-St-89123/home/29632712',
'https://www.redfin.com/NV/Las-Vegas/1635-Tristan-Flower-Ave-89183/home/29295915',
'https://www.redfin.com/NV/Henderson/1569-Bonner-Springs-Dr-89052/home/29740091',
'https://www.redfin.com/NV/Henderson/2710-Riceville-Dr-89052/home/29841823',
'https://www.redfin.com/NV/Las-Vegas/12300-Old-Muirfield-St-89141/home/113140575',
'https://www.redfin.com/NV/Henderson/371-Placer-Creek-Ln-89014/home/29657709',
'https://www.redfin.com/NV/Las-Vegas/7715-Radcliff-St-89123/home/29612337',
'https://www.redfin.com/NV/Henderson/3164-Majestic-Shadows-Ave-89052/home/29641611',
'https://www.redfin.com/NV/Las-Vegas/932-Rainbow-Rock-St-89123/home/29631477',
'https://www.redfin.com/NV/Henderson/2078-Rose-Pine-Ct-89052/home/28994631',
'https://www.redfin.com/NV/Las-Vegas/912-Serpentina-Ave-89123/home/29612817',
'https://www.redfin.com/NV/Las-Vegas/9107-Cedeno-St-89123/home/29631946',
'https://www.redfin.com/NV/Las-Vegas/9205-Cantana-St-89123/home/28992580',
'https://www.redfin.com/NV/Henderson/2182-Eaglecloud-Dr-89074/home/29679876',
'https://www.redfin.com/NV/Las-Vegas/479-Wendover-Hills-Ave-89123/home/28992719',
'https://www.redfin.com/NV/Henderson/1936-Grey-Eagle-St-89074/home/29675686',
'https://www.redfin.com/NV/Henderson/2827-Secret-Canyon-Rd-89074/home/29619382',
'https://www.redfin.com/NV/Henderson/2-Panther-Creek-Ct-89052/home/29738470',
'https://www.redfin.com/NV/Henderson/1828-Baton-Rouge-St-89052/home/29742909',
'https://www.redfin.com/NV/Henderson/3783-Canis-Minor-Ln-89052/unit-6103/home/170139489',
'https://www.redfin.com/NV/Henderson/1130-Pandora-Canyon-St-89052/home/109633039',
'https://www.redfin.com/NV/Henderson/1700-Ravanusa-Dr-89052/home/29884936',
'https://www.redfin.com/NV/Henderson/2553-Leighton-Ave-89052/home/29738729',
'https://www.redfin.com/NV/Las-Vegas/7667-Borealis-St-89123/home/29925051',
'https://www.redfin.com/NV/Henderson/3050-Hickory-Valley-Rd-89052/home/29947062',
'https://www.redfin.com/NV/Henderson/3237-Peaceful-Shadows-Ct-89052/home/29740786',
'https://www.redfin.com/NV/Henderson/3003-Panorama-Ridge-Dr-89052/home/29641351',
'https://www.redfin.com/NV/Las-Vegas/7414-Desert-Wildflower-St-89123/home/65562846',
'https://www.redfin.com/NV/Las-Vegas/9852-Snowy-Canyon-Ct-89183/home/29636871',
'https://www.redfin.com/NV/Las-Vegas/891-Sweeping-Vine-Ave-89183/home/29639813',
'https://www.redfin.com/NV/Henderson/1821-Eagle-Mesa-Ave-89012/home/29838888',
'https://www.redfin.com/NV/Henderson/1461-Bonner-Springs-Dr-89052/home/29738663',
'https://www.redfin.com/NV/Henderson/28-Tanglewood-Dr-89012/home/29683931',
'https://www.redfin.com/NV/Henderson/2272-Chestnut-Ranch-Ave-89052/home/29001834',
'https://www.redfin.com/NV/Henderson/1837-Birdie-Ln-89074/home/29677323',
'https://www.redfin.com/NV/Henderson/2003-High-Mesa-Dr-89012/home/29694384',
'https://www.redfin.com/NV/Henderson/2331-Thayer-Ave-89074/home/29658364',
'https://www.redfin.com/NV/Las-Vegas/875-E-Eldorado-Ln-89123/home/29611480',
'https://www.redfin.com/NV/Henderson/2214-Flowering-Cactus-Ave-89052/home/29696091',
'https://www.redfin.com/NV/Henderson/1230-Sonatina-Dr-89052/home/29904226',
'https://www.redfin.com/NV/Henderson/503-Carmel-Mesa-Dr-89012/home/29694187',
'https://www.redfin.com/NV/Henderson/206-Heiple-Ct-89052/home/29942681',
'https://www.redfin.com/NV/Henderson/766-Lighthall-Ct-89052/home/146418103',
'https://www.redfin.com/NV/Las-Vegas/7967-Boardwalk-Way-89123/home/29612515',
'https://www.redfin.com/NV/Las-Vegas/10795-Rising-Smoke-Ct-89183/home/29942812',
'https://www.redfin.com/NV/Henderson/1822-Dawn-Ridge-Ave-89074/home/29661731',
'https://www.redfin.com/NV/Henderson/2216-Alanhurst-Dr-89052/home/29696111',
'https://www.redfin.com/NV/Henderson/1652-Black-Fox-Canyon-Rd-89052/home/29886156',
'https://www.redfin.com/NV/Las-Vegas/423-Tanita-Ct-89123/home/29623580',
'https://www.redfin.com/NV/Henderson/2212-Lucerne-Dr-89014/home/29654171',
'https://www.redfin.com/NV/Henderson/108-Emerald-Dunes-Cir-89052/home/29839738',
'https://www.redfin.com/NV/Las-Vegas/7932-Crystal-Haven-Ln-89123/home/29611734',
'https://www.redfin.com/NV/Henderson/67-Desert-Rain-Ln-89074/home/29673916',
'https://www.redfin.com/NV/Henderson/403-Indigo-Springs-St-89014/home/29648314',
'https://www.redfin.com/NV/Henderson/4-Weston-Hills-Rd-89052/home/28994368',
'https://www.redfin.com/NV/Henderson/1648-Rockcrest-Hills-Ave-89052/home/29840652',
'https://www.redfin.com/NV/Henderson/133-McLaren-St-89074/home/29672891',
'https://www.redfin.com/NV/Henderson/2339-Brockton-Way-89074/home/29657327',
'https://www.redfin.com/NV/Henderson/238-Chestnut-Ridge-Ct-89012/home/29683412',
'https://www.redfin.com/NV/Henderson/15-Moraine-Dr-89052/home/29884140',
'https://www.redfin.com/NV/Henderson/953-Leadville-Meadows-Dr-89052/home/29839677',
'https://www.redfin.com/NV/Las-Vegas/1731-Spotted-Wolf-Ave-89123/home/29069763',
'https://www.redfin.com/NV/Las-Vegas/8556-Ebony-Hills-Way-89123/home/29625625',
'https://www.redfin.com/NV/Henderson/2053-Majestic-Peak-Dr-89074/home/29675736',
'https://www.redfin.com/NV/Henderson/342-Hutchings-Ln-89074/home/29657488',
'https://www.redfin.com/NV/Henderson/385-Manzanita-St-89014/home/29662815',
'https://www.redfin.com/NV/Las-Vegas/763-Castlebridge-Ave-89123/home/29875062',
'https://www.redfin.com/NV/Henderson/648-Doubleshot-Ln-89052/home/29294929',
'https://www.redfin.com/NV/Las-Vegas/10343-Perfect-Parsley-St-89183/home/29940663',
'https://www.redfin.com/NV/Las-Vegas/3110-Rabbit-Creek-Dr-89120/home/28978979',
'https://www.redfin.com/NV/Henderson/11153-Abbeyfield-Rose-Dr-89052/home/29040736',
'https://www.redfin.com/NV/Henderson/2389-Fayetteville-Ave-89052/home/28994599',
'https://www.redfin.com/NV/Henderson/2638-Douglas-Cir-89074/home/29616379',
'https://www.redfin.com/NV/Henderson/8952-Monteloma-Way-89074/home/29635670',
'https://www.redfin.com/NV/Henderson/2269-Aria-Dr-89052/home/29739391',
'https://www.redfin.com/NV/Henderson/588-Ione-Rd-89074/home/29636031',
'https://www.redfin.com/NV/Henderson/52-Attebury-Ct-89074/home/29677358',
'https://www.redfin.com/NV/Henderson/2838-Via-Stella-St-89074/home/29614976',
'https://www.redfin.com/NV/Las-Vegas/1571-Yellowwood-Dr-89123/home/29613711',
'https://www.redfin.com/NV/Henderson/468-Silverweed-Ave-89044/home/175529427',
'https://www.redfin.com/NV/Henderson/500-Carmel-Mesa-Dr-89012/home/29694163',
'https://www.redfin.com/NV/Henderson/141-Brightmoor-Ct-89074/home/29675608',
'https://www.redfin.com/NV/Henderson/380-Cavos-Way-89014/home/29662667',
'https://www.redfin.com/NV/Henderson/4-Hunt-Valley-Trl-89052/home/29946305',
'https://www.redfin.com/NV/Las-Vegas/77-Peachtree-Hill-Ave-89183/home/29303838',
'https://www.redfin.com/NV/Henderson/135-Primero-Way-89074/home/29618047',
'https://www.redfin.com/NV/Henderson/3196-Castle-Canyon-Ave-89052/home/28968276',
'https://www.redfin.com/NV/Henderson/1677-Warrington-Dr-89052/home/29742774',
'https://www.redfin.com/NV/Henderson/16-Holston-Hills-Rd-89052/home/29883894',
'https://www.redfin.com/NV/Las-Vegas/9803-Silver-Lasso-St-89183/home/29637586',
'https://www.redfin.com/NV/Henderson/2653-Langford-Ave-89052/home/29743218',
'https://www.redfin.com/NV/Las-Vegas/8184-Wildfire-St-89123/home/29624440',
'https://www.redfin.com/NV/Las-Vegas/2308-Crooked-Creek-Ave-89123/home/29835283',
'https://www.redfin.com/NV/Las-Vegas/10408-September-Flower-St-89183/home/29495105',
'https://www.redfin.com/NV/Las-Vegas/10282-Keystone-Pastures-St-89183/home/29875958',
'https://www.redfin.com/NV/Las-Vegas/2841-Montage-Sun-Rd-89183/unit-3/home/175528831',
'https://www.redfin.com/NV/Henderson/1156-Ekalaka-Rd-89052/home/29839871',
'https://www.redfin.com/NV/Las-Vegas/8986-Manalang-Rd-89123/home/29630132',
'https://www.redfin.com/NV/Henderson/1809-Eagle-Mesa-Ave-89012/home/29838725',
'https://www.redfin.com/NV/Las-Vegas/11987-Giles-St-89183/home/29303427',
'https://www.redfin.com/NV/Henderson/2013-Big-Bend-Way-89074/home/29658707',
'https://www.redfin.com/NV/Henderson/2365-Millcroft-Dr-89074/home/29655331',
'https://www.redfin.com/NV/Henderson/277-Via-Contata-St-89074/home/29615061',
'https://www.redfin.com/NV/Henderson/1880-High-Mesa-Dr-89012/home/29695202',
'https://www.redfin.com/NV/Henderson/2137-Tyler-Dr-89074/home/29675004',
'https://www.redfin.com/NV/Henderson/1136-Via-Fellini-89052/home/45356837',
'https://www.redfin.com/NV/Las-Vegas/8441-Palmada-Dr-89123/home/29623242',
'https://www.redfin.com/NV/Henderson/2004-Spruce-Brook-Dr-89074/home/29677404',
'https://www.redfin.com/NV/Las-Vegas/11997-Camden-Brook-St-89183/home/29057647',
'https://www.redfin.com/NV/Henderson/1620-Rockcrest-Hills-Ave-89052/home/29840630',
'https://www.redfin.com/NV/Henderson/2110-Club-Meadows-Dr-89074/home/29675662',
'https://www.redfin.com/NV/Las-Vegas/9384-Pinnacle-Cove-St-89123/home/29633386',
'https://www.redfin.com/NV/Henderson/1693-Ravanusa-Dr-89052/home/29840970',
'https://www.redfin.com/NV/Las-Vegas/860-Whispering-Grove-Ave-89123/home/40182581',
'https://www.redfin.com/NV/Henderson/2624-Peoria-Ave-89052/home/29743141',
'https://www.redfin.com/NV/Henderson/1286-Autumn-Wind-Way-89052/home/29739889',
'https://www.redfin.com/NV/Henderson/10-Old-Lake-Cir-89074/home/29679141',
'https://www.redfin.com/NV/Henderson/1540-Sabatini-Dr-89052/home/29840805',
'https://www.redfin.com/NV/Henderson/289-Cameron-Hill-Ct-89074/home/29658825',
'https://www.redfin.com/NV/Henderson/758-Faircliff-Ct-89052/home/166610086',
'https://www.redfin.com/NV/Las-Vegas/9531-Ruperts-Ct-89123/home/29631059',
'https://www.redfin.com/NV/Henderson/896-Keesey-Way-89052/home/166001958',
'https://www.redfin.com/NV/Henderson/2864-Mahogany-Grove-Ave-89074/home/29617869',
'https://www.redfin.com/NV/Henderson/1122-Majestic-Canyon-St-89052/home/29641167',
'https://www.redfin.com/NV/Henderson/505-First-Light-St-89052/home/29697299',
'https://www.redfin.com/NV/Henderson/3783-Canis-Minor-Ln-89052/unit-6205/home/170139491',
'https://www.redfin.com/NV/Henderson/1452-Via-Merano-St-89052/home/29741878',
'https://www.redfin.com/NV/Henderson/115-Blackberry-Ln-89074/home/29618267',
'https://www.redfin.com/NV/Henderson/365-Humboldt-North-Dr-89074/home/29616432',
'https://www.redfin.com/NV/Henderson/1830-June-Lake-Dr-89052/home/29885051',
'https://www.redfin.com/NV/Henderson/2005-Colvin-Run-Dr-89052/home/29889083',
'https://www.redfin.com/NV/Henderson/223-Drysdale-Cir-89074/home/29658651',
'https://www.redfin.com/NV/Las-Vegas/10628-Salmon-Leap-St-89183/home/29040723',
'https://www.redfin.com/NV/Las-Vegas/9406-Back-Bay-Cir-89123/home/28992665',
'https://www.redfin.com/NV/Las-Vegas/9787-Altadena-St-89183/home/29638179',
'https://www.redfin.com/NV/Henderson/2399-Cliffwood-Dr-89074/home/29678022',
'https://www.redfin.com/NV/Henderson/86-Paradise-Pkwy-89074/home/29678454',
'https://www.redfin.com/NV/Henderson/2704-Carolina-Blue-Ave-89052/home/29641828',
'https://www.redfin.com/NV/Henderson/2555-New-Morning-Ave-89052/home/29698120',
'https://www.redfin.com/NV/Las-Vegas/9622-Quarterhorse-Ln-89178/home/29182759',
'https://www.redfin.com/NV/Las-Vegas/1163-Regal-Lily-Way-89123/home/29874745',
'https://www.redfin.com/NV/Las-Vegas/686-Ribbon-Grass-Ave-89183/home/29836244',
'https://www.redfin.com/NV/Henderson/75-Fantasia-Ln-89074/home/29620954',
'https://www.redfin.com/NV/Las-Vegas/10647-Stronghold-Ct-89179/home/29769897',
'https://www.redfin.com/NV/Henderson/2406-Tour-Edition-Dr-89074/home/29679028',
'https://www.redfin.com/NV/Las-Vegas/133-Crooked-Putter-Dr-89148/home/29037326',
'https://www.redfin.com/NV/Las-Vegas/8275-Broad-Branch-Way-89113/home/145871408',
'https://www.redfin.com/NV/Las-Vegas/7756-Wishing-Well-Rd-89123/home/29614703',
'https://www.redfin.com/NV/Las-Vegas/8373-Bartholomew-Park-Ct-89139/home/166561394',
'https://www.redfin.com/NV/Las-Vegas/5251-Lake-Sonoma-Ave-89141/home/166356102',
'https://www.redfin.com/NV/Las-Vegas/6259-Spindrift-Foam-Ave-89139/home/29035982',
'https://www.redfin.com/NV/Henderson/2848-Patriot-Park-Pl-89052/home/29841833',
'https://www.redfin.com/NV/Henderson/2738-Carolina-Blue-Ave-89052/home/29641917',
'https://www.redfin.com/NV/Las-Vegas/5919-Rockway-Glen-Ave-89141/home/167763694',
'https://www.redfin.com/NV/Las-Vegas/385-Dog-Leg-Dr-89148/home/29287375',
'https://www.redfin.com/NV/Las-Vegas/7852-Patagonia-Point-Ct-89179/home/143190051',
'https://www.redfin.com/NV/Las-Vegas/7892-W-Shelbourne-Ave-89113/home/175526629',
'https://www.redfin.com/NV/Las-Vegas/7428-Bermuda-Island-St-89123/home/29600328',
'https://www.redfin.com/NV/Henderson/2404-Legacy-Island-Cir-89074/home/29678500',
'https://www.redfin.com/NV/Henderson/2551-Collinsville-Dr-89052/home/29886054',
'https://www.redfin.com/NV/Las-Vegas/534-Campus-Oaks-Ct-89183/home/29907594',
'https://www.redfin.com/NV/Las-Vegas/4327-Melrose-Abbey-Pl-89141/home/29040620',
'https://www.redfin.com/NV/Las-Vegas/8985-Castledowns-St-89148/home/29067597',
'https://www.redfin.com/NV/Las-Vegas/592-Pale-Pueblo-Ct-89183/home/29907451',
'https://www.redfin.com/NV/Las-Vegas/502-Pale-Pueblo-Ct-89183/home/29908549',
'https://www.redfin.com/NV/Henderson/2464-Cedar-Meadows-St-89052/home/29698223',
'https://www.redfin.com/NV/Las-Vegas/8109-Russell-Creek-Ct-89139/home/144164176',
'https://www.redfin.com/NV/Henderson/276-Horizon-Pointe-Cir-89012/home/29685710',
'https://www.redfin.com/NV/Henderson/1027-Aubrey-Springs-Ave-89014/home/113140153',
'https://www.redfin.com/NV/Henderson/861-Zinnia-Cir-89015/home/29704361',
'https://www.redfin.com/NV/Henderson/309-Andamento-Pl-89015/home/173637149',
'https://www.redfin.com/NV/Henderson/423-Barcarolle-Ln-89011/home/113713281',
'https://www.redfin.com/NV/Henderson/293-Helmsdale-Dr-89014/home/29664897',
'https://www.redfin.com/NV/Henderson/1413-Reebok-Ter-89014/home/29666038',
'https://www.redfin.com/NV/Henderson/113-Staplehurst-Ave-89002/home/29071338',
'https://www.redfin.com/NV/Henderson/715-N-Water-St-89015/home/184007787',
'https://www.redfin.com/NV/Henderson/713-N-Water-St-89015/home/184007786',
'https://www.redfin.com/NV/Henderson/357-Old-Erie-Pl-89011/home/184007708',
'https://www.redfin.com/NV/Henderson/709-Arrowhead-Trl-89002/home/29718788',
'https://www.redfin.com/NV/Henderson/231-E-Van-Wagenen-St-89015/home/29711014',
'https://www.redfin.com/NV/Henderson/851-Bergamont-Dr-89002/home/29724423',
'https://www.redfin.com/NV/Henderson/162-Emden-Dr-89015/home/29704638',
'https://www.redfin.com/NV/Henderson/1002-Black-Olive-St-89002/home/29839944',
'https://www.redfin.com/NV/Henderson/630-Federal-St-89015/home/29712155',
'https://www.redfin.com/NV/Henderson/852-Coastal-Beach-Rd-89002/home/29724360',
'https://www.redfin.com/NV/Henderson/401-Cross-St-89015/home/29690735',
'https://www.redfin.com/NV/Henderson/529-Blackridge-Rd-89015/home/29711328',
'https://www.redfin.com/NV/Henderson/678-Foam-Flower-LN-89015/unit-215/home/187643378',
'https://www.redfin.com/NV/Henderson/255-Kansas-Ave-89015/home/29706445',
'https://www.redfin.com/NV/Henderson/229-Inverbeg-St-89012/home/107501513',
'https://www.redfin.com/NV/Henderson/1096-Teal-Point-Dr-89074/home/29668348',
'https://www.redfin.com/NV/Henderson/126-Cricklewood-Ave-89002/home/29071768',
'https://www.redfin.com/NV/Henderson/669-Foam-Flower-Ln-89015/unit-221/home/187629454',
'https://www.redfin.com/NV/Henderson/132-Cabaletta-Ln-89015/home/173637181',
'https://www.redfin.com/NV/Henderson/261-Quail-Ranch-Dr-89015/home/29497020',
'https://www.redfin.com/NV/Henderson/236-Quail-Ranch-Dr-89015/home/30306711',
'https://www.redfin.com/NV/Henderson/216-Rocky-Basin-St-89012/home/52627137',
'https://www.redfin.com/NV/Henderson/1099-Jesse-Harbor-Ave-89014/home/92908575',
'https://www.redfin.com/NV/Henderson/800-College-Dr-89002/unit-18/home/187580888',
'https://www.redfin.com/NV/Henderson/601-E-Fairway-Rd-89015/home/29710909',
'https://www.redfin.com/NV/Henderson/526-Sellers-Pl-89011/home/29670110',
'https://www.redfin.com/NV/Henderson/780-Vortex-Ave-89002/home/29883429',
'https://www.redfin.com/NV/Henderson/482-Sellers-Pl-89011/home/29670191',
'https://www.redfin.com/NV/Henderson/203-W-Pacific-Ave-89015/home/29708661',
'https://www.redfin.com/NV/Henderson/203-Priority-Point-St-89012/home/28993799',
'https://www.redfin.com/NV/Henderson/276-Navajo-Dr-89015/home/29700130',
'https://www.redfin.com/NV/Henderson/711-Strawberry-Pl-89002/home/29722006',
'https://www.redfin.com/NV/Henderson/56-Nellywood-Ct-89012/home/29954966',
'https://www.redfin.com/NV/Henderson/81-Golden-Aria-Ave-89011/home/179983846',
'https://www.redfin.com/NV/Henderson/723-N-Water-St-89015/home/184007791',
'https://www.redfin.com/NV/Henderson/721-N-Water-St-89015/home/184007790',
'https://www.redfin.com/NV/Henderson/719-N-Water-St-89015/home/184007789',
'https://www.redfin.com/NV/Henderson/717-N-Water-St-89015/home/184007788',
'https://www.redfin.com/NV/Henderson/1404-Bugle-Boy-Dr-89014/home/29665593',
'https://www.redfin.com/NV/Henderson/631-Moonlight-Stroll-St-89002/home/28993930',
'https://www.redfin.com/NV/Henderson/637-Apollo-Ave-89002/home/29720179',
'https://www.redfin.com/NV/Henderson/1318-Hopewell-Ave-89012/home/29880928',
'https://www.redfin.com/NV/Henderson/642-Solitude-Point-Ave-89012/home/29688852',
'https://www.redfin.com/NV/Henderson/260-Queen-Marie-Ct-89015/home/29707410',
'https://www.redfin.com/NV/Henderson/1345-Chestnut-St-89011/home/29644274',
'https://www.redfin.com/NV/Henderson/361-Old-Erie-Pl-89011/home/184007705',
'https://www.redfin.com/NV/Henderson/648-Federal-St-89015/home/29712156',
'https://www.redfin.com/NV/Henderson/341-Alto-Avec-Ln-89011/home/172506052',
'https://www.redfin.com/NV/Henderson/285-Mandarin-Hill-Ln-89012/home/109635621',
'https://www.redfin.com/NV/Henderson/218-Augusta-Ridge-Ct-89015/home/177310175',
'https://www.redfin.com/NV/Henderson/375-Old-Erie-Pl-89011/home/184007627',
'https://www.redfin.com/NV/Henderson/658-Foam-Flower-Ln-89015/unit-205/home/187414034',
'https://www.redfin.com/NV/Henderson/654-Foam-Flower-Ln-89015/unit-203/home/187413985',
'https://www.redfin.com/NV/Henderson/87-Constitution-Ave-89015/home/29701991',
'https://www.redfin.com/NV/Henderson/292-Sidewheeler-St-89012/home/145871183',
'https://www.redfin.com/NV/Henderson/369-Old-Erie-Pl-89011/home/184007701',
'https://www.redfin.com/NV/Henderson/365-Old-Erie-Pl-89011/home/184007703',
'https://www.redfin.com/NV/Henderson/731-Scottsdale-St-89002/home/29722065',
'https://www.redfin.com/NV/Henderson/852-Coral-Cottage-Dr-89002/home/29724384',
'https://www.redfin.com/NV/Henderson/1432-Evening-Song-Ave-89012/home/29837551',
'https://www.redfin.com/NV/Henderson/601-Assurance-Pl-89011/home/171302384',
'https://www.redfin.com/NV/Henderson/266-Mariposa-Way-89015/home/29704144',
'https://www.redfin.com/NV/Henderson/359-Old-Erie-Pl-89011/home/184007707',
'https://www.redfin.com/NV/Henderson/320-E-Longacres-Dr-89015/home/29711264',
'https://www.redfin.com/NV/Henderson/258-Camino-Viejo-St-89012/home/29688347',
'https://www.redfin.com/NV/Henderson/26-Arkansas-Ave-89015/home/29710620',
'https://www.redfin.com/NV/Henderson/218-Twin-Creek-St-89074/home/29879989',
'https://www.redfin.com/NV/Henderson/363-Old-Erie-Pl-89011/home/184007704',
'https://www.redfin.com/NV/Henderson/945-Tami-Cir-89015/home/29703242',
'https://www.redfin.com/NV/Henderson/2802-Camelback-Ln-89074/home/29621286',
'https://www.redfin.com/NV/Henderson/2813-Cacto-Ct-89074/home/29618168',
'https://www.redfin.com/NV/Las-Vegas/10365-Mint-Leaves-St-89183/home/29940627',
'https://www.redfin.com/NV/Las-Vegas/9801-Bradford-Summit-St-89183/home/29638200',
'https://www.redfin.com/NV/Las-Vegas/5517-Prospectors-Creek-Way-89122/home/29062492',
'https://www.redfin.com/NV/Las-Vegas/10590-Pueblo-Springs-St-89183/home/28979740',
'https://www.redfin.com/NV/Henderson/39-E-Pacific-Ave-89015/home/29709540',
'https://www.redfin.com/NV/Las-Vegas/5049-Spencer-St-89119/unit-D/home/29512687',
'https://www.redfin.com/NV/Las-Vegas/3773-Casey-Dr-89120/home/29442244',
'https://www.redfin.com/NV/Henderson/132-Almendio-Ln-89074/home/29618196',
'https://www.redfin.com/NV/Las-Vegas/5243-Sunnywood-Dr-89120/home/29443892',
'https://www.redfin.com/NV/Henderson/3769-Toscanella-Ave-89052/home/52623871',
'https://www.redfin.com/NV/Las-Vegas/1019-Sandy-Shale-St-89123/home/29631306',
'https://www.redfin.com/NV/Henderson/1423-Orange-Jubilee-Rd-89014/home/29053001',
'https://www.redfin.com/NV/Las-Vegas/4855-Tamalpias-Ave-89120/home/29440425',
'https://www.redfin.com/NV/Las-Vegas/4760-Woodridge-Rd-89121/home/29509639',
'https://www.redfin.com/NV/Las-Vegas/5446-Birchbrook-Ct-89120/home/29511605',
'https://www.redfin.com/NV/Las-Vegas/1833-E-Eldorado-Ln-89123/home/29613781',
'https://www.redfin.com/NV/Henderson/3084-Evening-Mist-Ave-89052/home/29742098',
'https://www.redfin.com/NV/Henderson/2428-Greens-Ave-89014/home/29648373',
'https://www.redfin.com/NV/Henderson/1168-Aspen-Cliff-Dr-89011/home/29449076',
'https://www.redfin.com/NV/Las-Vegas/2818-Calle-del-Oro-89120/home/29509671',
'https://www.redfin.com/NV/Henderson/529-Quail-Bird-Pl-89052/home/29636689',
'https://www.redfin.com/NV/Henderson/158-Rocky-Star-St-89012/home/28995855',
'https://www.redfin.com/NV/Henderson/832-Stepney-Pl-89011/home/177309357',
'https://www.redfin.com/NV/Las-Vegas/130-Rhodium-Ave-89183/home/177144772',
'https://www.redfin.com/NV/Las-Vegas/6416-Jacobville-Ct-89122/home/92908020',
'https://www.redfin.com/NV/Henderson/366-Silva-Pl-89011/home/182480556',
'https://www.redfin.com/NV/Henderson/249-Jessica-Grove-St-89015/home/29070834',
'https://www.redfin.com/NV/Henderson/919-Shadowfax-Rd-89015/home/29705652',
'https://www.redfin.com/NV/Henderson/412-Jena-St-89015/home/185774949',
'https://www.redfin.com/NV/Henderson/563-Duran-St-89015/home/29839869',
'https://www.redfin.com/NV/Henderson/1015-New-Creek-Ave-89015/home/28994030',
'https://www.redfin.com/NV/Henderson/994-River-Walk-Ct-89015/home/166566376',
'https://www.redfin.com/NV/Henderson/1129-Devon-Downs-Ave-89015/home/29715423',
'https://www.redfin.com/NV/Henderson/Undisclosed-address-89015/home/29715802',
'https://www.redfin.com/NV/Henderson/1012-Zurich-Ave-89015/home/187471300',
'https://www.redfin.com/NV/Henderson/302-Bay-Village-Pl-89011/home/177309561',
'https://www.redfin.com/NV/Henderson/17-Laswell-St-89015/home/29708718',
'https://www.redfin.com/NV/Henderson/821-Zinnia-Cir-89015/home/29704045',
'https://www.redfin.com/NV/Henderson/805-E-Sunset-Rd-89011/home/167787130',
'https://www.redfin.com/NV/Henderson/725-N-Water-St-89015/home/174180656',
'https://www.redfin.com/NV/Henderson/116-Milicity-Rd-89012/home/29881240',
'https://www.redfin.com/NV/Henderson/1525-Spiced-Wine-Ave-89074/unit-15105/home/144151254',
'https://www.redfin.com/NV/Henderson/760-Viento-del-Montagna-Ave-89012/home/29671069',
'https://www.redfin.com/NV/Henderson/1020-Atticus-Ave-89015/home/29715805',
'https://www.redfin.com/NV/Henderson/178-Emerson-Hill-St-89012/home/52628539',
'https://www.redfin.com/NV/Henderson/111-Ibiza-Lake-Dr-89011/home/181102615',
'https://www.redfin.com/NV/Henderson/121-Barkset-Way-89011/home/181102634',
'https://www.redfin.com/NV/Henderson/114-Joshua-St-89015/home/29701139',
'https://www.redfin.com/NV/Henderson/3-Via-Dolcetto-89011/home/146416827',
'https://www.redfin.com/NV/Henderson/490-Anthurium-Pl-89015/home/175528566',
'https://www.redfin.com/NV/Henderson/270-Grand-Teton-Dr-89074/home/29658599',
'https://www.redfin.com/NV/Henderson/407-Bandstand-Pl-89011/home/182480567',
'https://www.redfin.com/NV/Henderson/409-Bandstand-Pl-89011/home/182480566',
'https://www.redfin.com/NV/Henderson/411-Bandstand-Pl-89011/home/182480565',
'https://www.redfin.com/NV/Henderson/693-Moonstone-Beach-Pl-89011/home/179983741',
'https://www.redfin.com/NV/Henderson/107-Antler-Square-Ln-89011/home/175528264',
'https://www.redfin.com/NV/Henderson/105-Antler-Square-Ln-89011/home/175528263',
'https://www.redfin.com/NV/Las-Vegas/3007-Ferndale-St-89121/home/29413963',
'https://www.redfin.com/NV/Henderson/42-Barrio-Alto-Ct-89011/home/183992905',
'https://www.redfin.com/NV/Las-Vegas/2811-Sesame-Dr-89142/home/29415288',
'https://www.redfin.com/NV/Henderson/609-Cervantes-Dr-89014/home/29650117',
'https://www.redfin.com/NV/Henderson/684-Swift-Eagle-Ave-89015/home/143190128',
'https://www.redfin.com/NV/Henderson/39-Casa-del-Fuego-St-89012/home/29671222',
'https://www.redfin.com/NV/Henderson/1124-E-Sunset-Rd-89011/home/144625041',
'https://www.redfin.com/NV/Henderson/2243-Marlboro-Dr-89014/home/29650943',
'https://www.redfin.com/NV/Henderson/689-Pepper-Tree-Cir-89014/home/29649833',
'https://www.redfin.com/NV/Henderson/625-Cadence-Vista-Dr-89011/home/167786702',
'https://www.redfin.com/NV/Henderson/223-Concho-Dr-89015/home/29702337',
'https://www.redfin.com/NV/Henderson/6-Via-Amarone-89011/home/166148148',
'https://www.redfin.com/NV/Henderson/1003-Geneva-Ave-89015/home/29700065',
'https://www.redfin.com/NV/Las-Vegas/4318-E-Twain-Ave-89121/home/29419830',
'https://www.redfin.com/NV/Henderson/484-Filaree-Pl-89015/home/175528525',
'https://www.redfin.com/NV/Henderson/266-Tropic-Tan-Ct-89074/home/29879810',
'https://www.redfin.com/NV/Henderson/1047-Armillaria-St-89011/home/29398587',
'https://www.redfin.com/NV/Henderson/38-Trailside-Ct-89012/home/29685737',
'https://www.redfin.com/NV/Henderson/5883-Crumbling-Ridge-St-89011/home/29053016',
'https://www.redfin.com/NV/Henderson/5-Tatahatso-Point-St-89011/home/177327873',
'https://www.redfin.com/NV/Henderson/51-Via-Di-Mello-89011/home/52627469',
'https://www.redfin.com/NV/Henderson/788-Aurora-Vale-Pl-89011/home/177309501',
'https://www.redfin.com/NV/Henderson/84-Tatahatso-Point-St-89011/home/177327818',
'https://www.redfin.com/NV/Henderson/753-Skipjack-Dr-89015/home/29704515',
'https://www.redfin.com/NV/Henderson/218-Wewatta-AVE-Unknown/home/186732926',
'https://www.redfin.com/NV/Henderson/6233-Overhang-Ave-89011/home/29885799',
'https://www.redfin.com/NV/Henderson/105-Elm-St-89015/home/29701278',
'https://www.redfin.com/NV/Henderson/929-Kyanite-Ln-89011/unit-163/home/186677715',
'https://www.redfin.com/NV/Henderson/45-Desert-Juniper-Ln-89011/home/170372487',
'https://www.redfin.com/NV/Henderson/708-N-Water-St-89015/home/170579455',
'https://www.redfin.com/NV/Las-Vegas/4015-Russian-Rider-Dr-89122/home/29181997',
'https://www.redfin.com/NV/Henderson/860-Moncure-St-89011/home/185107590',
'https://www.redfin.com/NV/Henderson/34-Sun-Lily-Ln-89011/home/175524616',
'https://www.redfin.com/NV/Henderson/867-Moncure-St-89011/home/185107579',
'https://www.redfin.com/NV/Henderson/864-Moncure-St-89011/home/185107588',
'https://www.redfin.com/NV/Henderson/859-Moncure-St-89011/home/185107574',
'https://www.redfin.com/NV/Henderson/217-Jaldena-Ln-Unknown/home/185107555',
'https://www.redfin.com/NV/Henderson/1016-Skyforest-Dr-89011/home/29447602',
'https://www.redfin.com/NV/Henderson/313-Royal-Legato-Ct-89015/home/169241537',
'https://www.redfin.com/NV/Henderson/496-Sellers-Pl-89011/home/29670188',
'https://www.redfin.com/NV/Henderson/454-Sellers-Pl-89011/home/29670178',
'https://www.redfin.com/NV/Henderson/8-Via-del-Fiume-89011/home/166148095',
'https://www.redfin.com/NV/Las-Vegas/4567-Dennis-Way-89121/home/29428164',
'https://www.redfin.com/NV/Las-Vegas/6089-Forest-Pony-Ave-89122/home/29480900',
'https://www.redfin.com/NV/Henderson/6342-Heavy-Gorge-Ave-89011/unit-103/home/29887005',
'https://www.redfin.com/NV/Las-Vegas/4753-El-Tesoro-Ave-89121/home/29428302',
'https://www.redfin.com/NV/Henderson/465-Punto-Vallata-Dr-89011/home/29028281',
'https://www.redfin.com/NV/Henderson/870-N-Water-St-89011/home/109635128',
'https://www.redfin.com/NV/Henderson/1408-Evening-Song-Ave-89012/home/29837559',
'https://www.redfin.com/NV/Las-Vegas/6348-Jacobville-Ct-89122/home/65557063',
'https://www.redfin.com/NV/Henderson/215-Jaldena-Ln-Unknown/home/185107383',
'https://www.redfin.com/NV/Henderson/221-Jaldena-Ln-Unknown/home/185107557',
'https://www.redfin.com/NV/Henderson/The-Bluffs-II/Residence-2230/home/186520423',
'https://www.redfin.com/NV/Henderson/The-Bluffs-I/Residence-1831/home/186520459',
'https://www.redfin.com/NV/Henderson/The-Bluffs-II/Residence-2301/home/186520561',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Founders-Village-II/Leighton-Next-Gen/home/186926640',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Founders-Village-II/Victoria/home/186926643',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Founders-Village-I/Kennedy/home/186926645',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Founders-Village-I/Hamilton/home/186926648',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Founders-Village-II/Charles/home/186926660',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Founders-Village-II/Eleanor-Next-Gen/home/186926671',
'https://www.redfin.com/NV/Henderson/Modena-I/Residence-1675/home/187315605',
'https://www.redfin.com/NV/Henderson/Modena-II/Residence-1792/home/187315667',
'https://www.redfin.com/NV/Henderson/Modena-II/Residence-2119/home/187315699',
'https://www.redfin.com/NV/Henderson/Modena-I/Residence-2001/home/187315804',
'https://www.redfin.com/NV/Henderson/Modena-I/Residence-1859/home/187315805',
'https://www.redfin.com/NV/Henderson/Modena-II/Residence-1943/home/187315827',
'https://www.redfin.com/NV/Henderson/Modena-I/Residence-2114/home/187316008',
'https://www.redfin.com/NV/Henderson/The-Amara-Collection-at-Portofino-at-Lake-Las-Vegas/Amaryllis/home/187364514',
'https://www.redfin.com/NV/Henderson/The-Vista-Pointe-Collection-at-Portofino-at-Lake-Las-Vegas/Lotus/home/187364603',
'https://www.redfin.com/NV/Henderson/The-Vista-Pointe-Collection-at-Portofino-at-Lake-Las-Vegas/Allium/home/187364615',
'https://www.redfin.com/NV/Henderson/The-Amara-Collection-at-Portofino-at-Lake-Las-Vegas/Calla-Lilly/home/187364639',
'https://www.redfin.com/NV/Henderson/The-Amara-Collection-at-Portofino-at-Lake-Las-Vegas/Daphne/home/187364699',
'https://www.redfin.com/NV/Henderson/The-Vista-Pointe-Collection-at-Portofino-at-Lake-Las-Vegas/Peony/home/187383584',
'https://www.redfin.com/NV/Henderson/878-Silverton-SPGS-89011/unit-St/home/185154349',
'https://www.redfin.com/NV/Henderson/867-Klamath-SPGS-89011/unit-St/home/185507044',
'https://www.redfin.com/NV/Henderson/2312-Janesville-Ln-89044/home/29498307',
'https://www.redfin.com/NV/Henderson/3577-Sorridere-Ln-89044/home/177148770',
'https://www.redfin.com/NV/Henderson/2213-Bon-Papa-Ct-89044/home/29301768',
'https://www.redfin.com/NV/Henderson/2131-Fort-Sanders-St-89052/home/29906267',
'https://www.redfin.com/NV/Henderson/2520-Lochmaben-St-89044/home/29043981',
'https://www.redfin.com/NV/Henderson/2604-Centaurus-St-89044/home/29072083',
'https://www.redfin.com/NV/Henderson/2421-Padulle-Pl-89044/home/170374977',
'https://www.redfin.com/NV/Henderson/2836-Blythswood-Sq-89044/home/29013548',
'https://www.redfin.com/NV/Henderson/2488-Sturrock-Dr-89044/home/29001202',
'https://www.redfin.com/NV/Henderson/2563-Evening-Twilight-Ave-89044/home/28996700',
'https://www.redfin.com/NV/Henderson/2503-Stardust-Valley-Dr-89044/home/28995046',
'https://www.redfin.com/NV/Henderson/2123-Burtonsville-Dr-89044/home/29042587',
'https://www.redfin.com/NV/Henderson/3175-Arco-Ave-89044/home/146417515',
'https://www.redfin.com/NV/Henderson/2233-Gondi-Castle-Ave-89044/home/29498700',
'https://www.redfin.com/NV/Henderson/2729-Auchmull-St-89044/home/29072179',
'https://www.redfin.com/NV/Henderson/3243-Fountaintree-Ave-89044/home/179110041',
'https://www.redfin.com/NV/Henderson/3242-Fountaintree-Ave-89044/home/179110025',
'https://www.redfin.com/NV/Henderson/2531-Findlater-St-89044/home/29043013',
'https://www.redfin.com/NV/Henderson/3364-Aultmore-Ln-89044/home/148732007',
'https://www.redfin.com/NV/Henderson/2268-Merrimack-Valley-Ave-89044/home/29300685',
'https://www.redfin.com/NV/Henderson/2350-Hardin-Ridge-Dr-89052/home/29948390',
'https://www.redfin.com/NV/Henderson/2612-Solar-Corona-Ln-89044/home/29611170',
'https://www.redfin.com/NV/Henderson/2802-Meadow-Park-Ave-89052/home/29906320',
'https://www.redfin.com/NV/Henderson/2974-Gramsci-Ave-89044/home/166613188',
'https://www.redfin.com/NV/Henderson/2350-Via-Barranca-St-89044/home/52627343',
'https://www.redfin.com/NV/Henderson/3185-Bitetto-Walk-89044/home/144626533',
'https://www.redfin.com/NV/Henderson/2143-Twin-Falls-Dr-89044/home/29007372',
'https://www.redfin.com/NV/Henderson/2366-Ecliptic-St-89044/home/29008403',
'https://www.redfin.com/NV/Henderson/2765-Drumlanrig-St-89044/home/29072398',
'https://www.redfin.com/NV/Henderson/2357-Aztec-Ruin-Way-89044/home/29498696',
'https://www.redfin.com/NV/Henderson/2759-Lochleven-Way-89044/home/29012618',
'https://www.redfin.com/NV/Henderson/2795-Olivia-Heights-Ave-89052/home/29906178',
'https://www.redfin.com/NV/Henderson/2804-Goldcreek-St-89052/home/29909735',
'https://www.redfin.com/NV/Henderson/3046-Tara-Murphy-Dr-89044/home/29001991',
'https://www.redfin.com/NV/Henderson/2899-Ainslie-Lake-Ave-89044/home/28997589',
'https://www.redfin.com/NV/Henderson/2296-Aztec-Ruin-Way-89044/home/29498315',
'https://www.redfin.com/NV/Henderson/2933-Gallarate-Dr-89044/home/181144935',
'https://www.redfin.com/NV/Henderson/2633-Cooper-Gallery-St-89044/home/147874494',
'https://www.redfin.com/NV/Henderson/2510-Verdello-St-89044/home/171288980',
'https://www.redfin.com/NV/Henderson/2169-De-Narvik-Dr-89044/home/29302854',
'https://www.redfin.com/NV/Henderson/3208-Color-Collage-Walk-89044/home/30522110',
'https://www.redfin.com/NV/Henderson/2941-Gallarate-Dr-89044/home/181144938',
'https://www.redfin.com/NV/Henderson/2929-Gallarate-Dr-89044/home/181144934',
'https://www.redfin.com/NV/Henderson/2275-Keego-Harbor-St-89052/home/28995641',
'https://www.redfin.com/NV/Henderson/2163-Savannah-River-St-89044/home/29043146',
'https://www.redfin.com/NV/Henderson/1140-Chislehurst-Ct-89002/home/29071375',
'https://www.redfin.com/NV/Henderson/2131-Cumberland-Hill-Dr-89052/home/28965698',
'https://www.redfin.com/NV/Las-Vegas/1355-Silver-Perch-Ave-89123/home/29633932',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89183/home/29056338',
'https://www.redfin.com/NV/Henderson/1123-Cathedral-Ridge-St-89052/home/29880107',
'https://www.redfin.com/NV/Henderson/3140-Cartanda-Ave-89044/home/143190522',
'https://www.redfin.com/NV/Henderson/2482-Lower-Meadows-Ave-89052/home/29698211',
'https://www.redfin.com/NV/Henderson/2325-Shorewood-Hills-Ave-89052/home/28994884',
'https://www.redfin.com/NV/Henderson/1170-Pandora-Canyon-St-89052/home/166499161',
'https://www.redfin.com/NV/Henderson/2899-Carmelo-Dr-89052/home/29905641',
'https://www.redfin.com/NV/Henderson/2445-Hardin-Ridge-Dr-89052/home/29915993',
'https://www.redfin.com/NV/Henderson/1833-Tarrant-City-St-89052/home/29742999',
'https://www.redfin.com/NV/Henderson/2198-Shadow-Canyon-Dr-89044/home/29071994',
'https://www.redfin.com/NV/Henderson/478-Carmel-Mesa-Dr-89012/home/29694173',
'https://www.redfin.com/NV/Henderson/1555-Via-Della-Scala-89052/home/29905442',
'https://www.redfin.com/NV/Las-Vegas/12281-Hidden-Green-Ct-89141/home/144625548',
'https://www.redfin.com/NV/Henderson/3-Caprington-Rd-89052/home/28994892',
'https://www.redfin.com/NV/Henderson/2248-Camargue-Ln-89044/home/29301579',
'https://www.redfin.com/NV/Henderson/2748-Canarsy-Ct-89052/home/29635755',
'https://www.redfin.com/NV/Henderson/3034-Scenic-Valley-Way-89052/home/29641314',
'https://www.redfin.com/NV/Henderson/1676-Ravanusa-Dr-89052/home/29840766',
'https://www.redfin.com/NV/Henderson/2236-Shadow-Canyon-Dr-89044/home/29191038',
'https://www.redfin.com/NV/Henderson/1519-Camino-Summit-Ave-89012/home/29688094',
'https://www.redfin.com/NV/Henderson/2419-Bellinzona-Pl-89044/home/170374921',
'https://www.redfin.com/NV/Henderson/48-Hunt-Valley-Trl-89052/home/29946254',
'https://www.redfin.com/NV/Henderson/2200-Twin-Falls-Dr-89044/home/29013793',
'https://www.redfin.com/NV/Henderson/1502-Fieldbrook-St-89052/home/29738395',
'https://www.redfin.com/NV/Las-Vegas/9852-Altadena-St-89183/home/29638572',
'https://www.redfin.com/NV/Henderson/2644-Dirleton-Pl-89044/home/29044103',
'https://www.redfin.com/NV/Henderson/2401-Paveene-Ave-89052/home/29887810',
'https://www.redfin.com/NV/Henderson/3650-Modico-Ln-89044/home/175529775',
'https://www.redfin.com/NV/Henderson/1210-Sonatina-Dr-89052/home/29904233',
'https://www.redfin.com/NV/Henderson/2052-Powder-Springs-St-89052/home/28994560',
'https://www.redfin.com/NV/Henderson/2651-Soda-Butte-Ct-89052/home/109634078',
'https://www.redfin.com/NV/Henderson/2007-Joy-View-Ln-89012/home/29693681',
'https://www.redfin.com/NV/Las-Vegas/10053-Fine-Fern-St-89183/home/29876903',
'https://www.redfin.com/NV/Henderson/2025-Sinfonia-Ave-89052/home/29946405',
'https://www.redfin.com/NV/Henderson/3668-Via-Certaldo-Ave-89052/home/52623881',
'https://www.redfin.com/NV/Henderson/564-Broken-Shale-Cir-89052/home/29890730',
'https://www.redfin.com/NV/Henderson/3719-Scafati-Ave-89044/home/177148971',
'https://www.redfin.com/NV/Henderson/2100-Canvas-Edge-Dr-89044/home/109633772',
'https://www.redfin.com/NV/Henderson/2525-Hamonah-Dr-89044/home/29499222',
'https://www.redfin.com/NV/Henderson/2851-Maffie-St-89052/home/29916802',
'https://www.redfin.com/NV/Henderson/1633-Ravanusa-Dr-89052/home/29840789',
'https://www.redfin.com/NV/Henderson/2737-Drumlanrig-St-89044/home/29072387',
'https://www.redfin.com/NV/Henderson/2729-Carolina-Blue-Ave-89052/home/29641848',
'https://www.redfin.com/NV/Henderson/2313-Fossil-Canyon-Dr-89052/home/29920967',
'https://www.redfin.com/NV/Henderson/2714-Port-Lewis-Ave-89052/home/29880211',
'https://www.redfin.com/NV/Henderson/766-Faircliff-Ct-89052/home/166034230',
'https://www.redfin.com/NV/Henderson/1570-Livingston-Dr-89012/home/29687311',
'https://www.redfin.com/NV/Henderson/2024-High-Mesa-Dr-89012/home/29694378',
'https://www.redfin.com/NV/Henderson/1428-Via-Merano-St-89052/home/29741864',
'https://www.redfin.com/NV/Henderson/3639-Bitonto-Ln-89044/home/175529717',
'https://www.redfin.com/NV/Henderson/2165-Indigo-Creek-Ave-89012/home/29693619',
'https://www.redfin.com/NV/Henderson/1701-Warrington-Dr-89052/home/29742776',
'https://www.redfin.com/NV/Las-Vegas/1928-Canvas-Edge-Dr-89044/home/29778402',
'https://www.redfin.com/NV/Henderson/2557-Terrytown-Ave-89052/home/29738383',
'https://www.redfin.com/NV/Henderson/2531-Ballatore-St-89044/home/144625779',
'https://www.redfin.com/NV/Henderson/1272-Moon-Vision-St-89052/home/29742163',
'https://www.redfin.com/NV/Henderson/2663-Marvel-Astoria-St-89044/home/113358292',
'https://www.redfin.com/NV/Henderson/969-Spring-Haven-St-89052/home/144164594',
'https://www.redfin.com/NV/Henderson/1054-Bellevue-Peak-St-89002/home/145873856',
'https://www.redfin.com/NV/Henderson/2161-Sawtooth-Mountain-Dr-89044/home/29012263',
'https://www.redfin.com/NV/Henderson/3021-Via-Meridiana-89052/home/29641212',
'https://www.redfin.com/NV/Henderson/2726-Carolina-Blue-Ave-89052/home/29641862',
'https://www.redfin.com/NV/Henderson/1821-Tarrant-City-St-89052/home/29743001',
'https://www.redfin.com/NV/Henderson/2247-Tedesca-Dr-89052/home/29739665',
'https://www.redfin.com/NV/Henderson/2839-Lochbroom-Way-89044/home/29004304',
'https://www.redfin.com/NV/Las-Vegas/9935-Mystic-Dance-St-89183/home/29639927',
'https://www.redfin.com/NV/Henderson/2685-Cooper-Gallery-St-89044/home/146417166',
'https://www.redfin.com/NV/Las-Vegas/507-Dairy-Creek-Ave-89183/home/29922366',
'https://www.redfin.com/NV/Henderson/2734-Alfena-Pl-89044/home/179984954',
'https://www.redfin.com/NV/Henderson/3196-Monte-Luna-Ave-89044/home/179110053',
'https://www.redfin.com/NV/Henderson/3772-Battipaglia-Ave-89044/home/177149118',
'https://www.redfin.com/NV/Henderson/2267-Lazlo-Point-St-89044/home/184815961',
'https://www.redfin.com/NV/Las-Vegas/10441-Bent-River-St-89183/home/175527243',
'https://www.redfin.com/NV/Henderson/3543-Luciana-Glen-Ave-89044/home/184815927',
'https://www.redfin.com/NV/Henderson/2512-Vittora-Pointe-St-89044/home/182486662',
'https://www.redfin.com/NV/Henderson/2269-Lazlo-Point-St-89044/home/184815962',
'https://www.redfin.com/NV/Henderson/3539-Luciana-Glen-Ave-89044/home/184815929',
'https://www.redfin.com/NV/Henderson/662-Tyler-Ridge-Ave-89012/home/29688755',
'https://www.redfin.com/NV/Henderson/2356-Hardin-Ridge-Dr-89052/home/29948393',
'https://www.redfin.com/NV/Henderson/2692-La-Boutique-St-89044/home/29499043',
'https://www.redfin.com/NV/Las-Vegas/6020-Ancient-Peaks-Ave-89141/home/167763720',
'https://www.redfin.com/NV/Henderson/3235-Avernus-Ave-89044/home/166439756',
'https://www.redfin.com/NV/Las-Vegas/10531-Mann-St-89141/home/29771037',
'https://www.redfin.com/NV/Henderson/2755-Fort-Myer-Ave-89052/home/29906350',
'https://www.redfin.com/NV/Henderson/2716-Robust-Ct-89052/home/40233076',
'https://www.redfin.com/NV/Henderson/2867-Anaheim-Ave-89074/home/29635910',
'https://www.redfin.com/NV/Henderson/246-Calle-Palacio-St-89012/home/29002474',
'https://www.redfin.com/NV/Henderson/2021-Poppywood-Ave-89012/home/29694324',
'https://www.redfin.com/NV/Las-Vegas/10281-Desert-Barrel-Ct-89141/home/184024789',
'https://www.redfin.com/NV/Las-Vegas/10218-Glowing-Amber-St-89141/home/185796026',
'https://www.redfin.com/NV/Las-Vegas/9448-Lugo-St-89123/home/29634971',
'https://www.redfin.com/NV/Henderson/2707-Cool-Lilac-Ave-89052/home/29642051',
'https://www.redfin.com/NV/Henderson/3360-Solento-Ln-89044/home/161755239',
'https://www.redfin.com/NV/N-Las-Vegas/10215-Glowing-Amber-St-89081/home/185795994',
'https://www.redfin.com/NV/Las-Vegas/59-Belle-Springs-Ave-89123/home/29628387',
'https://www.redfin.com/NV/Henderson/78-Rattlesnake-Grass-Ct-89002/home/29015620',
'https://www.redfin.com/NV/Henderson/2683-Neon-Moon-Ct-89052/home/29637171',
'https://www.redfin.com/NV/Las-Vegas/10970-Winter-Rise-St-89141/home/145873379',
'https://www.redfin.com/NV/Henderson/2309-Aztec-Ruin-Way-89044/home/29498303',
'https://www.redfin.com/NV/Henderson/2350-Corte-Pescado-St-89044/home/52627388',
'https://www.redfin.com/NV/Henderson/39-Emerald-Dunes-Cir-89052/home/29883891',
'https://www.redfin.com/NV/Henderson/328-Aravalli-Crest-St-89015/home/179984342',
'https://www.redfin.com/NV/Henderson/1900-Magic-Canyon-Dr-89002/home/28969088',
'https://www.redfin.com/NV/Henderson/314-Robson-Ridge-Ln-89015/home/184469353',
'https://www.redfin.com/NV/Henderson/960-Harbor-Ave-89002/home/101339940',
'https://www.redfin.com/NV/Henderson/1950-Red-Sand-Ct-89002/home/29716547',
'https://www.redfin.com/NV/Henderson/1343-Graphite-Ave-89002/home/179984549',
'https://www.redfin.com/NV/Henderson/1339-Graphite-Ave-89002/home/179984548',
'https://www.redfin.com/NV/Henderson/1533-Eliana-Crossing-PL-Unknown/home/187661833',
'https://www.redfin.com/NV/Henderson/652-Foam-Flower-Ln-89015/home/184008646',
'https://www.redfin.com/NV/Henderson/658-Foam-Flower-Ln-89015/home/184008650',
'https://www.redfin.com/NV/Henderson/272-Mount-Williamson-St-89015/home/179983971',
'https://www.redfin.com/NV/Henderson/678-Foam-Flower-LN-89015/home/186319203',
'https://www.redfin.com/NV/Henderson/1352-Graphite-Ave-89002/unit-2/home/185633623',
'https://www.redfin.com/NV/Henderson/627-Prentice-St-89015/home/169242388',
'https://www.redfin.com/NV/Henderson/1070-Sheer-Paradise-Ln-89002/unit-2/home/29041841',
'https://www.redfin.com/NV/Henderson/676-Foam-Flower-LN-89015/home/186319228',
'https://www.redfin.com/NV/Henderson/1167-Paradise-Mountain-Trl-89002/home/29945215',
'https://www.redfin.com/NV/Henderson/2233-Sky-Island-Dr-89002/home/144164608',
'https://www.redfin.com/NV/Henderson/1156-Paradise-River-Rd-89002/home/29945356',
'https://www.redfin.com/NV/Henderson/1623-Lefty-Garcia-Way-89002/home/29951997',
'https://www.redfin.com/NV/Henderson/277-Mount-Williamson-St-89015/home/179983977',
'https://www.redfin.com/NV/Henderson/288-Fossil-Falls-St-89015/home/179984115',
'https://www.redfin.com/NV/Henderson/110-Barclay-Heights-Ave-89015/home/184008316',
'https://www.redfin.com/NV/Henderson/1352-Graphite-Ave-89002/home/179984516',
'https://www.redfin.com/NV/Henderson/1355-Graphite-Ave-89002/home/179984552',
'https://www.redfin.com/NV/Henderson/330-Owens-Peak-Dr-89015/home/179984203',
'https://www.redfin.com/NV/Henderson/1143-Safari-Creek-Dr-89002/home/29945059',
'https://www.redfin.com/NV/Henderson/1023-Benton-Ave-89015/home/169242401',
'https://www.redfin.com/NV/Henderson/480-Waterfall-Cove-Ct-89002/home/185775160',
'https://www.redfin.com/NV/Henderson/322-Owens-Peak-Dr-89015/home/179984207',
'https://www.redfin.com/NV/Henderson/471-Secret-Cove-Ct-89002/home/185775157',
'https://www.redfin.com/NV/Henderson/1343-Graphite-Ave-89002/unit-40/home/187332777',
'https://www.redfin.com/NV/Henderson/965-Nevada-State-Dr-89002/unit-18201/home/169242831',
'https://www.redfin.com/NV/Henderson/1161-Amarillo-Sky-Pl-89002/unit-1/home/29042487',
'https://www.redfin.com/NV/Henderson/207-Cimarron-Ct-89002/home/29723396',
'https://www.redfin.com/NV/Henderson/1355-Graphite-Ave-89002/unit-43/home/187182821',
'https://www.redfin.com/NV/Henderson/326-Owens-Peak-Dr-89015/home/179984205',
'https://www.redfin.com/NV/Henderson/831-Cherry-Dr-89002/home/28968984',
'https://www.redfin.com/NV/Henderson/467-Cove-Ct-89002/unit-46/home/187075787',
'https://www.redfin.com/NV/Henderson/709-Bighorn-Creek-St-89002/home/29056725',
'https://www.redfin.com/NV/Henderson/1194-Seaboard-Ct-89002/home/167794938',
'https://www.redfin.com/NV/Henderson/165-Coastal-Pier-Ct-89002/home/29724314',
'https://www.redfin.com/NV/Henderson/708-Morrocco-Dr-89002/home/29722078',
'https://www.redfin.com/NV/Henderson/111-Ibiza-LK-Unknown/unit-Dr/home/186995268',
'https://www.redfin.com/NV/Henderson/323-Owens-Peak-Dr-89015/home/179984192',
'https://www.redfin.com/NV/Henderson/1221-Grove-Park-St-89002/home/29056666',
'https://www.redfin.com/NV/Henderson/708-Greenway-Rd-89002/home/29723508',
'https://www.redfin.com/NV/Henderson/1022-Boundary-Line-St-89002/home/166610912',
'https://www.redfin.com/NV/Henderson/391-E-Kimberly-Dr-89015/home/29711297',
'https://www.redfin.com/NV/Henderson/2572-Lazy-Saddle-Dr-89002/home/29950984',
'https://www.redfin.com/NV/Henderson/838-Brahma-Ln-89002/home/29719269',
'https://www.redfin.com/NV/Henderson/1654-Lefty-Garcia-Way-89002/home/28972500',
'https://www.redfin.com/NV/Henderson/1016-Zurich-Ave-89015/home/186364061',
'https://www.redfin.com/NV/Henderson/528-Big-Lake-St-89002/home/29042151',
'https://www.redfin.com/NV/Henderson/34-Arkansas-Ave-89015/home/29710856',
'https://www.redfin.com/NV/Henderson/940-San-Carlos-Creek-Ln-89002/home/29013388',
'https://www.redfin.com/NV/Henderson/802-Mesa-Pine-Ct-89015/home/29706573',
'https://www.redfin.com/NV/Henderson/130-Cloudcrest-Dr-89015/home/29705743',
'https://www.redfin.com/NV/Henderson/312-Shimmering-Moon-St-89015/home/29071068',
'https://www.redfin.com/NV/Henderson/534-Eresma-St-89015/home/29042141',
'https://www.redfin.com/NV/Henderson/601-Sandy-Dr-89002/home/29726261',
'https://www.redfin.com/NV/Henderson/1507-Eliana-Crossing-PL-Unknown/home/187661956',
'https://www.redfin.com/NV/Henderson/310-Robson-Ridge-Ln-89015/home/184469354',
'https://www.redfin.com/NV/Henderson/530-E-Fairway-Rd-89015/home/29710953',
'https://www.redfin.com/NV/Henderson/624-Desert-Passage-St-89002/home/28994293',
'https://www.redfin.com/NV/Henderson/1147-Grass-Pond-Pl-89002/unit-102/home/108200482',
'https://www.redfin.com/NV/Henderson/1408-Colt-Dr-89002/home/29716743',
'https://www.redfin.com/NV/Henderson/244-Pioneers-Peak-Ave-89002/home/29057486',
'https://www.redfin.com/NV/Henderson/1146-Spottswoode-St-89002/home/29071927',
'https://www.redfin.com/NV/Henderson/976-Buffalo-River-Ave-89002/home/29008972',
'https://www.redfin.com/NV/Henderson/324-Aravalli-Crest-St-89015/home/179984343',
'https://www.redfin.com/NV/Henderson/228-Wewatta-AVE-Unknown/home/185575671',
'https://www.redfin.com/NV/Henderson/232-Wewatta-AVE-Unknown/home/185575605',
'https://www.redfin.com/NV/Henderson/230-Wewatta-AVE-Unknown/home/185575595',
'https://www.redfin.com/NV/Henderson/1626-Lefty-Garcia-Way-89002/home/29953339',
'https://www.redfin.com/NV/Henderson/55-Ruffled-Feather-Way-89012/home/29041640',
'https://www.redfin.com/NV/Henderson/Undisclosed-address-89002/home/29724740',
'https://www.redfin.com/NV/Henderson/1561-Ward-Frontier-Ln-89002/home/28994592',
'https://www.redfin.com/NV/Henderson/286-Robson-Ridge-Ln-89015/home/177310162',
'https://www.redfin.com/NV/Henderson/76-Eagle-Scout-Way-89012/home/29041680',
'https://www.redfin.com/NV/Henderson/640-Badlands-Ave-89012/home/29688712',
'https://www.redfin.com/NV/Henderson/1052-New-Creek-Ave-89015/home/28977326',
'https://www.redfin.com/NV/Henderson/908-Monte-Nerone-Ave-89012/home/40549439',
'https://www.redfin.com/NV/Henderson/112-Ibiza-LK-Unknown/unit-Dr/home/185575643',
'https://www.redfin.com/NV/Henderson/741-Heritage-Vista-Ave-89015/home/29712915',
'https://www.redfin.com/NV/Henderson/956-Open-Plains-Way-89002/home/108217237',
'https://www.redfin.com/NV/Henderson/43-Pangloss-St-89002/home/29883380',
'https://www.redfin.com/NV/Henderson/164-Golden-Crown-Ave-89002/home/29883691',
'https://www.redfin.com/NV/Henderson/2484-Crafty-Clint-Ln-89002/home/28994159',
'https://www.redfin.com/NV/Henderson/1031-Barker-Ridge-St-89002/home/187593582',
'https://www.redfin.com/NV/Henderson/739-Camino-del-Ray-89012/home/29881068',
'https://www.redfin.com/NV/Henderson/103-Magnesium-St-89015/home/29708818',
'https://www.redfin.com/NV/Henderson/300-E-Delamar-Dr-89015/home/29711317',
'https://www.redfin.com/NV/Henderson/732-Brick-Dr-89002/home/29721972',
'https://www.redfin.com/NV/Henderson/991-Alverstone-Ct-89002/home/166432399',
'https://www.redfin.com/NV/Henderson/361-Lime-Cir-89015/home/29707615',
'https://www.redfin.com/NV/Henderson/774-Wigan-Pier-Dr-89002/home/29944948',
'https://www.redfin.com/NV/Henderson/Juniper-Trails/Barclay/home/177039439',
'https://www.redfin.com/NV/Henderson/Juniper-Trails/Cambridge/home/177046958',
'https://www.redfin.com/NV/Henderson/Juniper-Trails/Albany/home/177046959',
'https://www.redfin.com/NV/Henderson/Juniper-Trails/Griffin/home/177482925',
'https://www.redfin.com/NV/Henderson/Juniper-Trails/Bedford/home/177482926',
'https://www.redfin.com/NV/Henderson/Auburn-Ridge/Bayview/home/179852923',
'https://www.redfin.com/NV/Henderson/Auburn-Ridge/Tiburon/home/179852924',
'https://www.redfin.com/NV/Henderson/Auburn-Ridge/Sausalito/home/179852925',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Highpointe/Rhett/home/181007230',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Highpointe/Reid/home/181007231',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Highpointe/Franklin-with-Next-Gen-Suite/home/181007232',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Highpointe/Ramsey/home/181007233',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Highpointe/Fletcher/home/181007234',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Highpointe/Finley/home/181007235',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Ridgeview/Evan/home/182951844',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Ridgeview/Liberty-NextGen/home/182951845',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Ridgeview/Pioneer-NextGen/home/182951847',
'https://www.redfin.com/NV/Henderson/The-Presley/Ramsey/home/183702443',
'https://www.redfin.com/NV/Henderson/The-Presley/Reid/home/183702510',
'https://www.redfin.com/NV/Henderson/The-Presley/Rhett/home/183702512',
'https://www.redfin.com/NV/Henderson/The-Presley/Franklin-Next-Gen/home/183702681',
'https://www.redfin.com/NV/Henderson/The-Presley/Fletcher/home/183702755',
'https://www.redfin.com/NV/Henderson/The-Presley/Finley/home/183703023',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Daybreak/Arwen/home/183703108',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Daybreak/Glenbrook/home/183703138',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Daybreak/Mitchell-NextGen/home/183703197',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Landmark/Nora/home/183703308',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Landmark/Elliot/home/183703312',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Landmark/Diana/home/183703456',
'https://www.redfin.com/NV/Henderson/Horizon-Trails/1417-Plan/home/186068884',
'https://www.redfin.com/NV/Henderson/Horizon-Trails/1319-Plan/home/186069590',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Heritage/Carson/home/187182707',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Heritage/Claremont/home/187233984',
'https://www.redfin.com/NV/Henderson/Black-Mountain-Ranch-Heritage/Connery/home/187234006',
'https://www.redfin.com/NV/Henderson/328-Owens-Peak-Dr-89015/home/179984204',
'https://www.redfin.com/NV/Henderson/325-Aravalli-Crest-St-89015/home/179984353',
'https://www.redfin.com/NV/Henderson/624-Sylvan-St-89002/home/183233550',
'https://www.redfin.com/NV/Henderson/1328-Graphite-Ave-89002/unit-8/home/187047273',
'https://www.redfin.com/NV/North-Las-Vegas/6322-Wind-Loft-St-89081/home/170137294',
'https://www.redfin.com/NV/North-Las-Vegas/6357-Supernova-Hill-St-89031/home/166075615',
'https://www.redfin.com/NV/North-Las-Vegas/6603-Ruddock-Dr-89084/home/28976883',
'https://www.redfin.com/NV/North-Las-Vegas/119-Queenswreath-Dr-89031/home/29086898',
'https://www.redfin.com/NV/Las-Vegas/7480-W-Verde-Way-89149/home/29099645',
'https://www.redfin.com/NV/Las-Vegas/6317-Guadalupe-Ave-89108/home/29172977',
'https://www.redfin.com/NV/Las-Vegas/7432-Saybrook-Point-Dr-89128/home/29197204',
'https://www.redfin.com/NV/Las-Vegas/7508-Maiden-Run-Ave-89130/home/29094632',
'https://www.redfin.com/NV/Las-Vegas/2521-Apricot-Ln-89108/home/187740207',
'https://www.redfin.com/NV/North-Las-Vegas/1717-James-St-89030/home/29331008',
'https://www.redfin.com/NV/Las-Vegas/4400-Dunlap-Crossing-St-89129/home/29118596',
'https://www.redfin.com/NV/Las-Vegas/2832-Geist-Cir-89115/home/29363034',
'https://www.redfin.com/NV/North-Las-Vegas/6013-Leaping-Foal-St-89081/home/29009651',
'https://www.redfin.com/NV/Las-Vegas/6513-Painted-Desert-Dr-89108/home/29117313',
'https://www.redfin.com/NV/North-Las-Vegas/5320-Chino-Heights-St-89081/home/29020408',
'https://www.redfin.com/NV/Las-Vegas/4005-Broadriver-Dr-89108/home/29119675',
'https://www.redfin.com/NV/Las-Vegas/8201-Bermuda-Beach-Dr-89128/home/29161687',
'https://www.redfin.com/NV/Las-Vegas/6802-Briarwood-Bend-Ave-89130/home/29478321',
'https://www.redfin.com/NV/Las-Vegas/4048-Woodgreen-Dr-89108/home/29119704',
'https://www.redfin.com/NV/Las-Vegas/2633-Ohio-Ct-89128/home/29146019',
'https://www.redfin.com/NV/North-Las-Vegas/4113-Keasberry-Ave-89081/home/30166906',
'https://www.redfin.com/NV/North-Las-Vegas/1835-Giant-Rock-Pl-89031/home/29077857',
'https://www.redfin.com/NV/North-Las-Vegas/3836-Van-Ness-Ave-89081/home/29265099',
'https://www.redfin.com/NV/North-Las-Vegas/1724-Hoover-St-89030/home/29326629',
'https://www.redfin.com/NV/North-Las-Vegas/6824-Satin-Silver-Ct-89086/unit-138/home/187684546',
'https://www.redfin.com/NV/North-Las-Vegas/2506-Angel-Field-St-89030/home/29305336',
'https://www.redfin.com/NV/North-Las-Vegas/2302-Superior-Position-St-89032/home/29308711',
'https://www.redfin.com/NV/Las-Vegas/1820-Jack-Rabbit-Way-89128/home/29166456',
'https://www.redfin.com/NV/Las-Vegas/1225-Pacific-Terrace-Dr-89128/home/29198205',
'https://www.redfin.com/NV/North-Las-Vegas/5116-Lydfort-Ct-89031/home/29081889',
'https://www.redfin.com/NV/Las-Vegas/8476-Stoney-Bluff-Ave-89129/home/29127105',
'https://www.redfin.com/NV/North-Las-Vegas/524-Princess-Ave-89030/home/29027651',
'https://www.redfin.com/NV/Las-Vegas/4840-Bedrock-Cove-Ct-89131/home/29006114',
'https://www.redfin.com/NV/Las-Vegas/8328-Sea-Glen-Dr-89128/home/29143048',
'https://www.redfin.com/NV/Las-Vegas/5236-Paradise-Skies-Ave-89156/home/29922194',
'https://www.redfin.com/NV/North-Las-Vegas/5017-Blue-Rose-St-89081/home/29178430',
'https://www.redfin.com/NV/North-Las-Vegas/1525-Ascension-Cir-89031/home/29083754',
'https://www.redfin.com/NV/North-Las-Vegas/4820-Nestled-Grove-Dr-89031/home/29081118',
'https://www.redfin.com/NV/Las-Vegas/4609-Pencester-St-89115/home/65559455',
'https://www.redfin.com/NV/North-Las-Vegas/4408-Fenton-Ln-89032/home/29245130',
'https://www.redfin.com/NV/Las-Vegas/6212-Leafy-St-89130/home/29024493',
'https://www.redfin.com/NV/North-Las-Vegas/3506-Shonna-Way-89032/home/29252459',
'https://www.redfin.com/NV/North-Las-Vegas/3909-Royal-Stone-Ct-89032/home/29061863',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89106/home/29324613',
'https://www.redfin.com/NV/North-Las-Vegas/3304-Brayton-Mist-Dr-89081/home/29003739',
'https://www.redfin.com/NV/North-Las-Vegas/1841-Luna-Vista-Pl-89084/home/29267817',
'https://www.redfin.com/NV/Las-Vegas/4611-Woolcomber-St-89115/home/55664591',
'https://www.redfin.com/NV/Las-Vegas/6429-Chippindale-Ln-89108/home/29137917',
'https://www.redfin.com/NV/North-Las-Vegas/4028-Farmdale-Ave-89031/home/29917894',
'https://www.redfin.com/NV/Las-Vegas/7500-Orange-Haze-Way-89149/home/29099505',
'https://www.redfin.com/NV/Las-Vegas/8137-Cimarron-Ridge-Dr-89128/home/29202544',
'https://www.redfin.com/NV/Las-Vegas/7705-Allano-Way-89128/home/29165281',
'https://www.redfin.com/NV/North-Las-Vegas/3129-Crawford-St-89030/home/29258140',
'https://www.redfin.com/NV/North-Las-Vegas/2909-Salado-Creek-Ave-89081/home/29018851',
'https://www.redfin.com/NV/Las-Vegas/5023-Tranquil-View-St-89130/home/179964075',
'https://www.redfin.com/NV/North-Las-Vegas/6121-Mott-Smith-St-89081/home/29178243',
'https://www.redfin.com/NV/Las-Vegas/7301-Fall-River-Cir-89129/home/29129264',
'https://www.redfin.com/NV/Las-Vegas/5540-Singing-Hills-Dr-89130/home/28965077',
'https://www.redfin.com/NV/North-Las-Vegas/6033-Hunter-Jumper-St-89081/home/28982308',
'https://www.redfin.com/NV/North-Las-Vegas/509-W-Colton-Ave-89032/home/29044385',
'https://www.redfin.com/NV/North-Las-Vegas/4008-Bola-Dr-89032/home/29242815',
'https://www.redfin.com/NV/Las-Vegas/7225-High-Cascade-Ave-89129/home/28986703',
'https://www.redfin.com/NV/North-Las-Vegas/Undisclosed-address-89031/home/29081862',
'https://www.redfin.com/NV/North-Las-Vegas/6128-Casper-Sands-Ct-89031/home/29842814',
'https://www.redfin.com/NV/North-Las-Vegas/4335-Twin-Peaks-Dr-89032/home/29240803',
'https://www.redfin.com/NV/North-Las-Vegas/1616-E-Evans-Ave-89030/home/29267907',
'https://www.redfin.com/NV/Las-Vegas/8429-Sewards-Bluff-Ave-89129/home/29127160',
'https://www.redfin.com/NV/Las-Vegas/5424-Cove-Point-Dr-89130/home/29102840',
'https://www.redfin.com/NV/Las-Vegas/5197-Silica-Chalk-Ave-89115/home/166540847',
'https://www.redfin.com/NV/Las-Vegas/5616-Desert-Creek-Way-89149/home/29096228',
'https://www.redfin.com/NV/North-Las-Vegas/3221-Orange-Orchid-Pl-89084/unit-3/home/29010065',
'https://www.redfin.com/NV/North-Las-Vegas/6524-Butterfly-Sky-St-89084/home/28998839',
'https://www.redfin.com/NV/Las-Vegas/4915-Chidlaw-St-89115/unit-1/home/183991520',
'https://www.redfin.com/NV/North-Las-Vegas/5210-Baltimore-Station-St-89031/home/29926789',
'https://www.redfin.com/NV/Las-Vegas/7764-Howard-Dade-Ave-89129/home/29128226',
'https://www.redfin.com/NV/North-Las-Vegas/6428-Spiced-Butter-Rum-St-89084/home/166572554',
'https://www.redfin.com/NV/Las-Vegas/1984-Nitida-St-89108/unit-6/home/187640953',
'https://www.redfin.com/NV/North-Las-Vegas/166-Macadamia-Nut-Ave-89084/home/165737133',
'https://www.redfin.com/NV/North-Las-Vegas/645-Roberta-Alecia-Ave-89031/home/29084879',
'https://www.redfin.com/NV/North-Las-Vegas/5820-Petrified-Tree-Ln-89081/home/144412533',
'https://www.redfin.com/NV/North-Las-Vegas/3320-Castlefields-Dr-89081/home/181098959',
'https://www.redfin.com/NV/North-Las-Vegas/6358-Ironville-Dr-89081/home/181098942',
'https://www.redfin.com/NV/North-Las-Vegas/4442-Rolls-Royce-Rd-89031/home/29238447',
'https://www.redfin.com/NV/North-Las-Vegas/2379-Arrington-Ave-89086/home/170466247',
'https://www.redfin.com/NV/Las-Vegas/8012-Hilliard-Ave-89128/home/29164507',
'https://www.redfin.com/NV/North-Las-Vegas/3328-Beca-Faith-Dr-89032/home/29252716',
'https://www.redfin.com/NV/Las-Vegas/8320-Bermuda-Beach-Dr-89128/home/29161590',
'https://www.redfin.com/NV/North-Las-Vegas/4206-Mountain-Rose-Ave-89031/home/29951320',
'https://www.redfin.com/NV/North-Las-Vegas/3324-Castlefields-Dr-89081/home/181098958',
'https://www.redfin.com/NV/North-Las-Vegas/6351-Ironville-Dr-89081/home/181099008',
'https://www.redfin.com/NV/North-Las-Vegas/2041-Rose-St-89032/home/29311823',
'https://www.redfin.com/NV/North-Las-Vegas/113-Barton-Bay-Ave-89031/home/179957041',
'https://www.redfin.com/NV/North-Las-Vegas/6354-Ironville-Dr-89081/home/181098941',
'https://www.redfin.com/NV/North-Las-Vegas/109-Barton-Bay-Ave-89031/home/179957040',
'https://www.redfin.com/NV/North-Las-Vegas/6048-La-Coste-St-89081/home/177334450',
'https://www.redfin.com/NV/Las-Vegas/2736-Chokeberry-Ct-89108/home/29137832',
'https://www.redfin.com/NV/North-Las-Vegas/1231-Chestnut-Sparrow-Ave-89084/home/179956479',
'https://www.redfin.com/NV/North-Las-Vegas/6820-Boulder-Canyon-St-89084/home/166071783',
'https://www.redfin.com/NV/North-Las-Vegas/1577-Cowboy-Path-Ave-89032/home/179978564',
'https://www.redfin.com/NV/North-Las-Vegas/4120-Golden-Sol-St-89032/home/179978570',
'https://www.redfin.com/NV/North-Las-Vegas/6616-Sea-Swallow-St-89084/home/28977602',
'https://www.redfin.com/NV/Las-Vegas/4348-Fox-Point-Dr-89108/home/29119073',
'https://www.redfin.com/NV/North-Las-Vegas/5825-Pirate-Ship-Dr-89031/home/29597274',
'https://www.redfin.com/NV/North-Las-Vegas/1201-Cobblestone-Cove-Rd-89081/home/29926561',
'https://www.redfin.com/NV/North-Las-Vegas/1602-Morning-Bell-Ave-89081/home/167613999',
'https://www.redfin.com/NV/Las-Vegas/4420-Dover-Straight-St-89115/home/29813962',
'https://www.redfin.com/NV/North-Las-Vegas/716-Newbridge-Way-89032/home/29255705',
'https://www.redfin.com/NV/North-Las-Vegas/1321-Red-Torador-Cir-89031/home/29084532',
'https://www.redfin.com/NV/North-Las-Vegas/6220-Wildcat-Brook-Ct-89081/home/29016966',
'https://www.redfin.com/NV/Las-Vegas/5413-Alabaster-Ct-89149/home/29098292',
'https://www.redfin.com/NV/North-Las-Vegas/5713-Lawrence-St-89081/home/28997471',
'https://www.redfin.com/NV/North-Las-Vegas/6749-Lookout-Lodge-Ln-89084/unit-2/home/29018050',
'https://www.redfin.com/NV/North-Las-Vegas/5224-Painted-Pebble-St-89081/home/28974927',
'https://www.redfin.com/NV/North-Las-Vegas/3625-Rocklin-Peak-Ave-89081/home/29264062',
'https://www.redfin.com/NV/Las-Vegas/3300-Tuscany-Village-Dr-89129/home/29127954',
'https://www.redfin.com/NV/North-Las-Vegas/1621-Dream-Canyon-Ave-89084/home/166572272',
'https://www.redfin.com/NV/North-Las-Vegas/6635-Palmer-Heights-St-89086/home/174789366',
'https://www.redfin.com/NV/Las-Vegas/4916-Virginia-Falls-Ln-89130/home/29116046',
'https://www.redfin.com/NV/North-Las-Vegas/6322-Antelope-Creek-Ct-89031/home/29852626',
'https://www.redfin.com/NV/North-Las-Vegas/4724-Calming-Garden-St-89031/home/176107236',
'https://www.redfin.com/NV/Las-Vegas/4530-Dover-Straight-St-89115/home/166121101',
'https://www.redfin.com/NV/Las-Vegas/2320-Viva-Cir-89108/home/29172200',
'https://www.redfin.com/NV/Las-Vegas/5608-Balzar-Ave-89108/home/29175504',
'https://www.redfin.com/NV/North-Las-Vegas/3355-Dragon-Fly-St-89032/home/29912017',
'https://www.redfin.com/NV/North-Las-Vegas/5540-Grand-Rapids-St-89031/home/29008362',
'https://www.redfin.com/NV/Las-Vegas/6132-Peggotty-Ave-89130/home/29094456',
'https://www.redfin.com/NV/North-Las-Vegas/6643-Montezuma-Castle-Ln-89084/home/29851645',
'https://www.redfin.com/NV/North-Las-Vegas/3556-Morgan-Springs-Ave-89081/home/29263933',
'https://www.redfin.com/NV/North-Las-Vegas/6633-Pass-Catcher-St-89086/home/170969176',
'https://www.redfin.com/NV/North-Las-Vegas/6629-Pass-Catcher-St-89086/home/170969175',
'https://www.redfin.com/NV/North-Las-Vegas/6625-Pass-Catcher-St-89086/home/170969174',
'https://www.redfin.com/NV/North-Las-Vegas/6621-Pass-Catcher-St-89086/home/170969173',
'https://www.redfin.com/NV/Las-Vegas/7709-Sandridge-Ct-89149/home/29096143',
'https://www.redfin.com/NV/North-Las-Vegas/5340-Santa-Fe-Heights-St-89081/home/29007216',
'https://www.redfin.com/NV/North-Las-Vegas/2803-Flower-Ave-89030/home/29328468',
'https://www.redfin.com/NV/North-Las-Vegas/2715-Amazing-Meadows-Ave-89031/unit-1185/home/165829518',
'https://www.redfin.com/NV/Las-Vegas/4060-E-Tonopah-Ave-89115/home/178669331',
'https://www.redfin.com/NV/North-Las-Vegas/1300-Moorpoint-Dr-89031/home/29078912',
'https://www.redfin.com/NV/Las-Vegas/7432-Stoney-Shore-Dr-89128/home/29167077',
'https://www.redfin.com/NV/North-Las-Vegas/3509-Golden-Sage-Dr-89032/home/29246231',
'https://www.redfin.com/NV/North-Las-Vegas/950-Twincrest-Ave-89032/home/175853840',
'https://www.redfin.com/NV/North-Las-Vegas/4456-Acropolis-Ave-89031/home/28977665',
'https://www.redfin.com/NV/North-Las-Vegas/4717-Vista-del-Rey-Ct-89031/home/29952052',
'https://www.redfin.com/NV/North-Las-Vegas/3420-Mercury-St-89030/unit-G/home/29255244',
'https://www.redfin.com/NV/Las-Vegas/4552-Vincente-Ln-89130/home/29116534',
'https://www.redfin.com/NV/North-Las-Vegas/4325-Bob-White-Dr-89032/home/29240853',
'https://www.redfin.com/NV/North-Las-Vegas/4220-Donato-Cir-89032/home/29243081',
'https://www.redfin.com/NV/Las-Vegas/6408-Eagle-Point-Rd-89108/home/29131459',
'https://www.redfin.com/NV/North-Las-Vegas/6040-Sea-Cliff-Cove-St-89031/home/29477202',
'https://www.redfin.com/NV/North-Las-Vegas/3737-Corte-Bella-Hills-Ave-89081/home/29263805',
'https://www.redfin.com/NV/Las-Vegas/5361-Brevin-Ct-89131/home/173319221',
'https://www.redfin.com/NV/North-Las-Vegas/3425-Palatine-Hills-Ave-89081/home/29019184',
'https://www.redfin.com/NV/Las-Vegas/3805-Twig-Ln-89108/home/29130835',
'https://www.redfin.com/NV/North-Las-Vegas/33-Newburg-Ave-89032/home/29237982',
'https://www.redfin.com/NV/North-Las-Vegas/3136-Iroquois-Ln-89030/home/29260521',
'https://www.redfin.com/NV/North-Las-Vegas/5201-Anneke-Way-89031/home/29853362',
'https://www.redfin.com/NV/North-Las-Vegas/5940-Whalers-Drift-St-89031/home/29843955',
'https://www.redfin.com/NV/Las-Vegas/4530-Rabnor-Cir-89115/home/166471170',
'https://www.redfin.com/NV/Las-Vegas/3004-Morning-Ridge-Dr-89134/home/29149342',
'https://www.redfin.com/NV/Las-Vegas/4229-Beth-Ave-89108/home/29306341',
'https://www.redfin.com/NV/Las-Vegas/2971-N-Gateway-Rd-89115/home/29364052',
'https://www.redfin.com/NV/Las-Vegas/7800-Paper-Flower-Ct-89128/home/29166539',
'https://www.redfin.com/NV/North-Las-Vegas/3300-E-Tonopah-Ave-89030/home/29331171',
'https://www.redfin.com/NV/North-Las-Vegas/1573-Cowboy-Path-Ave-89032/home/179978565',
'https://www.redfin.com/NV/Las-Vegas/6640-Crosstimber-Ct-89108/home/29131436',
'https://www.redfin.com/NV/Las-Vegas/6713-Silver-Spoon-Dr-89108/home/29138096',
'https://www.redfin.com/NV/Las-Vegas/5013-Pacific-Grove-Dr-89130/home/29102568',
'https://www.redfin.com/NV/Las-Vegas/3520-Miramar-Dr-89108/home/29133657',
'https://www.redfin.com/NV/Las-Vegas/7882-W-Gilmore-Ave-89129/home/29128056',
'https://www.redfin.com/NV/North-Las-Vegas/3437-Dewitt-Dr-89032/home/29252572',
'https://www.redfin.com/NV/Las-Vegas/3957-Eblick-Wash-Dr-89115/home/29353685',
'https://www.redfin.com/NV/North-Las-Vegas/2609-Positive-Ct-89031/home/29083400',
'https://www.redfin.com/NV/Las-Vegas/3013-Sungold-Dr-89134/home/29148942',
'https://www.redfin.com/NV/North-Las-Vegas/4738-Challenger-St-89031/home/170372058',
'https://www.redfin.com/NV/North-Las-Vegas/416-Rossmoyne-Ave-89030/home/29293601',
'https://www.redfin.com/NV/North-Las-Vegas/4736-Grand-Rock-Dr-89081/home/29235937',
'https://www.redfin.com/NV/Las-Vegas/3321-Michelangelo-Ct-89129/home/29127521',
'https://www.redfin.com/NV/North-Las-Vegas/5904-Willowcreek-Rd-89031/home/29079157',
'https://www.redfin.com/NV/North-Las-Vegas/628-Civic-Holiday-Ave-89031/home/28982447',
'https://www.redfin.com/NV/North-Las-Vegas/5717-Bullhead-St-89031/home/29477233',
'https://www.redfin.com/NV/Las-Vegas/7613-Hackensack-Ct-89128/home/29145755',
'https://www.redfin.com/NV/North-Las-Vegas/6125-Benchmark-Way-89031/home/29078769',
'https://www.redfin.com/NV/North-Las-Vegas/2236-Superior-Position-St-89032/home/29308712',
'https://www.redfin.com/NV/North-Las-Vegas/6608-Towerstone-St-89084/home/113141194',
'https://www.redfin.com/NV/North-Las-Vegas/6122-Circling-Hawk-Dr-89031/home/29077317',
'https://www.redfin.com/NV/Las-Vegas/720-Kasper-Ave-89106/home/29324362',
'https://www.redfin.com/NV/Las-Vegas/8405-Blazing-Sun-Ave-89129/home/29127689',
'https://www.redfin.com/NV/North-Las-Vegas/6103-Harvest-Dance-St-89031/home/29916845',
'https://www.redfin.com/NV/North-Las-Vegas/2100-Dogwood-Ave-89030/home/29263931',
'https://www.redfin.com/NV/Las-Vegas/6408-Rancho-Santa-Fe-Dr-89130/home/29100571',
'https://www.redfin.com/NV/North-Las-Vegas/6016-Vivanco-St-89081/home/177334421',
'https://www.redfin.com/NV/North-Las-Vegas/6020-Vivanco-St-89081/home/177334420',
'https://www.redfin.com/NV/Las-Vegas/6160-Browning-Way-89130/home/29101491',
'https://www.redfin.com/NV/North-Las-Vegas/6852-Sunflower-Fields-St-89086/home/181098801',
'https://www.redfin.com/NV/Las-Vegas/1936-Susan-St-89106/home/29324636',
'https://www.redfin.com/NV/North-Las-Vegas/3129-Emeritus-Cir-89032/home/29249510',
'https://www.redfin.com/NV/North-Las-Vegas/5908-Beluga-Bay-St-89081/home/161579583',
'https://www.redfin.com/NV/North-Las-Vegas/6024-Vivanco-St-89081/home/177334419',
'https://www.redfin.com/NV/North-Las-Vegas/3621-Covewick-Dr-89032/home/29252281',
'https://www.redfin.com/NV/North-Las-Vegas/6909-Arcadia-Creek-St-89084/home/29267732',
'https://www.redfin.com/NV/North-Las-Vegas/3836-Carisbrook-Dr-89081/home/29015068',
'https://www.redfin.com/NV/Las-Vegas/3864-Battery-St-89129/home/29127645',
'https://www.redfin.com/NV/Las-Vegas/8309-Shad-Bush-Ave-89149/home/29098015',
'https://www.redfin.com/NV/Las-Vegas/4620-Windy-Hollow-St-89130/home/29117986',
'https://www.redfin.com/NV/North-Las-Vegas/1228-Bluff-Ave-89030/home/29304350',
'https://www.redfin.com/NV/North-Las-Vegas/149-Thorntree-Ave-89031/home/167609310',
'https://www.redfin.com/NV/North-Las-Vegas/4816-Captain-McDonald-Ct-89031/home/29060456',
'https://www.redfin.com/NV/North-Las-Vegas/3816-W-Colton-Ave-89032/home/28987360',
'https://www.redfin.com/NV/Las-Vegas/7837-Silver-Plateau-Ave-89128/home/28970528',
'https://www.redfin.com/NV/North-Las-Vegas/4505-Galapagos-Ave-89084/home/29013469',
'https://www.redfin.com/NV/North-Las-Vegas/727-Oakbridge-Ct-89032/home/29255438',
'https://www.redfin.com/NV/North-Las-Vegas/5028-Jacarilla-Ln-89031/home/29083982',
'https://www.redfin.com/NV/Las-Vegas/4629-Spider-Ct-89108/home/29307098',
'https://www.redfin.com/NV/North-Las-Vegas/3138-Crownline-Ct-89031/home/29852466',
'https://www.redfin.com/NV/North-Las-Vegas/4029-Spring-Line-St-89032/home/29010477',
'https://www.redfin.com/NV/Las-Vegas/2140-Bavington-Dr-89108/unit-C/home/29170753',
'https://www.redfin.com/NV/Las-Vegas/6208-Minerva-Dr-89130/home/29117688',
'https://www.redfin.com/NV/North-Las-Vegas/913-Rocky-Stone-Ave-89081/home/29235984',
'https://www.redfin.com/NV/North-Las-Vegas/3618-Dutchmans-Vine-Ct-89081/home/29058813',
'https://www.redfin.com/NV/North-Las-Vegas/2556-Dollison-Ave-89081/home/170137187',
'https://www.redfin.com/NV/Las-Vegas/1964-H-St-89106/home/29324381',
'https://www.redfin.com/NV/Las-Vegas/4205-Topsider-St-89129/home/29120388',
'https://www.redfin.com/NV/North-Las-Vegas/5313-Dawn-Break-Canyon-St-89031/home/29926727',
'https://www.redfin.com/NV/Las-Vegas/4325-Bacara-Ridge-Ave-89115/home/29046559',
'https://www.redfin.com/NV/Las-Vegas/6245-Hawthorn-Woods-Ave-89130/home/29007056',
'https://www.redfin.com/NV/North-Las-Vegas/1585-Gold-Sunset-Ave-89032/home/179978604',
'https://www.redfin.com/NV/North-Las-Vegas/1619-Gold-Sunset-Ave-89032/home/179978602',
'https://www.redfin.com/NV/Las-Vegas/5313-Fountain-Palm-St-89130/home/29102377',
'https://www.redfin.com/NV/Las-Vegas/2633-Los-Palos-St-89108/home/28974702',
'https://www.redfin.com/NV/Las-Vegas/5045-N-Bonita-Vista-St-89149/home/29097405',
'https://www.redfin.com/NV/North-Las-Vegas/204-Frapuccino-Ave-89084/home/29059474',
'https://www.redfin.com/NV/Las-Vegas/7721-Kasmere-Falls-Dr-89149/home/29096152',
'https://www.redfin.com/NV/Las-Vegas/3651-Starry-Beach-Ave-89115/home/29932237',
'https://www.redfin.com/NV/Las-Vegas/4520-Crimson-Leaf-Dr-89130/home/28966182',
'https://www.redfin.com/NV/Las-Vegas/1425-Bow-Creek-Ct-89128/home/29197569',
'https://www.redfin.com/NV/North-Las-Vegas/6449-Raptor-Ct-89084/home/28980907',
'https://www.redfin.com/NV/North-Las-Vegas/6232-Double-Oak-St-89031/home/29019470',
'https://www.redfin.com/NV/Las-Vegas/8500-Festival-Dr-89134/home/29148838',
'https://www.redfin.com/NV/Las-Vegas/7224-Sublime-Ave-89130/home/29095979',
'https://www.redfin.com/NV/Las-Vegas/8463-Vacarez-Dr-89149/home/170654116',
'https://www.redfin.com/NV/Las-Vegas/6925-Coral-Rock-Dr-89108/home/29119834',
'https://www.redfin.com/NV/Las-Vegas/4341-Panther-Cove-Dr-89115/home/169230413',
'https://www.redfin.com/NV/Las-Vegas/7717-Tinted-Mesa-Ct-89149/home/29098341',
'https://www.redfin.com/NV/Las-Vegas/833-Hart-Ave-89106/home/29323916',
'https://www.redfin.com/NV/Las-Vegas/8453-Vast-Horizon-Ave-89129/home/29127765',
'https://www.redfin.com/NV/Las-Vegas/6444-Deadwood-Rd-89108/home/29139041',
'https://www.redfin.com/NV/Las-Vegas/7709-Bauble-Ave-89128/home/29166658',
'https://www.redfin.com/NV/Las-Vegas/1701-Goodrich-Cir-89108/home/29171678',
'https://www.redfin.com/NV/Las-Vegas/1612-Barrel-Cactus-Ct-89108/home/29174230',
'https://www.redfin.com/NV/Las-Vegas/7936-Ben-Hogan-Dr-89149/home/29099128',
'https://www.redfin.com/NV/Las-Vegas/5923-Bromley-Ave-89107/home/29188360',
'https://www.redfin.com/NV/Las-Vegas/7766-Vista-Sunrise-Dr-89149/home/29098489',
'https://www.redfin.com/NV/Las-Vegas/5345-Big-Fawn-Ct-89130/home/29093616',
'https://www.redfin.com/NV/Las-Vegas/1635-Dwayne-Stedman-Ave-89106/home/29861044',
'https://www.redfin.com/NV/Las-Vegas/6309-Gambetta-St-89115/home/179954149',
'https://www.redfin.com/NV/North-Las-Vegas/6861-Desert-Finch-St-89084/home/179956407',
'https://www.redfin.com/NV/North-Las-Vegas/2721-River-Ranch-Pl-89081/home/29048332',
'https://www.redfin.com/NV/Las-Vegas/100-S-Crestline-Dr-89107/unit-54/home/29229166',
'https://www.redfin.com/NV/North-Las-Vegas/4212-Karma-Dr-89032/home/29238375',
'https://www.redfin.com/NV/Las-Vegas/5524-Goldbrush-St-89130/home/29099684',
'https://www.redfin.com/NV/North-Las-Vegas/4441-Ivory-Jade-Ave-89031/home/29080377',
'https://www.redfin.com/NV/North-Las-Vegas/3325-Crawford-St-89030/home/29257178',
'https://www.redfin.com/NV/North-Las-Vegas/3123-Inlet-Bay-Ave-89031/home/29852822',
'https://www.redfin.com/NV/Las-Vegas/7251-Daily-Double-Ave-89129/home/29955542',
'https://www.redfin.com/NV/Las-Vegas/909-Shady-Creek-Dr-89108/home/29196047',
'https://www.redfin.com/NV/Las-Vegas/5709-Mossman-Ave-89108/home/29135166',
'https://www.redfin.com/NV/Las-Vegas/7744-Silver-Wells-Rd-89149/home/29099059',
'https://www.redfin.com/NV/Las-Vegas/4178-Story-Rock-St-89115/home/29599079',
'https://www.redfin.com/NV/North-Las-Vegas/4142-Nipp-Ave-89081/home/177334457',
'https://www.redfin.com/NV/Las-Vegas/6209-Gambetta-St-89115/home/179954139',
'https://www.redfin.com/NV/Las-Vegas/6241-Gambetta-St-89115/home/179954147',
'https://www.redfin.com/NV/North-Las-Vegas/2341-Whitney-Peak-Way-89031/home/29239482',
'https://www.redfin.com/NV/Las-Vegas/3816-Apache-Ln-89107/home/29342751',
'https://www.redfin.com/NV/Las-Vegas/3714-Coleus-AVE-89108/unit-2/home/185911204',
'https://www.redfin.com/NV/Las-Vegas/1342-N-Jones-Blvd-89108/home/29184092',
'https://www.redfin.com/NV/Las-Vegas/5569-Holcomb-Bridge-Ct-89149/home/29276491',
'https://www.redfin.com/NV/Las-Vegas/3782-Norton-Dr-89129/home/29128342',
'https://www.redfin.com/NV/North-Las-Vegas/1216-Fox-Grove-Ct-89031/home/143190716',
'https://www.redfin.com/NV/Las-Vegas/10468-Nordic-Cliff-Ave-89129/home/29277322',
'https://www.redfin.com/NV/Las-Vegas/9821-Aspen-Grove-Pl-89134/home/29155127',
'https://www.redfin.com/NV/Las-Vegas/8717-Prairie-Hill-Dr-89134/home/29159298',
'https://www.redfin.com/NV/Las-Vegas/2521-Banora-Point-Dr-89134/home/28969573',
'https://www.redfin.com/NV/Las-Vegas/2501-Palmridge-Dr-89134/home/29150239',
'https://www.redfin.com/NV/Las-Vegas/10317-Pompei-Pl-89144/home/29112354',
'https://www.redfin.com/NV/Las-Vegas/9305-Magic-Flower-Ave-89134/home/29156918',
'https://www.redfin.com/NV/Las-Vegas/544-Lacabana-Beach-Dr-89138/home/29951339',
'https://www.redfin.com/NV/Las-Vegas/9324-Chilly-Pond-Ave-89129/home/29122661',
'https://www.redfin.com/NV/Las-Vegas/8939-Dove-Cove-Dr-89129/home/29123898',
'https://www.redfin.com/NV/Las-Vegas/2384-Open-Valley-St-89138/home/179981719',
'https://www.redfin.com/NV/Las-Vegas/11616-Observation-Point-Ave-89138/home/179981739',
'https://www.redfin.com/NV/Las-Vegas/863-Elmstone-Pl-89138/home/163613771',
'https://www.redfin.com/NV/Las-Vegas/654-Pyrite-Will-St-89138/home/176105833',
'https://www.redfin.com/NV/Las-Vegas/11777-Pandion-Ave-89138/home/183997333',
'https://www.redfin.com/NV/Las-Vegas/2825-Greenview-Ct-89134/home/29147994',
'https://www.redfin.com/NV/Las-Vegas/10033-Bonterra-Ave-89129/home/28966612',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89134/home/29108790',
'https://www.redfin.com/NV/Las-Vegas/8805-Sundial-Dr-89134/home/29150618',
'https://www.redfin.com/NV/Las-Vegas/11856-Dragontail-Peak-Ave-89138/home/173182002',
'https://www.redfin.com/NV/Las-Vegas/660-Clowder-Spring-Way-89138/home/176105820',
'https://www.redfin.com/NV/Las-Vegas/10325-Summer-River-Ave-89144/home/29110421',
'https://www.redfin.com/NV/Las-Vegas/10936-Free-Flow-Pl-89138/home/147876279',
'https://www.redfin.com/NV/Las-Vegas/336-McLoughlin-St-89138/home/173181954',
'https://www.redfin.com/NV/Las-Vegas/3913-Copperhead-Hills-St-89129/home/29121978',
'https://www.redfin.com/NV/Las-Vegas/2125-Whitebirch-Ln-89134/home/29156543',
'https://www.redfin.com/NV/Las-Vegas/617-Ivy-Spring-St-89138/home/29007707',
'https://www.redfin.com/NV/Las-Vegas/3105-Moongold-Ct-89134/home/29146943',
'https://www.redfin.com/NV/Las-Vegas/11569-Earth-Stone-Ave-89138/home/176105900',
'https://www.redfin.com/NV/Las-Vegas/10534-Midnight-Gleam-Ave-89129/home/29277397',
'https://www.redfin.com/NV/Las-Vegas/169-Agnew-St-89138/home/176106608',
'https://www.redfin.com/NV/Las-Vegas/2612-Tumble-Brook-Dr-89134/home/29152161',
'https://www.redfin.com/NV/Las-Vegas/2101-Red-Dawn-Sky-St-89134/home/29107695',
'https://www.redfin.com/NV/Las-Vegas/11740-San-Rosarita-Ct-89138/home/28986102',
'https://www.redfin.com/NV/Las-Vegas/8801-Smokey-Dr-89134/home/29159262',
'https://www.redfin.com/NV/Las-Vegas/11748-Corrigan-Pl-89138/home/28973923',
'https://www.redfin.com/NV/Las-Vegas/12248-Montura-Rosa-Pl-89138/home/29025107',
'https://www.redfin.com/NV/Las-Vegas/11765-Pandion-Ave-89138/home/183997335',
'https://www.redfin.com/NV/Las-Vegas/11783-Pandion-Ave-89138/home/183997331',
'https://www.redfin.com/NV/Las-Vegas/786-Carlon-Hills-Ave-89138/home/179981856',
'https://www.redfin.com/NV/Las-Vegas/8961-Fort-Crestwood-Dr-89129/home/29123725',
'https://www.redfin.com/NV/Las-Vegas/11795-Pandion-Ave-89138/home/183997329',
'https://www.redfin.com/NV/Las-Vegas/11801-Pandion-Ave-89138/home/183997328',
'https://www.redfin.com/NV/Las-Vegas/11807-Pandion-Ave-89138/home/183997327',
'https://www.redfin.com/NV/Las-Vegas/11813-Pandion-Ave-89138/home/183997326',
'https://www.redfin.com/NV/Las-Vegas/11789-Pandion-Ave-89138/home/183997330',
'https://www.redfin.com/NV/Las-Vegas/4225-Sago-Lily-St-89129/home/183996674',
'https://www.redfin.com/NV/Las-Vegas/11612-Observation-Point-Ave-89138/home/179981740',
'https://www.redfin.com/NV/Las-Vegas/10637-Heritage-Hills-Dr-89134/home/29108867',
'https://www.redfin.com/NV/Las-Vegas/189-Springbough-Ln-89138/home/170970267',
'https://www.redfin.com/NV/Las-Vegas/8713-Autumn-Wreath-Ave-89129/home/29125685',
'https://www.redfin.com/NV/Las-Vegas/10360-Faustine-Ave-89129/home/29001285',
'https://www.redfin.com/NV/Las-Vegas/9417-Amber-Valley-Ln-89134/home/29156949',
'https://www.redfin.com/NV/Las-Vegas/8657-Raindrop-Canyon-Ave-89129/home/29125880',
'https://www.redfin.com/NV/Las-Vegas/10624-Night-Magic-Ct-89129/home/29024386',
'https://www.redfin.com/NV/Las-Vegas/11449-Parkersburg-Ave-89138/home/29006506',
'https://www.redfin.com/NV/Las-Vegas/10970-Terra-Azul-Pl-89138/home/145873050',
'https://www.redfin.com/NV/Las-Vegas/12309-Fanwood-Ln-89138/home/146415770',
'https://www.redfin.com/NV/Las-Vegas/4367-Western-Front-St-89129/home/166090747',
'https://www.redfin.com/NV/Las-Vegas/600-Bachelor-Button-St-89138/home/29025631',
'https://www.redfin.com/NV/Las-Vegas/9081-Herrera-Ave-89129/home/28987186',
'https://www.redfin.com/NV/Las-Vegas/600-N-Carriage-Hill-Dr-89138/unit-1099/home/181147792',
'https://www.redfin.com/NV/Las-Vegas/10512-Jeremy-Ridge-Ave-89144/home/29108434',
'https://www.redfin.com/NV/Las-Vegas/11563-Desert-Hollow-Ave-89138/home/179981647',
'https://www.redfin.com/NV/Las-Vegas/2108-Sun-Cliffs-St-89134/home/29107718',
'https://www.redfin.com/NV/Las-Vegas/2409-Floyd-Ct-89134/home/29149081',
'https://www.redfin.com/NV/Las-Vegas/3948-Total-Eclipse-St-89129/home/55518951',
'https://www.redfin.com/NV/Las-Vegas/10604-Fassbinder-Ct-89129/home/29024824',
'https://www.redfin.com/NV/Las-Vegas/9332-Fresh-Spring-Dr-89134/home/29152409',
'https://www.redfin.com/NV/Las-Vegas/2213-Fawn-Ridge-St-89134/home/29109389',
'https://www.redfin.com/NV/Las-Vegas/11677-Hatchling-Ave-89138/home/181147147',
'https://www.redfin.com/NV/Las-Vegas/1908-Vero-Dr-89134/home/29160458',
'https://www.redfin.com/NV/Las-Vegas/2048-Wandering-Doe-Ln-89134/home/29109489',
'https://www.redfin.com/NV/Las-Vegas/8800-Sundial-Dr-89134/home/29150589',
'https://www.redfin.com/NV/Las-Vegas/11837-Pippa-Ave-89138/home/173620939',
'https://www.redfin.com/NV/Las-Vegas/10636-Clarion-Ln-89134/home/29108870',
'https://www.redfin.com/NV/Las-Vegas/3401-Conan-St-89129/unit-103/home/29121601',
'https://www.redfin.com/NV/Las-Vegas/211-Robledo-St-89138/home/177121595',
'https://www.redfin.com/NV/Las-Vegas/9709-Cameo-Rose-Ln-89134/home/29154185',
'https://www.redfin.com/NV/Las-Vegas/9213-Villa-Ridge-Dr-89134/home/29146929',
'https://www.redfin.com/NV/Las-Vegas/921-Percy-Arms-St-89138/home/28986596',
'https://www.redfin.com/NV/Las-Vegas/2637-Tumble-Brook-Dr-89134/home/29152168',
'https://www.redfin.com/NV/Las-Vegas/2409-Indigo-Valley-St-89134/home/29105475',
'https://www.redfin.com/NV/Las-Vegas/2421-Desert-Butte-Dr-89134/home/29150707',
'https://www.redfin.com/NV/Las-Vegas/9138-Sparklewood-Ct-89129/home/29124174',
'https://www.redfin.com/NV/Las-Vegas/4529-Dawn-Peak-St-89129/home/29928861',
'https://www.redfin.com/NV/Las-Vegas/10408-Longwood-Dr-89134/home/29109689',
'https://www.redfin.com/NV/Las-Vegas/9100-Sundial-Dr-89134/home/29148173',
'https://www.redfin.com/NV/Las-Vegas/2304-Sierra-Heights-Dr-89134/home/29107959',
'https://www.redfin.com/NV/Las-Vegas/2109-Jadeleaf-Ct-89134/home/29157285',
'https://www.redfin.com/NV/Las-Vegas/2401-Indigo-Valley-St-89134/home/29105457',
'https://www.redfin.com/NV/Las-Vegas/9936-Bundella-Dr-89134/home/29151022',
'https://www.redfin.com/NV/Las-Vegas/10413-Long-Leaf-Pl-89134/home/29108389',
'https://www.redfin.com/NV/Las-Vegas/361-Purple-Sandpiper-St-89138/home/169228433',
'https://www.redfin.com/NV/Las-Vegas/10508-Mission-Lakes-Ave-89134/home/29108894',
'https://www.redfin.com/NV/Las-Vegas/2021-Summer-Spruce-Pl-89134/unit-101/home/29109762',
'https://www.redfin.com/NV/Las-Vegas/3470-White-Mission-Dr-89129/home/29124063',
'https://www.redfin.com/NV/Las-Vegas/121-Agnew-St-89138/home/176106590',
'https://www.redfin.com/NV/Las-Vegas/2737-Valley-Downs-Dr-89134/home/29106239',
'https://www.redfin.com/NV/Las-Vegas/154-Nine-Mile-Creek-Dr-89138/home/173620808',
'https://www.redfin.com/NV/Las-Vegas/8741-Castle-Hill-Ave-89129/home/29124943',
'https://www.redfin.com/NV/Las-Vegas/1952-Faywood-St-89134/home/29157646',
'https://www.redfin.com/NV/Las-Vegas/2820-Tumble-Brook-Dr-89134/home/29150953',
'https://www.redfin.com/NV/Las-Vegas/486-Fairview-Hills-St-89138/home/173620758',
'https://www.redfin.com/NV/Las-Vegas/2912-Vista-Butte-Dr-89134/home/29105851',
'https://www.redfin.com/NV/Las-Vegas/1600-Tranquil-Meadows-Ln-89128/home/29163625',
'https://www.redfin.com/NV/Las-Vegas/2537-Desert-Glen-Dr-89134/home/29150331',
'https://www.redfin.com/NV/Las-Vegas/2328-Huntsville-Dr-89134/home/29151808',
'https://www.redfin.com/NV/Las-Vegas/4025-Treasured-Note-Ct-89129/home/28972975',
'https://www.redfin.com/NV/Las-Vegas/2913-Abercorn-Dr-89134/home/29105978',
'https://www.redfin.com/NV/Las-Vegas/2840-Bluff-Point-Dr-89134/home/29152576',
'https://www.redfin.com/NV/Las-Vegas/231-Carmel-Sky-St-89138/home/176106114',
'https://www.redfin.com/NV/Las-Vegas/10301-Horseback-Ridge-Ave-89144/home/29111436',
'https://www.redfin.com/NV/Las-Vegas/5166-N-Chieftain-St-89149/home/29097239',
'https://www.redfin.com/NV/Las-Vegas/1186-Blossom-Point-ST-89135/unit-21/home/179250259',
'https://www.redfin.com/NV/Las-Vegas/6227-Don-Gaspar-Ave-89108/home/29173362',
'https://www.redfin.com/NV/Las-Vegas/6717-Old-Castle-Dr-89108/home/29119160',
'https://www.redfin.com/NV/Las-Vegas/1045-Puerta-del-Sol-Dr-89138/home/29025984',
'https://www.redfin.com/NV/Las-Vegas/1601-Cathedral-Peak-Ct-89134/home/29156062',
'https://www.redfin.com/NV/Las-Vegas/7629-Delaware-Bay-Dr-89128/home/29145813',
'https://www.redfin.com/NV/Las-Vegas/2436-Palm-Shore-Ct-89128/home/29143888',
'https://www.redfin.com/NV/Las-Vegas/5015-Jessica-Joy-St-89149/home/29024538',
'https://www.redfin.com/NV/Las-Vegas/2604-Palmridge-Dr-89134/home/29150249',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89134/home/29153367',
'https://www.redfin.com/NV/Las-Vegas/3705-Bronco-Billy-Ct-89129/home/29127038',
'https://www.redfin.com/NV/Las-Vegas/10025-Hemet-Dr-89134/home/29151182',
'https://www.redfin.com/NV/Las-Vegas/1344-Silver-Sierra-St-89128/home/29203700',
'https://www.redfin.com/NV/Las-Vegas/11557-Earth-Stone-Ave-89138/home/176105902',
'https://www.redfin.com/NV/Las-Vegas/668-Tranquil-Jade-St-89138/home/176105852',
'https://www.redfin.com/NV/Las-Vegas/11826-Corenzio-Ave-89138/home/143190582',
'https://www.redfin.com/NV/Las-Vegas/654-Solar-Citrine-St-89138/home/176105845',
'https://www.redfin.com/NV/Las-Vegas/668-Solar-Citrine-St-89138/home/176105843',
'https://www.redfin.com/NV/Las-Vegas/675-Solar-Citrine-St-89138/home/176105841',
'https://www.redfin.com/NV/Las-Vegas/643-Solar-Citrine-St-89138/home/176105836',
'https://www.redfin.com/NV/Las-Vegas/7505-Cobal-Canyon-Ln-89129/home/29118547',
'https://www.redfin.com/NV/Las-Vegas/321-Corsicana-St-89138/home/28976075',
'https://www.redfin.com/NV/Las-Vegas/12433-Alpenglow-Ct-89138/home/170970443',
'https://www.redfin.com/NV/Las-Vegas/10437-Broom-Hill-Dr-89134/home/29106498',
'https://www.redfin.com/NV/Las-Vegas/8513-Glenmount-Dr-89134/home/29149554',
'https://www.redfin.com/NV/Las-Vegas/2412-Twin-Flower-Cir-89134/home/29148996',
'https://www.redfin.com/NV/Las-Vegas/5035-Tranquil-View-St-89130/home/179964072',
'https://www.redfin.com/NV/Las-Vegas/5040-Serene-Skies-St-89130/home/179964052',
'https://www.redfin.com/NV/Las-Vegas/5112-Peaceful-Paradise-St-89130/home/179963896',
'https://www.redfin.com/NV/Las-Vegas/5128-Peaceful-Paradise-St-89130/home/179963894',
'https://www.redfin.com/NV/Las-Vegas/5120-Peaceful-Paradise-St-89130/home/179963895',
'https://www.redfin.com/NV/Las-Vegas/10702-Sky-Meadows-Ave-89134/home/29108137',
'https://www.redfin.com/NV/Las-Vegas/8073-Mackenzie-Ct-89129/home/29120002',
'https://www.redfin.com/NV/Las-Vegas/11817-Stone-Run-Ave-89138/home/176105635',
'https://www.redfin.com/NV/Las-Vegas/2593-Evolutionary-Ln-89138/home/167633614',
'https://www.redfin.com/NV/Las-Vegas/10041-Netherton-Dr-89134/home/29106694',
'https://www.redfin.com/NV/Las-Vegas/8301-Spring-Arts-Ave-89129/home/29126422',
'https://www.redfin.com/NV/Las-Vegas/1309-Bittersweet-Cir-89128/home/29200097',
'https://www.redfin.com/NV/Las-Vegas/12483-Point-Sierra-St-89138/home/167636040',
'https://www.redfin.com/NV/Las-Vegas/12446-Foxtail-Run-Ave-89138/home/170970416',
'https://www.redfin.com/NV/Las-Vegas/11817-Pied-Harrier-Ave-89138/home/185463254',
'https://www.redfin.com/NV/Las-Vegas/6804-Atwood-Ave-89108/home/29131022',
'https://www.redfin.com/NV/Las-Vegas/6564-Bradford-Ln-89108/home/29138813',
'https://www.redfin.com/NV/Las-Vegas/3609-Julia-Waldene-Ct-89129/home/29127007',
'https://www.redfin.com/NV/Las-Vegas/4425-N-Chieftain-St-89129/home/29120634',
'https://www.redfin.com/NV/Las-Vegas/7802-Howard-Dade-Ave-89129/home/29128083',
'https://www.redfin.com/NV/Las-Vegas/12141-Las-Hurdes-Ave-89138/home/29033418',
'https://www.redfin.com/NV/Las-Vegas/1905-Trail-Peak-Ln-89134/home/29110056',
'https://www.redfin.com/NV/Las-Vegas/10831-Far-Country-Ave-89129/home/167631313',
'https://www.redfin.com/NV/Las-Vegas/5045-Saint-Annes-Dr-89149/home/29100252',
'https://www.redfin.com/NV/Las-Vegas/6849-Coral-Rock-Dr-89108/home/29119875',
'https://www.redfin.com/NV/Las-Vegas/1468-Reef-Ridge-Ct-89128/home/29197540',
'https://www.redfin.com/NV/Las-Vegas/9637-Buckhorn-Dr-89134/home/29153332',
'https://www.redfin.com/NV/Las-Vegas/2024-Summer-Cove-Ct-89134/unit-102/home/29109849',
'https://www.redfin.com/NV/Las-Vegas/5028-Serene-Skies-St-89130/home/179964049',
'https://www.redfin.com/NV/Las-Vegas/6516-Peace-Dove-Ave-89130/home/179963870',
'https://www.redfin.com/NV/Las-Vegas/5044-Serene-Skies-St-89130/home/179964053',
'https://www.redfin.com/NV/Las-Vegas/6471-Perch-Coo-Ave-89130/home/179964027',
'https://www.redfin.com/NV/Las-Vegas/5016-Tranquil-View-St-89130/home/179963972',
'https://www.redfin.com/NV/Las-Vegas/1833-Spanish-Sun-Ln-89128/home/29162373',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Stirling/Sidney/home/183344065',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Stirling/Sawyer/home/183344066',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Evander/Everly/home/183344069',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Evander/Elizabeth/home/183344071',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Evander/Ethan/home/183344073',
'https://www.redfin.com/NV/Las-Vegas/Blacktail/The-Kendall/home/183357744',
'https://www.redfin.com/NV/Las-Vegas/Vertex/Plan-3/home/185428774',
'https://www.redfin.com/NV/Las-Vegas/Vertex/Plan-2/home/185428793',
'https://www.redfin.com/NV/Las-Vegas/Vertex/Plan-2X/home/185428882',
'https://www.redfin.com/NV/Las-Vegas/Vertex/Plan-1/home/185428994',
'https://www.redfin.com/NV/Las-Vegas/Arroyos-Edge/Plan-3/home/185551507',
'https://www.redfin.com/NV/Las-Vegas/Arroyos-Edge/Plan-2/home/185551577',
'https://www.redfin.com/NV/Las-Vegas/Arroyos-Edge/Plan-1/home/185551579',
'https://www.redfin.com/NV/Las-Vegas/Obsidian-in-Summerlin/Onyx-2A/home/185690631',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Cromwell/Claremont/home/186926558',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Cromwell/Connery/home/186926559',
'https://www.redfin.com/NV/Las-Vegas/Heritage-at-Stonebridge-Cromwell/Carson/home/186926577',
'https://www.redfin.com/NV/Las-Vegas/Durango-Reserve/Violet/home/186926639',
'https://www.redfin.com/NV/Las-Vegas/Durango-Reserve/Vance/home/186926661',
'https://www.redfin.com/NV/Las-Vegas/10321-Anniston-Ln-89144/home/29110177',
'https://www.redfin.com/NV/Las-Vegas/3120-Sonata-Dr-89121/home/29475558',
'https://www.redfin.com/NV/Las-Vegas/4501-Buena-Vista-Dr-89102/home/29463013',
'https://www.redfin.com/NV/Las-Vegas/3736-Paseo-de-Paz-Ave-89115/home/172505592',
'https://www.redfin.com/NV/Las-Vegas/3035-El-Camino-Rd-89146/home/29538666',
'https://www.redfin.com/NV/Las-Vegas/3285-Lindell-Rd-89146/home/29539235',
'https://www.redfin.com/NV/Las-Vegas/2870-Belcastro-St-89117/home/29537181',
'https://www.redfin.com/NV/Las-Vegas/1414-S-13th-St-89104/home/29451509',
'https://www.redfin.com/NV/Las-Vegas/1988-Nitida-ST-89108/unit-5/home/185923899',
'https://www.redfin.com/NV/Las-Vegas/340-S-Jones-Blvd-89107/home/29231951',
'https://www.redfin.com/NV/Las-Vegas/2214-Spring-Water-Dr-89134/home/29109257',
'https://www.redfin.com/NV/Las-Vegas/624-N-Bruce-St-89101/home/29336164',
'https://www.redfin.com/NV/North-Las-Vegas/3857-Chopin-Ct-89032/home/29250691',
'https://www.redfin.com/NV/Las-Vegas/3852-Remount-Dr-89121/home/29423322',
'https://www.redfin.com/NV/Las-Vegas/1518-Silver-Mesa-Way-89169/home/29468421',
'https://www.redfin.com/NV/Las-Vegas/9064-Sunpine-Ct-89129/home/29124263',
'https://www.redfin.com/NV/Las-Vegas/1389-Commanche-Dr-89169/home/29475771',
'https://www.redfin.com/NV/North-Las-Vegas/3736-Cima-Dr-89032/home/29248358',
'https://www.redfin.com/NV/Las-Vegas/3709-Metter-St-89129/home/29024420',
'https://www.redfin.com/NV/Las-Vegas/1228-Fascination-St-89128/home/29204445',
'https://www.redfin.com/NV/North-Las-Vegas/3556-Valley-Lily-St-89032/home/179100007',
'https://www.redfin.com/NV/Las-Vegas/1912-S-Valley-View-Blvd-89102/home/29461356',
'https://www.redfin.com/NV/Las-Vegas/2851-S-Valley-View-Blvd-89102/unit-1191/home/29464548',
'https://www.redfin.com/NV/Las-Vegas/1417-Winwood-St-89108/home/166580936',
'https://www.redfin.com/NV/Las-Vegas/2136-Bavington-Dr-89108/unit-A/home/29170763',
'https://www.redfin.com/NV/Las-Vegas/425-Blush-Creek-Pl-89144/home/29112814',
'https://www.redfin.com/NV/Las-Vegas/4000-Fulton-Pl-89107/home/29345376',
'https://www.redfin.com/NV/Las-Vegas/424-Stanford-St-89107/home/29190193',
'https://www.redfin.com/NV/Las-Vegas/3453-Del-Marino-Dr-89121/home/29411851',
'https://www.redfin.com/NV/Las-Vegas/2410-Poplar-Ave-89101/home/29349865',
'https://www.redfin.com/NV/Las-Vegas/6770-Irish-Sea-Ave-89146/home/29538451',
'https://www.redfin.com/NV/Las-Vegas/8200-Bright-Dr-89117/home/29521909',
'https://www.redfin.com/NV/Las-Vegas/85-Southern-Peach-St-89138/home/173620943',
'https://www.redfin.com/NV/Las-Vegas/2859-Glass-Vine-Ct-89117/home/65558629',
'https://www.redfin.com/NV/Las-Vegas/11807-Pippa-Ave-89138/home/173620934',
'https://www.redfin.com/NV/Las-Vegas/3059-Lenoir-St-89135/home/29870623',
'https://www.redfin.com/NV/Las-Vegas/9569-Teton-Diablo-Ave-89117/home/29556135',
'https://www.redfin.com/NV/Las-Vegas/285-Molinetto-St-89138/home/165790222',
'https://www.redfin.com/NV/Las-Vegas/700-Lacey-Tree-St-89145/home/29213644',
'https://www.redfin.com/NV/Las-Vegas/8108-Tower-Bridge-Ave-89117/home/29522023',
'https://www.redfin.com/NV/Las-Vegas/9713-Horizon-Hills-Dr-89117/home/29529377',
'https://www.redfin.com/NV/Las-Vegas/412-Dazzling-Gold-St-89138/home/28986037',
'https://www.redfin.com/NV/Las-Vegas/9304-Angelfish-Dr-89117/home/29530346',
'https://www.redfin.com/NV/Las-Vegas/1005-Olive-Mill-Ln-89134/home/29211871',
'https://www.redfin.com/NV/Las-Vegas/919-Baronet-Dr-89138/home/39657276',
'https://www.redfin.com/NV/Las-Vegas/11485-Vibrant-Heights-DR-89135/unit-32/home/179250313',
'https://www.redfin.com/NV/Las-Vegas/11331-Rising-Ridge-Ave-89135/home/29937158',
'https://www.redfin.com/NV/Las-Vegas/121-Sapodilla-Ln-89144/home/29847590',
'https://www.redfin.com/NV/Las-Vegas/1056-Neil-Armstrong-Cir-89145/home/29224722',
'https://www.redfin.com/NV/Las-Vegas/8949-Diamond-Falls-Dr-89117/home/29523213',
'https://www.redfin.com/NV/Las-Vegas/10928-Sutter-Hills-Ave-89144/home/29113719',
'https://www.redfin.com/NV/Las-Vegas/2450-Tribeca-St-89135/home/29936390',
'https://www.redfin.com/NV/Las-Vegas/509-N-Torrey-Pines-Dr-89107/home/29193054',
'https://www.redfin.com/NV/Las-Vegas/10138-Prairie-Dove-Ave-89117/home/29554806',
'https://www.redfin.com/NV/Las-Vegas/11426-Ogden-Mills-Dr-89135/unit-101/home/29286165',
'https://www.redfin.com/NV/Las-Vegas/409-Holcomb-Cir-89107/home/29227609',
'https://www.redfin.com/NV/Las-Vegas/9856-Gray-Sea-Eagle-Ave-89117/home/29555038',
'https://www.redfin.com/NV/Las-Vegas/432-Anatolia-Ln-89145/home/29222568',
'https://www.redfin.com/NV/Las-Vegas/709-Purcell-Dr-89107/home/29193367',
'https://www.redfin.com/NV/Las-Vegas/1328-Cranston-Ct-89135/home/29032633',
'https://www.redfin.com/NV/Las-Vegas/6352-Peppermill-Dr-89146/home/29520131',
'https://www.redfin.com/NV/Las-Vegas/205-Derwent-Pl-89144/home/29113414',
'https://www.redfin.com/NV/Las-Vegas/8701-Cremona-Dr-89117/home/29523796',
'https://www.redfin.com/NV/Las-Vegas/2804-Shadow-Creek-Cir-89117/home/29531666',
'https://www.redfin.com/NV/Las-Vegas/7292-White-Bloom-Ave-89117/home/65558571',
'https://www.redfin.com/NV/Las-Vegas/2725-Beachside-Ct-89117/home/29528353',
'https://www.redfin.com/NV/Las-Vegas/11576-Aruba-Beach-Ave-89138/home/28986371',
'https://www.redfin.com/NV/Las-Vegas/11469-Vibrant-Heights-DR-89135/unit-28/home/178973210',
'https://www.redfin.com/NV/Las-Vegas/9717-Camden-Hills-Ave-89145/home/29212549',
'https://www.redfin.com/NV/Las-Vegas/701-Vincent-Way-89145/home/29225856',
'https://www.redfin.com/NV/Las-Vegas/8429-Stapleton-Ave-89145/home/29214568',
'https://www.redfin.com/NV/Las-Vegas/2310-Malaga-Peak-St-89135/home/29054302',
'https://www.redfin.com/NV/Las-Vegas/7975-Lapis-Harbor-Ave-89117/home/29536373',
'https://www.redfin.com/NV/Las-Vegas/11060-Village-Ridge-Ln-89135/home/29916742',
'https://www.redfin.com/NV/Las-Vegas/1404-Ibizo-St-89117/home/29525277',
'https://www.redfin.com/NV/Las-Vegas/10017-Camino-Loma-Verde-Ave-89117/home/29525338',
'https://www.redfin.com/NV/Las-Vegas/2761-Lodestone-Dr-89117/unit-63/home/29531128',
'https://www.redfin.com/NV/Las-Vegas/10561-Haywood-Dr-89135/home/29872558',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89117/home/29533566',
'https://www.redfin.com/NV/Las-Vegas/3075-Santa-Margarita-St-89146/home/29538173',
'https://www.redfin.com/NV/Las-Vegas/11518-Belmont-Lake-Dr-89135/unit-105/home/29286131',
'https://www.redfin.com/NV/Las-Vegas/1164-Prospering-Creek-ST-89135/unit-6/home/186002342',
'https://www.redfin.com/NV/Las-Vegas/11287-Glowing-Ave-89135/unit-103/home/113672890',
'https://www.redfin.com/NV/Las-Vegas/11531-Sailing-Stones-St-89138/home/173620863',
'https://www.redfin.com/NV/Las-Vegas/3540-Moraga-Dr-89103/home/29543408',
'https://www.redfin.com/NV/Las-Vegas/8924-Rainbow-Ridge-Dr-89117/home/29523490',
'https://www.redfin.com/NV/Las-Vegas/10021-Via-Toro-Ave-89117/home/29525262',
'https://www.redfin.com/NV/Las-Vegas/1871-Granemore-St-89135/home/29320603',
'https://www.redfin.com/NV/Las-Vegas/10753-Turquoise-Valley-Dr-89144/home/29111699',
'https://www.redfin.com/NV/Las-Vegas/1208-Lucia-Dr-89128/home/29198171',
'https://www.redfin.com/NV/Las-Vegas/6607-Costa-Brava-Rd-89146/home/29543574',
'https://www.redfin.com/NV/Las-Vegas/2303-Malaga-Peak-St-89135/home/29054325',
'https://www.redfin.com/NV/Las-Vegas/549-Ivy-Spring-St-89138/home/29008314',
'https://www.redfin.com/NV/Las-Vegas/8665-Catalonia-Dr-89117/home/29534537',
'https://www.redfin.com/NV/Las-Vegas/7664-Angel-Crest-Cir-89117/home/29537044',
'https://www.redfin.com/NV/Las-Vegas/1509-Canyon-Ledge-Ct-89117/home/29285505',
'https://www.redfin.com/NV/Las-Vegas/9820-Cross-Creek-Way-89117/home/29529371',
'https://www.redfin.com/NV/Las-Vegas/2399-Malaga-Peak-St-89135/home/29054231',
'https://www.redfin.com/NV/Las-Vegas/321-Bent-Creek-Dr-89107/home/29225745',
'https://www.redfin.com/NV/Las-Vegas/11216-Playa-Bonita-Ave-89138/home/28978515',
'https://www.redfin.com/NV/Las-Vegas/600-Baysville-Ct-89144/home/29114379',
'https://www.redfin.com/NV/Las-Vegas/317-Lake-Windemere-St-89138/home/28985934',
'https://www.redfin.com/NV/Las-Vegas/1837-Ardilea-St-89135/home/29320661',
'https://www.redfin.com/NV/Las-Vegas/10417-Niagara-Falls-Ln-89144/home/29113171',
'https://www.redfin.com/NV/Las-Vegas/1213-Daytona-Ln-89117/home/29526373',
'https://www.redfin.com/NV/Las-Vegas/1049-Nevada-Sky-St-89128/home/29204548',
'https://www.redfin.com/NV/Las-Vegas/10236-Amber-Hue-Ln-89144/home/29112789',
'https://www.redfin.com/NV/Las-Vegas/8139-Retriever-Ave-89147/home/29001741',
'https://www.redfin.com/NV/Las-Vegas/121-Sarabeth-St-89138/home/173620960',
'https://www.redfin.com/NV/Las-Vegas/128-Farkas-Ln-89145/home/29209367',
'https://www.redfin.com/NV/Las-Vegas/225-Robledo-St-89138/home/177121596',
'https://www.redfin.com/NV/Las-Vegas/7221-Tempest-Pl-89145/home/29222474',
'https://www.redfin.com/NV/Las-Vegas/1152-Prospering-Creek-ST-89135/unit-8/home/179516165',
'https://www.redfin.com/NV/Las-Vegas/587-Del-Giorno-St-89138/home/29891901',
'https://www.redfin.com/NV/Las-Vegas/3482-Straight-Arrow-Dr-89117/home/29552103',
'https://www.redfin.com/NV/Las-Vegas/Acadia-Ridge/Alba/home/177029618',
'https://www.redfin.com/NV/Las-Vegas/Acadia-Ridge/Torre-Elite/home/177057244',
'https://www.redfin.com/NV/Las-Vegas/Acadia-Ridge/Torre/home/177057245',
'https://www.redfin.com/NV/Las-Vegas/Acadia-Ridge/Estella-Elite/home/177057246',
'https://www.redfin.com/NV/Las-Vegas/Acadia-Ridge/Estella/home/177057247',
'https://www.redfin.com/NV/Las-Vegas/Acadia-Ridge/Alba-Elite/home/177057248',
'https://www.redfin.com/NV/Las-Vegas/2078-Scanlon-Ferry-Ct-89156/unit-101/home/29480532',
'https://www.redfin.com/NV/Las-Vegas/2050-Los-Feliz-St-89156/unit-173/home/28987789',
'https://www.redfin.com/NV/Las-Vegas/6918-Canary-Ivy-Way-89156/home/29281662',
'https://www.redfin.com/NV/Las-Vegas/2230-Shatz-St-89156/home/29371292',
'https://www.redfin.com/NV/Las-Vegas/6953-Hathaway-Dr-89156/home/29374288',
'https://www.redfin.com/NV/Las-Vegas/6772-Warthog-Ave-89156/home/183043871',
'https://www.redfin.com/NV/Las-Vegas/2050-Los-Feliz-St-89156/unit-121/home/28978310',
'https://www.redfin.com/NV/Las-Vegas/6423-Moonglow-Dr-89156/home/29372479',
'https://www.redfin.com/NV/Las-Vegas/3053-Forest-Falls-Ct-89156/home/28970548',
'https://www.redfin.com/NV/Las-Vegas/6706-Warthog-Ave-89156/home/183043840',
'https://www.redfin.com/NV/Las-Vegas/6737-Warthog-Ave-89156/home/183043847',
'https://www.redfin.com/NV/North-Las-Vegas/2204-Bennett-St-89030/home/29324768',
'https://www.redfin.com/NV/North-Las-Vegas/3252-Cheyenne-Gardens-Way-89032/home/28973747',
'https://www.redfin.com/NV/North-Las-Vegas/4125-Thomas-Patrick-Ave-89032/home/29027510',
'https://www.redfin.com/NV/Las-Vegas/1936-Neoga-St-89115/home/148024860',
'https://www.redfin.com/NV/Las-Vegas/4429-Marlena-Cir-89108/home/29307637',
'https://www.redfin.com/NV/North-Las-Vegas/3318-Trotting-Horse-Rd-89032/home/29245856',
'https://www.redfin.com/NV/Las-Vegas/1821-Verde-Mirada-Dr-89115/home/29062266',
'https://www.redfin.com/NV/North-Las-Vegas/708-Lillis-Ave-89030/home/29326297',
'https://www.redfin.com/NV/Las-Vegas/6510-Viewpoint-Dr-89156/home/29372601',
'https://www.redfin.com/NV/Las-Vegas/4343-Harristown-Dr-89115/home/163845999',
'https://www.redfin.com/NV/Las-Vegas/1316-Lawry-Ave-89106/home/29322854',
'https://www.redfin.com/NV/North-Las-Vegas/3640-Lake-Victoria-Dr-89032/home/29254356',
'https://www.redfin.com/NV/Las-Vegas/4170-Ripple-River-Ave-89115/home/29364919',
'https://www.redfin.com/NV/North-Las-Vegas/4409-Erica-Dr-89032/home/29245230',
'https://www.redfin.com/NV/Las-Vegas/1828-Walnut-Hollow-St-89115/home/178669320',
'https://www.redfin.com/NV/North-Las-Vegas/3453-Lonesome-Drum-St-89032/home/28987075',
'https://www.redfin.com/NV/Las-Vegas/6464-Bubbling-Springs-Ave-89156/home/29360804',
'https://www.redfin.com/NV/Las-Vegas/5465-Madre-Mesa-Dr-89108/home/29136133',
'https://www.redfin.com/NV/Las-Vegas/6123-Canter-Glen-Ave-89122/home/29482185',
'https://www.redfin.com/NV/Las-Vegas/6895-E-Washington-Ave-89110/home/29375283',
'https://www.redfin.com/NV/Las-Vegas/5518-Avenida-Vaquero-89108/home/29134816',
'https://www.redfin.com/NV/Las-Vegas/5308-Gipsy-Ave-89107/home/29235575',
'https://www.redfin.com/NV/Las-Vegas/6259-Santander-Ave-89103/home/29546232',
'https://www.redfin.com/NV/Las-Vegas/4010-Evesham-Ct-89121/home/29424773',
'https://www.redfin.com/NV/Las-Vegas/628-N-21st-St-89101/home/29335997',
'https://www.redfin.com/NV/Las-Vegas/4641-Plata-del-Sol-Dr-89121/home/29428825',
'https://www.redfin.com/NV/Las-Vegas/928-Pinehurst-Dr-89109/home/29467006',
'https://www.redfin.com/NV/Las-Vegas/4640-Sheppard-Dr-89122/home/29430208',
'https://www.redfin.com/NV/Las-Vegas/2499-Swan-Ln-89121/home/29507107',
'https://www.redfin.com/NV/Las-Vegas/6531-Jordan-River-Dr-89156/home/29358644',
'https://www.redfin.com/NV/Las-Vegas/1779-Sycamore-Trl-89108/home/29307496',
'https://www.redfin.com/NV/Las-Vegas/1691-Morning-Quiet-St-89156/home/29189436',
'https://www.redfin.com/NV/Las-Vegas/4907-Chidlaw-St-89115/unit-3/home/183991530',
'https://www.redfin.com/NV/Las-Vegas/4907-Chidlaw-St-89115/unit-2/home/183991527',
'https://www.redfin.com/NV/North-Las-Vegas/3841-Canary-Cedar-St-89032/home/29859921',
'https://www.redfin.com/NV/North-Las-Vegas/220-Buck-Ranch-Ave-89032/home/30175563',
'https://www.redfin.com/NV/North-Las-Vegas/4553-Queens-Hill-St-89031/home/179978367',
'https://www.redfin.com/NV/North-Las-Vegas/4545-Queens-Hill-St-89031/home/179978365',
'https://www.redfin.com/NV/North-Las-Vegas/4554-Queens-Hill-St-89031/home/179978393',
'https://www.redfin.com/NV/North-Las-Vegas/4549-Queens-Hill-St-89031/home/179978366',
'https://www.redfin.com/NV/North-Las-Vegas/4541-Queens-Hill-St-89031/home/179978364',
'https://www.redfin.com/NV/North-Las-Vegas/3911-McGregor-Way-89032/home/29253677',
'https://www.redfin.com/NV/Las-Vegas/2116-Lost-Maple-St-89115/home/28987514',
'https://www.redfin.com/NV/Las-Vegas/6479-Elwood-Mead-Ave-89156/unit-101/home/29480521',
'https://www.redfin.com/NV/Las-Vegas/4289-Studio-St-89115/home/29351334',
'https://www.redfin.com/NV/Las-Vegas/6328-Kell-Ln-89156/home/29372566',
'https://www.redfin.com/NV/North-Las-Vegas/2731-Perliter-Ave-89030/home/28968274',
'https://www.redfin.com/NV/North-Las-Vegas/4542-Queens-Hill-St-89031/home/179978391',
'https://www.redfin.com/NV/North-Las-Vegas/4540-Anspach-St-89031/home/177318090',
'https://www.redfin.com/NV/North-Las-Vegas/4536-Anspach-St-89031/home/177318089',
'https://www.redfin.com/NV/North-Las-Vegas/2724-Graceful-Grove-Ave-89032/home/54559855',
'https://www.redfin.com/NV/North-Las-Vegas/2600-Daley-St-89030/home/29259001',
'https://www.redfin.com/NV/North-Las-Vegas/2009-Perliter-Ave-89030/home/29326969',
'https://www.redfin.com/NV/Las-Vegas/1566-Del-Mar-Ave-89169/home/29467562',
'https://www.redfin.com/NV/Las-Vegas/3036-Sierra-Ridge-Dr-89156/home/29359014',
'https://www.redfin.com/NV/Las-Vegas/6570-Viewpoint-Dr-89156/home/29372600',
'https://www.redfin.com/NV/Las-Vegas/3674-Via-Sonja-Ave-89115/home/145869772',
'https://www.redfin.com/NV/Las-Vegas/1517-McGrail-St-89110/home/29383097',
'https://www.redfin.com/NV/North-Las-Vegas/4046-Aaron-Scott-St-89032/home/29894090',
'https://www.redfin.com/NV/North-Las-Vegas/4123-Jessica-Marie-St-89032/home/29894147',
'https://www.redfin.com/NV/Las-Vegas/2805-Alcoa-Ave-89102/home/29464958',
'https://www.redfin.com/NV/Las-Vegas/2105-Jansen-Ave-89101/home/29335474',
'https://www.redfin.com/NV/Las-Vegas/5208-Starter-Ave-89156/home/29045531',
'https://www.redfin.com/NV/Las-Vegas/6796-Warthog-Ave-89156/home/183043867',
'https://www.redfin.com/NV/North-Las-Vegas/1615-Harewood-Ave-89030/home/29261798',
'https://www.redfin.com/NV/North-Las-Vegas/3708-Taylor-Ave-89030/home/29331237',
'https://www.redfin.com/NV/Las-Vegas/4000-Emerson-Ave-89121/home/29423861',
'https://www.redfin.com/NV/Las-Vegas/6350-Waterdragon-Ave-89110/home/29893962',
'https://www.redfin.com/NV/Las-Vegas/4508-Pencester-St-89115/home/166471178',
'https://www.redfin.com/NV/Las-Vegas/1408-E-Carson-Ave-89101/home/29348303',
'https://www.redfin.com/NV/North-Las-Vegas/616-Marcella-Ave-89030/home/29324057',
'https://www.redfin.com/NV/North-Las-Vegas/4533-Queens-Hill-St-89031/home/179978362',
'https://www.redfin.com/NV/North-Las-Vegas/3404-Siler-Ave-89030/home/29256759',
'https://www.redfin.com/NV/North-Las-Vegas/4112-Solar-System-St-89032/home/29061950',
'https://www.redfin.com/NV/Las-Vegas/277-Pecos-Way-89121/home/29412740',
'https://www.redfin.com/NV/Las-Vegas/2266-Hidden-View-St-89156/home/29908947',
'https://www.redfin.com/NV/Las-Vegas/2357-Bridle-Wreath-Ln-89156/home/29371243',
'https://www.redfin.com/NV/Las-Vegas/4319-Saddle-Brook-Park-Dr-89115/home/166541268',
'https://www.redfin.com/NV/Las-Vegas/4312-Heavenly-Bamboo-Way-89115/home/165846825',
'https://www.redfin.com/NV/Las-Vegas/3608-Hudson-Bay-Ave-89110/home/29386134',
'https://www.redfin.com/NV/Las-Vegas/1282-Laguna-Ave-89169/home/29467787',
'https://www.redfin.com/NV/Las-Vegas/5207-Paradise-Valley-Ave-89156/home/29911656',
'https://www.redfin.com/NV/Las-Vegas/1514-S-13th-St-89104/home/29451505',
'https://www.redfin.com/NV/North-Las-Vegas/4128-Golden-Sol-St-89032/home/179978572',
'https://www.redfin.com/NV/North-Las-Vegas/4136-Golden-Sol-St-89032/home/179978574',
'https://www.redfin.com/NV/North-Las-Vegas/1569-Cowboy-Path-Ave-89032/home/179978566',
'https://www.redfin.com/NV/Las-Vegas/6790-Warthog-Ave-89156/home/183043868',
'https://www.redfin.com/NV/Las-Vegas/3591-Driving-Range-St-89122/home/29865354',
'https://www.redfin.com/NV/Las-Vegas/500-Greenbriar-Townhouse-Way-89121/home/29411729',
'https://www.redfin.com/NV/North-Las-Vegas/2525-Vana-Ave-89030/home/29258044',
'https://www.redfin.com/NV/Las-Vegas/6027-Peach-Orchard-Rd-89142/home/29401825',
'https://www.redfin.com/NV/Las-Vegas/4259-Puebla-St-89115/home/29352194',
'https://www.redfin.com/NV/Las-Vegas/4625-E-Monroe-Ave-89110/home/29381375',
'https://www.redfin.com/NV/Las-Vegas/1700-Atlantic-St-89104/home/29451026',
'https://www.redfin.com/NV/Las-Vegas/236-Vista-Valley-St-89110/home/29397057',
'https://www.redfin.com/NV/North-Las-Vegas/4529-Anspach-St-89031/home/177318039',
'https://www.redfin.com/NV/Las-Vegas/1728-Wendell-Williams-Ave-89106/home/29860941',
'https://www.redfin.com/NV/Las-Vegas/6791-Globemaster-Ct-89156/home/183043878',
'https://www.redfin.com/NV/Las-Vegas/1518-Golden-Arrow-Dr-89169/home/29468459',
'https://www.redfin.com/NV/Las-Vegas/6092-Fox-Creek-Ave-89122/home/29482127',
'https://www.redfin.com/NV/Las-Vegas/2999-Bel-Air-Dr-89109/home/29466831',
'https://www.redfin.com/NV/Las-Vegas/5956-Sandra-Field-Ct-89110/home/29933198',
'https://www.redfin.com/NV/North-Las-Vegas/4521-Anspach-St-89031/home/177318037',
'https://www.redfin.com/NV/Las-Vegas/2943-Gunnery-Forge-St-89156/home/175524373',
'https://www.redfin.com/NV/North-Las-Vegas/4157-Sandia-Star-Dr-89032/home/179978582',
'https://www.redfin.com/NV/North-Las-Vegas/1569-Gold-Sunset-Ave-89032/home/179978608',
'https://www.redfin.com/NV/North-Las-Vegas/2540-Spear-St-89030/home/29280904',
'https://www.redfin.com/NV/North-Las-Vegas/2745-Salt-Lake-St-89030/home/29271740',
'https://www.redfin.com/NV/North-Las-Vegas/2729-Saint-George-St-89030/home/29270704',
'https://www.redfin.com/NV/Las-Vegas/601-Frederick-Ave-89106/home/29325399',
'https://www.redfin.com/NV/Las-Vegas/431-N-Fogg-St-89110/home/29395542',
'https://www.redfin.com/NV/Las-Vegas/1944-Randa-Ln-89104/home/29410526',
'https://www.redfin.com/NV/Las-Vegas/4544-Hearts-Desire-Ave-89115/home/29028063',
'https://www.redfin.com/NV/Las-Vegas/213-N-18th-St-89101/unit-C/home/29349810',
'https://www.redfin.com/NV/Las-Vegas/945-Vegas-Valley-Dr-89109/home/29465945',
'https://www.redfin.com/NV/Las-Vegas/3332-Lovell-Ct-89121/home/29475518',
'https://www.redfin.com/NV/Las-Vegas/3372-Sheep-Canyon-St-89122/home/29029045',
'https://www.redfin.com/NV/Las-Vegas/388-Dolorosa-St-89110/home/29393950',
'https://www.redfin.com/NV/North-Las-Vegas/2110-Keller-Ct-89032/home/29250606',
'https://www.redfin.com/NV/Las-Vegas/2990-Spokane-Dr-89121/home/29472651',
'https://www.redfin.com/NV/Las-Vegas/1530-Bonita-Ave-89104/home/29452558',
'https://www.redfin.com/NV/Las-Vegas/5730-Coleridge-Way-89142/home/29416625',
'https://www.redfin.com/NV/Las-Vegas/4657-Warner-Pl-89115/home/29352437',
'https://www.redfin.com/NV/Las-Vegas/2561-Early-Sky-Ct-89142/home/45356807',
'https://www.redfin.com/NV/Las-Vegas/1165-Panorama-Heights-St-89110/home/29916687',
'https://www.redfin.com/NV/Las-Vegas/2799-Heritage-Dr-89121/home/29472438',
'https://www.redfin.com/NV/Las-Vegas/2106-Spencer-St-89104/home/29454794',
'https://www.redfin.com/NV/Las-Vegas/2500-Sierra-Bello-Ave-89106/unit-102/home/29310855',
'https://www.redfin.com/NV/North-Las-Vegas/1590-Cowboy-Path-Ave-89032/home/179978596',
'https://www.redfin.com/NV/North-Las-Vegas/4558-Queens-Hill-St-89031/home/179978394',
'https://www.redfin.com/NV/North-Las-Vegas/4557-Queens-Hill-St-89031/home/179978368',
'https://www.redfin.com/NV/North-Las-Vegas/4561-Queens-Hill-St-89031/home/179978369',
'https://www.redfin.com/NV/North-Las-Vegas/4562-Queens-Hill-St-89031/home/179978395',
'https://www.redfin.com/NV/Las-Vegas/2859-Gunnery-Forge-St-89156/home/179957937',
'https://www.redfin.com/NV/Las-Vegas/10386-Vanhoy-Crest-AVE-Unknown/home/187592129',
'https://www.redfin.com/NV/Las-Vegas/231-Dougram-Ave-89101/home/50268281',
'https://www.redfin.com/NV/Las-Vegas/10378-Vanhoy-Crest-AVE-Unknown/home/187593507',
'https://www.redfin.com/NV/Las-Vegas/1808-Windchime-Dr-89106/home/29308824',
'https://www.redfin.com/NV/Las-Vegas/2897-Overlook-Brook-St-89156/home/179957896',
'https://www.redfin.com/NV/Las-Vegas/1813-Lamplighter-Ln-89104/home/29454772',
'https://www.redfin.com/NV/Las-Vegas/1819-Beverly-Way-89104/home/29455610',
'https://www.redfin.com/NV/Las-Vegas/6730-Warthog-Ave-89156/home/183043835',
'https://www.redfin.com/NV/Las-Vegas/6778-Warthog-Ave-89156/home/183043870',
'https://www.redfin.com/NV/Las-Vegas/6718-Warthog-Ave-89156/home/183043837',
'https://www.redfin.com/NV/Las-Vegas/976-Balzar-Ave-89106/home/29321097',
'https://www.redfin.com/NV/Las-Vegas/974-Balzar-Ave-89106/home/29321095',
'https://www.redfin.com/NV/Las-Vegas/3128-Theresa-Ave-89101/home/29334456',
'https://www.redfin.com/NV/North-Las-Vegas/3482-Bassler-Ct-89030/home/29257119',
'https://www.redfin.com/NV/North-Las-Vegas/2720-Graceful-Grove-Ave-89032/home/54559841',
'https://www.redfin.com/NV/Las-Vegas/524-Boxerwood-Dr-89110/home/29864417',
'https://www.redfin.com/NV/Las-Vegas/1315-S-17th-St-89104/home/29451463',
'https://www.redfin.com/NV/North-Las-Vegas/4012-Laurel-Hill-Dr-89032/home/29240421',
'https://www.redfin.com/NV/Las-Vegas/5625-E-Monroe-Ave-89110/home/29379999',
'https://www.redfin.com/NV/Las-Vegas/3544-Tack-St-89122/home/29482195',
'https://www.redfin.com/NV/Las-Vegas/749-Rossmore-Dr-89110/home/29397333',
'https://www.redfin.com/NV/Las-Vegas/3732-Cannon-Ave-89121/home/28966399',
'https://www.redfin.com/NV/Las-Vegas/6240-Viewpoint-Dr-89156/home/29370964',
'https://www.redfin.com/NV/Las-Vegas/2683-Regency-Cove-Ct-89121/home/29828661',
'https://www.redfin.com/NV/Las-Vegas/3358-Knollwood-Ct-89121/home/29475445',
'https://www.redfin.com/NV/Las-Vegas/6742-Warthog-Ave-89156/home/183043833',
'https://www.redfin.com/NV/Las-Vegas/6680-Evening-Rain-Ave-89156/home/29360611',
'https://www.redfin.com/NV/Las-Vegas/3422-Kensbrook-St-89121/home/29474620',
'https://www.redfin.com/NV/Las-Vegas/1183-Devon-Lake-St-89110/home/29889829',
'https://www.redfin.com/NV/Las-Vegas/5268-Harmony-Ave-89107/home/29233976',
'https://www.redfin.com/NV/Las-Vegas/10538-Sopra-Ct-89135/home/29873728',
'https://www.redfin.com/NV/Las-Vegas/4737-Woodlake-Ave-89147/home/29569058',
'https://www.redfin.com/NV/Las-Vegas/7600-Beverly-Hills-Dr-89147/home/29565555',
'https://www.redfin.com/NV/Las-Vegas/4204-El-Cederal-Ave-89102/home/29461555',
'https://www.redfin.com/NV/Las-Vegas/9050-W-Tropicana-Ave-89147/unit-1123/home/29487659',
'https://www.redfin.com/NV/Las-Vegas/452-Astillero-St-89138/home/109635134',
'https://www.redfin.com/NV/Las-Vegas/6301-Clarice-Ave-89107/home/29196987',
'https://www.redfin.com/NV/Las-Vegas/4409-Palloni-Ct-89135/home/29033394',
'https://www.redfin.com/NV/Las-Vegas/2640-Crystal-Quartz-St-89146/home/167096543',
'https://www.redfin.com/NV/Las-Vegas/6227-Meadowgrass-Ln-89103/home/29545676',
'https://www.redfin.com/NV/Las-Vegas/8436-Diamond-Dust-Dr-89147/home/29560378',
'https://www.redfin.com/NV/Las-Vegas/6317-Dayton-Ave-89107/home/29228942',
'https://www.redfin.com/NV/Las-Vegas/4998-Schumann-Dr-89146/home/29540264',
'https://www.redfin.com/NV/Las-Vegas/4481-Sirius-Ave-89102/home/29463016',
'https://www.redfin.com/NV/Las-Vegas/4008-Delos-Dr-89103/home/29543673',
'https://www.redfin.com/NV/Las-Vegas/355-Purple-Sandpiper-St-89138/home/169228432',
'https://www.redfin.com/NV/Las-Vegas/3194-Cereus-Ave-89146/home/29540255',
'https://www.redfin.com/NV/Las-Vegas/5700-Harmony-Ave-89107/home/29232303',
'https://www.redfin.com/NV/Las-Vegas/8957-Tempest-Point-Ct-89147/home/29908958',
'https://www.redfin.com/NV/Las-Vegas/7053-Mountain-Meadow-Ln-89147/home/29569065',
'https://www.redfin.com/NV/Las-Vegas/12064-Avery-Meadows-Ave-89138/home/29025204',
'https://www.redfin.com/NV/Las-Vegas/6325-Lawton-Ave-89107/home/29197139',
'https://www.redfin.com/NV/Las-Vegas/4028-Nook-Way-89103/home/29543653',
'https://www.redfin.com/NV/Las-Vegas/5940-Yorba-Ct-89103/home/29540498',
'https://www.redfin.com/NV/Las-Vegas/12263-La-Prada-Pl-89138/home/29025050',
'https://www.redfin.com/NV/Las-Vegas/889-Adair-Grove-Ln-89138/home/181148145',
'https://www.redfin.com/NV/Las-Vegas/961-Norwood-Heights-Ln-89138/home/181148051',
'https://www.redfin.com/NV/Las-Vegas/969-Norwood-Heights-Ln-89138/home/181148049',
'https://www.redfin.com/NV/Las-Vegas/889-Norwood-Heights-Ln-89138/home/181148159',
'https://www.redfin.com/NV/Las-Vegas/4132-Coneflower-St-89135/home/167753632',
'https://www.redfin.com/NV/Las-Vegas/885-Norwood-Heights-Ln-89138/home/181148160',
'https://www.redfin.com/NV/Las-Vegas/8402-Tibana-Way-89147/home/29560932',
'https://www.redfin.com/NV/Las-Vegas/7694-Selby-Ct-89147/home/29566943',
'https://www.redfin.com/NV/Las-Vegas/4705-Dover-Pl-89107/home/29344194',
'https://www.redfin.com/NV/Las-Vegas/7823-Windward-Rd-89147/home/29564295',
'https://www.redfin.com/NV/Las-Vegas/12137-Vista-Linda-Ave-89138/home/29025089',
'https://www.redfin.com/NV/Las-Vegas/4354-Alderbrook-Ct-89103/home/29570205',
'https://www.redfin.com/NV/Las-Vegas/4678-El-Camino-Cabos-Dr-89147/home/29559464',
'https://www.redfin.com/NV/Las-Vegas/3818-Silent-Hill-Dr-89147/home/29547619',
'https://www.redfin.com/NV/Las-Vegas/7931-Slip-Point-Ave-89147/home/45356940',
'https://www.redfin.com/NV/Las-Vegas/4618-El-Camino-Cabos-Dr-89147/home/29559517',
'https://www.redfin.com/NV/Las-Vegas/228-Native-Stone-St-89138/home/170971033',
'https://www.redfin.com/NV/Las-Vegas/2625-Crystal-Quartz-St-89146/home/167727584',
'https://www.redfin.com/NV/North-Las-Vegas/7229-Bedazzle-St-89084/home/167602214',
'https://www.redfin.com/NV/Las-Vegas/7012-Delta-Falls-St-89166/home/40548286',
'https://www.redfin.com/NV/Las-Vegas/7809-Ellendale-Ct-89149/home/33484351',
'https://www.redfin.com/NV/Las-Vegas/7095-Alabaster-Peak-ST-89166/unit-68/home/187432382',
'https://www.redfin.com/NV/Las-Vegas/8833-Big-Bear-Pines-Ave-89143/home/29091061',
'https://www.redfin.com/NV/Las-Vegas/5005-Whistling-Acres-Ave-89131/home/28975488',
'https://www.redfin.com/NV/Las-Vegas/8940-David-Allen-Ct-89149/home/29242160',
'https://www.redfin.com/NV/Las-Vegas/10751-Hammett-Park-Ave-89166/home/56542769',
'https://www.redfin.com/NV/Las-Vegas/8959-Sandy-Isle-Ct-89131/home/29477503',
'https://www.redfin.com/NV/Las-Vegas/8929-Windgate-Ranch-St-89166/home/177117949',
'https://www.redfin.com/NV/Las-Vegas/7563-Moose-River-Ct-89166/home/109862488',
'https://www.redfin.com/NV/Las-Vegas/4916-Morning-Splash-Ave-89131/home/29854968',
'https://www.redfin.com/NV/Las-Vegas/7853-Autumn-Gate-Ave-89131/home/29889053',
'https://www.redfin.com/NV/Las-Vegas/9239-Albus-Hills-Ave-89143/home/177326391',
'https://www.redfin.com/NV/Las-Vegas/8572-Calomeria-Ct-89149/home/29045430',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89149/home/28984966',
'https://www.redfin.com/NV/Las-Vegas/5001-Diamond-Ranch-Ave-89131/home/28974541',
'https://www.redfin.com/NV/Las-Vegas/7820-Wavering-Pine-Dr-89143/home/29822748',
'https://www.redfin.com/NV/North-Las-Vegas/7321-Dazzle-Point-St-89084/home/167601699',
'https://www.redfin.com/NV/Las-Vegas/8755-Scalpay-St-89166/home/177117993',
'https://www.redfin.com/NV/Las-Vegas/8612-Hidden-Pines-Ave-89143/home/29844507',
'https://www.redfin.com/NV/Las-Vegas/5760-Bedrock-Springs-Ave-89131/home/29049914',
'https://www.redfin.com/NV/North-Las-Vegas/7834-Lily-Trotter-St-89084/home/28973132',
'https://www.redfin.com/NV/Las-Vegas/9893-Cullin-Range-Ave-89166/home/176105013',
'https://www.redfin.com/NV/Las-Vegas/10024-Rams-Leap-Ave-89166/home/161755122',
'https://www.redfin.com/NV/Las-Vegas/9037-Rendon-St-89143/home/28983892',
'https://www.redfin.com/NV/North-Las-Vegas/7653-Rosebark-Cliffs-St-89084/home/167598952',
'https://www.redfin.com/NV/Las-Vegas/7060-N-Eula-St-89149/home/29598058',
'https://www.redfin.com/NV/Las-Vegas/7432-Wandering-St-89131/home/29089275',
'https://www.redfin.com/NV/Las-Vegas/9004-Picket-Fence-Ave-89143/home/29087155',
'https://www.redfin.com/NV/Las-Vegas/7848-Regency-Park-St-89149/home/29022869',
'https://www.redfin.com/NV/Las-Vegas/10285-Foramore-Ct-89166/home/166089292',
'https://www.redfin.com/NV/Las-Vegas/10277-Cosmic-Latte-Ct-89143/home/184013236',
'https://www.redfin.com/NV/Las-Vegas/8108-Terracotta-Gulf-Ct-89143/home/29088061',
'https://www.redfin.com/NV/Las-Vegas/10253-Cosmic-Latte-Ct-89143/home/184013232',
'https://www.redfin.com/NV/Las-Vegas/8009-Tailwind-Ave-89131/home/29089365',
'https://www.redfin.com/NV/Las-Vegas/10225-Black-Friar-Ct-89166/home/39586850',
'https://www.redfin.com/NV/Las-Vegas/8430-Footpath-St-89166/home/109872090',
'https://www.redfin.com/NV/Las-Vegas/8616-Vivid-Violet-Ave-89143/home/29087729',
'https://www.redfin.com/NV/North-Las-Vegas/8132-Pink-Desert-St-89085/home/40429177',
'https://www.redfin.com/NV/Las-Vegas/9081-Spoonbill-Ridge-Pl-89143/home/28983517',
'https://www.redfin.com/NV/Las-Vegas/8332-Winterchase-Pl-89143/home/29273090',
'https://www.redfin.com/NV/Las-Vegas/7812-Rhodora-Peak-St-89166/home/40348635',
'https://www.redfin.com/NV/Las-Vegas/7130-Dogan-Ridge-St-89131/home/29092425',
'https://www.redfin.com/NV/Las-Vegas/7515-Bedford-Ridge-Ct-89166/home/92907829',
'https://www.redfin.com/NV/Las-Vegas/10241-Elias-Child-Ct-89166/home/45492413',
'https://www.redfin.com/NV/Las-Vegas/8117-Caramel-Gorge-Ct-89143/home/29088460',
'https://www.redfin.com/NV/Las-Vegas/5504-Lawhon-Canyon-Ct-89131/home/28976901',
'https://www.redfin.com/NV/Las-Vegas/10372-Thurso-Ave-89166/home/179980930',
'https://www.redfin.com/NV/Las-Vegas/8128-Walnut-Roast-Way-89131/home/29006187',
'https://www.redfin.com/NV/Las-Vegas/7613-Falconwing-Ave-89131/home/29090471',
'https://www.redfin.com/NV/Las-Vegas/7244-Royal-Melbourne-Dr-89131/home/29020939',
'https://www.redfin.com/NV/Las-Vegas/11450-Rocky-Ave-89166/home/167628847',
'https://www.redfin.com/NV/Las-Vegas/7625-Highland-Pony-St-89149/home/29023458',
'https://www.redfin.com/NV/Las-Vegas/6098-Golden-Arowana-Way-89149/home/92908209',
'https://www.redfin.com/NV/Las-Vegas/6424-Trautman-Ct-89149/home/40067868',
'https://www.redfin.com/NV/Las-Vegas/7909-Thorne-Pine-Ave-89131/home/29090928',
'https://www.redfin.com/NV/Las-Vegas/8162-Desert-Madera-St-89166/home/169227655',
'https://www.redfin.com/NV/North-Las-Vegas/7020-Manzanares-Dr-89084/home/29048027',
'https://www.redfin.com/NV/Las-Vegas/8641-Palomino-Ranch-St-89131/home/29004340',
'https://www.redfin.com/NV/North-Las-Vegas/7533-Brody-Meadows-St-89084/home/167600480',
'https://www.redfin.com/NV/Las-Vegas/7817-Falconwing-Ave-89131/home/29090453',
'https://www.redfin.com/NV/Las-Vegas/10245-Cosmic-Latte-Ct-89143/home/184013230',
'https://www.redfin.com/NV/Las-Vegas/7145-Eyebright-St-89131/home/29091651',
'https://www.redfin.com/NV/Las-Vegas/10634-Mount-Blackburn-Ave-89166/home/65555737',
'https://www.redfin.com/NV/Las-Vegas/7111-Thistle-Ridge-St-89166/home/51678829',
'https://www.redfin.com/NV/North-Las-Vegas/3773-Bowers-Hollow-Ave-89085/home/29266078',
'https://www.redfin.com/NV/Las-Vegas/10026-Auburn-Rose-Ave-89143/unit-278/home/187590987',
'https://www.redfin.com/NV/Las-Vegas/10074-Auburn-Rose-Ave-89143/unit-272/home/187590867',
'https://www.redfin.com/NV/Las-Vegas/10082-Auburn-Rose-Ave-89143/unit-271/home/187590866',
'https://www.redfin.com/NV/Las-Vegas/10057-Auburn-Rose-Ave-89143/unit-269/home/187590566',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89143/home/28983785',
'https://www.redfin.com/NV/Las-Vegas/8745-Deering-Bay-Dr-89131/home/29020945',
'https://www.redfin.com/NV/Las-Vegas/9584-Jadeite-Ln-89143/home/176643636',
'https://www.redfin.com/NV/Las-Vegas/9781-Free-Rappel-Ave-89166/home/183995676',
'https://www.redfin.com/NV/Las-Vegas/8024-Calico-Mesa-St-89166/home/109871631',
'https://www.redfin.com/NV/Las-Vegas/7930-Formitch-Ct-89166/home/144626726',
'https://www.redfin.com/NV/Las-Vegas/8024-Quail-Head-Ave-89131/home/29856216',
'https://www.redfin.com/NV/Las-Vegas/9507-Diamond-Bridge-Ave-89166/home/29020464',
'https://www.redfin.com/NV/Las-Vegas/9049-Pine-Mission-Ave-89143/home/28998485',
'https://www.redfin.com/NV/Las-Vegas/9113-Spoonbill-Ridge-Pl-89143/home/28983509',
'https://www.redfin.com/NV/Las-Vegas/4832-Whispering-Spring-Ave-89131/home/29908981',
'https://www.redfin.com/NV/Las-Vegas/8100-Artistic-Heights-Ct-89143/home/29088052',
'https://www.redfin.com/NV/Las-Vegas/8213-Jo-Marcy-Dr-89131/home/29856153',
'https://www.redfin.com/NV/North-Las-Vegas/7337-Redhead-Dr-89084/home/29017376',
'https://www.redfin.com/NV/North-Las-Vegas/4165-Vibrant-Star-Ave-89084/home/167602034',
'https://www.redfin.com/NV/Las-Vegas/9488-Lunar-Phase-St-89143/home/177326605',
'https://www.redfin.com/NV/Las-Vegas/5418-Basset-Hound-Ave-89131/home/29855243',
'https://www.redfin.com/NV/Las-Vegas/7424-Forest-Ivy-St-89131/home/29090098',
'https://www.redfin.com/NV/Las-Vegas/10148-Skye-Saddle-Ave-89166/home/169227802',
'https://www.redfin.com/NV/Las-Vegas/9844-Outer-Hebrides-Ave-89166/home/171825316',
'https://www.redfin.com/NV/North-Las-Vegas/7748-Broadwing-Dr-89084/home/28979535',
'https://www.redfin.com/NV/Las-Vegas/6729-Pleasant-Flower-St-89149/home/113140968',
'https://www.redfin.com/NV/North-Las-Vegas/7318-Glimmer-Point-St-89084/home/167601485',
'https://www.redfin.com/NV/Las-Vegas/10179-Abseil-St-89166/home/183995463',
'https://www.redfin.com/NV/Las-Vegas/9343-Lunar-Effect-St-89143/home/177326648',
'https://www.redfin.com/NV/Las-Vegas/9511-Lunar-Phase-St-89143/home/177326598',
'https://www.redfin.com/NV/Las-Vegas/10118-Bismarck-Palms-Ave-89166/home/176105256',
'https://www.redfin.com/NV/North-Las-Vegas/3718-Snow-Fawn-Ave-89084/home/171301047',
'https://www.redfin.com/NV/Las-Vegas/9576-Balais-Dr-89143/home/176643523',
'https://www.redfin.com/NV/Las-Vegas/10308-Walworth-Ave-89166/home/29759485',
'https://www.redfin.com/NV/Las-Vegas/5430-Sharpei-Ct-89131/home/29888040',
'https://www.redfin.com/NV/Las-Vegas/7010-Lakota-Summit-Ct-89149/home/167620953',
'https://www.redfin.com/NV/Las-Vegas/9321-Hampstead-Hills-Ave-89149/home/29004991',
'https://www.redfin.com/NV/Las-Vegas/5304-Grand-Teton-Dr-89131/home/29088293',
'https://www.redfin.com/NV/Las-Vegas/7220-Rustic-Meadow-St-89131/home/29844806',
'https://www.redfin.com/NV/Las-Vegas/6431-Winecup-St-89149/home/167621108',
'https://www.redfin.com/NV/Las-Vegas/6640-Weeping-Pine-St-89149/home/29313531',
'https://www.redfin.com/NV/Las-Vegas/9037-Spinning-Wheel-Ave-89143/home/29087605',
'https://www.redfin.com/NV/Las-Vegas/7348-Zion-Falls-St-89131/home/28996638',
'https://www.redfin.com/NV/Las-Vegas/8718-Lavender-Ridge-St-89131/home/52627182',
'https://www.redfin.com/NV/Las-Vegas/7724-Blended-Stitch-Ct-89149/home/55514464',
'https://www.redfin.com/NV/Las-Vegas/10827-Mason-Hill-Ave-89166/home/30515838',
'https://www.redfin.com/NV/Las-Vegas/10215-Copper-Meadow-Ave-89166/home/177117801',
'https://www.redfin.com/NV/North-Las-Vegas/3609-Fledgling-Dr-89084/home/29016840',
'https://www.redfin.com/NV/Las-Vegas/10446-Muro-Russo-Ave-89166/home/177299570',
'https://www.redfin.com/NV/Las-Vegas/9045-Dolce-Brezza-St-89166/home/179980813',
'https://www.redfin.com/NV/Las-Vegas/9044-Dolce-Brezza-St-89166/home/179980811',
'https://www.redfin.com/NV/Las-Vegas/8050-California-Pine-St-89166/home/169227915',
'https://www.redfin.com/NV/Las-Vegas/7653-Tender-Tassels-St-89149/home/29060314',
'https://www.redfin.com/NV/Las-Vegas/8620-Rosey-Ct-89149/home/29096861',
'https://www.redfin.com/NV/North-Las-Vegas/3625-Tiger-Ridge-Ln-89084/home/29048017',
'https://www.redfin.com/NV/Las-Vegas/8162-Skye-Quarry-St-89166/home/167628922',
'https://www.redfin.com/NV/Las-Vegas/10954-Fennel-AVE-89166/unit-194/home/187414402',
'https://www.redfin.com/NV/Las-Vegas/10954-Parsley-Ave-89166/unit-87/home/187414241',
'https://www.redfin.com/NV/North-Las-Vegas/7469-Garnet-Moon-St-89084/home/167600314',
'https://www.redfin.com/NV/North-Las-Vegas/7620-Mojave-Evening-St-89084/home/171301129',
'https://www.redfin.com/NV/Las-Vegas/8605-Blowing-Pines-Dr-89143/home/29824703',
'https://www.redfin.com/NV/Las-Vegas/5945-Wasatch-Ridge-Cir-89149/home/29097133',
'https://www.redfin.com/NV/Las-Vegas/6315-N-El-Capitan-Way-89149/home/29096284',
'https://www.redfin.com/NV/Las-Vegas/8505-Cheerful-Brook-Ave-89143/home/28998816',
'https://www.redfin.com/NV/Las-Vegas/5942-Sunset-River-Ave-89131/home/165757315',
'https://www.redfin.com/NV/Las-Vegas/9956-Yetti-Stroll-Ave-89166/home/171301428',
'https://www.redfin.com/NV/Las-Vegas/9008-Rolling-Pietra-St-89166/home/179980766',
'https://www.redfin.com/NV/Las-Vegas/10197-Bismarck-Palms-Ave-89166/home/177117875',
'https://www.redfin.com/NV/Las-Vegas/10190-Sequoia-Glen-Ave-89166/home/177117861',
'https://www.redfin.com/NV/Las-Vegas/9001-Powell-Springs-St-89166/home/177117849',
'https://www.redfin.com/NV/Las-Vegas/8878-Sasquatch-Run-Ave-89166/home/176643691',
'https://www.redfin.com/NV/Las-Vegas/4805-Palm-Tree-Ct-89131/home/29024649',
'https://www.redfin.com/NV/Las-Vegas/8941-Marmot-Ridge-Ct-89149/home/29022527',
'https://www.redfin.com/NV/Las-Vegas/9009-Garden-Fox-Ct-89149/home/40163303',
'https://www.redfin.com/NV/Las-Vegas/9904-Alpine-Style-Dr-89166/home/183995377',
'https://www.redfin.com/NV/Las-Vegas/6491-Vanhoven-Crest-St-89166/home/177119184',
'https://www.redfin.com/NV/Las-Vegas/6570-Grand-Concourse-St-89166/home/30470004',
'https://www.redfin.com/NV/North-Las-Vegas/7405-Bugler-Swan-Way-89084/home/29016851',
'https://www.redfin.com/NV/Las-Vegas/6614-Fence-Jumper-Ave-89131/home/29891978',
'https://www.redfin.com/NV/Las-Vegas/7619-Redcloud-Peak-St-89166/home/52625305',
'https://www.redfin.com/NV/Las-Vegas/7900-Pale-Rider-St-89131/home/29927864',
'https://www.redfin.com/NV/Las-Vegas/6808-Tarpon-Springs-Ct-89131/home/29021765',
'https://www.redfin.com/NV/North-Las-Vegas/7737-Wishbone-Falls-St-89084/home/167599722',
'https://www.redfin.com/NV/North-Las-Vegas/4521-Bell-Cord-Ave-89031/unit-101/home/29597383',
'https://www.redfin.com/NV/Las-Vegas/8873-Felker-St-89166/home/179964317',
'https://www.redfin.com/NV/Las-Vegas/8886-Felker-St-89166/home/179964314',
'https://www.redfin.com/NV/Las-Vegas/10723-Lenore-Park-Ct-89166/home/29598551',
'https://www.redfin.com/NV/North-Las-Vegas/4321-Westmere-Ave-89084/home/170579205',
'https://www.redfin.com/NV/Las-Vegas/7824-Marbledoe-St-89149/home/29023196',
'https://www.redfin.com/NV/Las-Vegas/4617-Savin-Cir-89130/home/29115441',
'https://www.redfin.com/NV/North-Las-Vegas/4702-Steeplechase-Ave-89031/home/29240403',
'https://www.redfin.com/NV/Las-Vegas/10279-Ansley-Bay-Ave-89166/home/181146177',
'https://www.redfin.com/NV/North-Las-Vegas/3419-Ocatillo-Mesa-Way-89031/home/29892102',
'https://www.redfin.com/NV/Las-Vegas/10287-Ansley-Bay-Ave-89166/home/181146176',
'https://www.redfin.com/NV/North-Las-Vegas/7073-Seabirds-Pl-89084/home/29018192',
'https://www.redfin.com/NV/Las-Vegas/6463-Sean-Creek-St-89149/home/29598001',
'https://www.redfin.com/NV/Las-Vegas/10553-Skye-Paseo-Ave-89166/home/169227429',
'https://www.redfin.com/NV/Las-Vegas/7071-Shady-Palms-St-89131/home/166086034',
'https://www.redfin.com/NV/North-Las-Vegas/4635-Frasers-Owl-Ave-89084/unit-102/home/167600872',
'https://www.redfin.com/NV/Las-Vegas/10723-Myrtle-Grove-Ave-89166/home/29759557',
'https://www.redfin.com/NV/North-Las-Vegas/7716-Fruit-Dove-St-89084/home/28980886',
'https://www.redfin.com/NV/Las-Vegas/6781-Hyla-Roman-Ave-89131/home/33459192',
'https://www.redfin.com/NV/North-Las-Vegas/3835-Enchanted-Wells-Ave-89031/home/166573387',
'https://www.redfin.com/NV/Las-Vegas/6228-Autumn-Creek-Dr-89130/home/29093008',
'https://www.redfin.com/NV/Las-Vegas/6891-Goose-River-Ave-89131/home/28974413',
'https://www.redfin.com/NV/Las-Vegas/6200-Peggotty-Ave-89130/home/29094460',
'https://www.redfin.com/NV/Las-Vegas/7409-Timber-Run-St-89149/home/29023306',
'https://www.redfin.com/NV/Las-Vegas/10426-Lilac-Square-Ave-89166/home/30469472',
'https://www.redfin.com/NV/Las-Vegas/8826-Arroyo-Azul-St-89131/home/29006196',
'https://www.redfin.com/NV/Las-Vegas/8705-Stokestone-St-89166/home/177118116',
'https://www.redfin.com/NV/Las-Vegas/10229-Cosmic-Latte-Ct-89143/home/184013228',
'https://www.redfin.com/NV/Las-Vegas/10221-Cosmic-Latte-Ct-89143/home/184013227',
'https://www.redfin.com/NV/Las-Vegas/10244-Cosmic-Latte-Ct-89143/home/184013221',
'https://www.redfin.com/NV/North-Las-Vegas/4510-Turkey-Ridge-Ct-89031/home/29890891',
'https://www.redfin.com/NV/Las-Vegas/7869-Solid-Horn-Ct-89149/home/29023156',
'https://www.redfin.com/NV/Las-Vegas/5200-Future-Dr-89130/home/29103648',
'https://www.redfin.com/NV/Las-Vegas/5537-Sentinel-Bridge-St-89130/home/29102925',
'https://www.redfin.com/NV/Las-Vegas/8165-Starling-View-Ct-89166/home/28983284',
'https://www.redfin.com/NV/North-Las-Vegas/6513-Grayback-Dr-89084/home/29011027',
'https://www.redfin.com/NV/North-Las-Vegas/3902-Champagne-Wood-Dr-89031/home/29853242',
'https://www.redfin.com/NV/North-Las-Vegas/3808-Specula-Wing-Dr-89084/home/29017433',
'https://www.redfin.com/NV/North-Las-Vegas/4229-Great-Egret-Ln-89084/home/29059514',
'https://www.redfin.com/NV/North-Las-Vegas/3826-Birch-River-St-89032/home/166108138',
'https://www.redfin.com/NV/Las-Vegas/7539-Hornblower-Ave-89131/home/29892874',
'https://www.redfin.com/NV/Las-Vegas/5305-Little-Fawn-Ct-89130/home/29093974',
'https://www.redfin.com/NV/Las-Vegas/8413-Starstruck-Ave-89143/home/29854018',
'https://www.redfin.com/NV/Las-Vegas/8916-El-Diablo-St-89131/home/29011680',
'https://www.redfin.com/NV/Las-Vegas/6227-Wittig-Ave-89131/home/29092576',
'https://www.redfin.com/NV/Las-Vegas/7917-Brighton-Summit-Ave-89131/home/187224437',
'https://www.redfin.com/NV/Las-Vegas/6425-Bullring-Ln-89130/home/183056805',
'https://www.redfin.com/NV/Las-Vegas/7344-Savannah-Falls-St-89131/home/29005169',
'https://www.redfin.com/NV/Las-Vegas/6504-N-Tee-Pee-Ln-89149/home/29792610',
'https://www.redfin.com/NV/Las-Vegas/10166-Bismarck-Palms-Ave-89166/home/177117891',
'https://www.redfin.com/NV/Las-Vegas/10396-Thurso-Ave-89166/home/179980927',
'https://www.redfin.com/NV/North-Las-Vegas/3415-W-Red-Coach-Ave-89031/home/29238702',
'https://www.redfin.com/NV/Las-Vegas/9025-Reindeer-Lake-St-89143/home/28983948',
'https://www.redfin.com/NV/Las-Vegas/5950-N-Tee-Pee-Ln-89149/home/29096778',
'https://www.redfin.com/NV/Las-Vegas/10576-Pelican-Island-Ave-89166/home/143190798',
'https://www.redfin.com/NV/Las-Vegas/5683-Beguiling-Ct-89149/unit-45/home/187182301',
'https://www.redfin.com/NV/North-Las-Vegas/3204-Knight-Hill-Pl-89031/home/29921322',
'https://www.redfin.com/NV/Las-Vegas/8843-Pauley-Hills-St-89131/home/143189656',
'https://www.redfin.com/NV/North-Las-Vegas/4660-Ridgeley-Ave-89084/unit-101/home/167601021',
'https://www.redfin.com/NV/Las-Vegas/6145-Copper-Crest-Dr-89130/home/29100788',
'https://www.redfin.com/NV/Las-Vegas/8332-Fritzen-Ave-89131/home/29844646',
'https://www.redfin.com/NV/Las-Vegas/6459-Red-Garnet-Ct-89131/home/29007682',
'https://www.redfin.com/NV/Las-Vegas/10957-Fennel-ST-89166/unit-181/home/187077532',
'https://www.redfin.com/NV/Las-Vegas/6145-Misty-Brook-Ct-89149/home/29096787',
'https://www.redfin.com/NV/Las-Vegas/10973-Nicholas-Jay-Ave-89166/home/179981147',
'https://www.redfin.com/NV/Las-Vegas/6559-Benbrook-Springs-Dr-89131/home/29051274',
'https://www.redfin.com/NV/Las-Vegas/10966-Parsley-Ave-89166/unit-84/home/187075144',
'https://www.redfin.com/NV/Las-Vegas/5704-Sligo-St-89130/home/29094334',
'https://www.redfin.com/NV/Las-Vegas/10420-Thurso-Ave-89166/home/179980924',
'https://www.redfin.com/NV/Las-Vegas/10972-Nicholas-Jay-Ave-89166/home/179981130',
'https://www.redfin.com/NV/Las-Vegas/10428-Thurso-Ave-89166/home/179980923',
'https://www.redfin.com/NV/Las-Vegas/10388-Thurso-Ave-89166/home/179980928',
'https://www.redfin.com/NV/Las-Vegas/4094-Sparrow-Rock-St-89129/home/29024324',
'https://www.redfin.com/NV/Las-Vegas/4172-Swept-Plains-St-89129/home/29024360',
'https://www.redfin.com/NV/Las-Vegas/6647-Weeping-Pine-St-89149/home/29313588',
'https://www.redfin.com/NV/North-Las-Vegas/3425-Fledgling-Dr-89084/home/29016811',
'https://www.redfin.com/NV/Las-Vegas/8325-W-La-Madre-Way-89149/home/29097728',
'https://www.redfin.com/NV/Las-Vegas/7927-Bartlett-Peak-St-89166/home/52625258',
'https://www.redfin.com/NV/Las-Vegas/8560-Longmire-Ave-89143/home/179962231',
'https://www.redfin.com/NV/Las-Vegas/10419-Bush-Mountain-Ave-89166/home/30479913',
'https://www.redfin.com/NV/Las-Vegas/5924-McLennan-Ranch-Ave-89131/home/29598700',
'https://www.redfin.com/NV/Las-Vegas/3741-Cactus-Wheel-Ct-89129/home/29126630',
'https://www.redfin.com/NV/Las-Vegas/6633-Wheelbarrow-Peak-Dr-89108/home/29117369',
'https://www.redfin.com/NV/Las-Vegas/4905-Morning-Falls-Ave-89131/home/29893192',
'https://www.redfin.com/NV/Las-Vegas/5433-Bridgehampton-Ave-89130/home/29093285',
'https://www.redfin.com/NV/Las-Vegas/6732-Accent-Ct-89108/home/29119039',
'https://www.redfin.com/NV/Las-Vegas/Skye-Hills-Sage-Reserve/Somerset/home/177619437',
'https://www.redfin.com/NV/Las-Vegas/Luna-at-Sunstone/Ariel-Plan-4/home/178311937',
'https://www.redfin.com/NV/Las-Vegas/Luna-at-Sunstone/Selene-Plan-5/home/178311938',
'https://www.redfin.com/NV/Las-Vegas/Skye-Canyon-Belterra-II/Glenbrook/home/179437435',
'https://www.redfin.com/NV/Las-Vegas/Toll-Brothers-at-Skye-Canyon-Montrose-Collection/Evora/home/180412877',
'https://www.redfin.com/NV/Las-Vegas/Toll-Brothers-at-Skye-Canyon-Montrose-Collection/Clavel/home/180412879',
'https://www.redfin.com/NV/Las-Vegas/Toll-Brothers-at-Skye-Canyon-Montrose-Collection/Braga/home/180412881',
'https://www.redfin.com/NV/Las-Vegas/Toll-Brothers-at-Skye-Canyon-Vista-Rossa-Collection/Montoro/home/180412882',
'https://www.redfin.com/NV/Las-Vegas/Toll-Brothers-at-Skye-Canyon-Vista-Rossa-Collection/Loma/home/180412883',
'https://www.redfin.com/NV/Las-Vegas/Citrine/Plan-3/home/186302985',
'https://www.redfin.com/NV/Las-Vegas/Citrine/Plan-1/home/186303516',
'https://www.redfin.com/NV/Las-Vegas/Citrine/Plan-2/home/186303629',
'https://www.redfin.com/NV/North-Las-Vegas/4371-Outlook-Vista-Ave-89084/home/170367555',
'https://www.redfin.com/NV/Las-Vegas/9256-Aiden-Valley-Ave-89143/home/177326438',
'https://www.redfin.com/NV/North-Las-Vegas/7232-N-Decatur-Blvd-89084/unit-3/home/185192655',
'https://www.redfin.com/NV/North-Las-Vegas/7308-N-Decatur-Blvd-89084/unit-5/home/185741753',
'https://www.redfin.com/NV/North-Las-Vegas/7135-Stockton-Dunes-St-89084/home/179956247',
'https://www.redfin.com/NV/North-Las-Vegas/7127-Stockton-Dunes-St-89084/home/179956245',
'https://www.redfin.com/NV/North-Las-Vegas/7065-Tilligerry-St-89084/home/179956328',
'https://www.redfin.com/NV/North-Las-Vegas/7052-Tilligerry-St-89084/home/179956322',
'https://www.redfin.com/NV/North-Las-Vegas/7123-Stockton-Dunes-St-89084/home/179956244',
'https://www.redfin.com/NV/North-Las-Vegas/363-Free-Fall-Ave-89084/home/172238571',
'https://www.redfin.com/NV/North-Las-Vegas/1049-Williamtown-Ave-89084/home/179956288',
'https://www.redfin.com/NV/North-Las-Vegas/1045-Williamtown-Ave-89084/home/179956287',
'https://www.redfin.com/NV/Las-Vegas/7305-Breezy-Night-Ct-89129/home/29118249',
'https://www.redfin.com/NV/Las-Vegas/667-Grey-Saker-St-89138/home/176105673',
'https://www.redfin.com/NV/Las-Vegas/670-Spotted-Falcon-St-89138/home/176105693',
'https://www.redfin.com/NV/Las-Vegas/5121-Portraits-Pl-89149/home/29100259',
'https://www.redfin.com/NV/North-Las-Vegas/916-Pine-Vista-Ct-89084/home/113140099',
'https://www.redfin.com/NV/Las-Vegas/7061-Lakota-Summit-Ct-89149/home/167620927',
'https://www.redfin.com/NV/Las-Vegas/10741-Marathon-Bell-Ln-89129/home/167631132',
'https://www.redfin.com/NV/Las-Vegas/8033-Divernon-Ave-89149/home/29095970',
'https://www.redfin.com/NV/Las-Vegas/6647-Brooklyn-Heights-St-89166/home/29760681',
'https://www.redfin.com/NV/Las-Vegas/4300-Coran-Ln-89108/home/29306270',
'https://www.redfin.com/NV/Las-Vegas/2508-Springridge-Dr-89134/home/29150369',
'https://www.redfin.com/NV/Las-Vegas/2504-Sunset-Beach-Ln-89128/home/29143211',
'https://www.redfin.com/NV/Las-Vegas/8716-Autumn-Wreath-Ave-89129/home/29125594',
'https://www.redfin.com/NV/Las-Vegas/5221-Mambo-Vista-Ave-89108/home/29932403',
'https://www.redfin.com/NV/Las-Vegas/Undisclosed-address-89134/home/29105858',
'https://www.redfin.com/NV/Las-Vegas/6341-Shadow-Glen-Ln-89108/home/29132581',
'https://www.redfin.com/NV/North-Las-Vegas/108-Delighted-Ave-89031/home/29476889',
'https://www.redfin.com/NV/Las-Vegas/6520-W-Cheyenne-Ave-89108/home/29131245',
'https://www.redfin.com/NV/Las-Vegas/4794-Pencarrow-Ct-89130/home/29258112',
'https://www.redfin.com/NV/Las-Vegas/7132-Antique-Olive-St-89149/home/28985146',
'https://www.redfin.com/NV/North-Las-Vegas/513-Dolphin-Peak-Ct-89084/home/29953179',
'https://www.redfin.com/NV/Las-Vegas/4325-Skylake-Ct-89129/home/177305318',
'https://www.redfin.com/NV/North-Las-Vegas/4408-Duck-Harbor-Ave-89031/home/60738039',
'https://www.redfin.com/NV/Las-Vegas/8036-Divernon-Ave-89149/home/29096026',
'https://www.redfin.com/NV/Las-Vegas/11608-Observation-Point-Ave-89138/home/179981741',
'https://www.redfin.com/NV/North-Las-Vegas/7139-Stockton-Dunes-St-89084/home/179956248',
'https://www.redfin.com/NV/North-Las-Vegas/7143-Stockton-Dunes-St-89084/home/179956249',
'https://www.redfin.com/NV/North-Las-Vegas/324-Coldwell-Station-Rd-89084/home/166517134',
'https://www.redfin.com/NV/North-Las-Vegas/7851-Nestor-Creek-Ln-89084/unit-32/home/187675484',
'https://www.redfin.com/NV/North-Las-Vegas/1054-Williamtown-Ave-89084/home/179956252',
'https://www.redfin.com/NV/North-Las-Vegas/1046-Williamtown-Ave-89084/home/179956254',
'https://www.redfin.com/NV/North-Las-Vegas/1050-Williamtown-Ave-89084/home/179956253',
'https://www.redfin.com/NV/North-Las-Vegas/7139-Calvert-Cliffs-St-89084/home/167606546',
'https://www.redfin.com/NV/North-Las-Vegas/1130-Brilliant-Meadow-Ave-89086/home/169224490',
'https://www.redfin.com/NV/North-Las-Vegas/1226-Corral-Bluffs-Ave-89084/home/167597414',
'https://www.redfin.com/NV/North-Las-Vegas/7620-Widewing-Dr-89084/home/29017695',
'https://www.redfin.com/NV/North-Las-Vegas/6929-Noche-Calida-St-89086/home/179640032',
'https://www.redfin.com/NV/North-Las-Vegas/2358-Brisa-Fresca-Ave-89086/home/179639933',
'https://www.redfin.com/NV/North-Las-Vegas/7002-Noche-Calida-St-89086/home/179639952',
'https://www.redfin.com/NV/North-Las-Vegas/6939-Feather-Hill-St-89086/home/175522502',
'https://www.redfin.com/NV/North-Las-Vegas/1122-Brilliant-Meadow-Ave-89086/home/169224492',
'https://www.redfin.com/NV/North-Las-Vegas/7539-Marie-Brook-LN-89084/unit-10/home/187550538',
'https://www.redfin.com/NV/North-Las-Vegas/7640-Homing-Pigeon-St-89084/home/29266902',
'https://www.redfin.com/NV/North-Las-Vegas/7440-Goosander-St-89084/home/29059284',
'https://www.redfin.com/NV/North-Las-Vegas/7543-Marie-Brook-LN-89084/unit-11/home/187428078',
'https://www.redfin.com/NV/North-Las-Vegas/3129-Winter-Sunset-Ave-89081/home/29049601',
'https://www.redfin.com/NV/Las-Vegas/5527-Breecher-Ave-89131/home/29002811',
'https://www.redfin.com/NV/Las-Vegas/5200-Silent-Hope-Ave-89131/home/29003339',
'https://www.redfin.com/NV/North-Las-Vegas/6831-Laguna-Blanca-St-89086/home/175522546',
'https://www.redfin.com/NV/North-Las-Vegas/5750-Sunset-Downs-St-89081/home/29264411',
'https://www.redfin.com/NV/Las-Vegas/7500-Real-Quiet-Dr-89131/home/29844759',
'https://www.redfin.com/NV/North-Las-Vegas/6340-Sereno-Springs-St-89081/home/29016189',
'https://www.redfin.com/NV/North-Las-Vegas/3413-Fledgling-Dr-89084/home/29016816',
'https://www.redfin.com/NV/Las-Vegas/7925-Formitch-Ct-89166/home/144626731',
'https://www.redfin.com/NV/North-Las-Vegas/1006-Junipine-Ave-89081/home/170369169',
'https://www.redfin.com/NV/North-Las-Vegas/3017-Little-Crimson-Ave-89081/home/29048805',
'https://www.redfin.com/NV/Las-Vegas/8941-Windgate-Ranch-St-89166/home/177117764',
'https://www.redfin.com/NV/Las-Vegas/8150-Deer-Clan-Ct-89131/home/29855055',
'https://www.redfin.com/NV/Las-Vegas/10004-Pine-Trail-Ave-89166/home/171301550',
'https://www.redfin.com/NV/North-Las-Vegas/1717-Silent-Sunset-Ave-89084/home/29924562',
'https://www.redfin.com/NV/North-Las-Vegas/2117-Mystic-Wave-Rd-89031/home/167610529',
'https://www.redfin.com/NV/North-Las-Vegas/930-Cornerstone-Pl-89031/home/29078787',
'https://www.redfin.com/NV/Las-Vegas/7340-Marbury-St-89166/home/65558826',
'https://www.redfin.com/NV/North-Las-Vegas/944-E-Dorrell-Ln-89086/home/161117285',
'https://www.redfin.com/NV/North-Las-Vegas/320-San-Antonio-River-St-89084/home/145871107',
'https://www.redfin.com/NV/North-Las-Vegas/6358-Selston-Dr-89081/home/181098997',
'https://www.redfin.com/NV/North-Las-Vegas/5905-Feral-Garden-St-89031/home/29269947',
'https://www.redfin.com/NV/North-Las-Vegas/213-Tainted-Berry-Ave-89031/home/29009812',
'https://www.redfin.com/NV/Las-Vegas/6320-Gambetta-St-89115/home/179954163',
'https://www.redfin.com/NV/North-Las-Vegas/121-Barton-Bay-Ave-89031/home/179957043',
'https://www.redfin.com/NV/Las-Vegas/6316-Gambetta-St-89115/home/179954164',
'https://www.redfin.com/NV/North-Las-Vegas/5336-Talaverde-Heights-St-89081/home/29020376',
'https://www.redfin.com/NV/North-Las-Vegas/5771-Summit-Greens-St-89081/home/29264436',
'https://www.redfin.com/NV/North-Las-Vegas/1025-Old-Creek-Way-89084/home/145306832',
'https://www.redfin.com/NV/North-Las-Vegas/2421-Sunrise-Springs-Ct-89031/home/29082364',
'https://www.redfin.com/NV/North-Las-Vegas/6342-Riddings-Wood-Dr-89081/home/181098987',
'https://www.redfin.com/NV/North-Las-Vegas/2640-Torch-Ave-89081/home/29018895',
'https://www.redfin.com/NV/North-Las-Vegas/7010-Skyline-Blue-St-89086/home/173619166',
'https://www.redfin.com/NV/North-Las-Vegas/325-Casmailia-Ave-89031/home/143156568',
'https://www.redfin.com/NV/North-Las-Vegas/5808-Sagamore-Canyon-St-89081/home/29263821',
'https://www.redfin.com/NV/North-Las-Vegas/3125-Bounding-Home-Rd-89086/home/174789363',
'https://www.redfin.com/NV/North-Las-Vegas/324-Glory-Ridge-Ave-89031/home/29761557',
'https://www.redfin.com/NV/North-Las-Vegas/2016-Marathon-Keys-Ave-89031/home/29083461',
'https://www.redfin.com/NV/Las-Vegas/6931-Leon-Ave-89131/home/29092643',
'https://www.redfin.com/NV/Las-Vegas/5501-Wells-Cathedral-Ave-89130/home/29093147',
'https://www.redfin.com/NV/North-Las-Vegas/6420-Indian-Peak-Ct-89084/home/29009661',
'https://www.redfin.com/NV/North-Las-Vegas/7002-Crimson-Shadow-St-89086/home/173619137',
'https://www.redfin.com/NV/North-Las-Vegas/1205-Granite-Falls-Pl-89084/home/167606956',
'https://www.redfin.com/NV/North-Las-Vegas/4305-Red-Fan-Palm-Ct-89032/home/109828049',
'https://www.redfin.com/NV/North-Las-Vegas/1918-Wild-Grain-Ave-89086/home/175522309',
'https://www.redfin.com/NV/Las-Vegas/2021-San-Simeon-St-89108/home/29173281',
'https://www.redfin.com/NV/North-Las-Vegas/1023-Sunny-Acres-Ave-89081/home/29597139',
'https://www.redfin.com/NV/North-Las-Vegas/5217-Grand-Lake-St-89081/home/29312425',
'https://www.redfin.com/NV/North-Las-Vegas/4162-Nipp-Ave-89081/home/177334462',
'https://www.redfin.com/NV/North-Las-Vegas/336-River-Glider-Ave-89084/home/29851537',
'https://www.redfin.com/NV/Las-Vegas/6420-Break-Point-Ave-89130/home/29100963',
'https://www.redfin.com/NV/North-Las-Vegas/6320-Legend-Falls-St-89081/home/29007632',
'https://www.redfin.com/NV/Las-Vegas/4329-San-Gabriel-Hill-Ave-89115/home/29046339',
'https://www.redfin.com/NV/North-Las-Vegas/Undisclosed-address-89031/home/177318010',
'https://www.redfin.com/NV/Las-Vegas/6533-Grand-Stand-Ave-89131/home/29892036',
'https://www.redfin.com/NV/North-Las-Vegas/6508-Gentle-Falls-Ln-89084/home/29924618',
'https://www.redfin.com/NV/North-Las-Vegas/3945-Buteo-Ln-89084/home/29047892',
'https://www.redfin.com/NV/North-Las-Vegas/6051-Black-Gold-St-89031/home/29925764',
'https://www.redfin.com/NV/North-Las-Vegas/4508-W-San-Miguel-Ave-89032/home/29241125',
'https://www.redfin.com/NV/North-Las-Vegas/4406-Cinema-Ave-89031/home/29843663',
'https://www.redfin.com/NV/Las-Vegas/2244-Welsey-Manor-Dr-89156/home/29374386',
'https://www.redfin.com/NV/Las-Vegas/2564-San-Marcos-St-89115/home/29363828',
'https://www.redfin.com/NV/Las-Vegas/6733-E-Carey-Ave-89156/home/29372318',
'https://www.redfin.com/NV/Las-Vegas/2729-Cathedral-Ln-89108/home/29138950',
'https://www.redfin.com/NV/North-Las-Vegas/3656-Bella-Legato-Ave-89081/home/29264729',
'https://www.redfin.com/NV/Las-Vegas/5701-Avenida-Tampico-89108/home/29134873',
'https://www.redfin.com/NV/North-Las-Vegas/5308-La-Quinta-Hills-St-89081/home/29020446',
'https://www.redfin.com/NV/Las-Vegas/2306-Maverick-St-89108/home/29172177',
'https://www.redfin.com/NV/Las-Vegas/3372-Death-Valley-Dr-89122/home/29418477',
'https://www.redfin.com/NV/Las-Vegas/3386-Ewa-Beach-Dr-89122/unit-762/home/29418549',
'https://www.redfin.com/NV/Las-Vegas/4295-Chafer-Dr-89121/home/29420197',
'https://www.redfin.com/NV/Las-Vegas/3712-Garden-South-Dr-89121/home/29423126',
'https://www.redfin.com/NV/Las-Vegas/3543-Legendary-Dr-89121/home/29423277',
'https://www.redfin.com/NV/Las-Vegas/3546-Bounty-Ave-89121/home/29423334',
'https://www.redfin.com/NV/Las-Vegas/3743-Cannon-Ave-89121/home/29423473',
'https://www.redfin.com/NV/Las-Vegas/3833-Rio-Blanco-Pl-89121/home/29424714',
'https://www.redfin.com/NV/Las-Vegas/1405-Hillside-Pl-89104/home/29452794',
'https://www.redfin.com/NV/Las-Vegas/1704-Griffith-Ave-89104/home/29454180',
'https://www.redfin.com/NV/Las-Vegas/1605-5th-Pl-89104/home/29454579',
'https://www.redfin.com/NV/Las-Vegas/1913-E-Saint-Louis-Ave-89104/home/29454745',
'https://www.redfin.com/NV/Las-Vegas/3506-Westleigh-Ave-89102/home/29458768',
'https://www.redfin.com/NV/Las-Vegas/1355-Douglas-Dr-89102/home/29458770',
'https://www.redfin.com/NV/Las-Vegas/2101-Plaza-del-Cerro-89102/home/29459184',
'https://www.redfin.com/NV/Las-Vegas/2724-Miraflores-Ave-89102/home/29460615',
'https://www.redfin.com/NV/Las-Vegas/4557-Sirius-Ave-89102/home/29462921',
'https://www.redfin.com/NV/Las-Vegas/2824-Alcoa-Ave-89102/home/29464997',
'https://www.redfin.com/NV/Las-Vegas/2616-Yardley-St-89102/home/29465095',
'https://www.redfin.com/NV/Las-Vegas/924-Vegas-Valley-Dr-89109/home/29465816',
'https://www.redfin.com/NV/Las-Vegas/1405-Vegas-Valley-Dr-89169/unit-225/home/29467482',
'https://www.redfin.com/NV/Las-Vegas/1405-Vegas-Valley-Dr-89169/unit-331/home/29468736',
'https://www.redfin.com/NV/Las-Vegas/1934-Capistrano-Ave-89169/home/29469316',
'https://www.redfin.com/NV/Las-Vegas/2740-San-Lago-Ct-89121/home/29473256',
'https://www.redfin.com/NV/Las-Vegas/3571-Calvert-Ct-89121/home/29475269',
'https://www.redfin.com/NV/Las-Vegas/1655-Commanche-Dr-89169/home/29476887',
'https://www.redfin.com/NV/Las-Vegas/6124-Tokara-Ave-89122/home/29480874',
'https://www.redfin.com/NV/Las-Vegas/125-E-Harmon-Ave-89109/unit-2914/home/29485313',
'https://www.redfin.com/NV/Las-Vegas/510-Elm-Dr-89169/unit-106/home/29494065',
'https://www.redfin.com/NV/Las-Vegas/4746-Via-San-Andros-89103/home/29502067',
'https://www.redfin.com/NV/Las-Vegas/4673-Via-Torino-89103/home/29502315',
'https://www.redfin.com/NV/Las-Vegas/4150-Sanderling-Cir-89103/unit-455/home/29502906',
'https://www.redfin.com/NV/Las-Vegas/4339-Spencer-St-89119/unit-8/home/29504555',
'https://www.redfin.com/NV/Las-Vegas/3083-Van-Buskirk-Cir-89121/home/29508090',
'https://www.redfin.com/NV/Las-Vegas/5405-Mountain-View-Dr-89146/home/29517000',
'https://www.redfin.com/NV/Las-Vegas/5228-Del-Rey-Ave-89146/home/29517335',
'https://www.redfin.com/NV/Las-Vegas/5401-Del-Rey-Ave-89146/home/29517447',
'https://www.redfin.com/NV/Las-Vegas/8217-Gillette-St-89117/home/29521745',
'https://www.redfin.com/NV/Las-Vegas/8817-Borla-Dr-89117/home/29523704',
'https://www.redfin.com/NV/Las-Vegas/10001-Pinnacle-Pass-Dr-89117/home/29525532',
'https://www.redfin.com/NV/Las-Vegas/2052-Jade-Creek-St-89117/unit-203/home/29525970',
'https://www.redfin.com/NV/Las-Vegas/2740-Monrovia-Dr-89117/home/29528387',
'https://www.redfin.com/NV/Las-Vegas/9733-Fern-Canyon-Ave-89117/home/29528900',
'https://www.redfin.com/NV/Las-Vegas/3220-Beacon-Shores-Cir-89117/home/29530099',
'https://www.redfin.com/NV/Las-Vegas/3304-Paragon-Dr-89117/home/29534664',
'https://www.redfin.com/NV/Las-Vegas/8455-W-Sahara-Ave-89117/unit-173/home/29534716',
'https://www.redfin.com/NV/Las-Vegas/3309-Mariner-Bay-St-89117/home/29534751',
'https://www.redfin.com/NV/Las-Vegas/7355-Edna-Ave-89117/home/29536948',
'https://www.redfin.com/NV/Las-Vegas/2845-Montessouri-St-89117/home/29537098',
'https://www.redfin.com/NV/Las-Vegas/5865-Edna-Ave-89146/home/29539015',
'https://www.redfin.com/NV/Las-Vegas/5735-Eldora-Ave-89146/home/29539071',
'https://www.redfin.com/NV/Las-Vegas/3236-Lindell-Rd-89146/home/29539945',
'https://www.redfin.com/NV/Las-Vegas/5085-Via-de-Palma-Dr-89146/home/29540064',
'https://www.redfin.com/NV/Las-Vegas/3955-Glory-Ct-89103/home/29541009',
'https://www.redfin.com/NV/Las-Vegas/3669-Redwood-St-89103/home/29543387',
'https://www.redfin.com/NV/Las-Vegas/9342-W-Viking-Rd-89147/home/29552869',
'https://www.redfin.com/NV/Las-Vegas/9222-Snow-Flower-Ave-89147/home/29553034',
'https://www.redfin.com/NV/Las-Vegas/8965-Pomona-Ct-89147/home/29553637',
'https://www.redfin.com/NV/Las-Vegas/3826-Windansea-St-89147/home/29553931',
'https://www.redfin.com/NV/Las-Vegas/10090-Diving-Duck-Ave-89117/home/29554864',
'https://www.redfin.com/NV/Las-Vegas/9740-Pine-Thickett-Ave-89147/home/29556165',
'https://www.redfin.com/NV/Las-Vegas/9652-Camino-Capistrano-Ln-89147/home/29556938',
'https://www.redfin.com/NV/Las-Vegas/4210-Night-Star-St-89147/home/29557864',
'https://www.redfin.com/NV/Las-Vegas/8501-W-University-Ave-89147/unit-2001/home/29560765',
'https://www.redfin.com/NV/Las-Vegas/8501-W-University-Ave-89147/unit-1014/home/29560776',
'https://www.redfin.com/NV/Las-Vegas/8501-W-University-Ave-89147/unit-1049/home/29561088',
'https://www.redfin.com/NV/Las-Vegas/8101-W-Flamingo-Rd-89147/unit-2099/home/29562479',
'https://www.redfin.com/NV/Las-Vegas/7924-Laurena-Ave-89147/home/29563294',
'https://www.redfin.com/NV/Las-Vegas/7653-Gallant-Cir-89147/home/29564426',
'https://www.redfin.com/NV/Las-Vegas/4436-Sandcastle-Dr-89147/home/29565476',
'https://www.redfin.com/NV/Las-Vegas/4447-Fernbrook-Rd-89103/home/29569196',
'https://www.redfin.com/NV/Las-Vegas/9144-Beautiful-Flower-Ct-89149/home/29598356',
'https://www.redfin.com/NV/Las-Vegas/3676-Asbury-Hill-Ave-89110/home/29599148',
'https://www.redfin.com/NV/Las-Vegas/1773-Molly-Meadows-St-89115/home/29781213',
'https://www.redfin.com/NV/Las-Vegas/9103-Alta-Dr-89145/unit-1407/home/29792359',
'https://www.redfin.com/NV/Las-Vegas/2000-N-Fashion-Show-Dr-89109/unit-2127/home/29804200',
'https://www.redfin.com/NV/Las-Vegas/2000-N-Fashion-Show-Dr-89109/unit-2604/home/29804375',
'https://www.redfin.com/NV/Las-Vegas/150-Las-Vegas-Blvd-N-89101/unit-906/home/29812975',
'https://www.redfin.com/NV/Las-Vegas/4471-Dean-Martin-Dr-89103/unit-1208/home/29813888',
'https://www.redfin.com/NV/Las-Vegas/353-E-Bonneville-Ave-89101/unit-567/home/29816775',
'https://www.redfin.com/NV/Las-Vegas/5509-Raven-Creek-Ave-89130/home/29844845',
'https://www.redfin.com/NV/Las-Vegas/3350-Eagle-Bend-St-89122/home/29848672',
'https://www.redfin.com/NV/Las-Vegas/7431-Mystic-Stream-St-89131/home/29856043',
'https://www.redfin.com/NV/Las-Vegas/6728-Alpine-Brooks-Ave-89130/home/29857376',
'https://www.redfin.com/NV/Las-Vegas/932-Coatbridge-St-89145/home/29859037',
'https://www.redfin.com/NV/Las-Vegas/4355-Micahs-Canyon-Ct-89129/home/29859368',
'https://www.redfin.com/NV/Las-Vegas/1165-Warm-Sun-Ct-89110/home/29863548',
'https://www.redfin.com/NV/Las-Vegas/5650-E-Sahara-Ave-89142/unit-1035/home/29864625',
'https://www.redfin.com/NV/Las-Vegas/3400-Cabana-Dr-89122/unit-1119/home/29865221',
'https://www.redfin.com/NV/Las-Vegas/8436-Cobble-Village-Ct-89117/home/29866630',
'https://www.redfin.com/NV/Las-Vegas/2877-Paradise-Rd-89109/unit-2502/home/29866756',
'https://www.redfin.com/NV/Las-Vegas/1110-Azure-Heights-Pl-89110/home/29895034',
'https://www.redfin.com/NV/Las-Vegas/3224-Monaco-Shores-Dr-89117/home/29895806',
'https://www.redfin.com/NV/Las-Vegas/2877-Paradise-Rd-89109/unit-1505/home/29895918',
'https://www.redfin.com/NV/Las-Vegas/5001-River-Splash-Ave-89131/home/29908858',
'https://www.redfin.com/NV/Las-Vegas/7255-Diamond-Canyon-Ln-89149/unit-204/home/29910919',
'https://www.redfin.com/NV/Las-Vegas/7734-Morning-Lake-Dr-89131/home/29915915',
'https://www.redfin.com/NV/Las-Vegas/4928-Wild-Buffalo-Ave-89131/home/29927866',
'https://www.redfin.com/NV/Las-Vegas/5025-Hostetler-Ave-89131/home/29928010',
'https://www.redfin.com/NV/Las-Vegas/7817-Orchard-Port-Ave-89131/home/29949084',
'https://www.redfin.com/NV/Las-Vegas/2122-Clancy-St-89156/home/29952987',
'https://www.redfin.com/NV/Las-Vegas/3968-Jazzy-Ginger-Ct-89129/home/29955574',
'https://www.redfin.com/NV/Las-Vegas/3726-Las-Vegas-Blvd-S-89158/unit-703/home/30534120',
'https://www.redfin.com/NV/Las-Vegas/3726-Las-Vegas-Blvd-S-89158/unit-3009/home/30534296',
'https://www.redfin.com/NV/Las-Vegas/5802-Smithsonian-Way-89130/home/33551364',
'https://www.redfin.com/NV/Las-Vegas/9261-Spiderlily-Ct-89149/home/35626223',
'https://www.redfin.com/NV/Las-Vegas/3672-Asbury-Hill-Ave-89110/home/40233601',
'https://www.redfin.com/NV/Las-Vegas/9112-Picturesque-Ave-89149/home/52619985',
'https://www.redfin.com/NV/Las-Vegas/4123-Ivy-Russell-Way-89115/home/55222186',
'https://www.redfin.com/NV/Las-Vegas/2833-Rough-Green-St-89117/home/65558607',
'https://www.redfin.com/NV/Las-Vegas/2080-Karen-Ave-89169/unit-B42/home/77434308',
'https://www.redfin.com/NV/Las-Vegas/9936-Coyote-Echo-Ct-89166/home/109871169',
'https://www.redfin.com/NV/Las-Vegas/8048-Calico-Mesa-St-89166/home/109871930',
'https://www.redfin.com/NV/Las-Vegas/9052-Drummer-Bay-Ave-89149/home/113168918',
'https://www.redfin.com/NV/Las-Vegas/7084-Winstar-St-89131/home/147876257',
'https://www.redfin.com/NV/Las-Vegas/5200-Steep-Cliffs-Ave-89115/home/160676394',
'https://www.redfin.com/NV/Las-Vegas/6731-Sycamore-Pines-St-89149/home/167621517',
'https://www.redfin.com/NV/Las-Vegas/4943-Enlightenment-St-89130/home/167627501',
'https://www.redfin.com/NV/Las-Vegas/4381-W-Flamingo-Rd-89103/unit-30305/home/167869840',
'https://www.redfin.com/NV/Las-Vegas/7490-Tucker-William-St-89149/home/169225931',
'https://www.redfin.com/NV/Las-Vegas/7310-Fruitful-Harvest-Ave-89131/home/109627949',
'https://www.redfin.com/NV/Las-Vegas/9729-Treeline-Run-Ave-89166/home/109871848',
'https://www.redfin.com/NV/Las-Vegas/4597-Townwall-St-89115/home/144792621',
'https://www.redfin.com/NV/Las-Vegas/5251-Dubris-Dr-89115/home/166121066',
'https://www.redfin.com/NV/Las-Vegas/6691-Viewpoint-Dr-89156/home/166133253',
'https://www.redfin.com/NV/Las-Vegas/8479-Hickam-Ave-89129/home/167639210',
'https://www.redfin.com/NV/Las-Vegas/6499-Smithville-Ave-89142/home/169230515',
'https://www.redfin.com/NV/Las-Vegas/8359-Lagunilla-Ave-89149/home/170969773',
'https://www.redfin.com/NV/North-Las-Vegas/4660-Blaise-Ave-89084/unit-102/home/174788033',
'https://www.redfin.com/NV/Las-Vegas/4940-Chidlaw-St-89115/unit-1/home/183991560',
'https://www.redfin.com/NV/Las-Vegas/4940-Chidlaw-St-89115/unit-3/home/183991566',
'https://www.redfin.com/NV/Las-Vegas/1721-Chapman-Dr-89104/home/28968907',
'https://www.redfin.com/NV/Las-Vegas/2072-Sandy-Ln-89115/home/28987470',
'https://www.redfin.com/NV/Las-Vegas/2513-Huber-Heights-Dr-89128/home/29143035',
'https://www.redfin.com/NV/Las-Vegas/10000-Folsom-Dr-89134/home/29150813',
'https://www.redfin.com/NV/Las-Vegas/9629-Blue-Bell-Dr-89134/home/29153422',
'https://www.redfin.com/NV/Las-Vegas/9420-San-Laguna-Ct-89134/unit-104/home/29211600',
'https://www.redfin.com/NV/Las-Vegas/5520-Heron-Ave-89107/home/29232936',
'https://www.redfin.com/NV/North-Las-Vegas/2723-E-Webb-Ave-89030/home/29329361',
'https://www.redfin.com/NV/Las-Vegas/405-N-13th-St-89101/home/29348530',
'https://www.redfin.com/NV/Las-Vegas/2000-Walnut-Ave-89101/home/29349009',
'https://www.redfin.com/NV/Las-Vegas/246-N-23rd-St-89101/home/29350772',
'https://www.redfin.com/NV/Las-Vegas/3625-Valley-Forge-Ave-89110/home/29384226',
'https://www.redfin.com/NV/Las-Vegas/2972-Juniper-Hills-Blvd-89142/unit-102/home/29416224',
'https://www.redfin.com/NV/Las-Vegas/2714-Bridgepointe-Dr-89121/home/29474142',
'https://www.redfin.com/NV/Las-Vegas/3020-Island-View-Ct-89117/home/29532628',
'https://www.redfin.com/NV/Las-Vegas/3220-Lindell-Rd-89146/home/29539947',
'https://www.redfin.com/NV/Las-Vegas/5151-Pioneer-Ave-89146/unit-102/home/29542285',
'https://www.redfin.com/NV/Las-Vegas/6765-Painted-Morning-Ave-89142/home/104008439',
'https://www.redfin.com/NV/Las-Vegas/2198-E-Desert-Inn-Rd-89169/home/28968463',
'https://www.redfin.com/NV/Las-Vegas/6658-Chapel-Heights-Ct-89156/home/28987722',
'https://www.redfin.com/NV/Las-Vegas/6308-Barrington-Bridge-Ave-89122/home/29001418',
'https://www.redfin.com/NV/Las-Vegas/3504-Durant-River-Dr-89122/home/29006829',
'https://www.redfin.com/NV/Las-Vegas/6511-Flatwoods-Bay-Ct-89122/home/29028748',
'https://www.redfin.com/NV/Las-Vegas/3421-Greenwood-Springs-Dr-89122/home/29028858',
'https://www.redfin.com/NV/Las-Vegas/3338-Mendocino-Forest-St-89122/home/29028992',
'https://www.redfin.com/NV/Las-Vegas/900-Las-Vegas-Blvd-S-89101/unit-1414/home/29052400',
'https://www.redfin.com/NV/Las-Vegas/5280-Shreve-Ave-89156/home/29052664',
'https://www.redfin.com/NV/Las-Vegas/793-Oakmont-Ave-89109/unit-203/home/29063353',
'https://www.redfin.com/NV/Las-Vegas/714-Oakmont-Ave-89109/unit-1108/home/29063540',
'https://www.redfin.com/NV/Las-Vegas/2831-Geary-Pl-89109/unit-2933/home/29063732',
'https://www.redfin.com/NV/Las-Vegas/2831-Geary-Pl-89109/unit-2902/home/29063793',
'https://www.redfin.com/NV/Las-Vegas/2845-Loveland-Dr-89109/unit-3606/home/29063914',
'https://www.redfin.com/NV/Las-Vegas/2194-Dragon-Claw-Ln-89156/home/29180299',
'https://www.redfin.com/NV/Las-Vegas/6170-E-Sahara-Ave-89142/unit-1080/home/29181838',
'https://www.redfin.com/NV/Las-Vegas/6170-E-Sahara-Ave-89142/unit-1015/home/29281953',
'https://www.redfin.com/NV/Las-Vegas/221-George-Pl-89106/home/29325473',
'https://www.redfin.com/NV/Las-Vegas/417-Freeman-Ave-89106/home/29325589',
'https://www.redfin.com/NV/North-Las-Vegas/908-Stanley-Ave-89030/home/29326787',
'https://www.redfin.com/NV/Las-Vegas/1429-N-Mojave-Rd-89101/home/29331974',
'https://www.redfin.com/NV/Las-Vegas/1324-Arthur-Ave-89101/home/29332001',
'https://www.redfin.com/NV/Las-Vegas/1320-Arthur-Ave-89101/home/29332002',
'https://www.redfin.com/NV/Las-Vegas/3209-Theresa-Ave-89101/home/29333537',
'https://www.redfin.com/NV/Las-Vegas/3300-Marionette-Ave-89101/home/29333603',
'https://www.redfin.com/NV/Las-Vegas/1300-N-22nd-St-89101/home/29334320',
'https://www.redfin.com/NV/Las-Vegas/1613-Constantine-Ave-89101/home/29334450',
'https://www.redfin.com/NV/Las-Vegas/637-N-11th-St-89101/home/29334667',
'https://www.redfin.com/NV/Las-Vegas/2205-Melinda-Ave-89101/home/29335290',
'https://www.redfin.com/NV/Las-Vegas/904-N-24th-St-89101/home/29335525',
'https://www.redfin.com/NV/Las-Vegas/2221-Constantine-Ave-89101/home/29335602',
'https://www.redfin.com/NV/Las-Vegas/633-N-22nd-St-89101/home/29335999',
'https://www.redfin.com/NV/Las-Vegas/1909-E-Wilson-Ave-89101/home/29336178',
'https://www.redfin.com/NV/Las-Vegas/910-N-1st-St-89101/home/29336983',
'https://www.redfin.com/NV/Las-Vegas/317-N-Bruce-St-89101/home/29348188',
'https://www.redfin.com/NV/Las-Vegas/1921-Walnut-Ave-89101/home/29349044',
'https://www.redfin.com/NV/Las-Vegas/140-N-19th-St-89101/home/29349687',
'https://www.redfin.com/NV/Las-Vegas/1708-Poplar-Ave-89101/home/29350043',
'https://www.redfin.com/NV/Las-Vegas/2901-Stewart-Ave-89101/home/29351497',
'https://www.redfin.com/NV/Las-Vegas/4479-Amargosa-Way-89115/home/29357961',
'https://www.redfin.com/NV/Las-Vegas/6005-Casa-Loma-Ave-89156/home/29358825',
'https://www.redfin.com/NV/Las-Vegas/2690-Holy-Cross-Dr-89156/home/29360675',
'https://www.redfin.com/NV/Las-Vegas/2671-Pomona-Dr-89156/home/29360970',
'https://www.redfin.com/NV/Las-Vegas/3749-Carlyle-Dr-89115/unit-60/home/29362825',
'https://www.redfin.com/NV/Las-Vegas/2441-Oak-Glen-Way-89115/home/29363577',
'https://www.redfin.com/NV/Las-Vegas/2770-Lincoln-Rd-89115/home/29364057',
'https://www.redfin.com/NV/Las-Vegas/1830-N-Pecos-Rd-89115/unit-152/home/29364348',
'https://www.redfin.com/NV/Las-Vegas/4059-Alto-Ave-89115/home/29364606',
'https://www.redfin.com/NV/Las-Vegas/4275-Ripple-River-Ave-89115/home/29364835',
'https://www.redfin.com/NV/Las-Vegas/2334-N-Lamb-Blvd-89115/home/29366094',
'https://www.redfin.com/NV/Las-Vegas/1680-Royal-Estates-Dr-89115/home/29366173',
'https://www.redfin.com/NV/Las-Vegas/4200-E-Tonopah-Ave-89115/home/29366366',
'https://www.redfin.com/NV/Las-Vegas/2232-N-Gateway-Rd-89115/home/29366538',
'https://www.redfin.com/NV/Las-Vegas/2026-Castleberry-Ln-89156/home/29368364',
'https://www.redfin.com/NV/Las-Vegas/6201-E-Lake-Mead-Blvd-89156/unit-148/home/29370878',
'https://www.redfin.com/NV/Las-Vegas/6659-Network-Cir-89156/home/29372325',
'https://www.redfin.com/NV/Las-Vegas/6800-E-Lake-Mead-Blvd-89156/unit-2042/home/29373901',
'https://www.redfin.com/NV/Las-Vegas/6800-E-Lake-Mead-Blvd-89156/unit-2031/home/29373930',
'https://www.redfin.com/NV/Las-Vegas/6800-E-Lake-Mead-Blvd-89156/unit-2098/home/29374042',
'https://www.redfin.com/NV/Las-Vegas/7116-Wayfarer-Dr-89156/home/29374341',
'https://www.redfin.com/NV/Las-Vegas/6760-Mountridge-Dr-89110/home/29375298',
'https://www.redfin.com/NV/Las-Vegas/7022-Oakleigh-Dr-89110/home/29375331',
'https://www.redfin.com/NV/Las-Vegas/1880-Balzac-Dr-89156/home/29375415',
'https://www.redfin.com/NV/Las-Vegas/1555-Mount-Hood-St-89110/home/29376480',
'https://www.redfin.com/NV/Las-Vegas/1501-Sunvue-Cir-89110/home/29376567',
'https://www.redfin.com/NV/Las-Vegas/930-N-Sloan-Ln-89110/unit-102/home/29376620',
'https://www.redfin.com/NV/Las-Vegas/6037-Mirror-Lake-Dr-89110/home/29376747',
'https://www.redfin.com/NV/Las-Vegas/870-N-Sloan-Ln-89110/unit-104/home/29377110',
'https://www.redfin.com/NV/Las-Vegas/5485-E-Owens-Ave-89110/home/29378116',
'https://www.redfin.com/NV/Las-Vegas/1265-Lime-Point-St-89110/home/29378352',
'https://www.redfin.com/NV/Las-Vegas/5614-Stingaree-Cir-89110/home/29378659',
'https://www.redfin.com/NV/Las-Vegas/5655-Crown-Point-Ave-89110/home/29378741',
'https://www.redfin.com/NV/Las-Vegas/1259-Linn-Ln-89110/home/29379869',
'https://www.redfin.com/NV/Las-Vegas/4572-Bingham-Ave-89110/home/29382038',
'https://www.redfin.com/NV/Las-Vegas/4953-Sandra-Rd-89110/home/29382588',
'https://www.redfin.com/NV/Las-Vegas/3504-Chad-Cir-89110/home/29383001',
'https://www.redfin.com/NV/Las-Vegas/808-Hedge-Way-89110/unit-2/home/29384435',
'https://www.redfin.com/NV/Las-Vegas/1408-Henry-Dr-89110/home/29384665',
'https://www.redfin.com/NV/Las-Vegas/4221-Caliper-Dr-89110/home/29385749',
'https://www.redfin.com/NV/Las-Vegas/712-Brenner-Way-89110/home/29385791',
'https://www.redfin.com/NV/Las-Vegas/69-Maya-St-89110/home/29387116',
'https://www.redfin.com/NV/Las-Vegas/4301-Keithann-Cir-89110/home/29388560',
'https://www.redfin.com/NV/Las-Vegas/4232-Park-Ct-89110/home/29388635',
'https://www.redfin.com/NV/Las-Vegas/4313-Devlin-Green-Ct-89110/home/29388836',
'https://www.redfin.com/NV/Las-Vegas/97-Lupin-Ct-89110/home/29388876',
'https://www.redfin.com/NV/Las-Vegas/40-Sherrill-Cir-89110/home/29390001',
'https://www.redfin.com/NV/Las-Vegas/136-Romero-Dr-89110/home/29390285',
'https://www.redfin.com/NV/Las-Vegas/139-Romero-Dr-89110/home/29390294',
'https://www.redfin.com/NV/Las-Vegas/4717-Mahogany-Dr-89110/home/29390730',
'https://www.redfin.com/NV/Las-Vegas/4933-Carmichael-Ave-89110/home/29390962',
'https://www.redfin.com/NV/Las-Vegas/11-Kolmar-Ct-89110/home/29391065',
'https://www.redfin.com/NV/Las-Vegas/4901-Ronan-Dr-89110/home/29391435',
'https://www.redfin.com/NV/Las-Vegas/5330-E-Charleston-Blvd-89142/unit-35/home/29392604',
'https://www.redfin.com/NV/Las-Vegas/5236-Sir-James-Way-89110/home/29392863',
'https://www.redfin.com/NV/Las-Vegas/5330-E-Charleston-Blvd-89142/unit-137/home/29393744',
'https://www.redfin.com/NV/Las-Vegas/5832-Sutcliffe-Cir-89110/home/29394277',
'https://www.redfin.com/NV/Las-Vegas/98-Linn-Ln-89110/home/29395317',
'https://www.redfin.com/NV/Las-Vegas/635-N-Fogg-St-89110/home/29395918',
'https://www.redfin.com/NV/Las-Vegas/435-Los-Feliz-St-89110/home/29397048',
'https://www.redfin.com/NV/Las-Vegas/6047-Golden-Nectar-Way-89142/home/29400024',
'https://www.redfin.com/NV/Las-Vegas/1779-Pineapple-Tree-Ct-89142/home/29401327',
'https://www.redfin.com/NV/Las-Vegas/5348-Hollow-Brook-Ave-89142/home/29402995',
'https://www.redfin.com/NV/Las-Vegas/1405-S-Nellis-Blvd-89104/unit-2069/home/29404234',
'https://www.redfin.com/NV/Las-Vegas/1797-Winterwood-Blvd-89142/home/29404257',
'https://www.redfin.com/NV/Las-Vegas/5357-Silverheart-Ave-89142/home/29404359',
'https://www.redfin.com/NV/Las-Vegas/2135-Caravelle-St-89142/home/29404605',
'https://www.redfin.com/NV/Las-Vegas/1125-S-Christy-Ln-89142/home/29405554',
'https://www.redfin.com/NV/Las-Vegas/5684-Ballinger-Dr-89142/home/29406362',
'https://www.redfin.com/NV/Las-Vegas/5519-Myron-Cir-89142/home/29406415',
'https://www.redfin.com/NV/Las-Vegas/4610-Yuma-Ave-89104/home/29409052',
'https://www.redfin.com/NV/Las-Vegas/4808-E-Wyoming-Ave-89104/home/29409375',
'https://www.redfin.com/NV/Las-Vegas/1268-S-Honolulu-St-89104/home/29409880',
'https://www.redfin.com/NV/Las-Vegas/1044-May-Ave-89104/home/29409985',
'https://www.redfin.com/NV/Las-Vegas/1524-Calabria-Dr-89104/home/29410291',
'https://www.redfin.com/NV/Las-Vegas/1405-Nedla-Ct-89104/home/29410495',
'https://www.redfin.com/NV/Las-Vegas/3875-Saint-Mark-Ct-89104/home/29411246',
'https://www.redfin.com/NV/Las-Vegas/601-Greenbriar-Townhouse-Way-89121/home/29411910',
'https://www.redfin.com/NV/Las-Vegas/2511-Stratford-Ave-89121/home/29411989',
'https://www.redfin.com/NV/Las-Vegas/4305-Vegas-Valley-Dr-89121/home/29413136',
'https://www.redfin.com/NV/Las-Vegas/4555-E-Sahara-Ave-89104/unit-247/home/29413438',
'https://www.redfin.com/NV/Las-Vegas/4019-Espanita-Ave-89121/home/29413553',
'https://www.redfin.com/NV/Las-Vegas/2725-S-Nellis-Blvd-89121/unit-2043/home/29414943',
'https://www.redfin.com/NV/Las-Vegas/2654-Bryandouglas-Dr-89121/home/29415122',
'https://www.redfin.com/NV/Las-Vegas/2796-Rungsted-St-89142/home/29415167',
'https://www.redfin.com/NV/Las-Vegas/2725-S-Nellis-Blvd-89121/unit-2067/home/29415351',
'https://www.redfin.com/NV/Las-Vegas/2982-Juniper-Hills-Blvd-89142/unit-102/home/29416258',
'https://www.redfin.com/NV/Las-Vegas/2978-Juniper-Hills-Blvd-89142/unit-203/home/29416275',
'https://www.redfin.com/NV/Las-Vegas/3192-Tularosa-Ln-89122/home/29417418',
'https://www.redfin.com/NV/Las-Vegas/3212-Bellavista-Ln-89122/home/29417447',
'https://www.redfin.com/NV/Las-Vegas/3456-Fort-McHenry-Dr-89122/home/29418496',
'https://www.redfin.com/NV/Las-Vegas/3315-Lost-Hills-Dr-89122/home/29418610',
'https://www.redfin.com/NV/Las-Vegas/3671-Gulf-Shores-Dr-89122/home/29418688',
'https://www.redfin.com/NV/Las-Vegas/3601-Katmai-Dr-89122/home/29418781',
'https://www.redfin.com/NV/Las-Vegas/3520-Estes-Park-Dr-89122/home/29418868',
'https://www.redfin.com/NV/Las-Vegas/3520-Big-Sur-Dr-89122/home/29418938',
'https://www.redfin.com/NV/Las-Vegas/3436-Blue-Ash-Ln-89122/home/29419149',
'https://www.redfin.com/NV/Las-Vegas/3423-Wayne-St-89121/home/29422038',
'https://www.redfin.com/NV/Las-Vegas/3721-Edison-Ave-89121/home/29422706',
'https://www.redfin.com/NV/Las-Vegas/3460-Florrie-Ave-89121/home/29423772',
'https://www.redfin.com/NV/Las-Vegas/1201-Eastwood-Dr-89104/home/29448100',
'https://www.redfin.com/NV/Las-Vegas/1720-Weeping-Willow-Ln-89104/home/29451209',
'https://www.redfin.com/NV/Las-Vegas/2404-Canosa-Ave-89104/home/29451215',
'https://www.redfin.com/NV/Las-Vegas/2029-Bonita-Ave-89104/home/29454363',
'https://www.redfin.com/NV/Las-Vegas/1601-S-6th-St-89104/home/29455988',
'https://www.redfin.com/NV/Las-Vegas/1047-Franklin-Ave-89104/home/29456099',
'https://www.redfin.com/NV/Las-Vegas/1405-Vegas-Valley-Dr-89169/unit-179/home/29467429',
'https://www.redfin.com/NV/Las-Vegas/2080-Karen-Ave-89169/unit-20/home/29468331',
'https://www.redfin.com/NV/Las-Vegas/1430-E-Desert-Inn-Rd-89169/home/29468473',
'https://www.redfin.com/NV/Las-Vegas/1405-Vegas-Valley-Dr-89169/unit-306/home/29468771',
'https://www.redfin.com/NV/Las-Vegas/2571-Boise-St-89121/home/29471085',
'https://www.redfin.com/NV/Las-Vegas/2677-Ruthe-Duarte-Ave-89121/home/29471763',
'https://www.redfin.com/NV/Las-Vegas/3320-Bridgeport-Dr-89121/home/29472702',
'https://www.redfin.com/NV/Las-Vegas/3641-Laguna-Verde-Way-89121/home/29473944',
'https://www.redfin.com/NV/Las-Vegas/3457-Clandara-Ave-89121/home/29474574',
'https://www.redfin.com/NV/Las-Vegas/3385-Chikasaw-Way-89169/home/29475983',
'https://www.redfin.com/NV/Las-Vegas/3762-Halter-Dr-89122/home/29482144',
'https://www.redfin.com/NV/Las-Vegas/745-N-Royal-Crest-Cir-89169/unit-117/home/29496813',
'https://www.redfin.com/NV/Las-Vegas/3718-Hazelwood-St-89119/unit-19/home/29499922',
'https://www.redfin.com/NV/Las-Vegas/200-Hoover-Ave-89101/unit-904/home/29792159',
'https://www.redfin.com/NV/Las-Vegas/150-Las-Vegas-Blvd-N-89101/unit-1002/home/29812937',
'https://www.redfin.com/NV/Las-Vegas/150-Las-Vegas-Blvd-N-89101/unit-2007/home/29813721',
'https://www.redfin.com/NV/Las-Vegas/353-E-Bonneville-Ave-89101/unit-371/home/29815140',
'https://www.redfin.com/NV/Las-Vegas/353-E-Bonneville-Ave-89101/unit-1404/home/29816649',
'https://www.redfin.com/NV/Las-Vegas/353-E-Bonneville-Ave-89101/unit-613/home/29816826',
'https://www.redfin.com/NV/Las-Vegas/6295-Stone-Hollow-Ave-89156/home/29862635',
'https://www.redfin.com/NV/Las-Vegas/5650-E-Sahara-Ave-89142/unit-2046/home/29864596',
'https://www.redfin.com/NV/Las-Vegas/2126-Betty-Davis-St-89142/home/29864749',
'https://www.redfin.com/NV/Las-Vegas/1370-Morning-Sun-Way-89110/home/33529947',
'https://www.redfin.com/NV/Las-Vegas/1721-S-Eastern-Ave-89104/home/33554711',
'https://www.redfin.com/NV/Las-Vegas/1801-E-Fremont-St-89101/home/33562724',
'https://www.redfin.com/NV/Las-Vegas/6072-Corbin-Ave-89122/home/35656316',
'https://www.redfin.com/NV/Las-Vegas/2903-Gables-Vale-Ct-89121/home/108827918',
'https://www.redfin.com/NV/Las-Vegas/6788-Lavender-Sky-Ct-89142/home/109872198',
'https://www.redfin.com/NV/Las-Vegas/Unknown-Unknown/home/166120807',
'https://www.redfin.com/NV/Las-Vegas/2459-Silk-Tree-Mimosa-St-89115/home/169230409',
'https://www.redfin.com/NV/Las-Vegas/1811-Lincoln-Rd-89115/home/178669314',
'https://www.redfin.com/NV/Las-Vegas/2751-Ariel-Ocean-St-89156/home/183043823',
'https://www.redfin.com/NV/Las-Vegas/2739-Ariel-Ocean-St-89156/home/183043825',
'https://www.redfin.com/NV/Las-Vegas/2733-Ariel-Ocean-St-89156/home/183043826',
'https://www.redfin.com/NV/Las-Vegas/2727-Ariel-Ocean-St-89156/home/183043827',
'https://www.redfin.com/NV/Sunrise-Manor/6790-Globemaster-Ct-89032/home/183043881',
'https://www.redfin.com/NV/Sunrise-Manor/6786-Globemaster-Ct-89032/home/183043882',
'https://www.redfin.com/NV/Sunrise-Manor/6778-Globemaster-Ct-89032/home/183043884',
'https://www.redfin.com/NV/Las-Vegas/6777-Stratofortress-Ct-89156/home/183043886',
'https://www.redfin.com/NV/Las-Vegas/6789-Stratofortress-Ct-89156/home/183043889',
'https://www.redfin.com/NV/Las-Vegas/6784-Stratofortress-Ct-89156/home/183043895',
'https://www.redfin.com/NV/Las-Vegas/4005-Welter-Ave-89104/home/183049633',
'https://www.redfin.com/NV/Las-Vegas/3508-Sunrise-Ave-89110/home/184511081',
'https://www.redfin.com/NV/Las-Vegas/Unknown-89115/home/187463012',
'https://www.redfin.com/NV/Las-Vegas/5924-Sleepy-Fawn-Dr-89142/home/28988172',
'https://www.redfin.com/NV/Las-Vegas/421-Wyatt-Ave-89106/home/29325509',
'https://www.redfin.com/NV/Las-Vegas/2629-Jansen-Ave-89101/home/29332480',
'https://www.redfin.com/NV/Las-Vegas/3470-N-Pecos-Rd-89115/home/29354807',
'https://www.redfin.com/NV/Las-Vegas/3471-Purdue-Way-89115/home/29354837',
'https://www.redfin.com/NV/Las-Vegas/4508-Crystal-Peak-Dr-89115/home/29357770',
'https://www.redfin.com/NV/Las-Vegas/3415-Goldyke-St-89115/home/29357928',
'https://www.redfin.com/NV/Las-Vegas/2416-Cash-Ct-89156/home/29361608',
'https://www.redfin.com/NV/Las-Vegas/1748-Nutmeg-Ln-89115/home/29365098',
'https://www.redfin.com/NV/Las-Vegas/1700-N-Walnut-Rd-89115/home/29366244',
'https://www.redfin.com/NV/Las-Vegas/1833-Henson-Ln-89156/home/29368680',
'https://www.redfin.com/NV/Las-Vegas/1692-Eddingham-Ct-89156/home/29371575',
'https://www.redfin.com/NV/Las-Vegas/6247-Brookings-Ct-89110/home/29377426',
'https://www.redfin.com/NV/Las-Vegas/1561-Linnbaker-Ln-89110/unit-104/home/29384710',
'https://www.redfin.com/NV/Las-Vegas/4241-Brockton-Green-Ct-89110/home/29388823',
'https://www.redfin.com/NV/Las-Vegas/408-Agora-Ct-89110/home/29389305',
'https://www.redfin.com/NV/Las-Vegas/11-Hickory-St-89110/home/29391170',
'https://www.redfin.com/NV/Las-Vegas/5175-E-Bonanza-Rd-89110/home/29392218',
'https://www.redfin.com/NV/Las-Vegas/5330-E-Charleston-Blvd-89142/unit-102/home/29392470',
'https://www.redfin.com/NV/Las-Vegas/5762-Deep-Sea-Ct-89110/home/29393450',
'https://www.redfin.com/NV/Las-Vegas/2132-Seahurst-Dr-89142/home/29406611',
'https://www.redfin.com/NV/Las-Vegas/4461-Sun-Vista-Dr-89104/home/29407852',
'https://www.redfin.com/NV/Las-Vegas/1325-Arlee-Ct-89104/home/29410466',
'https://www.redfin.com/NV/Las-Vegas/4075-Perdido-Ave-89121/home/29413618',
'https://www.redfin.com/NV/Las-Vegas/3028-Gardendale-St-89121/home/29414013',
'https://www.redfin.com/NV/Las-Vegas/2020-Bonita-Ave-89104/home/29454382',
'https://www.redfin.com/NV/Las-Vegas/2839-Boise-St-89121/home/29471112',
'https://www.redfin.com/NV/Las-Vegas/3080-Garnet-Ct-89121/home/29471354',
'https://www.redfin.com/NV/Las-Vegas/2611-Bolero-Ave-89121/home/29471590',
'https://www.redfin.com/NV/Las-Vegas/3400-Cabana-Dr-89122/unit-1024/home/29865101',
'https://www.redfin.com/NV/Las-Vegas/5551-Cresent-Valley-St-89148/home/28964764',
'https://www.redfin.com/NV/Las-Vegas/3658-Moonlit-Rain-Dr-89135/home/28967870',
'https://www.redfin.com/NV/Las-Vegas/5973-Buff-Bay-St-89148/home/28973422',
'https://www.redfin.com/NV/Las-Vegas/11225-Sweet-Cicely-Ave-89138/home/28973945',
'https://www.redfin.com/NV/Las-Vegas/10517-Premia-Pl-89135/home/29000176',
'https://www.redfin.com/NV/Las-Vegas/5437-Progresso-St-89135/home/29009037',
'https://www.redfin.com/NV/Las-Vegas/4310-Sandy-River-Dr-89103/unit-54/home/29030262',
'https://www.redfin.com/NV/Las-Vegas/8142-Watauga-Ave-89147/home/29030343',
'https://www.redfin.com/NV/Las-Vegas/8171-Sailor-Point-Ave-89147/home/29030641',
'https://www.redfin.com/NV/Las-Vegas/4921-River-Glen-Dr-89103/unit-20/home/29030691',
'https://www.redfin.com/NV/Las-Vegas/5350-River-Glen-Dr-89103/unit-312/home/29030768',
'https://www.redfin.com/NV/Las-Vegas/4420-Sandy-River-Dr-89103/unit-37/home/29031250',
'https://www.redfin.com/NV/Las-Vegas/5097-Indian-River-Dr-89103/unit-172/home/29031377',
'https://www.redfin.com/NV/Las-Vegas/5140-Indian-River-Dr-89103/unit-307/home/29031461',
'https://www.redfin.com/NV/Las-Vegas/5020-Indian-River-Dr-89103/unit-425/home/29031530',
'https://www.redfin.com/NV/Las-Vegas/5383-Indian-River-Dr-89103/unit-328/home/29031880',
'https://www.redfin.com/NV/Las-Vegas/9580-W-Reno-Ave-89148/unit-205/home/29032103',
'https://www.redfin.com/NV/Las-Vegas/9708-Maple-Sugar-Leaf-Pl-89148/home/29032458',
'https://www.redfin.com/NV/Las-Vegas/9820-Hanover-Grove-Ave-89148/home/29032966',
'https://www.redfin.com/NV/Las-Vegas/10385-Brillare-Ave-89135/home/29033385',
'https://www.redfin.com/NV/Las-Vegas/11321-Corsica-Mist-Ave-89135/home/29033505',
'https://www.redfin.com/NV/Las-Vegas/5260-Mission-Monterey-Ln-89107/unit-202/home/29051933',
'https://www.redfin.com/NV/Las-Vegas/230-Mission-Catalina-Ln-89107/unit-107/home/29052116',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-2618/home/29053249',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-2708/home/29053270',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-3209/home/29053654',
'https://www.redfin.com/NV/Las-Vegas/10001-Peace-Way-89147/unit-1191/home/29053801',
'https://www.redfin.com/NV/Las-Vegas/10001-Peace-Way-89147/unit-2302/home/29053984',
'https://www.redfin.com/NV/Las-Vegas/5392-Candlespice-Way-89135/home/29054268',
'https://www.redfin.com/NV/Las-Vegas/5443-Bristol-Bend-Ct-89135/home/29054284',
'https://www.redfin.com/NV/Las-Vegas/5430-Bristol-Bend-Ct-89135/home/29054290',
'https://www.redfin.com/NV/Las-Vegas/2420-Cordoba-Bluff-Ct-89135/home/29054366',
'https://www.redfin.com/NV/Las-Vegas/2777-Paradise-Rd-89109/unit-1404/home/29063301',
'https://www.redfin.com/NV/Las-Vegas/2777-Paradise-Rd-89109/unit-2301/home/29063447',
'https://www.redfin.com/NV/Las-Vegas/230-E-Flamingo-Rd-89169/unit-308/home/29064393',
'https://www.redfin.com/NV/Las-Vegas/230-E-Flamingo-Rd-89169/unit-422/home/29064409',
'https://www.redfin.com/NV/Las-Vegas/270-E-Flamingo-Rd-89169/unit-215/home/29064574',
'https://www.redfin.com/NV/Las-Vegas/270-E-Flamingo-Rd-89169/unit-232/home/29064590',
'https://www.redfin.com/NV/Las-Vegas/5030-S-Rainbow-Blvd-89118/unit-201/home/29064796',
'https://www.redfin.com/NV/Las-Vegas/5016-S-Rainbow-Blvd-89118/unit-106/home/29064971',
'https://www.redfin.com/NV/Las-Vegas/5076-S-Rainbow-Blvd-89118/unit-206/home/29065065',
'https://www.redfin.com/NV/Las-Vegas/10519-Ardsley-Ln-89135/home/29065271',
'https://www.redfin.com/NV/Las-Vegas/5457-Nettle-Way-89135/home/29065578',
'https://www.redfin.com/NV/Las-Vegas/208-Swale-Ln-89144/home/29113496',
'https://www.redfin.com/NV/Las-Vegas/10620-Esk-Dr-89144/home/29113569',
'https://www.redfin.com/NV/Las-Vegas/10600-Amber-Ridge-Dr-89144/unit-104/home/29114759',
'https://www.redfin.com/NV/Las-Vegas/10600-Amber-Ridge-Dr-89144/unit-203/home/29114787',
'https://www.redfin.com/NV/Las-Vegas/10701-Pappas-Ln-89144/unit-202/home/29114820',
'https://www.redfin.com/NV/Las-Vegas/10207-Birch-Bluff-Ln-89145/home/29115107',
'https://www.redfin.com/NV/Las-Vegas/1013-Jennis-Silver-St-89145/home/29212457',
'https://www.redfin.com/NV/Las-Vegas/905-Old-Woostra-Ct-89145/home/29212558',
'https://www.redfin.com/NV/Las-Vegas/9721-Winter-Palace-Dr-89145/home/29212705',
'https://www.redfin.com/NV/Las-Vegas/1005-Bannockburn-St-89145/home/29213467',
'https://www.redfin.com/NV/Las-Vegas/216-Moose-Ln-89145/home/29214412',
'https://www.redfin.com/NV/Las-Vegas/225-Moose-Ln-89145/home/29214439',
'https://www.redfin.com/NV/Las-Vegas/8649-Inwood-Dr-89145/home/29214731',
'https://www.redfin.com/NV/Las-Vegas/8628-Highacre-Dr-89145/home/29214851',
'https://www.redfin.com/NV/Las-Vegas/900-Heavenly-Hills-Ct-89145/unit-220/home/29218728',
'https://www.redfin.com/NV/Las-Vegas/900-Heavenly-Hills-Ct-89145/unit-113/home/29218750',
'https://www.redfin.com/NV/Las-Vegas/328-S-Buffalo-Dr-89145/unit-102/home/29219877',
'https://www.redfin.com/NV/Las-Vegas/324-S-Buffalo-Dr-89145/unit-202/home/29219897',
'https://www.redfin.com/NV/Las-Vegas/7905-Lago-Vista-Ln-89145/unit-103/home/29220454',
'https://www.redfin.com/NV/Las-Vegas/725-Yacht-Harbor-Dr-89145/home/29220529',
'https://www.redfin.com/NV/Las-Vegas/740-Hunters-Creek-Ln-89145/unit-103/home/29220573',
'https://www.redfin.com/NV/Las-Vegas/7901-Idledale-Ct-89145/unit-102/home/29220607',
'https://www.redfin.com/NV/Las-Vegas/512-Yacht-Harbor-Dr-89145/unit-204/home/29220683',
'https://www.redfin.com/NV/Las-Vegas/7912-Foxwood-Pl-89145/unit-201/home/29220699',
'https://www.redfin.com/NV/Las-Vegas/7408-Alta-Dr-89145/home/29222530',
'https://www.redfin.com/NV/Las-Vegas/401-Pinecliff-Dr-89145/home/29223393',
'https://www.redfin.com/NV/Las-Vegas/6833-Sweetwater-Pl-89145/home/29225555',
'https://www.redfin.com/NV/Las-Vegas/6805-Midpark-Cir-89145/home/29225634',
'https://www.redfin.com/NV/Las-Vegas/6945-Surrey-Ct-89145/home/29225670',
'https://www.redfin.com/NV/Las-Vegas/808-Vincent-Way-89145/home/29226471',
'https://www.redfin.com/NV/Las-Vegas/6948-Marseilles-Cir-89145/home/29226556',
'https://www.redfin.com/NV/Las-Vegas/6716-Theus-Cir-89107/home/29226725',
'https://www.redfin.com/NV/Las-Vegas/6901-Alta-Dr-89145/home/29226798',
'https://www.redfin.com/NV/Las-Vegas/6612-Bristol-Way-89107/home/29227949',
'https://www.redfin.com/NV/Las-Vegas/6121-Borden-Cir-89107/home/29229939',
'https://www.redfin.com/NV/Las-Vegas/6329-Arlington-Ave-89107/home/29230332',
'https://www.redfin.com/NV/Las-Vegas/5509-Flower-Cir-89107/home/29233001',
'https://www.redfin.com/NV/Las-Vegas/413-Falcon-Ln-89107/home/29233119',
'https://www.redfin.com/NV/Las-Vegas/5321-Harmony-Ave-89107/home/29233939',
'https://www.redfin.com/NV/Las-Vegas/1020-Baronet-Dr-89138/home/29277281',
'https://www.redfin.com/NV/Las-Vegas/8600-W-Charleston-Blvd-89117/unit-2118/home/29279338',
'https://www.redfin.com/NV/Las-Vegas/8600-W-Charleston-Blvd-89117/unit-1157/home/29279381',
'https://www.redfin.com/NV/Las-Vegas/8600-W-Charleston-Blvd-89117/unit-1081/home/29279479',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-802/home/29283547',
'https://www.redfin.com/NV/Las-Vegas/4200-S-Valley-View-Blvd-89103/unit-3045/home/29283900',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-3220/home/29285159',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-2720/home/29285239',
'https://www.redfin.com/NV/Las-Vegas/3651-Arville-St-89103/unit-437/home/29315344',
'https://www.redfin.com/NV/Las-Vegas/3651-Arville-St-89103/unit-704/home/29317076',
'https://www.redfin.com/NV/Las-Vegas/5250-S-Rainbow-Blvd-89118/unit-1041/home/29319346',
'https://www.redfin.com/NV/Las-Vegas/5250-S-Rainbow-Blvd-89118/unit-2182/home/29319822',
'https://www.redfin.com/NV/Las-Vegas/4624-Fulton-Pl-89107/home/29343669',
'https://www.redfin.com/NV/Las-Vegas/413-Park-Way-E-89106/home/29346338',
'https://www.redfin.com/NV/Las-Vegas/331-S-Tonopah-Dr-89106/home/29346656',
'https://www.redfin.com/NV/Las-Vegas/1407-Westwood-Dr-89102/home/29457236',
'https://www.redfin.com/NV/Las-Vegas/3013-Colanthe-Ave-89102/home/29458458',
'https://www.redfin.com/NV/Las-Vegas/3009-Plaza-de-Monte-89102/home/29459981',
'https://www.redfin.com/NV/Las-Vegas/2217-Juana-Vista-St-89102/home/29461946',
'https://www.redfin.com/NV/Las-Vegas/4309-Esmeralda-Ave-89102/home/29462262',
'https://www.redfin.com/NV/Las-Vegas/4415-Del-Oro-Dr-89102/home/29463071',
'https://www.redfin.com/NV/Las-Vegas/2851-S-Valley-View-Blvd-89102/unit-1081/home/29463517',
'https://www.redfin.com/NV/Las-Vegas/2851-S-Valley-View-Blvd-89102/unit-1072/home/29463532',
'https://www.redfin.com/NV/Las-Vegas/3009-Wilmington-Way-89102/home/29465274',
'https://www.redfin.com/NV/Las-Vegas/125-E-Harmon-Ave-89109/unit-3214/home/29485364',
'https://www.redfin.com/NV/Las-Vegas/125-E-Harmon-Ave-89109/unit-2205/home/29485464',
'https://www.redfin.com/NV/Las-Vegas/2200-S-Fort-Apache-Rd-89117/unit-2171/home/29486705',
'https://www.redfin.com/NV/Las-Vegas/2200-S-Fort-Apache-Rd-89117/unit-2160/home/29486725',
'https://www.redfin.com/NV/Las-Vegas/2200-S-Fort-Apache-Rd-89117/unit-2083/home/29486836',
'https://www.redfin.com/NV/Las-Vegas/211-E-Flamingo-Rd-89169/unit-714/home/29486962',
'https://www.redfin.com/NV/Las-Vegas/6067-Honeysuckle-Ridge-St-89148/home/29488373',
'https://www.redfin.com/NV/Las-Vegas/3609-Melonies-Dr-89103/unit-95/home/29502130',
'https://www.redfin.com/NV/Las-Vegas/4210-Gannet-Cir-89103/unit-289/home/29502766',
'https://www.redfin.com/NV/Las-Vegas/797-E-Harmon-Ave-89119/unit-30/home/29504871',
'https://www.redfin.com/NV/Las-Vegas/1800-Edmond-St-89146/unit-132/home/29518363',
'https://www.redfin.com/NV/Las-Vegas/1860-Ski-Slope-Cir-89117/home/29522124',
'https://www.redfin.com/NV/Las-Vegas/8900-Borla-Dr-89117/home/29523515',
'https://www.redfin.com/NV/Las-Vegas/8700-Cremona-Dr-89117/home/29523797',
'https://www.redfin.com/NV/Las-Vegas/8812-Rainbow-Ridge-Dr-89117/home/29524014',
'https://www.redfin.com/NV/Las-Vegas/1624-British-Cup-Dr-89117/home/29527135',
'https://www.redfin.com/NV/Las-Vegas/9565-World-Cup-Dr-89117/home/29527159',
'https://www.redfin.com/NV/Las-Vegas/2705-Sattley-Cir-89117/home/29527675',
'https://www.redfin.com/NV/Las-Vegas/2817-Cotton-Cloud-Rd-89117/home/29528060',
'https://www.redfin.com/NV/Las-Vegas/2833-Edge-Rock-Cir-89117/home/29528210',
'https://www.redfin.com/NV/Las-Vegas/9829-Russett-Wood-Cir-89117/home/29529169',
'https://www.redfin.com/NV/Las-Vegas/2720-Tidewater-Ct-89117/home/29530994',
'https://www.redfin.com/NV/Las-Vegas/8932-Antioch-Way-89117/home/29531089',
'https://www.redfin.com/NV/Las-Vegas/3001-Beach-View-Ct-89117/home/29532625',
'https://www.redfin.com/NV/Las-Vegas/9009-Cape-Wood-Ct-89117/home/29532715',
'https://www.redfin.com/NV/Las-Vegas/3305-Grey-Dolphin-Dr-89117/home/29533177',
'https://www.redfin.com/NV/Las-Vegas/2750-S-Durango-Dr-89117/unit-1155/home/29535177',
'https://www.redfin.com/NV/Las-Vegas/8273-Ruby-Heights-Ave-89117/home/29535935',
'https://www.redfin.com/NV/Las-Vegas/7889-Mosaic-Harbor-Ave-89117/home/29536148',
'https://www.redfin.com/NV/Las-Vegas/8042-Earl-Grey-Ct-89117/home/29536662',
'https://www.redfin.com/NV/Las-Vegas/7225-Latour-Ct-89117/home/29538072',
'https://www.redfin.com/NV/Las-Vegas/6878-Edna-Ave-89146/home/29538264',
'https://www.redfin.com/NV/Las-Vegas/3150-S-Bronco-St-89146/home/29538548',
'https://www.redfin.com/NV/Las-Vegas/3858-Moongate-Cir-89103/home/29540882',
'https://www.redfin.com/NV/Las-Vegas/3797-Topawa-St-89103/home/29542434',
'https://www.redfin.com/NV/Las-Vegas/5280-Cabrito-Dr-89103/home/29542598',
'https://www.redfin.com/NV/Las-Vegas/6783-Baile-Rd-89146/home/29543729',
'https://www.redfin.com/NV/Las-Vegas/6576-Casa-Linda-Dr-89103/home/29543930',
'https://www.redfin.com/NV/Las-Vegas/6476-Scotch-Pine-Cir-89146/home/29544986',
'https://www.redfin.com/NV/Las-Vegas/6116-Meadowhaven-Ln-89103/home/29545621',
'https://www.redfin.com/NV/Las-Vegas/7400-W-Flamingo-Rd-89147/unit-2089/home/29546745',
'https://www.redfin.com/NV/Las-Vegas/3869-Bombastic-Ct-89147/home/29550220',
'https://www.redfin.com/NV/Las-Vegas/3519-Judah-Way-89147/home/29552180',
'https://www.redfin.com/NV/Las-Vegas/10015-Cambridge-Blue-Ave-89147/home/29555165',
'https://www.redfin.com/NV/Las-Vegas/9618-Windborne-Ave-89147/home/29555769',
'https://www.redfin.com/NV/Las-Vegas/3942-Starfield-Ln-89147/home/29557501',
'https://www.redfin.com/NV/Las-Vegas/8767-Crystal-Port-Ave-89147/home/29560610',
'https://www.redfin.com/NV/Las-Vegas/8501-W-University-Ave-89147/unit-2110/home/29560997',
'https://www.redfin.com/NV/Las-Vegas/7023-Forest-Vista-St-89147/home/29568085',
'https://www.redfin.com/NV/Las-Vegas/4383-Pineaire-St-89147/home/29568106',
'https://www.redfin.com/NV/Las-Vegas/7082-Acorn-Ct-89147/home/29568675',
'https://www.redfin.com/NV/Las-Vegas/7241-Girard-Dr-89147/home/29568703',
'https://www.redfin.com/NV/Las-Vegas/4749-Hackamore-Dr-89103/home/29570629',
'https://www.redfin.com/NV/Las-Vegas/6621-Shelter-Ln-89103/home/29570647',
'https://www.redfin.com/NV/Las-Vegas/6734-Wayburn-Way-89103/home/29570841',
'https://www.redfin.com/NV/Las-Vegas/6277-Fairbanks-Rd-89103/home/29571823',
'https://www.redfin.com/NV/Las-Vegas/4453-Ridgeville-St-89103/home/29572204',
'https://www.redfin.com/NV/Las-Vegas/6490-Gunderson-Blvd-89103/home/29572575',
'https://www.redfin.com/NV/Las-Vegas/6233-Bullion-Blvd-89103/home/29573247',
'https://www.redfin.com/NV/Las-Vegas/4659-Royal-Ridge-Blvd-89103/home/29575030',
'https://www.redfin.com/NV/Las-Vegas/4624-E-Royal-Club-Way-89103/home/29575387',
'https://www.redfin.com/NV/Las-Vegas/5237-Wave-Dancer-Ln-89118/home/29576578',
'https://www.redfin.com/NV/Las-Vegas/5226-Souvenir-Ln-89118/home/29576591',
'https://www.redfin.com/NV/Las-Vegas/5251-Lindell-Rd-89118/unit-205/home/29576887',
'https://www.redfin.com/NV/Las-Vegas/5263-Tropical-Peach-Dr-89118/home/29577204',
'https://www.redfin.com/NV/Las-Vegas/5330-Comstock-Lode-Ln-89118/home/29577440',
'https://www.redfin.com/NV/Las-Vegas/5756-Chisolm-Trl-89118/home/29577500',
'https://www.redfin.com/NV/Las-Vegas/5155-W-Tropicana-Ave-89103/unit-2175/home/29578267',
'https://www.redfin.com/NV/Las-Vegas/5229-Shasta-Lake-Way-89118/home/29579697',
'https://www.redfin.com/NV/Las-Vegas/5320-Sharon-Marie-Ct-89118/home/29579758',
'https://www.redfin.com/NV/Las-Vegas/6659-W-Tropicana-Ave-89103/unit-203/home/29579847',
'https://www.redfin.com/NV/Las-Vegas/6282-La-Palma-Pkwy-89118/home/29580121',
'https://www.redfin.com/NV/Las-Vegas/4914-Mission-Bay-Dr-89113/home/29580735',
'https://www.redfin.com/NV/Las-Vegas/6370-W-Dewey-Dr-89118/home/29580794',
'https://www.redfin.com/NV/Las-Vegas/7715-Top-Hat-Ave-89113/home/29581565',
'https://www.redfin.com/NV/Las-Vegas/5053-Mount-Pleasant-Ln-89113/home/29582195',
'https://www.redfin.com/NV/Las-Vegas/8271-Turtle-Creek-Cir-89113/home/29583121',
'https://www.redfin.com/NV/Las-Vegas/7825-Rancho-Mirage-Dr-89113/home/29583175',
'https://www.redfin.com/NV/Las-Vegas/5517-Morning-Swim-Ln-89113/home/29583970',
'https://www.redfin.com/NV/Las-Vegas/8555-W-Russell-Rd-89113/unit-1107/home/29584156',
'https://www.redfin.com/NV/Las-Vegas/7292-Purple-Shadow-Ave-89113/home/29585915',
'https://www.redfin.com/NV/Las-Vegas/10550-Cedar-Run-Ct-89135/home/29587749',
'https://www.redfin.com/NV/Las-Vegas/2890-Painted-Lilly-Dr-89135/home/29588864',
'https://www.redfin.com/NV/Las-Vegas/3249-River-Glorious-Ln-89135/home/29589401',
'https://www.redfin.com/NV/Las-Vegas/2700-Las-Vegas-Blvd-S-89109/unit-809/home/29784618',
'https://www.redfin.com/NV/Las-Vegas/2700-Las-Vegas-Blvd-S-89109/unit-505/home/29784676',
'https://www.redfin.com/NV/Las-Vegas/4381-W-Flamingo-Rd-89103/unit-5016/home/29789958',
'https://www.redfin.com/NV/Las-Vegas/4575-Dean-Martin-Dr-89103/unit-1407/home/29791283',
'https://www.redfin.com/NV/Las-Vegas/4575-Dean-Martin-Dr-89103/unit-3011/home/29792269',
'https://www.redfin.com/NV/Las-Vegas/4575-Dean-Martin-Dr-89103/unit-2107/home/29792288',
'https://www.redfin.com/NV/Las-Vegas/4471-Dean-Martin-Dr-89103/unit-801/home/29814037',
'https://www.redfin.com/NV/Las-Vegas/4471-Dean-Martin-Dr-89103/unit-3508/home/29815118',
'https://www.redfin.com/NV/Las-Vegas/8082-Planting-Fields-Pl-89117/home/29828876',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-2024/home/29829615',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-1068/home/29831069',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-1214/home/29831159',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-2041/home/29831278',
'https://www.redfin.com/NV/Las-Vegas/9895-Shadycrest-Ct-89148/home/29831330',
'https://www.redfin.com/NV/Las-Vegas/4424-San-Cascina-St-89135/home/29833228',
'https://www.redfin.com/NV/Las-Vegas/3050-Harbor-Heights-Dr-89117/home/29866688',
'https://www.redfin.com/NV/Las-Vegas/10094-San-Gervasio-Ave-89147/home/29867789',
'https://www.redfin.com/NV/Las-Vegas/4661-Altina-St-89147/home/29867814',
'https://www.redfin.com/NV/Las-Vegas/9907-Ridge-Manor-Ave-89148/home/29868756',
'https://www.redfin.com/NV/Las-Vegas/9778-W-Mesa-Vista-Ave-89148/home/29869423',
'https://www.redfin.com/NV/Las-Vegas/5902-Bellerive-St-89113/home/29870407',
'https://www.redfin.com/NV/Las-Vegas/8555-W-Russell-Rd-89113/unit-2057/home/29870560',
'https://www.redfin.com/NV/Las-Vegas/3562-Red-Fir-St-89135/home/29872337',
'https://www.redfin.com/NV/Las-Vegas/9674-Geiger-Peak-Ct-89148/home/29889880',
'https://www.redfin.com/NV/Las-Vegas/3143-Elk-Clover-St-89135/home/29896888',
'https://www.redfin.com/NV/Las-Vegas/10478-Melodia-Magico-Ave-89135/home/29897809',
'https://www.redfin.com/NV/Las-Vegas/10833-Silver-Lace-Ln-89135/home/29897933',
'https://www.redfin.com/NV/Las-Vegas/10267-Early-Morning-Ave-89135/home/29898052',
'https://www.redfin.com/NV/Las-Vegas/9578-Adobe-Arch-Ct-89148/home/29907188',
'https://www.redfin.com/NV/Las-Vegas/1659-Boundary-Peak-Way-89135/home/29908170',
'https://www.redfin.com/NV/Las-Vegas/1560-Boundary-Peak-Way-89135/home/29908374',
'https://www.redfin.com/NV/Las-Vegas/4530-Regalo-Bello-St-89135/home/29911988',
'https://www.redfin.com/NV/Las-Vegas/11072-Village-Ridge-Ln-89135/home/29916955',
'https://www.redfin.com/NV/Las-Vegas/10525-Bambola-Pl-89135/home/29917797',
'https://www.redfin.com/NV/Las-Vegas/35-Panorama-Crest-Ave-89135/home/29920056',
'https://www.redfin.com/NV/Las-Vegas/9495-Perennial-View-Ave-89148/home/29921080',
'https://www.redfin.com/NV/Las-Vegas/400-Rancho-La-Costa-St-89138/home/29931219',
'https://www.redfin.com/NV/Las-Vegas/3552-Ridge-Meadow-St-89135/home/29937336',
'https://www.redfin.com/NV/Las-Vegas/3750-Las-Vegas-Blvd-S-89158/unit-2406/home/30523235',
'https://www.redfin.com/NV/Las-Vegas/3726-Las-Vegas-Blvd-S-89158/unit-610/home/30534132',
'https://www.redfin.com/NV/Las-Vegas/3722-Las-Vegas-Blvd-S-89158/unit-1408/home/30534194',
'https://www.redfin.com/NV/Las-Vegas/10608-Tranquil-Glade-Ln-89135/home/35654738',
'https://www.redfin.com/NV/Las-Vegas/1870-Solvang-Mill-Dr-89135/home/40202891',
'https://www.redfin.com/NV/Las-Vegas/5623-Shadow-Bend-Dr-89135/home/49748897',
'https://www.redfin.com/NV/Las-Vegas/11362-Belmont-Lake-Dr-89135/unit-104/home/52622258',
'https://www.redfin.com/NV/Las-Vegas/8271-Buffalo-Ranch-Ave-89147/home/65557963',
'https://www.redfin.com/NV/Las-Vegas/8775-Cerrito-Canyon-Ct-89148/home/95953513',
'https://www.redfin.com/NV/Las-Vegas/81-Pristine-Glen-St-89135/home/101391854',
'https://www.redfin.com/NV/Las-Vegas/7714-Pyrenees-Park-Dr-89113/home/113035622',
'https://www.redfin.com/NV/Las-Vegas/8949-Brockhampton-Ct-89148/home/113713701',
'https://www.redfin.com/NV/Las-Vegas/5581-Cornadelo-Range-Rd-89113/home/144164027',
'https://www.redfin.com/NV/Las-Vegas/6474-Estrella-Hills-Ave-89118/home/144625082',
'https://www.redfin.com/NV/Las-Vegas/6360-Clovis-Point-St-89135/home/145294574',
'https://www.redfin.com/NV/Las-Vegas/11237-Rainbow-Peak-Ave-89135/unit-211/home/145873183',
'https://www.redfin.com/NV/Las-Vegas/11256-Rainbow-Peak-Ave-89135/unit-201/home/145873208',
'https://www.redfin.com/NV/Las-Vegas/5932-Wellington-Peak-Rd-89148/home/146417699',
'https://www.redfin.com/NV/Las-Vegas/4253-Sunrise-Flats-St-89135/home/146418250',
'https://www.redfin.com/NV/Las-Vegas/11436-Opal-Springs-Way-89135/home/160595682',
'https://www.redfin.com/NV/Las-Vegas/6216-Haiku-Stairs-St-89148/home/165915999',
'https://www.redfin.com/NV/Las-Vegas/4980-Vincitor-St-89135/home/28989513',
'https://www.redfin.com/NV/Las-Vegas/5619-Deer-Creek-Falls-Ct-89118/home/29031783',
'https://www.redfin.com/NV/Las-Vegas/6362-Whispering-Creek-St-89148/home/29032719',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-2711/home/29053260',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-2509/home/29053291',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-1802/home/29053372',
'https://www.redfin.com/NV/Las-Vegas/260-E-Flamingo-Rd-89169/unit-122/home/29064378',
'https://www.redfin.com/NV/Las-Vegas/5471-Indian-Cedar-Dr-89135/home/29065528',
'https://www.redfin.com/NV/Las-Vegas/10417-Orkiney-Dr-89144/home/29115187',
'https://www.redfin.com/NV/Las-Vegas/1021-Collingtree-St-89145/home/29213440',
'https://www.redfin.com/NV/Las-Vegas/7901-Aplin-Ave-89145/home/29218482',
'https://www.redfin.com/NV/Las-Vegas/105-Redstone-St-89145/home/29225641',
'https://www.redfin.com/NV/Las-Vegas/100-S-Crestline-Dr-89107/unit-12/home/29229155',
'https://www.redfin.com/NV/Las-Vegas/401-Mill-Hollow-Rd-89107/home/29235458',
'https://www.redfin.com/NV/Las-Vegas/5208-Alta-Dr-89107/home/29235759',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-804/home/29283549',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-221/home/29283658',
'https://www.redfin.com/NV/Las-Vegas/4224-Kay-Pl-89107/home/29343615',
'https://www.redfin.com/NV/Las-Vegas/4304-El-Conlon-Ave-89102/home/29463621',
'https://www.redfin.com/NV/Las-Vegas/211-E-Flamingo-Rd-89169/unit-715/home/29486967',
'https://www.redfin.com/NV/Las-Vegas/1430-Covelo-Ct-89146/home/29516479',
'https://www.redfin.com/NV/Las-Vegas/7481-Crystal-Forrest-Dr-89117/home/29520230',
'https://www.redfin.com/NV/Las-Vegas/1830-Casa-de-Elegante-Ct-89117/home/29520567',
'https://www.redfin.com/NV/Las-Vegas/8109-Via-del-Cerro-Ct-89117/home/29522285',
'https://www.redfin.com/NV/Las-Vegas/2541-Tumble-Brook-Dr-89134/home/28964528',
'https://www.redfin.com/NV/Las-Vegas/309-Gardenia-Ln-89107/home/28966001',
'https://www.redfin.com/NV/Las-Vegas/7200-Pirates-Cove-Rd-89145/unit-1103/home/28969570',
'https://www.redfin.com/NV/Las-Vegas/144-Macoby-Run-St-89148/home/28972126',
'https://www.redfin.com/NV/Las-Vegas/5611-Aurora-Crest-Ave-89139/home/28974208',
'https://www.redfin.com/NV/Las-Vegas/9667-Brooks-Lake-Ave-89148/home/28976812',
'https://www.redfin.com/NV/Las-Vegas/3898-Chase-Glen-Cir-89121/home/28976892',
'https://www.redfin.com/NV/Las-Vegas/7595-Ironwood-Knoll-Ave-89113/home/28978709',
'https://www.redfin.com/NV/Las-Vegas/7967-Delft-Blue-St-89139/home/28991574',
'https://www.redfin.com/NV/Las-Vegas/4839-Jubilee-Diamond-Ct-89139/home/29005868',
'https://www.redfin.com/NV/Las-Vegas/5848-Old-English-Ct-89139/home/29013361',
'https://www.redfin.com/NV/Las-Vegas/6255-Rock-Dove-Ave-89122/home/29029357',
'https://www.redfin.com/NV/Las-Vegas/3145-E-Flamingo-Rd-89121/unit-2105/home/29029740',
'https://www.redfin.com/NV/Las-Vegas/7702-Hampton-Willows-Ln-89113/home/29034777',
'https://www.redfin.com/NV/Las-Vegas/5741-Old-Colony-Dr-89139/home/29036208',
'https://www.redfin.com/NV/Las-Vegas/4989-Tioga-Pass-Ave-89139/home/29036350',
'https://www.redfin.com/NV/Las-Vegas/9216-Cazador-St-89123/home/29039308',
'https://www.redfin.com/NV/Las-Vegas/2451-N-Rainbow-Blvd-89108/unit-2110/home/29062106',
'https://www.redfin.com/NV/Las-Vegas/2451-N-Rainbow-Blvd-89108/unit-1073/home/29062176',
'https://www.redfin.com/NV/Las-Vegas/7411-Bedford-Pine-Ct-89113/home/29066287',
'https://www.redfin.com/NV/Las-Vegas/8994-Harmony-Grove-Ave-89148/home/29067605',
'https://www.redfin.com/NV/Las-Vegas/10605-Back-Plains-Dr-89134/home/29105003',
'https://www.redfin.com/NV/Las-Vegas/10801-Heritage-Hills-Dr-89134/home/29107513',
'https://www.redfin.com/NV/Las-Vegas/10117-Cardinal-View-Pl-89134/home/29109306',
'https://www.redfin.com/NV/Las-Vegas/10303-Sawmill-Ave-89134/home/29110005',
'https://www.redfin.com/NV/Las-Vegas/5613-Deodar-Dr-89108/home/29136185',
'https://www.redfin.com/NV/Las-Vegas/6625-Old-Oxford-Ave-89108/home/29137874',
'https://www.redfin.com/NV/Las-Vegas/6228-Heather-Mist-Ln-89108/home/29140191',
'https://www.redfin.com/NV/Las-Vegas/3049-Brigantine-Way-89128/home/29141154',
'https://www.redfin.com/NV/Las-Vegas/8125-Sapphire-Bay-Cir-89128/home/29142773',
'https://www.redfin.com/NV/Las-Vegas/2417-Sun-Shores-Dr-89128/home/29143904',
'https://www.redfin.com/NV/Las-Vegas/2648-Golden-Sands-Dr-89128/home/29146127',
'https://www.redfin.com/NV/Las-Vegas/2653-Seahorse-Ct-89128/home/29146145',
'https://www.redfin.com/NV/Las-Vegas/2656-Echo-Mesa-Dr-89134/home/29148092',
'https://www.redfin.com/NV/Las-Vegas/10033-Woodhouse-Dr-89134/home/29152031',
'https://www.redfin.com/NV/Las-Vegas/9748-Panorama-Cliff-Dr-89134/home/29156068',
'https://www.redfin.com/NV/Las-Vegas/1900-Mountain-Hills-Ct-89128/unit-202/home/29165238',
'https://www.redfin.com/NV/Las-Vegas/7925-Sierra-Largo-Dr-89128/home/29165298',
'https://www.redfin.com/NV/Las-Vegas/7820-Desert-Bell-Ave-89128/home/29166303',
'https://www.redfin.com/NV/Las-Vegas/1636-Mexican-Poppy-St-89128/home/29166928',
'https://www.redfin.com/NV/Las-Vegas/7212-Old-Mission-Dr-89128/home/29168865',
'https://www.redfin.com/NV/Las-Vegas/1800-Leaning-Pine-Way-89128/home/29170090',
'https://www.redfin.com/NV/Las-Vegas/2136-Willowbury-Dr-89108/unit-A/home/29170751',
'https://www.redfin.com/NV/Las-Vegas/1608-Burningwood-Ln-89108/unit-101/home/29172228',
'https://www.redfin.com/NV/Las-Vegas/6220-Yerba-Ln-89108/home/29173481',
'https://www.redfin.com/NV/Las-Vegas/5624-Eugene-Ave-89108/home/29175561',
'https://www.redfin.com/NV/Las-Vegas/5921-Pebble-Beach-Blvd-89108/home/29185102',
'https://www.redfin.com/NV/Las-Vegas/5213-Cannon-Blvd-89108/home/29189285',
'https://www.redfin.com/NV/Las-Vegas/6516-Moon-Roses-Ct-89108/home/29192022',
'https://www.redfin.com/NV/Las-Vegas/1021-Dreamland-Cir-89108/home/29195027',
'https://www.redfin.com/NV/Las-Vegas/1112-Spunsilk-Cir-89108/home/29195282',
'https://www.redfin.com/NV/Las-Vegas/1405-Grangeville-Dr-89108/home/29195326',
'https://www.redfin.com/NV/Las-Vegas/6304-Clarice-Ave-89107/home/29197015',
'https://www.redfin.com/NV/Las-Vegas/1150-N-Buffalo-Dr-89128/unit-1056/home/29199054',
'https://www.redfin.com/NV/Las-Vegas/7100-Pirates-Cove-Rd-89145/unit-1062/home/29200431',
'https://www.redfin.com/NV/Las-Vegas/7001-Alamosa-Way-89128/home/29201284',
'https://www.redfin.com/NV/Las-Vegas/8033-Celestial-Ave-89128/unit-201/home/29202804',
'https://www.redfin.com/NV/Las-Vegas/8021-Cimarron-Ridge-Dr-89128/unit-201/home/29203820',
'https://www.redfin.com/NV/Las-Vegas/7933-Decker-Canyon-Dr-89128/unit-202/home/29205629',
'https://www.redfin.com/NV/Las-Vegas/1312-Driscoll-Dr-89128/unit-202/home/29206525',
'https://www.redfin.com/NV/Las-Vegas/10116-Jacob-Pl-89144/unit-203/home/29209757',
'https://www.redfin.com/NV/Las-Vegas/9712-Amber-Peak-Ct-89144/home/29210466',
'https://www.redfin.com/NV/Las-Vegas/112-Tesoro-Dr-89144/home/29210597',
'https://www.redfin.com/NV/Las-Vegas/9208-Sunnyfield-Dr-89134/home/29211532',
'https://www.redfin.com/NV/Las-Vegas/5395-Raccoon-Valley-Ln-89122/home/29242720',
'https://www.redfin.com/NV/Las-Vegas/7701-W-Robindale-Rd-89113/unit-164/home/29287192',
'https://www.redfin.com/NV/Las-Vegas/8985-S-Durango-Dr-89113/unit-2059/home/29290707',
'https://www.redfin.com/NV/Las-Vegas/9517-Dawning-Heat-St-89178/home/29290807',
'https://www.redfin.com/NV/Las-Vegas/8256-River-Beach-Ave-89178/home/29292576',
'https://www.redfin.com/NV/Las-Vegas/4616-Eugene-Ave-89108/home/29305235',
'https://www.redfin.com/NV/Las-Vegas/2112-Havelina-St-89108/home/29305300',
'https://www.redfin.com/NV/Las-Vegas/4516-W-Lake-Mead-Blvd-89108/unit-102/home/29306176',
'https://www.redfin.com/NV/Las-Vegas/2213-Saxton-Hill-Ave-89106/home/29309694',
'https://www.redfin.com/NV/Las-Vegas/2520-Perryville-Ave-89106/unit-103/home/29310639',
'https://www.redfin.com/NV/Las-Vegas/5424-Bradford-Pear-Dr-89122/home/29315558',
'https://www.redfin.com/NV/Las-Vegas/1037-Lawry-Ave-89106/home/29323847',
'https://www.redfin.com/NV/Las-Vegas/4405-Saint-Andrews-Cir-89107/home/29340815',
'https://www.redfin.com/NV/Las-Vegas/2725-S-Nellis-Blvd-89121/unit-1197/home/29414678',
'https://www.redfin.com/NV/Las-Vegas/2725-S-Nellis-Blvd-89121/unit-2185/home/29414778',
'https://www.redfin.com/NV/Las-Vegas/4864-Rollingwood-Cir-89121/home/29426804',
'https://www.redfin.com/NV/Las-Vegas/5361-Simpson-Dr-89122/home/29431493',
'https://www.redfin.com/NV/Las-Vegas/5710-E-Tropicana-Ave-89122/unit-1177/home/29431744',
'https://www.redfin.com/NV/Las-Vegas/5710-E-Tropicana-Ave-89122/unit-1090/home/29431976',
'https://www.redfin.com/NV/Las-Vegas/5187-Midnight-Oil-Dr-89122/home/29434404',
'https://www.redfin.com/NV/Las-Vegas/5375-Shake-Ct-89122/unit-203/home/29435391',
'https://www.redfin.com/NV/Las-Vegas/5040-Haystack-Dr-89122/home/29435787',
'https://www.redfin.com/NV/Las-Vegas/5624-Wellesley-Dr-89122/home/29437629',
'https://www.redfin.com/NV/Las-Vegas/5439-Northridge-Ln-89122/home/29437922',
'https://www.redfin.com/NV/Las-Vegas/4855-Terra-Linda-Ave-89120/home/29440434',
'https://www.redfin.com/NV/Las-Vegas/5327-Lamoille-Cir-89120/home/29442155',
'https://www.redfin.com/NV/Las-Vegas/9300-Hosner-St-89178/home/29490136',
'https://www.redfin.com/NV/Las-Vegas/1300-Lorilyn-Ave-89119/unit-1/home/29505862',
'https://www.redfin.com/NV/Las-Vegas/4770-Topaz-St-89121/unit-57/home/29508187',
'https://www.redfin.com/NV/Las-Vegas/5447-Birchbrook-Ct-89120/home/29511589',
'https://www.redfin.com/NV/Las-Vegas/5216-Mandalay-Springs-Dr-89120/unit-101/home/29511813',
'https://www.redfin.com/NV/Las-Vegas/873-De-Met-Dr-89119/home/29515357',
'https://www.redfin.com/NV/Las-Vegas/1107-Westminster-Ave-89119/home/29516255',
'https://www.redfin.com/NV/Las-Vegas/1067-Westminster-Ave-89119/home/29516278',
'https://www.redfin.com/NV/Las-Vegas/3286-E-Oquendo-Rd-89120/home/29516745',
'https://www.redfin.com/NV/Las-Vegas/1770-Autumn-Rust-Dr-89119/home/29592403',
'https://www.redfin.com/NV/Las-Vegas/6992-Antell-Cir-89119/home/29592494',
'https://www.redfin.com/NV/Las-Vegas/1430-Recital-Way-89119/home/29592607',
'https://www.redfin.com/NV/Las-Vegas/7049-Morales-Cir-89119/home/29593635',
'https://www.redfin.com/NV/Las-Vegas/1730-Forest-Walk-Dr-89119/home/29593744',
'https://www.redfin.com/NV/Las-Vegas/7078-Knoll-View-Dr-89119/home/29593753',
'https://www.redfin.com/NV/Las-Vegas/7132-Paradise-Bay-Dr-89119/home/29594041',
'https://www.redfin.com/NV/Las-Vegas/7026-Troubador-Dr-89119/home/29594241',
'https://www.redfin.com/NV/Las-Vegas/6650-W-Warm-Springs-Rd-89118/unit-1176/home/29603979',
'https://www.redfin.com/NV/Las-Vegas/9308-Night-Mesa-St-89178/home/29606326',
'https://www.redfin.com/NV/Las-Vegas/490-E-Mardon-Ave-89123/home/29606896',
'https://www.redfin.com/NV/Las-Vegas/9777-Plane-Tree-Ct-89178/home/29607481',
'https://www.redfin.com/NV/Las-Vegas/950-Elysian-Dr-89123/home/29611623',
'https://www.redfin.com/NV/Las-Vegas/7450-S-Eastern-Ave-89123/unit-1004/home/29614448',
'https://www.redfin.com/NV/Las-Vegas/7450-S-Eastern-Ave-89123/unit-1051/home/29614591',
'https://www.redfin.com/NV/Las-Vegas/653-Almondale-Ave-89123/home/29622953',
'https://www.redfin.com/NV/Las-Vegas/578-Lennox-Dr-89123/home/29623763',
'https://www.redfin.com/NV/Las-Vegas/392-E-Shelbourne-Ave-89123/home/29626434',
'https://www.redfin.com/NV/Las-Vegas/130-E-Wigwam-Ave-89123/home/29626636',
'https://www.redfin.com/NV/Las-Vegas/9147-Briarthorne-St-89123/home/29629425',
'https://www.redfin.com/NV/Las-Vegas/9394-Graceful-Gold-St-89123/home/29632765',
'https://www.redfin.com/NV/Las-Vegas/1466-October-Oak-Ave-89123/home/29633592',
'https://www.redfin.com/NV/Las-Vegas/9836-Altadena-St-89183/home/29638570',
'https://www.redfin.com/NV/Las-Vegas/684-Gritty-Garnet-Ave-89183/home/29638814',
'https://www.redfin.com/NV/Las-Vegas/9320-Column-Cactus-St-89139/home/29770079',
'https://www.redfin.com/NV/Las-Vegas/6694-Chimes-Tower-Ave-89139/home/29834513',
'https://www.redfin.com/NV/Las-Vegas/3209-Turtle-Head-Peak-Dr-89135/home/29870712',
'https://www.redfin.com/NV/Las-Vegas/492-Kirkstone-Way-89123/home/29875134',
'https://www.redfin.com/NV/Las-Vegas/172-Staten-Island-Ave-89123/home/29875245',
'https://www.redfin.com/NV/Las-Vegas/9963-Fine-Fern-St-89183/home/29876920',
'https://www.redfin.com/NV/Las-Vegas/7243-Twin-Maples-Ct-89148/home/29898432',
'https://www.redfin.com/NV/Las-Vegas/173-Duck-Hollow-Ave-89148/home/29908380',
'https://www.redfin.com/NV/Las-Vegas/7590-Bear-River-Ct-89139/home/29939357',
'https://www.redfin.com/NV/Las-Vegas/9451-Golden-Bell-Dr-89139/home/30496436',
'https://www.redfin.com/NV/Las-Vegas/8445-Insignia-Ave-89178/unit-105/home/33373982',
'https://www.redfin.com/NV/Las-Vegas/62-Tribute-Peak-Way-89148/home/40234276',
'https://www.redfin.com/NV/Las-Vegas/9022-Winthrop-Springs-Rd-89139/home/40548087',
'https://www.redfin.com/NV/Las-Vegas/6423-W-Camero-Ave-89139/home/52625580',
'https://www.redfin.com/NV/Las-Vegas/1515-E-Reno-Ave-89119/unit-203/home/77433410',
'https://www.redfin.com/NV/Las-Vegas/6092-Crown-Palms-Ave-89139/home/92907917',
'https://www.redfin.com/NV/Las-Vegas/6760-Goose-Watch-Ct-89139/home/101992606',
'https://www.redfin.com/NV/Las-Vegas/6367-Beavertail-Pond-Ave-89122/home/109632554',
'https://www.redfin.com/NV/Las-Vegas/9049-Janice-Glen-Ave-89148/home/114093564',
'https://www.redfin.com/NV/Las-Vegas/6325-Point-Isabel-Way-89122/home/143188514',
'https://www.redfin.com/NV/Las-Vegas/4936-Quest-Tribe-St-89122/home/146084229',
'https://www.redfin.com/NV/Las-Vegas/9240-Black-Diamond-St-89178/home/161752134',
'https://www.redfin.com/NV/Las-Vegas/11280-Granite-Ridge-Dr-89135/unit-1006/home/165926155',
'https://www.redfin.com/NV/Las-Vegas/9746-W-Richmar-Ave-89178/home/167760986',
'https://www.redfin.com/NV/Las-Vegas/9196-Lynea-Ct-89178/home/167769246',
'https://www.redfin.com/NV/Las-Vegas/5400-Swenson-St-89119/unit-32/home/167955788',
'https://www.redfin.com/NV/Las-Vegas/6295-Whippet-Ave-89122/home/169230639',
'https://www.redfin.com/NV/Las-Vegas/5158-Peak-Climb-Dr-89135/home/169231663',
'https://www.redfin.com/NV/Las-Vegas/7809-Waterfalls-Ave-89128/home/171652555',
'https://www.redfin.com/NV/Las-Vegas/7110-Cass-Ave-89113/home/175525674',
'https://www.redfin.com/NV/Las-Vegas/7898-W-Agate-Ave-89113/unit-65/home/185869640',
'https://www.redfin.com/NV/Enterprise/4515-Ashley-Hills-Ave-89141/unit-27/home/186207631',
'https://www.redfin.com/NV/Las-Vegas/7925-Whisper-Words-Ave-89113/unit-37/home/186301980',
'https://www.redfin.com/NV/Enterprise/4499-Ashley-Hills-Ave-89141/unit-23/home/186302872',
'https://www.redfin.com/NV/Enterprise/9930-Florence-Hills-St-89141/unit-116/home/186636704',
'https://www.redfin.com/NV/Las-Vegas/9128-Rocky-Raccoon-ST-89113/unit-19/home/186902560',
'https://www.redfin.com/NV/Las-Vegas/7869-Sailor-Sam-Ave-89113/unit-24/home/186958351',
'https://www.redfin.com/NV/Las-Vegas/9762-Sage-Grove-Ct-89148/home/29032846',
'https://www.redfin.com/NV/Las-Vegas/5441-Scotch-Cap-Dr-89122/home/145870780',
'https://www.redfin.com/NV/Las-Vegas/8457-Insignia-Ave-89178/unit-102/home/147856117',
'https://www.redfin.com/NV/Las-Vegas/5282-Belmont-Mill-Ct-89122/home/161754850',
'https://www.redfin.com/NV/Las-Vegas/11280-Granite-Ridge-Dr-89135/unit-1023/home/166555086',
'https://www.redfin.com/NV/Las-Vegas/11238-Essence-Point-Ave-89135/unit-203/home/166601089',
'https://www.redfin.com/NV/Las-Vegas/5614-Calm-Season-St-89118/home/167745194',
'https://www.redfin.com/NV/Las-Vegas/7493-Triple-Knot-Ct-89113/home/170466528',
'https://www.redfin.com/NV/Las-Vegas/4228-Bayley-Skye-Ave-89141/home/173636482',
'https://www.redfin.com/NV/Las-Vegas/6244-Yerba-Ln-89108/home/183394749',
'https://www.redfin.com/NV/Enterprise/9935-Florence-Hills-St-89141/unit-107/home/186275806',
'https://www.redfin.com/NV/Las-Vegas/1375-E-Hacienda-Ave-89119/unit-105/home/29513412',
'https://www.redfin.com/NV/Las-Vegas/6165-Meadow-View-Ln-89103/home/29545653',
'https://www.redfin.com/NV/Las-Vegas/4909-Quiet-Morning-St-89122/home/166181309',
'https://www.redfin.com/NV/Las-Vegas/6820-Scarlet-Flax-St-89148/home/28972475',
'https://www.redfin.com/NV/Las-Vegas/5411-Pinosa-Ct-89141/home/28975799',
'https://www.redfin.com/NV/Las-Vegas/11424-Glowing-Sunset-Ln-89135/home/28989084',
'https://www.redfin.com/NV/Las-Vegas/11072-Pentland-Downs-St-89141/home/28992867',
'https://www.redfin.com/NV/Las-Vegas/11690-Vacanze-Ct-89183/home/28994826',
'https://www.redfin.com/NV/Las-Vegas/128-Struzzo-Ave-89183/home/28994856',
'https://www.redfin.com/NV/Las-Vegas/8465-Adams-Grove-St-89139/home/29007763',
'https://www.redfin.com/NV/Las-Vegas/6239-Bedding-Plane-Ct-89139/home/29010782',
'https://www.redfin.com/NV/Las-Vegas/6255-Peneplain-Ave-89139/home/29035228',
'https://www.redfin.com/NV/Las-Vegas/137-Water-Hazard-Ln-89148/home/29037651',
'https://www.redfin.com/NV/Las-Vegas/10288-Nolinas-St-89141/home/29038486',
'https://www.redfin.com/NV/Las-Vegas/10819-Vestone-St-89141/home/29038527',
'https://www.redfin.com/NV/Las-Vegas/10623-Lessona-St-89141/home/29038650',
'https://www.redfin.com/NV/Las-Vegas/10864-Fishers-Island-St-89141/home/29040149',
'https://www.redfin.com/NV/Las-Vegas/9158-Placer-Bullion-Ave-89178/home/29055222',
'https://www.redfin.com/NV/Las-Vegas/10082-Amber-Field-St-89178/home/29055927',
'https://www.redfin.com/NV/Las-Vegas/7255-W-Sunset-Rd-89113/unit-2124/home/29065894',
'https://www.redfin.com/NV/Las-Vegas/8000-Badura-Ave-89113/unit-1173/home/29066604',
'https://www.redfin.com/NV/Las-Vegas/8767-Brilliant-Star-Dr-89178/home/29068643',
'https://www.redfin.com/NV/Las-Vegas/10315-Eve-Springs-St-89178/home/29069378',
'https://www.redfin.com/NV/Las-Vegas/2300-E-Silverado-Ranch-Blvd-89183/unit-2061/home/29070141',
'https://www.redfin.com/NV/Las-Vegas/555-E-Silverado-Ranch-Blvd-89183/unit-2089/home/29070464',
'https://www.redfin.com/NV/Las-Vegas/7701-W-Robindale-Rd-89113/unit-226/home/29287225',
'https://www.redfin.com/NV/Las-Vegas/5040-Upper-Falls-Ct-89141/home/29291877',
'https://www.redfin.com/NV/Las-Vegas/9037-Castledowns-St-89148/home/29321168',
'https://www.redfin.com/NV/Las-Vegas/3851-Topaz-St-89121/home/29474075',
'https://www.redfin.com/NV/Las-Vegas/3110-Espanol-Dr-89121/home/29475684',
'https://www.redfin.com/NV/Las-Vegas/1455-E-Katie-Ave-89119/unit-D26/home/29483425',
'https://www.redfin.com/NV/Las-Vegas/8976-Lily-Touchstone-Ct-89148/home/29489724',
'https://www.redfin.com/NV/Las-Vegas/5553-Markley-Ave-89141/home/29490147',
'https://www.redfin.com/NV/Las-Vegas/3076-Bella-Verona-Ave-89141/home/29495849',
'https://www.redfin.com/NV/Las-Vegas/3930-University-Center-Dr-89119/unit-910/home/29501079',
'https://www.redfin.com/NV/Las-Vegas/1569-Hialeah-Dr-89119/home/29504833',
'https://www.redfin.com/NV/Las-Vegas/1386-Lorilyn-Ave-89119/unit-4/home/29505898',
'https://www.redfin.com/NV/Las-Vegas/1700-Gabriel-Dr-89119/home/29505915',
'https://www.redfin.com/NV/Las-Vegas/2007-Rockledge-Dr-89119/home/29507296',
'https://www.redfin.com/NV/Las-Vegas/4584-Casa-Mia-Cir-89121/home/29507756',
'https://www.redfin.com/NV/Las-Vegas/2565-Malabar-Ave-89121/home/29507845',
'https://www.redfin.com/NV/Las-Vegas/4770-Topaz-St-89121/unit-22/home/29508291',
'https://www.redfin.com/NV/Las-Vegas/2431-Paradise-Village-Way-89120/home/29509942',
'https://www.redfin.com/NV/Las-Vegas/5259-Caspian-Springs-Dr-89120/unit-204/home/29511038',
'https://www.redfin.com/NV/Las-Vegas/3215-Casey-Dr-89120/unit-204/home/29511080',
'https://www.redfin.com/NV/Las-Vegas/8475-Gagnier-Blvd-89113/home/29591832',
'https://www.redfin.com/NV/Las-Vegas/1677-Garden-Path-Ct-89119/home/29592389',
'https://www.redfin.com/NV/Las-Vegas/6010-Sierra-Medina-Ave-89139/home/29606150',
'https://www.redfin.com/NV/Las-Vegas/9992-Big-Sur-Mountain-St-89178/home/29606926',
'https://www.redfin.com/NV/Las-Vegas/6406-Fulton-Meadows-Ave-89141/home/29608844',
'https://www.redfin.com/NV/Las-Vegas/1586-Laurel-Oak-Dr-89123/home/29613664',
'https://www.redfin.com/NV/Las-Vegas/1555-Yellowwood-Dr-89123/home/29613712',
'https://www.redfin.com/NV/Las-Vegas/722-Irwindale-Ave-89123/home/29622284',
'https://www.redfin.com/NV/Las-Vegas/9462-Boulder-Creek-St-89123/home/29627883',
'https://www.redfin.com/NV/Las-Vegas/868-Rainbow-Rock-St-89123/home/29631268',
'https://www.redfin.com/NV/Las-Vegas/1865-Silver-Cactus-Ct-89183/home/29637565',
'https://www.redfin.com/NV/Las-Vegas/8255-Las-Vegas-Blvd-S-89123/unit-121/home/29800376',
'https://www.redfin.com/NV/Las-Vegas/8255-Las-Vegas-Blvd-S-89123/unit-505/home/29800446',
'https://www.redfin.com/NV/Las-Vegas/10578-Gambassi-Ct-89141/home/29823866',
'https://www.redfin.com/NV/Las-Vegas/46-Perennial-Border-Ct-89148/home/29833521',
'https://www.redfin.com/NV/Las-Vegas/7810-Nautilus-Shell-St-89139/home/29834340',
'https://www.redfin.com/NV/Las-Vegas/976-Pagoda-Tree-Ct-89183/home/29837112',
'https://www.redfin.com/NV/Las-Vegas/1050-E-Cactus-Ave-89183/unit-1034/home/29877282',
'https://www.redfin.com/NV/Las-Vegas/1050-E-Cactus-Ave-89183/unit-1045/home/29878157',
'https://www.redfin.com/NV/Las-Vegas/5644-Bracana-Ct-89141/home/29900427',
'https://www.redfin.com/NV/Las-Vegas/7977-Marble-City-Ct-89139/home/29901549',
'https://www.redfin.com/NV/Las-Vegas/3722-Caesars-Cir-89120/home/29903189',
'https://www.redfin.com/NV/Las-Vegas/10613-Eagle-Nest-St-89141/home/29909775',
'https://www.redfin.com/NV/Las-Vegas/10626-Eagle-Nest-St-89141/home/29910746',
'https://www.redfin.com/NV/Las-Vegas/10257-Bethel-Mill-St-89183/home/29917816',
'https://www.redfin.com/NV/Las-Vegas/7482-Aurora-Glow-St-89139/home/29939327',
'https://www.redfin.com/NV/Las-Vegas/5578-Casa-Palazzo-Ct-89141/home/29940313',
'https://www.redfin.com/NV/Las-Vegas/3049-Scalise-Ct-89141/home/29941825',
'https://www.redfin.com/NV/Las-Vegas/9325-Wild-Lariat-Ave-89178/home/30301948',
'https://www.redfin.com/NV/Las-Vegas/10016-Arwen-St-89178/home/30476230',
'https://www.redfin.com/NV/Las-Vegas/31-Bay-Course-Ct-89148/home/39609547',
'https://www.redfin.com/NV/Las-Vegas/11216-Jewel-Desert-Ct-89179/home/40219853',
'https://www.redfin.com/NV/Las-Vegas/10355-Copalito-Dr-89178/home/44585152',
'https://www.redfin.com/NV/Las-Vegas/342-Bridgeton-Cross-Ct-89148/home/49748640',
'https://www.redfin.com/NV/Las-Vegas/10236-Mopan-Rd-89178/home/52594043',
'https://www.redfin.com/NV/Las-Vegas/75-Buckthorn-Ridge-Ct-89183/home/52629721',
'https://www.redfin.com/NV/Las-Vegas/8917-Fox-Season-Ave-89178/home/92905366',
'https://www.redfin.com/NV/Las-Vegas/7583-Mallard-Bay-Ave-89179/home/109630406',
'https://www.redfin.com/NV/Las-Vegas/9055-Linden-Oak-St-89178/home/109633075',
'https://www.redfin.com/NV/Las-Vegas/3031-Versace-Ave-89141/home/109635400',
'https://www.redfin.com/NV/Las-Vegas/61-Olympia-Canyon-Way-89141/home/112855191',
'https://www.redfin.com/NV/Las-Vegas/8053-Green-River-Knob-St-89113/home/113712910',
'https://www.redfin.com/NV/Las-Vegas/7670-Monomoy-Bay-Ave-89179/home/143188810',
'https://www.redfin.com/NV/Las-Vegas/12239-Pro-Tour-Ct-89141/home/144686982',
'https://www.redfin.com/NV/Las-Vegas/102-Opulent-Rose-Ave-89183/home/147875410',
'https://www.redfin.com/NV/Las-Vegas/10860-Woods-Hole-Bay-St-89179/home/161713664',
'https://www.redfin.com/NV/Las-Vegas/12-Olympia-Ridge-Dr-89141/home/166057757',
'https://www.redfin.com/NV/Las-Vegas/12817-Slipknot-St-89141/home/166058055',
'https://www.redfin.com/NV/Las-Vegas/12888-Slipknot-St-89141/home/166438688',
'https://www.redfin.com/NV/Las-Vegas/10735-Galveston-Bay-St-89179/home/166605131',
'https://www.redfin.com/NV/Las-Vegas/10142-Eden-Mountain-St-89141/home/177143398',
'https://www.redfin.com/NV/Las-Vegas/11765-Discovery-Canyon-Dr-89135/home/181143275',
'https://www.redfin.com/NV/Las-Vegas/10112-Queens-Pearl-St-89141/home/182477817',
'https://www.redfin.com/NV/Las-Vegas/10118-Queens-Pearl-St-89141/home/182477818',
'https://www.redfin.com/NV/Las-Vegas/10279-Fragrant-Apple-Ct-89141/home/184024803',
'https://www.redfin.com/NV/Las-Vegas/10149-Cluny-Ave-89178/home/171321415',
'https://www.redfin.com/NV/Las-Vegas/9508-Medway-Towns-Ave-89178/home/172505844',
'https://www.redfin.com/NV/Las-Vegas/10389-Arroyo-Willow-St-89141/home/173454664',
'https://www.redfin.com/NV/Las-Vegas/9186-Theodore-Ct-89178/home/175526787',
'https://www.redfin.com/NV/Las-Vegas/10897-Lexus-Ct-89141/home/175527139',
'https://www.redfin.com/NV/Las-Vegas/234-Seven-Valleys-Ct-89183/home/175528772',
'https://www.redfin.com/NV/Las-Vegas/10250-Hairstreak-St-89141/home/178675152',
'https://www.redfin.com/NV/Las-Vegas/3965-Speckled-Wood-Ave-89141/home/178675168',
'https://www.redfin.com/NV/Las-Vegas/3465-Cosima-Ln-89120/home/179977221',
'https://www.redfin.com/NV/Las-Vegas/4472-Hidden-Stone-Ave-89141/home/182477813',
'https://www.redfin.com/NV/Las-Vegas/10124-Queens-Pearl-St-89141/home/182477819',
'https://www.redfin.com/NV/Las-Vegas/5262-Peak-Rock-Ct-89141/home/182484526',
'https://www.redfin.com/NV/Las-Vegas/9554-Dollardton-St-89139/home/183047229',
'https://www.redfin.com/NV/Las-Vegas/6219-Runestone-ST-89113/unit-19/home/185714454',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-3419-3421/home/186231679',
'https://www.redfin.com/NV/Las-Vegas/6211-Runestone-St-89113/unit-21/home/186339656',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-1801-1803/home/186342905',
'https://www.redfin.com/NV/Las-Vegas/5681-Steampunk-ST-89118/unit-13/home/186520991',
'https://www.redfin.com/NV/Las-Vegas/4120-Yucca-Bloom-St-89135/unit-258/home/186853214',
'https://www.redfin.com/NV/Las-Vegas/242-Fairway-Woods-Dr-89148/home/29014256',
'https://www.redfin.com/NV/Las-Vegas/6072-Lamotte-Ave-89141/home/29038542',
'https://www.redfin.com/NV/Las-Vegas/3750-Pilgrim-St-89121/home/29423252',
'https://www.redfin.com/NV/Las-Vegas/3089-Casey-Dr-89120/unit-202/home/29511948',
'https://www.redfin.com/NV/Las-Vegas/8925-Waymire-Creek-Ct-89147/home/29554095',
'https://www.redfin.com/NV/Las-Vegas/9990-Clear-Crystal-St-89183/home/29638723',
'https://www.redfin.com/NV/Las-Vegas/10675-Moss-Lake-St-89179/home/40234006',
'https://www.redfin.com/NV/Henderson/3136-Floral-Vista-Ave-89014/home/28963885',
'https://www.redfin.com/NV/Las-Vegas/6250-W-Arby-Ave-89118/unit-140/home/28964396',
'https://www.redfin.com/NV/Henderson/783-Rise-Canyon-Dr-89052/home/28972337',
'https://www.redfin.com/NV/Las-Vegas/4704-Sequoia-Park-Ave-89139/home/28978938',
'https://www.redfin.com/NV/Las-Vegas/6648-Oxendale-Ave-89139/home/28990998',
'https://www.redfin.com/NV/Las-Vegas/6692-Dunraven-Ave-89139/home/28991132',
'https://www.redfin.com/NV/Las-Vegas/6692-Hathersage-Ave-89139/home/28991302',
'https://www.redfin.com/NV/Las-Vegas/6629-Hurkling-Stone-Ave-89139/home/28991356',
'https://www.redfin.com/NV/Las-Vegas/6254-Arbor-Valley-Ave-89139/home/28992303',
'https://www.redfin.com/NV/Las-Vegas/11778-Weybrook-Park-Dr-89141/home/28995386',
'https://www.redfin.com/NV/Las-Vegas/118-Tansy-Ct-89183/home/28996606',
'https://www.redfin.com/NV/Las-Vegas/7397-Laramie-Ave-89113/home/28997827',
'https://www.redfin.com/NV/Las-Vegas/5031-Pine-Mountain-Ave-89139/home/29004067',
'https://www.redfin.com/NV/Henderson/1054-Astounding-Hills-Dr-89052/home/29005317',
'https://www.redfin.com/NV/Henderson/10843-Colour-Magic-St-89052/home/29005866',
'https://www.redfin.com/NV/Henderson/1052-Augusta-Wood-Ct-89052/home/29007575',
'https://www.redfin.com/NV/Henderson/311-Bass-Dr-89014/unit-D/home/29009036',
'https://www.redfin.com/NV/Henderson/2161-Big-Bar-Dr-89052/home/29012589',
'https://www.redfin.com/NV/Henderson/11128-Abbeyfield-Rose-Dr-89052/home/29014548',
'https://www.redfin.com/NV/Las-Vegas/6209-Alpine-Tree-Ave-89139/home/29015685',
'https://www.redfin.com/NV/Las-Vegas/7616-Poppy-Springs-Ave-89113/home/29034618',
'https://www.redfin.com/NV/Las-Vegas/7264-Larix-Rd-89113/home/29035050',
'https://www.redfin.com/NV/Las-Vegas/7158-Barton-Creek-Ct-89113/home/29035582',
'https://www.redfin.com/NV/Las-Vegas/7712-Coronado-Island-St-89139/home/29035729',
'https://www.redfin.com/NV/Las-Vegas/6621-Melodic-Ct-89139/home/29036616',
'https://www.redfin.com/NV/Las-Vegas/6258-Canebrake-Ct-89141/home/29038382',
'https://www.redfin.com/NV/Las-Vegas/1443-Corsica-Crest-Ct-89123/home/29038900',
'https://www.redfin.com/NV/Las-Vegas/4683-Ametrine-Ct-89139/home/29039047',
'https://www.redfin.com/NV/Henderson/10935-Scotch-Rose-St-89052/home/29040547',
'https://www.redfin.com/NV/Las-Vegas/10828-Rosabella-St-89141/home/29040643',
'https://www.redfin.com/NV/Las-Vegas/10576-Timber-Stand-St-89183/home/29040712',
'https://www.redfin.com/NV/Las-Vegas/8255-Cupertino-Heights-Way-89178/home/29056264',
'https://www.redfin.com/NV/Las-Vegas/8237-Campbell-Springs-Ave-89178/home/29056295',
'https://www.redfin.com/NV/Las-Vegas/9250-Valerie-Elaine-St-89139/home/29056416',
'https://www.redfin.com/NV/Las-Vegas/1841-Versante-Ave-89183/home/29057060',
'https://www.redfin.com/NV/Las-Vegas/7255-W-Sunset-Rd-89113/unit-1082/home/29065738',
'https://www.redfin.com/NV/Las-Vegas/9480-Swiftwater-Ct-89178/home/29068279',
'https://www.redfin.com/NV/Las-Vegas/2300-E-Silverado-Ranch-Blvd-89183/unit-1077/home/29070040',
'https://www.redfin.com/NV/Las-Vegas/2300-E-Silverado-Ranch-Blvd-89183/unit-2086/home/29070059',
'https://www.redfin.com/NV/Las-Vegas/555-E-Silverado-Ranch-Blvd-89183/unit-2037/home/29070177',
'https://www.redfin.com/NV/Las-Vegas/555-E-Silverado-Ranch-Blvd-89183/unit-2161/home/29070947',
'https://www.redfin.com/NV/Las-Vegas/9000-Las-Vegas-Blvd-S-89123/unit-1046/home/29184219',
'https://www.redfin.com/NV/Las-Vegas/9000-Las-Vegas-Blvd-S-89123/unit-1220/home/29184412',
'https://www.redfin.com/NV/Henderson/950-Seven-Hills-Dr-89052/unit-1016/home/29185327',
'https://www.redfin.com/NV/Henderson/950-Seven-Hills-Dr-89052/unit-1122/home/29185368',
'https://www.redfin.com/NV/Las-Vegas/9723-Skiers-Chalet-Ct-89178/home/29292506',
'https://www.redfin.com/NV/Las-Vegas/8054-Egypt-Meadows-Ave-89178/home/29292826',
'https://www.redfin.com/NV/Las-Vegas/9332-Vital-Crest-St-89123/home/29294891',
'https://www.redfin.com/NV/Henderson/1171-Adriana-Bend-Ct-89052/home/29296514',
'https://www.redfin.com/NV/Henderson/950-Seven-Hills-Dr-89052/unit-2522/home/29296966',
'https://www.redfin.com/NV/Las-Vegas/16-Harbor-Pines-Ct-89183/home/29303463',
'https://www.redfin.com/NV/Las-Vegas/11562-Elcadore-St-89183/home/29323046',
'https://www.redfin.com/NV/Henderson/251-S-Green-Valley-Pkwy-89012/unit-421/home/29450197',
'https://www.redfin.com/NV/Las-Vegas/7938-Homer-Ct-89139/home/29489118',
'https://www.redfin.com/NV/Las-Vegas/5708-Alington-Bend-Dr-89139/home/29490287',
'https://www.redfin.com/NV/Las-Vegas/10297-Burwood-St-89178/home/29490784',
'https://www.redfin.com/NV/Las-Vegas/481-Las-Alturas-Ave-89123/home/29493104',
'https://www.redfin.com/NV/Las-Vegas/68-E-Serene-Ave-89123/unit-416/home/29493777',
'https://www.redfin.com/NV/Las-Vegas/2405-W-Serene-Ave-89123/unit-735/home/29494050',
'https://www.redfin.com/NV/Las-Vegas/3163-Varenna-Ridge-Ave-89141/home/29495923',
'https://www.redfin.com/NV/Las-Vegas/259-Morning-Crest-Ave-89183/home/29499739',
'https://www.redfin.com/NV/Las-Vegas/9365-Lindell-Rd-89139/home/29591874',
'https://www.redfin.com/NV/Las-Vegas/6869-Tamarus-St-89119/unit-103/home/29592313',
'https://www.redfin.com/NV/Las-Vegas/1312-Finale-Ln-89119/home/29592353',
'https://www.redfin.com/NV/Las-Vegas/7218-Nordic-Lights-Dr-89119/home/29594180',
'https://www.redfin.com/NV/Las-Vegas/7123-Gunslinger-St-89119/home/29595084',
'https://www.redfin.com/NV/Las-Vegas/3345-W-Arby-Ave-89118/home/29595455',
'https://www.redfin.com/NV/Las-Vegas/4540-W-Pamalyn-Ave-89118/home/29595893',
'https://www.redfin.com/NV/Las-Vegas/4730-W-Robindale-Rd-89139/home/29596596',
'https://www.redfin.com/NV/Las-Vegas/3423-Beam-Dr-89139/home/29596986',
'https://www.redfin.com/NV/Las-Vegas/8025-Rancho-Destino-Rd-89123/home/29599387',
'https://www.redfin.com/NV/Las-Vegas/654-Ravenglen-Dr-89123/home/29607371',
'https://www.redfin.com/NV/Las-Vegas/354-Chaplin-Cove-Ave-89183/home/29609232',
'https://www.redfin.com/NV/Henderson/1168-Via-Piazza-89052/home/29610278',
'https://www.redfin.com/NV/Las-Vegas/932-Elysian-Dr-89123/home/29611604',
'https://www.redfin.com/NV/Las-Vegas/1170-Del-Lilly-Ln-89123/home/29611683',
'https://www.redfin.com/NV/Las-Vegas/7369-Cisco-Ln-89123/home/29611786',
'https://www.redfin.com/NV/Las-Vegas/917-Celebration-Dr-89123/home/29612750',
'https://www.redfin.com/NV/Henderson/2827-Via-Stella-St-89074/home/29614878',
'https://www.redfin.com/NV/Henderson/2764-Carnation-Ln-89074/home/29616151',
'https://www.redfin.com/NV/Henderson/219-Silado-Ct-89074/home/29617363',
'https://www.redfin.com/NV/Henderson/207-Chaco-Canyon-Dr-89074/home/29617554',
'https://www.redfin.com/NV/Henderson/141-Cologne-Ct-89074/home/29618029',
'https://www.redfin.com/NV/Henderson/2925-Wigwam-Pkwy-89074/unit-1321/home/29619183',
'https://www.redfin.com/NV/Henderson/2837-Yukon-Trail-Dr-89074/home/29619194',
'https://www.redfin.com/NV/Henderson/2925-Wigwam-Pkwy-89074/unit-1812/home/29619251',
'https://www.redfin.com/NV/Henderson/2925-Wigwam-Pkwy-89074/unit-223/home/29619515',
'https://www.redfin.com/NV/Henderson/103-Tapatio-St-89074/home/29620619',
'https://www.redfin.com/NV/Henderson/134-Coventry-Dr-89074/home/29620703',
'https://www.redfin.com/NV/Henderson/144-Coventry-Dr-89074/home/29620778',
'https://www.redfin.com/NV/Henderson/61-Tidwell-Ln-89074/home/29620873',
'https://www.redfin.com/NV/Las-Vegas/8863-Haviland-Rd-89123/home/29624528',
'https://www.redfin.com/NV/Las-Vegas/8845-Wallaby-Ln-89123/home/29624680',
'https://www.redfin.com/NV/Las-Vegas/832-Clear-Diamond-Ave-89123/home/29625543',
'https://www.redfin.com/NV/Las-Vegas/1129-Country-Shadows-Way-89123/home/29625690',
'https://www.redfin.com/NV/Las-Vegas/361-Cascade-Mist-Ave-89123/home/29629380',
'https://www.redfin.com/NV/Las-Vegas/9570-Soloshine-St-89123/home/29631068',
'https://www.redfin.com/NV/Las-Vegas/1025-Clover-Tip-Ct-89123/home/29633008',
'https://www.redfin.com/NV/Las-Vegas/2196-Madica-Ave-89123/home/29634934',
'https://www.redfin.com/NV/Henderson/8959-Sierra-Palms-Way-89074/home/29635474',
'https://www.redfin.com/NV/Henderson/502-Escoto-Pl-89052/home/29636717',
'https://www.redfin.com/NV/Las-Vegas/10665-Gilespie-St-89183/home/29640260',
'https://www.redfin.com/NV/Henderson/3158-Castle-Canyon-Ave-89052/home/29641192',
'https://www.redfin.com/NV/Henderson/3007-Scenic-Valley-Way-89052/home/29641291',
'https://www.redfin.com/NV/Henderson/3038-Scenic-Valley-Way-89052/home/29641312',
'https://www.redfin.com/NV/Henderson/1128-Via-Toscana-89052/home/29641699',
'https://www.redfin.com/NV/Henderson/2745-Middleburgh-Ct-89052/home/29641749',
'https://www.redfin.com/NV/Henderson/2427-Pickwick-Dr-89014/home/29647257',
'https://www.redfin.com/NV/Henderson/15-Quail-Hollow-Dr-89014/home/29651414',
'https://www.redfin.com/NV/Henderson/2356-Aqua-Vista-Ave-89014/home/29653105',
'https://www.redfin.com/NV/Henderson/364-Heath-Ct-89074/home/29654275',
'https://www.redfin.com/NV/Henderson/2532-El-Paso-Grande-Ave-89074/home/29655272',
'https://www.redfin.com/NV/Henderson/270-El-Camino-Verde-St-89074/home/29655410',
'https://www.redfin.com/NV/Henderson/264-El-Camino-Verde-St-89074/home/29655417',
'https://www.redfin.com/NV/Henderson/213-Malcolm-St-89074/home/29656365',
'https://www.redfin.com/NV/Henderson/2239-Le-Conte-Ave-89074/home/29657167',
'https://www.redfin.com/NV/Henderson/314-Pissarro-Pl-89074/home/29657561',
'https://www.redfin.com/NV/Henderson/2044-Waverly-Cir-89014/home/29657599',
'https://www.redfin.com/NV/Henderson/240-Windsong-Dr-89074/home/29658757',
'https://www.redfin.com/NV/Henderson/2379-Cliffwood-Dr-89074/home/29677069',
'https://www.redfin.com/NV/Henderson/193-Renville-Ct-89074/home/29677092',
'https://www.redfin.com/NV/Henderson/2484-Ram-Crossing-Way-89074/home/29677463',
'https://www.redfin.com/NV/Henderson/2503-Hacker-Dr-89074/home/29678293',
'https://www.redfin.com/NV/Henderson/2325-Windmill-Pkwy-89074/unit-912/home/29679265',
'https://www.redfin.com/NV/Henderson/216-Ultra-Dr-89074/home/29679343',
'https://www.redfin.com/NV/Henderson/2251-Wigwam-Pkwy-89074/unit-926/home/29679594',
'https://www.redfin.com/NV/Henderson/2251-Wigwam-Pkwy-89074/unit-1915/home/29679774',
'https://www.redfin.com/NV/Henderson/2455-Avenida-Cataluna-89074/home/29680533',
'https://www.redfin.com/NV/Henderson/78-Fortuna-Ct-89074/home/29680807',
'https://www.redfin.com/NV/Henderson/75-Motu-Ct-89074/home/29680825',
'https://www.redfin.com/NV/Henderson/16-Rue-de-Parc-89074/home/29681563',
'https://www.redfin.com/NV/Henderson/2267-Nashville-Ave-89052/home/29682267',
'https://www.redfin.com/NV/Henderson/25-Lantern-Glow-Cir-89074/home/29683006',
'https://www.redfin.com/NV/Henderson/2146-High-Mesa-Dr-89012/home/29692633',
'https://www.redfin.com/NV/Henderson/557-Carmel-Valley-St-89012/home/29693354',
'https://www.redfin.com/NV/Henderson/569-Fox-Links-St-89012/home/29693364',
'https://www.redfin.com/NV/Henderson/2526-Silver-Beach-Dr-89052/home/29696238',
'https://www.redfin.com/NV/Henderson/819-Adagio-St-89052/home/29697137',
'https://www.redfin.com/NV/Henderson/2578-Anchorgate-Dr-89052/home/29697154',
'https://www.redfin.com/NV/Henderson/2383-Falsetto-Ave-89052/home/29737279',
'https://www.redfin.com/NV/Henderson/2280-Lyrical-Rd-89052/home/29739606',
'https://www.redfin.com/NV/Henderson/1526-Villa-Rica-Dr-89052/home/29740897',
'https://www.redfin.com/NV/Henderson/1322-Imperia-Dr-89052/home/29740985',
'https://www.redfin.com/NV/Henderson/1376-European-Dr-89052/home/29742012',
'https://www.redfin.com/NV/Las-Vegas/8123-Amy-Springs-St-89113/home/29769023',
'https://www.redfin.com/NV/Las-Vegas/7295-Charreado-Ct-89179/home/29769697',
'https://www.redfin.com/NV/Las-Vegas/7190-La-Cadena-Ave-89179/home/29794959',
'https://www.redfin.com/NV/Las-Vegas/8255-Las-Vegas-Blvd-S-89123/unit-911/home/29800270',
'https://www.redfin.com/NV/Las-Vegas/2055-Dragonslayer-Ave-89183/home/29825045',
'https://www.redfin.com/NV/Las-Vegas/7633-Calm-Passage-Ct-89139/home/29834243',
'https://www.redfin.com/NV/Las-Vegas/17-Belle-La-Blanc-Ave-89123/home/29835417',
'https://www.redfin.com/NV/Henderson/691-Great-Dane-Ct-89052/home/29836043',
'https://www.redfin.com/NV/Henderson/2737-Bass-Lake-St-89052/home/29836999',
'https://www.redfin.com/NV/Henderson/2307-Silver-Crew-Pass-89052/home/29839918',
'https://www.redfin.com/NV/Henderson/1475-Cilento-Ct-89052/home/29840861',
'https://www.redfin.com/NV/Las-Vegas/10932-Sospel-Pl-89141/home/29848484',
'https://www.redfin.com/NV/Las-Vegas/4348-Val-Dechiana-Ave-89141/home/29849591',
'https://www.redfin.com/NV/Henderson/2662-Sunday-Grace-Dr-89052/home/29875857',
'https://www.redfin.com/NV/Las-Vegas/10010-Swimming-Hole-St-89183/home/29876812',
'https://www.redfin.com/NV/Henderson/700-Carnegie-St-89052/unit-913/home/29882606',
'https://www.redfin.com/NV/Las-Vegas/10621-Bardilino-St-89141/home/29889070',
'https://www.redfin.com/NV/Las-Vegas/595-Campus-Oaks-Ct-89183/home/29890978',
'https://www.redfin.com/NV/Las-Vegas/10502-Bonchester-Hill-St-89141/home/29900348',
'https://www.redfin.com/NV/Henderson/700-Carnegie-St-89052/unit-4712/home/29903610',
'https://www.redfin.com/NV/Henderson/1240-Sonatina-Dr-89052/home/29904218',
'https://www.redfin.com/NV/Las-Vegas/5557-San-Florentine-Ave-89141/home/29909868',
'https://www.redfin.com/NV/Henderson/2146-Handel-Ave-89052/home/29913686',
'https://www.redfin.com/NV/Las-Vegas/11011-Santorini-Dr-89141/home/29918661',
'https://www.redfin.com/NV/Las-Vegas/5773-Lazy-Days-Ct-89141/home/29920834',
'https://www.redfin.com/NV/Las-Vegas/5137-Numidia-Ave-89141/home/29940062',
'https://www.redfin.com/NV/Las-Vegas/10779-Barnard-Bee-Ct-89183/home/29942835',
'https://www.redfin.com/NV/Henderson/830-Carnegie-St-89052/unit-1222/home/29943773',
'https://www.redfin.com/NV/Henderson/2144-Paganini-Ave-89052/home/29945403',
'https://www.redfin.com/NV/Henderson/2144-Volante-Cir-89052/home/29946319',
'https://www.redfin.com/NV/Las-Vegas/9131-Marlia-St-89123/home/29950193',
'https://www.redfin.com/NV/Las-Vegas/7123-Hickory-Post-Ave-89179/home/30182713',
'https://www.redfin.com/NV/Las-Vegas/5695-Dorian-Black-Ave-89139/home/30475645',
'https://www.redfin.com/NV/Las-Vegas/7086-Positano-Hill-Ave-89178/home/30476247',
'https://www.redfin.com/NV/Las-Vegas/7596-Brisk-Ocean-Ave-89178/home/39657329',
'https://www.redfin.com/NV/Henderson/3664-Via-Vicchio-89052/home/45356842',
'https://www.redfin.com/NV/Henderson/1193-Via-Monticano-89052/home/52628249',
'https://www.redfin.com/NV/Las-Vegas/6728-Philharmonic-Ave-89139/home/65553205',
'https://www.redfin.com/NV/Las-Vegas/7588-Brisk-Ocean-Ave-89178/home/65553257',
'https://www.redfin.com/NV/Las-Vegas/5441-Ledgewood-Creek-Ave-89141/home/65557289',
'https://www.redfin.com/NV/Las-Vegas/6771-Medovina-Ct-89139/home/101992513',
'https://www.redfin.com/NV/Las-Vegas/9607-Gray-Cap-St-89123/home/105538903',
'https://www.redfin.com/NV/Las-Vegas/8175-Arville-St-89139/unit-305/home/108927532',
'https://www.redfin.com/NV/Las-Vegas/7283-Shannon-Ridge-Ct-89118/home/109871047',
'https://www.redfin.com/NV/Las-Vegas/3977-Jacob-Lake-Cir-89118/home/143026488',
'https://www.redfin.com/NV/Las-Vegas/11446-Quantum-Brooks-St-89183/home/145871212',
'https://www.redfin.com/NV/Las-Vegas/81-Kaku-Ridge-Way-89183/home/145871217',
'https://www.redfin.com/NV/Henderson/869-Bluebell-Brook-St-89052/home/146417769',
'https://www.redfin.com/NV/Las-Vegas/7125-Steele-Canyon-St-89118/home/165931045',
'https://www.redfin.com/NV/Henderson/2100-Havensight-Ln-89052/home/166035301',
'https://www.redfin.com/NV/Las-Vegas/3960-W-Torino-Ave-89139/home/166396644',
'https://www.redfin.com/NV/Las-Vegas/11296-Villa-Bellagio-Dr-89141/home/166408286',
'https://www.redfin.com/NV/Las-Vegas/8512-Baltimore-Heights-Ct-89113/home/167759790',
'https://www.redfin.com/NV/Henderson/3783-Canis-Minor-Ln-89052/unit-6206/home/170139492',
'https://www.redfin.com/NV/Henderson/1256-Jaya-Peak-Ct-89052/home/170976927',
'https://www.redfin.com/NV/Las-Vegas/6264-Dolosonte-ST-89148/unit-44/home/185368268',
'https://www.redfin.com/NV/Las-Vegas/10562-Pueblo-Springs-St-89183/home/28993598',
'https://www.redfin.com/NV/Las-Vegas/9106-Mount-Wilson-St-89113/home/29321134',
'https://www.redfin.com/NV/Las-Vegas/9137-Hunting-Arrow-St-89123/home/29629667',
'https://www.redfin.com/NV/Las-Vegas/1438-Silver-Knoll-Ave-89123/home/29633863',
'https://www.redfin.com/NV/Las-Vegas/2186-Lipari-Ct-89123/home/29634124',
'https://www.redfin.com/NV/Henderson/215-Animated-Ct-89052/home/29942638',
'https://www.redfin.com/NV/Las-Vegas/9920-Mills-Reef-Ct-89141/home/39658664',
'https://www.redfin.com/NV/Las-Vegas/6350-White-Heron-Ct-89139/home/65556076',
'https://www.redfin.com/NV/Las-Vegas/7610-W-Arby-Ave-89113/home/166316659',
'https://www.redfin.com/NV/Las-Vegas/4390-W-Cactus-Ave-89141/home/170139122',
'https://www.redfin.com/NV/Las-Vegas/8318-Waverton-Ct-89113/home/183993368',
'https://www.redfin.com/NV/Henderson/2970-W-Horizon-Ridge-Pkwy-89052/unit-C/home/184802741',
'https://www.redfin.com/NV/Henderson/1475-Cilento-Ct-89052/home/186909961',
'https://www.redfin.com/NV/Henderson/167-White-Butte-St-89012/home/28967797',
'https://www.redfin.com/NV/Henderson/522-Kristin-Ln-89011/home/28969942',
'https://www.redfin.com/NV/Henderson/1561-Buffalo-Brubaker-Ln-89002/home/28974050',
'https://www.redfin.com/NV/Henderson/697-Blooming-Sage-Ct-89015/home/28979442',
'https://www.redfin.com/NV/Henderson/30-Strada-Di-Villaggio-89011/unit-349/home/29028316',
'https://www.redfin.com/NV/Henderson/1093-Pleasing-Plateau-St-89002/home/29041933',
'https://www.redfin.com/NV/Henderson/515-Moses-Lake-Ct-89002/home/29042156',
'https://www.redfin.com/NV/Henderson/995-Klamath-River-Ave-89002/home/29042173',
'https://www.redfin.com/NV/Henderson/2104-Savannah-River-St-89044/home/29042661',
'https://www.redfin.com/NV/Henderson/2310-Sandstone-Cliffs-Dr-89044/home/29042691',
'https://www.redfin.com/NV/Henderson/1979-Sunset-Bend-Dr-89014/home/29056832',
'https://www.redfin.com/NV/Henderson/1130-Yellow-Orchid-St-89002/home/29057270',
'https://www.redfin.com/NV/Las-Vegas/5263-Tipper-Ave-89122/home/29282799',
'https://www.redfin.com/NV/Las-Vegas/5710-E-Tropicana-Ave-89122/unit-1053/home/29431906',
'https://www.redfin.com/NV/Las-Vegas/5094-Mascaro-Dr-89122/home/29434163',
'https://www.redfin.com/NV/Las-Vegas/5215-Suzy-Saly-Pl-89122/home/29435336',
'https://www.redfin.com/NV/Las-Vegas/6312-Sabora-Ave-89122/home/29435995',
'https://www.redfin.com/NV/Las-Vegas/5336-Sunbury-Ave-89122/home/29436707',
'https://www.redfin.com/NV/Las-Vegas/5448-Wellesley-Dr-89122/home/29437542',
'https://www.redfin.com/NV/Henderson/721-Rising-Brook-Dr-89011/home/29447695',
'https://www.redfin.com/NV/Henderson/728-Rising-Brook-Dr-89011/home/29447746',
'https://www.redfin.com/NV/Henderson/621-Moontide-Ct-89011/home/29447940',
'https://www.redfin.com/NV/Henderson/45-Maleena-Mesa-St-89074/unit-925/home/29610177',
'https://www.redfin.com/NV/Henderson/45-Maleena-Mesa-St-89074/unit-1623/home/29610936',
'https://www.redfin.com/NV/Henderson/1929-Bearden-Ave-89011/home/29641790',
'https://www.redfin.com/NV/Henderson/554-Kristin-Ln-89011/home/29643014',
'https://www.redfin.com/NV/Henderson/1719-Empire-Mine-Dr-89014/home/29644495',
'https://www.redfin.com/NV/Henderson/400-Raindance-Dr-89014/home/29645248',
'https://www.redfin.com/NV/Henderson/727-Apple-Tree-Ct-89014/home/29649625',
'https://www.redfin.com/NV/Henderson/604-Cervantes-Dr-89014/home/29650086',
'https://www.redfin.com/NV/Henderson/2711-Osborne-Ln-89014/home/29650731',
'https://www.redfin.com/NV/Henderson/340-Fuente-Dr-89014/home/29660596',
'https://www.redfin.com/NV/Henderson/384-Seine-Way-89014/home/29662776',
'https://www.redfin.com/NV/Henderson/1709-Sequoia-Dr-89014/home/29664002',
'https://www.redfin.com/NV/Henderson/321-Perry-Ellis-Dr-89014/home/29665676',
'https://www.redfin.com/NV/Henderson/304-Saint-Augustine-Ln-89014/home/29666658',
'https://www.redfin.com/NV/Henderson/1194-Golden-Spike-Ct-89014/home/29668070',
'https://www.redfin.com/NV/Henderson/1149-Light-Sky-Ave-89074/home/29668245',
'https://www.redfin.com/NV/Henderson/237-Collindale-St-89074/home/29668309',
'https://www.redfin.com/NV/Henderson/196-Boothbay-St-89074/home/29669216',
'https://www.redfin.com/NV/Henderson/277-Ventana-Heights-St-89074/home/29669290',
'https://www.redfin.com/NV/Henderson/1134-Evening-Canyon-Ave-89014/home/29669453',
'https://www.redfin.com/NV/Henderson/205-Carson-Way-89015/home/29670288',
'https://www.redfin.com/NV/Henderson/245-Carson-Way-89015/home/29670485',
'https://www.redfin.com/NV/Henderson/11-Desert-Dawn-Ln-89074/home/29673696',
'https://www.redfin.com/NV/Henderson/194-Webster-Way-89074/home/29674736',
'https://www.redfin.com/NV/Henderson/2002-Spruce-Brook-Dr-89074/home/29677405',
'https://www.redfin.com/NV/Henderson/1951-Thunder-Ridge-Cir-89012/home/29685482',
'https://www.redfin.com/NV/Henderson/1722-Summerwood-Cir-89012/home/29686023',
'https://www.redfin.com/NV/Henderson/1828-Thunder-Mountain-Dr-89012/home/29686318',
'https://www.redfin.com/NV/Henderson/428-Wright-Way-89015/home/29689939',
'https://www.redfin.com/NV/Henderson/410-Daffodil-Dr-89015/home/29690912',
'https://www.redfin.com/NV/Henderson/81-Zimmerman-St-89002/home/29691586',
'https://www.redfin.com/NV/Henderson/361-Island-Reef-Ave-89012/home/29691631',
'https://www.redfin.com/NV/Henderson/2122-High-Mesa-Dr-89012/home/29692610',
'https://www.redfin.com/NV/Henderson/480-Eagle-Vista-Dr-89012/home/29694141',
'https://www.redfin.com/NV/Henderson/1816-Cypress-Mesa-Dr-89012/home/29699072',
'https://www.redfin.com/NV/Henderson/221-Fir-St-89015/home/29702926',
'https://www.redfin.com/NV/Henderson/1015-Brookside-Ct-89015/home/29703073',
'https://www.redfin.com/NV/Henderson/1003-Center-St-89015/home/29703118',
'https://www.redfin.com/NV/Henderson/431-Bell-Ave-89015/home/29703316',
'https://www.redfin.com/NV/Henderson/265-Dahlia-St-89015/home/29704209',
'https://www.redfin.com/NV/Henderson/211-Oleander-Pl-89015/home/29704350',
'https://www.redfin.com/NV/Henderson/220-Pretty-Sunset-Ter-89015/home/29704652',
'https://www.redfin.com/NV/Henderson/814-Anchor-Dr-89015/home/29704723',
'https://www.redfin.com/NV/Henderson/173-Skytop-Dr-89015/home/29705536',
'https://www.redfin.com/NV/Henderson/111-Maple-St-89015/home/29706518',
'https://www.redfin.com/NV/Henderson/601-Cabrillo-Cir-89015/unit-317/home/29706545',
'https://www.redfin.com/NV/Henderson/241-Queen-Marie-Ct-89015/home/29707404',
'https://www.redfin.com/NV/Henderson/425-McKenzie-Dr-89015/home/29707667',
'https://www.redfin.com/NV/Henderson/103-E-Pacific-Ave-89015/home/29709409',
'https://www.redfin.com/NV/Henderson/246-Tungsten-St-89015/home/29709785',
'https://www.redfin.com/NV/Henderson/203-E-Van-Wagenen-St-89015/home/29711049',
'https://www.redfin.com/NV/Henderson/315-E-Van-Wagenen-St-89015/home/29711096',
'https://www.redfin.com/NV/Henderson/984-Broken-Reed-Ct-89015/home/29714128',
'https://www.redfin.com/NV/Henderson/989-Trackers-Glen-Ave-89015/home/29714404',
'https://www.redfin.com/NV/Henderson/2042-Buckeye-Reef-St-89002/home/29716825',
'https://www.redfin.com/NV/Henderson/1581-Equestrian-Dr-89002/home/29717557',
'https://www.redfin.com/NV/Henderson/906-Big-Horn-St-89002/home/29719027',
'https://www.redfin.com/NV/Henderson/406-Scenic-Dr-89002/home/29719844',
'https://www.redfin.com/NV/Henderson/723-Strawberry-Pl-89002/home/29721948',
'https://www.redfin.com/NV/Henderson/124-Mint-Orchard-Dr-89002/home/29722416',
'https://www.redfin.com/NV/Henderson/113-Tamarack-Dr-89002/home/29722493',
'https://www.redfin.com/NV/Henderson/825-Cypress-Pines-Way-89002/home/29724520',
'https://www.redfin.com/NV/Henderson/847-Woodtack-Cove-Way-89002/home/29725328',
'https://www.redfin.com/NV/Boulder-City/1310-Marita-Dr-89005/home/29732402',
'https://www.redfin.com/NV/Boulder-City/768-Christina-Dr-89005/home/29733571',
'https://www.redfin.com/NV/Boulder-City/1504-Irene-Dr-89005/home/29735883',
'https://www.redfin.com/NV/Boulder-City/889-Shirley-Ln-89005/home/29736900',
'https://www.redfin.com/NV/Henderson/231-W-Horizon-Ridge-Pkwy-89012/unit-211/home/29772328',
'https://www.redfin.com/NV/Henderson/50-Aura-de-Blanco-St-89074/unit-17104/home/29772579',
'https://www.redfin.com/NV/Henderson/560-Carmel-Mesa-Dr-89012/home/29838580',
'https://www.redfin.com/NV/Henderson/201-Crown-Imperial-St-89074/home/29880007',
'https://www.redfin.com/NV/Henderson/2556-Forest-City-Dr-89052/home/29886070',
'https://www.redfin.com/NV/Henderson/2008-Colvin-Run-Dr-89052/home/29888844',
'https://www.redfin.com/NV/Henderson/6412-Saddle-Up-Ave-89011/unit-102/home/29894805',
'https://www.redfin.com/NV/Henderson/16-Wade-Hampton-Trl-89052/home/29905296',
'https://www.redfin.com/NV/Henderson/3019-Stratmoor-Hills-Ave-89052/home/29905347',
'https://www.redfin.com/NV/Henderson/91-Stock-Option-St-89074/unit-2/home/29915734',
'https://www.redfin.com/NV/Henderson/1285-Large-Cap-Dr-89074/unit-2/home/29942565',
'https://www.redfin.com/NV/Henderson/728-Tossa-de-Mar-Ave-89002/home/29943660',
'https://www.redfin.com/NV/Henderson/2092-Mountain-City-St-89052/home/29947255',
'https://www.redfin.com/NV/Boulder-City/375-Crystal-Ct-89005/home/33538666',
'https://www.redfin.com/NV/Henderson/789-Calamus-Palm-Pl-89011/home/52623922',
'https://www.redfin.com/NV/Henderson/1309-Jewelstone-Cir-89012/home/60188060',
'https://www.redfin.com/NV/Henderson/1977-Via-Firenze-89044/home/65556563',
'https://www.redfin.com/NV/Henderson/50-Aura-de-Blanco-St-89074/unit-5201/home/109621648',
'https://www.redfin.com/NV/Henderson/1109-Via-Della-Costrella-89011/home/109633497',
'https://www.redfin.com/NV/Henderson/701-Tidal-Flats-St-89002/home/113140376',
'https://www.redfin.com/NV/Henderson/1525-Spiced-Wine-Ave-89074/unit-16104/home/143400805',
'https://www.redfin.com/NV/Las-Vegas/6435-Point-Isabel-Way-89122/home/145870763',
'https://www.redfin.com/NV/Henderson/257-Via-Morelli-89011/home/146416200',
'https://www.redfin.com/NV/Henderson/1072-Upper-Springs-St-89002/home/147876643',
'https://www.redfin.com/NV/Henderson/338-Espressivo-St-89011/home/161754406',
'https://www.redfin.com/NV/Henderson/378-Badinerie-St-89011/home/161755161',
'https://www.redfin.com/NV/Henderson/4-Cloudloft-Ct-89012/home/166426816',
'https://www.redfin.com/NV/Henderson/211-E-Kimberly-Dr-89015/home/166566234',
'https://www.redfin.com/NV/Henderson/403-Andy-Wheeler-Dr-89011/home/167787119',
'https://www.redfin.com/NV/Boulder-City/1535-Bryce-Canyon-St-89005/home/167795877',
'https://www.redfin.com/NV/Henderson/624-Cultures-St-89011/home/169241339',
'https://www.redfin.com/NV/Henderson/965-Nevada-State-Dr-89002/unit-21102/home/169242844',
'https://www.redfin.com/NV/Henderson/89-Lake-Oasis-St-89011/home/171301906',
'https://www.redfin.com/NV/Henderson/932-E-Sunset-Rd-89011/home/171302356',
'https://www.redfin.com/NV/Henderson/361-Seine-Way-89014/unit-1311/home/172345414',
'https://www.redfin.com/NV/Henderson/731-Cottonwood-Hill-Pl-89011/home/175528274',
'https://www.redfin.com/NV/Henderson/729-Cottonwood-Hill-Pl-89011/home/175528275',
'https://www.redfin.com/NV/Henderson/307-Bay-Village-Pl-89011/home/177309564',
'https://www.redfin.com/NV/Henderson/57-Tatahatso-Point-St-89011/home/177327882',
'https://www.redfin.com/NV/Henderson/377-Old-Erie-Pl-89011/home/184007626',
'https://www.redfin.com/NV/Henderson/48-Cadence-Crest-Ave-89011/home/185121364',
'https://www.redfin.com/NV/Henderson/879-Mary-Baer-Way-89011/home/185121370',
'https://www.redfin.com/NV/Henderson/227-Wewatta-AVE-Unknown/home/185575654',
'https://www.redfin.com/NV/Henderson/384-Winter-Dove-ST-89011/unit-298/home/186023685',
'https://www.redfin.com/NV/Henderson/388-Blanca-Springs-Dr-89014/unit-341/home/186133092',
'https://www.redfin.com/NV/Henderson/479-Waterfall-Cove-Ct-89002/home/185775167',
'https://www.redfin.com/NV/Henderson/342-Larksong-AVE-89011/unit-81/home/186090400',
'https://www.redfin.com/NV/Henderson/344-Larksong-AVE-89011/unit-82/home/186138150',
'https://www.redfin.com/NV/Henderson/340-Larksong-AVE-89011/unit-80/home/186139372',
'https://www.redfin.com/NV/Henderson/885-Almandine-Pl-89011/unit-112/home/186161032',
'https://www.redfin.com/NV/Henderson/889-Almandine-Pl-89011/unit-113/home/186161049',
'https://www.redfin.com/NV/Henderson/905-Almandine-Pl-89011/unit-117/home/186161050',
'https://www.redfin.com/NV/Henderson/348-Nesting-Dove-ST-89011/unit-289/home/186200010',
'https://www.redfin.com/NV/Henderson/495-Blue-Ridge-CV-89002/unit-63/home/186205322',
'https://www.redfin.com/NV/Henderson/334-Larksong-AVE-89011/unit-77/home/186207851',
'https://www.redfin.com/NV/Henderson/338-Larksong-AVE-89011/unit-79/home/186208477',
'https://www.redfin.com/NV/Henderson/667-Anemone-Ln-89015/unit-249/home/186215558',
'https://www.redfin.com/NV/Henderson/1187-King-St-89011/unit-37/home/186274557',
'https://www.redfin.com/NV/Henderson/336-Larksong-AVE-89011/unit-78/home/186274617',
'https://www.redfin.com/NV/Henderson/1185-King-St-89011/unit-38/home/186298924',
'https://www.redfin.com/NV/Henderson/333-Seine-Way-89014/unit-1722/home/186318427',
'https://www.redfin.com/NV/Henderson/661-Anemone-Ln-89015/unit-252/home/186335917',
'https://www.redfin.com/NV/Henderson/1181-King-St-89011/unit-40/home/186373631',
'https://www.redfin.com/NV/Las-Vegas/4555-Winter-Place-St-89122/home/29014966',
'https://www.redfin.com/NV/Henderson/717-Oak-Field-Ln-89011/home/29448320',
'https://www.redfin.com/NV/Henderson/416-Bottle-Brush-Way-89015/home/29690847',
'https://www.redfin.com/NV/Boulder-City/667-Avenue-F-89005/home/29732587',
'https://www.redfin.com/NV/Henderson/856-Chameleon-Star-Ave-89015/home/29943837',
'https://www.redfin.com/NV/Henderson/1071-Fish-Pond-Ave-89014/home/64091870',
'https://www.redfin.com/NV/North-Las-Vegas/5816-Michael-Dean-St-89081/home/28964405',
'https://www.redfin.com/NV/North-Las-Vegas/3533-Rubio-Sun-Ave-89081/home/28968439',
'https://www.redfin.com/NV/North-Las-Vegas/323-River-Glider-Ave-89084/home/28969589',
'https://www.redfin.com/NV/North-Las-Vegas/4604-Checker-Way-89031/home/28970257',
'https://www.redfin.com/NV/North-Las-Vegas/1412-Iris-Kelly-Ave-89081/home/28971917',
'https://www.redfin.com/NV/North-Las-Vegas/5744-Dawson-Creek-St-89031/home/28972356',
'https://www.redfin.com/NV/North-Las-Vegas/4643-Mountain-Snow-St-89031/home/28972640',
'https://www.redfin.com/NV/North-Las-Vegas/5855-Valley-Dr-89031/unit-2119/home/28974895',
'https://www.redfin.com/NV/North-Las-Vegas/633-Claxton-Ave-89084/home/28975847',
'https://www.redfin.com/NV/North-Las-Vegas/6516-Copper-Smith-Ct-89084/home/28977295',
'https://www.redfin.com/NV/North-Las-Vegas/3012-Fern-Crest-Ave-89031/home/28977338',
'https://www.redfin.com/NV/North-Las-Vegas/3208-Tanagrine-Dr-89084/home/28980511',
'https://www.redfin.com/NV/North-Las-Vegas/3033-Dowitcher-Ave-89084/home/28980619',
'https://www.redfin.com/NV/North-Las-Vegas/3516-Glorious-Iris-Pl-89084/unit-1/home/28981019',
'https://www.redfin.com/NV/North-Las-Vegas/6625-Lavender-Lilly-Ln-89084/unit-1/home/28981029',
'https://www.redfin.com/NV/North-Las-Vegas/2128-Saddlebill-Ct-89084/home/28981184',
'https://www.redfin.com/NV/North-Las-Vegas/6740-Sand-Swallow-St-89084/home/28981223',
'https://www.redfin.com/NV/North-Las-Vegas/6624-Tumbler-St-89084/home/28981944',
'https://www.redfin.com/NV/North-Las-Vegas/2509-Silvereye-Dr-89084/home/28981947',
'https://www.redfin.com/NV/North-Las-Vegas/4925-Celsion-Rock-St-89081/home/28982641',
'https://www.redfin.com/NV/North-Las-Vegas/4811-Donna-St-89081/home/28982658',
'https://www.redfin.com/NV/North-Las-Vegas/3441-Brolio-Valley-Ct-89032/home/28987404',
'https://www.redfin.com/NV/North-Las-Vegas/5913-Horsehair-Blanket-Dr-89081/home/28997329',
'https://www.redfin.com/NV/North-Las-Vegas/5245-Coral-Hills-St-89081/home/28998014',
'https://www.redfin.com/NV/North-Las-Vegas/1151-Sunny-Acres-Ave-89081/home/28998494',
'https://www.redfin.com/NV/North-Las-Vegas/3128-Gnatcatcher-Ave-89084/home/28999391',
'https://www.redfin.com/NV/North-Las-Vegas/3116-Gnatcatcher-Ave-89084/home/28999547',
'https://www.redfin.com/NV/North-Las-Vegas/6237-Olympic-Gold-St-89031/home/29001655',
'https://www.redfin.com/NV/Las-Vegas/4106-Pistachio-Nut-Ave-89115/home/29007135',
'https://www.redfin.com/NV/North-Las-Vegas/6016-Leaping-Foal-St-89081/home/29010427',
'https://www.redfin.com/NV/North-Las-Vegas/4604-Grey-Heron-Dr-89084/home/29010786',
'https://www.redfin.com/NV/North-Las-Vegas/6208-Darby-Creek-Ct-89081/home/29011450',
'https://www.redfin.com/NV/North-Las-Vegas/1238-Shades-End-Ave-89081/home/29013683',
'https://www.redfin.com/NV/North-Las-Vegas/3832-Alpine-Bypass-Ave-89081/home/29016184',
'https://www.redfin.com/NV/North-Las-Vegas/3613-Fledgling-Dr-89084/home/29016841',
'https://www.redfin.com/NV/North-Las-Vegas/6224-Sun-Seed-Ct-89081/home/29016935',
'https://www.redfin.com/NV/North-Las-Vegas/4444-Carrier-Dove-Ave-89084/home/29017322',
'https://www.redfin.com/NV/North-Las-Vegas/4124-Erinbird-Ave-89084/home/29017779',
'https://www.redfin.com/NV/North-Las-Vegas/4021-California-Condor-Ave-89084/home/29017800',
'https://www.redfin.com/NV/North-Las-Vegas/4605-White-Stork-Dr-89084/home/29017958',
'https://www.redfin.com/NV/North-Las-Vegas/7082-Seabirds-Pl-89084/home/29018200',
'https://www.redfin.com/NV/North-Las-Vegas/3320-Majestic-Sand-Ave-89031/home/29019316',
'https://www.redfin.com/NV/North-Las-Vegas/5841-Vista-Luna-St-89031/home/29019341',
'https://www.redfin.com/NV/North-Las-Vegas/4324-Hera-Temple-Ave-89031/home/29020124',
'https://www.redfin.com/NV/North-Las-Vegas/4117-Boston-Bell-Ct-89031/home/29020137',
'https://www.redfin.com/NV/North-Las-Vegas/113-Frad-Ave-89031/home/29020641',
'https://www.redfin.com/NV/Las-Vegas/5115-Thataboy-Ct-89130/home/29024580',
'https://www.redfin.com/NV/Las-Vegas/4555-Hearts-Desire-Ave-89115/home/29028084',
'https://www.redfin.com/NV/North-Las-Vegas/713-W-Colton-Ave-89032/home/29044051',
'https://www.redfin.com/NV/North-Las-Vegas/5909-Addy-Ln-89081/home/29048533',
'https://www.redfin.com/NV/North-Las-Vegas/3021-Red-Imp-Ave-89081/home/29049078',
'https://www.redfin.com/NV/North-Las-Vegas/6213-Capehart-Falls-St-89081/home/29049214',
'https://www.redfin.com/NV/North-Las-Vegas/6012-Valley-Flower-St-89081/home/29049437',
'https://www.redfin.com/NV/North-Las-Vegas/3129-Little-Crimson-Ave-89081/home/29049592',
'https://www.redfin.com/NV/North-Las-Vegas/5450-Tea-Leaf-St-89031/home/29049976',
'https://www.redfin.com/NV/North-Las-Vegas/1944-W-Hammer-Ln-89031/home/29050197',
'https://www.redfin.com/NV/Las-Vegas/6070-Burleson-Ranch-Rd-89131/home/29051414',
'https://www.redfin.com/NV/North-Las-Vegas/5721-Leonidas-St-89031/home/29059830',
'https://www.redfin.com/NV/North-Las-Vegas/1217-Checkmark-Ave-89032/home/29061929',
'https://www.redfin.com/NV/North-Las-Vegas/5828-Meadow-Dale-Dr-89031/home/29076973',
'https://www.redfin.com/NV/North-Las-Vegas/1704-Black-Hills-Way-89031/home/29077663',
'https://www.redfin.com/NV/North-Las-Vegas/1601-Heather-Ridge-Rd-89031/home/29078367',
'https://www.redfin.com/NV/North-Las-Vegas/908-Crescent-Moon-Dr-89031/home/29078861',
'https://www.redfin.com/NV/North-Las-Vegas/1227-Dover-Glen-Dr-89031/home/29078905',
'https://www.redfin.com/NV/North-Las-Vegas/5604-Indian-Ridge-Dr-89031/home/29079374',
'https://www.redfin.com/NV/North-Las-Vegas/1440-Walstone-Rd-89031/home/29079510',
'https://www.redfin.com/NV/North-Las-Vegas/2603-Sweet-Leilani-Ave-89031/home/29079609',
'https://www.redfin.com/NV/North-Las-Vegas/2123-Maui-Surf-Ave-89031/home/29080087',
'https://www.redfin.com/NV/North-Las-Vegas/2113-Port-Antonio-Ct-89031/home/29083142',
'https://www.redfin.com/NV/North-Las-Vegas/2108-Spanish-Town-Ave-89031/home/29083184',
'https://www.redfin.com/NV/North-Las-Vegas/4836-Abundance-St-89031/home/29083234',
'https://www.redfin.com/NV/North-Las-Vegas/5117-Sierra-Blanca-Ln-89031/home/29083741',
'https://www.redfin.com/NV/North-Las-Vegas/5119-El-Nino-Ln-89031/home/29083744',
'https://www.redfin.com/NV/North-Las-Vegas/1133-Turnstone-Ct-89031/home/29084617',
'https://www.redfin.com/NV/North-Las-Vegas/1125-Egret-Ct-89031/home/29084728',
'https://www.redfin.com/NV/North-Las-Vegas/1217-Stonesthrow-Cir-89031/home/29084754',
'https://www.redfin.com/NV/North-Las-Vegas/5539-Jose-Leon-St-89031/home/29085104',
'https://www.redfin.com/NV/North-Las-Vegas/5241-Jose-Ernesto-St-89031/home/29085326',
'https://www.redfin.com/NV/North-Las-Vegas/220-Breezy-Shore-Ave-89031/home/29086727',
'https://www.redfin.com/NV/Las-Vegas/6016-Morning-Creek-Ct-89130/home/29092774',
'https://www.redfin.com/NV/Las-Vegas/6320-Whispering-Meadow-Ct-89130/home/29093064',
'https://www.redfin.com/NV/Las-Vegas/6317-Winter-Valley-Ct-89130/home/29093066',
'https://www.redfin.com/NV/Las-Vegas/6016-Winter-Meadow-St-89130/home/29093120',
'https://www.redfin.com/NV/Las-Vegas/5916-Trumbull-St-89130/home/29093246',
'https://www.redfin.com/NV/Las-Vegas/6101-Shallow-Springs-St-89130/home/29093517',
'https://www.redfin.com/NV/Las-Vegas/5421-Singing-Hills-Dr-89130/home/29102028',
'https://www.redfin.com/NV/Las-Vegas/5317-Braemar-Dr-89130/home/29102324',
'https://www.redfin.com/NV/Las-Vegas/5533-Port-Barrington-Way-89130/home/29102467',
'https://www.redfin.com/NV/Las-Vegas/4937-Rancho-Bernardo-Way-89130/home/29102624',
'https://www.redfin.com/NV/Las-Vegas/5604-Ocean-Pines-Cir-89130/home/29102698',
'https://www.redfin.com/NV/Las-Vegas/5037-Cedar-Lawn-Way-89130/home/29102964',
'https://www.redfin.com/NV/Las-Vegas/4533-Mancilla-St-89130/home/29115744',
'https://www.redfin.com/NV/Las-Vegas/4504-Plano-Dr-89130/home/29115859',
'https://www.redfin.com/NV/Las-Vegas/5424-Del-Lago-Dr-89130/home/29116287',
'https://www.redfin.com/NV/Las-Vegas/4980-W-San-Miguel-Ave-89130/home/29116772',
'https://www.redfin.com/NV/Las-Vegas/5908-Duncan-Dr-89108/home/29132405',
'https://www.redfin.com/NV/Las-Vegas/5812-Robin-Ln-89108/home/29133318',
'https://www.redfin.com/NV/Las-Vegas/3205-Point-Lobos-Dr-89108/home/29134056',
'https://www.redfin.com/NV/Las-Vegas/3916-Wharton-St-89130/home/29134258',
'https://www.redfin.com/NV/Las-Vegas/5515-Sheila-Ave-89108/home/29134938',
'https://www.redfin.com/NV/Las-Vegas/5425-Geneive-Ln-89108/unit-104/home/29137142',
'https://www.redfin.com/NV/Las-Vegas/5401-Geneive-Ln-89108/unit-101/home/29137198',
'https://www.redfin.com/NV/Las-Vegas/2204-Obispo-Cir-89108/home/29172566',
'https://www.redfin.com/NV/Las-Vegas/5917-Smoke-Ranch-Rd-89108/unit-D/home/29174373',
'https://www.redfin.com/NV/Las-Vegas/1628-N-Jones-Blvd-89108/home/29177264',
'https://www.redfin.com/NV/Las-Vegas/5378-W-Lake-Mead-Blvd-89108/home/29177321',
'https://www.redfin.com/NV/North-Las-Vegas/4012-Yellow-Mandarin-Ave-89081/home/29178219',
'https://www.redfin.com/NV/Las-Vegas/5921-Vegas-Dr-89108/home/29181770',
'https://www.redfin.com/NV/North-Las-Vegas/1103-Stoneypeak-Ave-89081/home/29236127',
'https://www.redfin.com/NV/North-Las-Vegas/4506-Minaret-Way-89031/home/29239506',
'https://www.redfin.com/NV/North-Las-Vegas/4028-Cricket-Ln-89032/home/29240817',
'https://www.redfin.com/NV/North-Las-Vegas/3733-Calumet-Farm-Cir-89031/home/29242138',
'https://www.redfin.com/NV/North-Las-Vegas/3709-Bordley-Way-89032/home/29244834',
'https://www.redfin.com/NV/North-Las-Vegas/4339-Sparta-Way-89032/home/29245206',
'https://www.redfin.com/NV/North-Las-Vegas/3516-Barrel-Race-Ct-89032/home/29245933',
'https://www.redfin.com/NV/North-Las-Vegas/4407-Red-Blanket-Rd-89032/home/29245989',
'https://www.redfin.com/NV/North-Las-Vegas/3716-Fountain-Ct-89032/home/29247433',
'https://www.redfin.com/NV/North-Las-Vegas/3820-Iverson-Ln-89032/home/29250323',
'https://www.redfin.com/NV/North-Las-Vegas/1881-W-Alexander-Rd-89032/unit-2149/home/29251690',
'https://www.redfin.com/NV/North-Las-Vegas/1516-Bluestar-Dr-89032/home/29252258',
'https://www.redfin.com/NV/North-Las-Vegas/3528-Beca-Faith-Dr-89032/home/29252430',
'https://www.redfin.com/NV/North-Las-Vegas/1532-Highfield-Ct-89032/home/29252706',
'https://www.redfin.com/NV/North-Las-Vegas/1324-Frost-Flower-Dr-89032/home/29253392',
'https://www.redfin.com/NV/North-Las-Vegas/3343-Outlook-Point-St-89032/home/29253559',
'https://www.redfin.com/NV/North-Las-Vegas/3301-Outlook-Point-St-89032/home/29253564',
'https://www.redfin.com/NV/North-Las-Vegas/3625-Rose-Canyon-Dr-89032/home/29254537',
'https://www.redfin.com/NV/North-Las-Vegas/3412-Mercury-St-89030/unit-I/home/29255374',
'https://www.redfin.com/NV/North-Las-Vegas/2200-Comet-Ave-89030/unit-C/home/29256236',
'https://www.redfin.com/NV/North-Las-Vegas/2120-Sun-Ave-89030/unit-C/home/29256298',
'https://www.redfin.com/NV/North-Las-Vegas/2112-Sun-Ave-89030/unit-F/home/29256306',
'https://www.redfin.com/NV/North-Las-Vegas/3470-Berg-St-89030/home/29256616',
'https://www.redfin.com/NV/North-Las-Vegas/2515-Bassler-St-89030/home/29258867',
'https://www.redfin.com/NV/North-Las-Vegas/2712-Holmes-St-89030/home/29259050',
'https://www.redfin.com/NV/North-Las-Vegas/2617-N-Bruce-St-89030/home/29261856',
'https://www.redfin.com/NV/Las-Vegas/4425-Cormorant-Ave-89115/home/29263467',
'https://www.redfin.com/NV/North-Las-Vegas/1871-Renada-Cir-89030/home/29263641',
'https://www.redfin.com/NV/North-Las-Vegas/3725-Citrus-Heights-Ave-89081/home/29264190',
'https://www.redfin.com/NV/North-Las-Vegas/3609-Inverness-Grove-Ave-89081/home/29264283',
'https://www.redfin.com/NV/North-Las-Vegas/3540-Turquoise-Waters-Ave-89081/home/29264707',
'https://www.redfin.com/NV/North-Las-Vegas/3612-Turquoise-Waters-Ave-89081/home/29264888',
'https://www.redfin.com/NV/North-Las-Vegas/2209-E-Piper-Ave-89030/home/29264893',
'https://www.redfin.com/NV/North-Las-Vegas/6745-Caporetto-Ln-89084/unit-102/home/29267106',
'https://www.redfin.com/NV/North-Las-Vegas/7070-Arcadia-Glen-Ct-89084/home/29267752',
'https://www.redfin.com/NV/North-Las-Vegas/316-Whitney-Breeze-Ave-89031/home/29268867',
'https://www.redfin.com/NV/North-Las-Vegas/216-Gemstone-Hill-Ave-89031/home/29269091',
'https://www.redfin.com/NV/North-Las-Vegas/2524-Spear-St-89030/home/29280897',
'https://www.redfin.com/NV/North-Las-Vegas/435-Dorchester-Bend-Ave-89032/unit-3/home/29280967',
'https://www.redfin.com/NV/Las-Vegas/2332-Bristol-View-Ct-89108/home/29305157',
'https://www.redfin.com/NV/Las-Vegas/4428-W-Lake-Mead-Blvd-89108/unit-202/home/29306179',
'https://www.redfin.com/NV/Las-Vegas/2117-Valley-Dr-89108/home/29306377',
'https://www.redfin.com/NV/Las-Vegas/1816-N-Decatur-Blvd-89108/unit-201/home/29307159',
'https://www.redfin.com/NV/Las-Vegas/4413-Coran-Ln-89108/home/29307372',
'https://www.redfin.com/NV/Las-Vegas/3008-Morningside-Ave-89106/home/29307757',
'https://www.redfin.com/NV/Las-Vegas/1812-Windchime-Dr-89106/home/29308825',
'https://www.redfin.com/NV/Las-Vegas/1816-Luning-Dr-89106/home/29310496',
'https://www.redfin.com/NV/Las-Vegas/2521-Perryville-Ave-89106/unit-104/home/29310634',
'https://www.redfin.com/NV/North-Las-Vegas/2037-Rose-St-89032/home/29311817',
'https://www.redfin.com/NV/Las-Vegas/875-Balzar-Ave-89106/home/29321865',
'https://www.redfin.com/NV/North-Las-Vegas/1912-Hoover-St-89030/home/29326589',
'https://www.redfin.com/NV/North-Las-Vegas/3208-Tabor-Ave-89030/home/29330401',
'https://www.redfin.com/NV/North-Las-Vegas/3309-Thomas-Ave-89030/home/29330529',
'https://www.redfin.com/NV/North-Las-Vegas/3701-Judson-Ave-89030/home/29330593',
'https://www.redfin.com/NV/North-Las-Vegas/3301-Judson-Ave-89030/home/29330642',
'https://www.redfin.com/NV/North-Las-Vegas/3615-Perliter-Ave-89030/home/29330934',
'https://www.redfin.com/NV/North-Las-Vegas/3420-E-Tonopah-Ave-89030/home/29331162',
'https://www.redfin.com/NV/Las-Vegas/1334-Smoke-Tree-Ave-89108/home/29338357',
'https://www.redfin.com/NV/Las-Vegas/1300-Smoke-Tree-Ave-89108/home/29338443',
'https://www.redfin.com/NV/Las-Vegas/3700-Briarglen-Ln-89108/home/29341997',
'https://www.redfin.com/NV/Las-Vegas/4248-Vornsand-Dr-89115/home/29352257',
'https://www.redfin.com/NV/Las-Vegas/4284-Vornsand-Dr-89115/home/29352271',
'https://www.redfin.com/NV/Las-Vegas/3730-Twinkle-Star-Dr-89115/home/29352775',
'https://www.redfin.com/NV/Las-Vegas/4270-Wendy-Ln-89115/home/29352909',
'https://www.redfin.com/NV/Las-Vegas/4022-Wendy-Ln-89115/home/29353796',
'https://www.redfin.com/NV/Las-Vegas/3871-Lancome-St-89115/home/29356974',
'https://www.redfin.com/NV/Las-Vegas/3757-Longfellow-St-89115/home/29357073',
'https://www.redfin.com/NV/North-Las-Vegas/5104-Harold-St-89081/home/29449056',
'https://www.redfin.com/NV/North-Las-Vegas/5948-Bellows-Beach-St-89081/home/29476800',
'https://www.redfin.com/NV/North-Las-Vegas/1010-Sunny-Acres-Ave-89081/home/29597120',
'https://www.redfin.com/NV/North-Las-Vegas/5062-Bayberry-Crest-St-89031/home/29598269',
'https://www.redfin.com/NV/North-Las-Vegas/3936-Fuselier-Dr-89032/home/29599396',
'https://www.redfin.com/NV/North-Las-Vegas/3829-Alpine-Track-Ave-89032/home/29781515',
'https://www.redfin.com/NV/North-Las-Vegas/5358-Ferrell-St-89031/home/29843745',
'https://www.redfin.com/NV/North-Las-Vegas/6662-Montezuma-Castle-Ln-89084/home/29851718',
'https://www.redfin.com/NV/North-Las-Vegas/1122-Granite-Ash-Ave-89081/home/29854357',
'https://www.redfin.com/NV/North-Las-Vegas/3411-Fort-Niagara-Ave-89032/home/29859302',
'https://www.redfin.com/NV/North-Las-Vegas/2430-Country-Orchard-St-89030/home/29859913',
'https://www.redfin.com/NV/North-Las-Vegas/2809-Albata-St-89030/home/29860046',
'https://www.redfin.com/NV/Las-Vegas/1631-Morse-Arberry-Ave-89106/home/29860968',
'https://www.redfin.com/NV/Las-Vegas/1704-Dwayne-Stedman-Ave-89106/home/29861006',
'https://www.redfin.com/NV/Las-Vegas/4730-E-Craig-Rd-89115/unit-1166/home/29862551',
'https://www.redfin.com/NV/North-Las-Vegas/2926-Walrus-Islands-Ct-89031/home/29886895',
'https://www.redfin.com/NV/North-Las-Vegas/5302-Big-Range-St-89031/home/29892203',
'https://www.redfin.com/NV/North-Las-Vegas/1102-Elliot-Park-Ave-89032/home/29894093',
'https://www.redfin.com/NV/North-Las-Vegas/1113-Christopher-View-Ave-89032/home/29894100',
'https://www.redfin.com/NV/North-Las-Vegas/5326-Kadena-Garden-Ct-89031/home/29919735',
'https://www.redfin.com/NV/Las-Vegas/6229-Eagle-Crossing-St-89130/home/29921118',
'https://www.redfin.com/NV/Las-Vegas/6493-Yellow-Bells-Ct-89131/home/29928568',
'https://www.redfin.com/NV/Las-Vegas/6493-Lantana-Falls-Ct-89131/home/29928645',
'https://www.redfin.com/NV/North-Las-Vegas/4716-Ferrell-St-89031/home/29932495',
'https://www.redfin.com/NV/North-Las-Vegas/6528-Bright-Morning-St-89084/home/29952095',
'https://www.redfin.com/NV/North-Las-Vegas/2112-Meadow-Green-Ave-89031/home/29954918',
'https://www.redfin.com/NV/North-Las-Vegas/5345-Brazelton-St-89081/home/30192840',
'https://www.redfin.com/NV/North-Las-Vegas/108-Elks-Peak-Ave-89084/home/33507294',
'https://www.redfin.com/NV/North-Las-Vegas/4137-Keasberry-Ave-89081/home/40232910',
'https://www.redfin.com/NV/Las-Vegas/4437-Dover-Straight-St-89115/unit-202/home/40234735',
'https://www.redfin.com/NV/Las-Vegas/4437-Dover-Straight-St-89115/unit-207/home/40234738',
'https://www.redfin.com/NV/North-Las-Vegas/3513-Storks-Bundle-Ln-89084/home/40534226',
'https://www.redfin.com/NV/North-Las-Vegas/5840-Montina-Vines-St-89081/home/52622144',
'https://www.redfin.com/NV/North-Las-Vegas/3808-Citrus-Heights-Ave-89081/home/65549955',
'https://www.redfin.com/NV/North-Las-Vegas/5741-Keystone-Crest-St-89081/home/92905186',
'https://www.redfin.com/NV/North-Las-Vegas/5741-Clear-Haven-Ln-89081/home/109620962',
'https://www.redfin.com/NV/North-Las-Vegas/5682-Colbert-St-89081/home/113139411',
'https://www.redfin.com/NV/North-Las-Vegas/6454-Cinnamon-Hazlenut-St-89084/home/143128831',
'https://www.redfin.com/NV/North-Las-Vegas/3796-Nerine-Pass-Way-89032/home/144626944',
'https://www.redfin.com/NV/North-Las-Vegas/5632-Akula-Bay-St-89081/home/145866395',
'https://www.redfin.com/NV/North-Las-Vegas/5853-Petrified-Tree-Ln-89081/home/145869556',
'https://www.redfin.com/NV/North-Las-Vegas/4556-Roaming-Vines-St-89031/home/145873671',
'https://www.redfin.com/NV/North-Las-Vegas/6239-Orions-Belt-Peak-St-89031/home/146417819',
'https://www.redfin.com/NV/North-Las-Vegas/1826-Yale-St-89030/home/147676348',
'https://www.redfin.com/NV/North-Las-Vegas/6508-Towerstone-St-89084/home/147875061',
'https://www.redfin.com/NV/North-Las-Vegas/6234-Orions-Belt-Peak-St-89031/home/147875671',
'https://www.redfin.com/NV/North-Las-Vegas/2416-Evening-Snow-Ct-89030/home/160497039',
'https://www.redfin.com/NV/North-Las-Vegas/3941-Aracari-Dr-89031/home/161751726',
'https://www.redfin.com/NV/North-Las-Vegas/5837-Delaney-Falls-St-89081/home/165730939',
'https://www.redfin.com/NV/North-Las-Vegas/537-Monarch-Ridge-Ct-89084/home/165736470',
'https://www.redfin.com/NV/North-Las-Vegas/4212-Juanita-May-Ave-89032/home/166108325',
'https://www.redfin.com/NV/North-Las-Vegas/3621-Zug-Bug-St-89032/home/166109615',
'https://www.redfin.com/NV/North-Las-Vegas/317-Stone-Fly-Rd-89032/home/166109624',
'https://www.redfin.com/NV/Las-Vegas/4481-Dover-Straight-St-89115/home/166121075',
'https://www.redfin.com/NV/North-Las-Vegas/4482-Brasada-Ranch-Ct-89031/home/166447829',
'https://www.redfin.com/NV/North-Las-Vegas/6938-Denio-Island-St-89084/home/166572294',
'https://www.redfin.com/NV/Las-Vegas/1285-Blankenship-Ave-89106/home/166583795',
'https://www.redfin.com/NV/North-Las-Vegas/7114-Red-Gulch-St-89084/home/167606567',
'https://www.redfin.com/NV/North-Las-Vegas/129-Mountain-Cedar-Ct-89031/home/167609307',
'https://www.redfin.com/NV/North-Las-Vegas/4531-Cradle-Cliff-St-89031/home/167654721',
'https://www.redfin.com/NV/North-Las-Vegas/6645-Evander-St-89086/home/169224926',
'https://www.redfin.com/NV/North-Las-Vegas/2316-Mystic-Wave-Rd-89031/home/170137348',
'https://www.redfin.com/NV/North-Las-Vegas/4355-Countryside-Glen-Ct-89084/home/170367594',
'https://www.redfin.com/NV/North-Las-Vegas/1006-Crystal-Grotto-Ave-89081/home/170368844',
'https://www.redfin.com/NV/North-Las-Vegas/4731-Agave-Cactus-St-89031/home/170372237',
'https://www.redfin.com/NV/North-Las-Vegas/6817-Toll-Mountain-St-89086/home/170968601',
'https://www.redfin.com/NV/North-Las-Vegas/2912-Sweeping-Palm-Rd-89086/home/170968949',
'https://www.redfin.com/NV/North-Las-Vegas/2908-Sweeping-Palm-Rd-89086/home/170968951',
'https://www.redfin.com/NV/North-Las-Vegas/2874-Roaming-Palmer-Rd-89086/home/170969037',
'https://www.redfin.com/NV/North-Las-Vegas/1201-Crystal-Grotto-Ave-89081/home/171287930',
'https://www.redfin.com/NV/North-Las-Vegas/6804-Surf-Green-Ct-89086/home/172603120',
'https://www.redfin.com/NV/North-Las-Vegas/6711-Palmer-Crest-St-89086/home/173453305',
'https://www.redfin.com/NV/North-Las-Vegas/4623-Observed-St-89031/home/173453834',
'https://www.redfin.com/NV/North-Las-Vegas/6734-Palmer-Estates-St-89086/home/174789327',
'https://www.redfin.com/NV/North-Las-Vegas/6722-Palmer-Estates-St-89086/home/174789330',
'https://www.redfin.com/NV/North-Las-Vegas/6959-Wildhorse-Creek-St-89086/home/175522419',
'https://www.redfin.com/NV/North-Las-Vegas/605-El-Gusto-Ave-89081/home/175522900',
'https://www.redfin.com/NV/North-Las-Vegas/2113-Dalewood-Ave-89086/home/176104742',
'https://www.redfin.com/NV/North-Las-Vegas/1062-Twincrest-Ave-89032/home/176107447',
'https://www.redfin.com/NV/North-Las-Vegas/2654-Fawn-Beige-Ct-89086/home/177325231',
'https://www.redfin.com/NV/North-Las-Vegas/6948-Crescent-Gold-St-89086/home/177325291',
'https://www.redfin.com/NV/North-Las-Vegas/6021-Torciano-St-89081/home/177334414',
'https://www.redfin.com/NV/North-Las-Vegas/7103-Stockton-Dunes-St-89084/home/179956239',
'https://www.redfin.com/NV/North-Las-Vegas/50-Gosia-Grove-Ave-89031/home/179957007',
'https://www.redfin.com/NV/North-Las-Vegas/46-Gosia-Grove-Ave-89031/home/179957008',
'https://www.redfin.com/NV/North-Las-Vegas/104-Barton-Bay-Ave-89031/home/179957017',
'https://www.redfin.com/NV/North-Las-Vegas/4517-Queens-Hill-St-89031/home/179978358',
'https://www.redfin.com/NV/North-Las-Vegas/4521-Queens-Hill-St-89031/home/179978359',
'https://www.redfin.com/NV/North-Las-Vegas/4526-Queens-Hill-St-89031/home/179978387',
'https://www.redfin.com/NV/North-Las-Vegas/1639-Oro-Grande-Dr-89032/home/179978478',
'https://www.redfin.com/NV/North-Las-Vegas/1635-Oro-Grande-Dr-89032/home/179978479',
'https://www.redfin.com/NV/North-Las-Vegas/1631-Oro-Grande-Dr-89032/home/179978480',
'https://www.redfin.com/NV/North-Las-Vegas/1590-Gold-Sunset-Ave-89032/home/179978615',
'https://www.redfin.com/NV/North-Las-Vegas/4144-Sandia-Star-Dr-89032/home/179978618',
'https://www.redfin.com/NV/North-Las-Vegas/4148-Sandia-Star-Dr-89032/home/179978619',
'https://www.redfin.com/NV/North-Las-Vegas/4156-Sandia-Star-Dr-89032/home/179978621',
'https://www.redfin.com/NV/North-Las-Vegas/6355-Selston-Dr-89081/home/181098995',
'https://www.redfin.com/NV/Las-Vegas/4932-Chidlaw-St-89115/unit-1/home/183991552',
'https://www.redfin.com/NV/Las-Vegas/4932-Chidlaw-St-89115/unit-3/home/183991559',
'https://www.redfin.com/NV/North-Las-Vegas/128-Junction-Peak-Ave-89031/home/185108853',
'https://www.redfin.com/NV/Las-Vegas/3710-Coleus-AVE-89108/unit-3/home/185911190',
'https://www.redfin.com/NV/North-Las-Vegas/6220-Demille-Ct-89081/home/28981990',
'https://www.redfin.com/NV/North-Las-Vegas/3012-Gnatcatcher-Ave-89084/home/29003013',
'https://www.redfin.com/NV/North-Las-Vegas/431-Yarnell-Dr-89031/home/29085276',
'https://www.redfin.com/NV/Las-Vegas/5904-Edrene-Ave-89108/home/29176233',
'https://www.redfin.com/NV/North-Las-Vegas/3520-Flower-Ave-89030/home/29330949',
'https://www.redfin.com/NV/North-Las-Vegas/1612-Logan-Valley-Ln-89081/home/29013124',
'https://www.redfin.com/NV/North-Las-Vegas/3932-Pepper-Thorn-Ave-89081/unit-2/home/29016303',
'https://www.redfin.com/NV/North-Las-Vegas/5239-Sugar-Maple-Ct-89031/home/29085148',
'https://www.redfin.com/NV/Las-Vegas/5609-Reiter-Ave-89108/home/29175388',
'https://www.redfin.com/NV/North-Las-Vegas/1304-E-Brooks-Ave-89030/home/29262132',
'https://www.redfin.com/NV/N-Las-Vegas/Address-Redacted-89031/home/29268765',
'https://www.redfin.com/NV/North-Las-Vegas/4350-Westbury-Square-St-89032/unit-1/home/29280978',
'https://www.redfin.com/NV/Las-Vegas/5344-Standing-Rock-Pl-89130/home/28967715',
'https://www.redfin.com/NV/Las-Vegas/5455-Cardinal-Ridge-Ct-89149/unit-202/home/28969903',
'https://www.redfin.com/NV/Las-Vegas/9421-Eagle-River-Ct-89166/home/28972613',
'https://www.redfin.com/NV/Las-Vegas/7917-Orchard-Port-Ave-89131/home/28977263',
'https://www.redfin.com/NV/Las-Vegas/8820-Saddle-Red-Ave-89143/home/28977819',
'https://www.redfin.com/NV/Las-Vegas/7104-Via-Campanile-Ave-89131/home/28979924',
'https://www.redfin.com/NV/Las-Vegas/9724-Red-Deer-St-89143/home/28983073',
'https://www.redfin.com/NV/Las-Vegas/8136-Nestled-Ct-89143/home/28983558',
'https://www.redfin.com/NV/Las-Vegas/8112-Loma-del-Ray-St-89131/home/28984267',
'https://www.redfin.com/NV/Las-Vegas/8036-Pasadera-St-89131/home/28984316',
'https://www.redfin.com/NV/Las-Vegas/5908-Audria-Falls-Ave-89131/home/28984945',
'https://www.redfin.com/NV/Las-Vegas/9546-Michelle-Falls-Ave-89149/home/29003683',
'https://www.redfin.com/NV/Las-Vegas/9305-Hampstead-Hills-Ave-89149/home/29004995',
'https://www.redfin.com/NV/Las-Vegas/9515-Havasu-Canyon-Ave-89166/home/29005497',
'https://www.redfin.com/NV/Las-Vegas/6476-Sandy-Copper-Ct-89131/home/29006117',
'https://www.redfin.com/NV/Las-Vegas/8065-Villa-Armando-St-89131/home/29008235',
'https://www.redfin.com/NV/Las-Vegas/8640-Painted-Horseshoe-St-89131/home/29009717',
'https://www.redfin.com/NV/Las-Vegas/9069-Adorable-Ave-89149/home/29011602',
'https://www.redfin.com/NV/Las-Vegas/7257-Royal-Melbourne-Dr-89131/home/29011630',
'https://www.redfin.com/NV/Las-Vegas/7404-Via-Fiorentino-St-89131/home/29015148',
'https://www.redfin.com/NV/Las-Vegas/8917-Monte-Oro-Dr-89131/home/29020290',
'https://www.redfin.com/NV/Las-Vegas/8917-Briar-Bay-Dr-89131/home/29020874',
'https://www.redfin.com/NV/Las-Vegas/8731-James-Raul-Ave-89143/home/29021009',
'https://www.redfin.com/NV/Las-Vegas/9136-Captivating-Ave-89149/home/29022297',
'https://www.redfin.com/NV/Las-Vegas/9141-Captivating-Ave-89149/home/29022318',
'https://www.redfin.com/NV/Las-Vegas/7913-Arden-Point-St-89149/home/29022805',
'https://www.redfin.com/NV/Las-Vegas/7852-Solid-Horn-Ct-89149/home/29023048',
'https://www.redfin.com/NV/Las-Vegas/9928-Kudo-Ct-89149/home/29023121',
'https://www.redfin.com/NV/Las-Vegas/6820-Sleepy-Pine-St-89130/home/29024467',
'https://www.redfin.com/NV/Las-Vegas/7580-Prancing-Pony-Ct-89131/home/29050606',
'https://www.redfin.com/NV/Las-Vegas/6632-Bluebell-Garden-St-89149/home/29051201',
'https://www.redfin.com/NV/Las-Vegas/8529-Caladium-Ct-89149/home/29051227',
'https://www.redfin.com/NV/Las-Vegas/6086-Denton-Ranch-Rd-89131/home/29051241',
'https://www.redfin.com/NV/Las-Vegas/6232-Tempting-Choice-Ave-89131/home/29060248',
'https://www.redfin.com/NV/Las-Vegas/8137-Satin-Carnation-Ln-89166/home/29060435',
'https://www.redfin.com/NV/Las-Vegas/6125-Tarrant-Ranch-Rd-89131/home/29060816',
'https://www.redfin.com/NV/Las-Vegas/6955-N-Durango-Dr-89149/unit-1047/home/29061019',
'https://www.redfin.com/NV/Las-Vegas/6955-N-Durango-Dr-89149/unit-1075/home/29061034',
'https://www.redfin.com/NV/Las-Vegas/3975-N-Hualapai-Way-89129/unit-256/home/29061259',
'https://www.redfin.com/NV/Las-Vegas/3975-N-Hualapai-Way-89129/unit-228/home/29061315',
'https://www.redfin.com/NV/Las-Vegas/8709-Grazing-Hill-Ct-89143/home/29087101',
'https://www.redfin.com/NV/Las-Vegas/8937-Iron-Hitch-Ave-89143/home/29087218',
'https://www.redfin.com/NV/Las-Vegas/9775-Homestead-Rd-89143/home/29087373',
'https://www.redfin.com/NV/Las-Vegas/8921-Tumblewood-Ave-89143/home/29087512',
'https://www.redfin.com/NV/Las-Vegas/8729-Burning-Hide-Ave-89143/home/29087661',
'https://www.redfin.com/NV/Las-Vegas/8424-Wild-Diamond-Ave-89143/home/29087946',
'https://www.redfin.com/NV/Las-Vegas/7820-Thorne-Pine-Ave-89131/home/29090088',
'https://www.redfin.com/NV/Las-Vegas/7809-Amulet-St-89131/home/29090263',
'https://www.redfin.com/NV/Las-Vegas/7905-Elk-Stone-Ave-89131/home/29090763',
'https://www.redfin.com/NV/Las-Vegas/7921-Lovely-Pine-Pl-89143/home/29091078',
'https://www.redfin.com/NV/Las-Vegas/8112-Boxberry-Ave-89131/home/29091590',
'https://www.redfin.com/NV/Las-Vegas/7804-Purple-Mountain-Ave-89131/home/29091702',
'https://www.redfin.com/NV/Las-Vegas/8205-Antler-Pines-Ct-89149/home/29091914',
'https://www.redfin.com/NV/Las-Vegas/6812-Tartarus-St-89131/home/29092130',
'https://www.redfin.com/NV/Las-Vegas/6924-Bienville-St-89131/home/29092930',
'https://www.redfin.com/NV/Las-Vegas/7213-Grand-Palace-Ave-89130/home/29095278',
'https://www.redfin.com/NV/Las-Vegas/5900-Taj-Mahal-Dr-89130/home/29095508',
'https://www.redfin.com/NV/Las-Vegas/5924-Kane-Holly-St-89130/home/29095513',
'https://www.redfin.com/NV/Las-Vegas/8040-Edward-Baher-Ave-89149/home/29095775',
'https://www.redfin.com/NV/Las-Vegas/9662-Azure-Dr-89149/home/29096996',
'https://www.redfin.com/NV/Las-Vegas/9665-W-Regena-Ave-89149/home/29096999',
'https://www.redfin.com/NV/Las-Vegas/5245-N-Riley-St-89149/home/29097445',
'https://www.redfin.com/NV/Las-Vegas/9045-W-Rosada-Way-89149/home/29097602',
'https://www.redfin.com/NV/Las-Vegas/9085-W-Rosada-Way-89149/home/29097603',
'https://www.redfin.com/NV/Las-Vegas/8324-Green-Clover-Ave-89149/home/29098000',
'https://www.redfin.com/NV/Las-Vegas/8221-Green-Clover-Ave-89149/home/29098049',
'https://www.redfin.com/NV/Las-Vegas/8101-Kimwood-Ave-89149/home/29098193',
'https://www.redfin.com/NV/Las-Vegas/7620-Sudan-Ct-89149/home/29098822',
'https://www.redfin.com/NV/Las-Vegas/5124-N-Valadez-St-89149/home/29099131',
'https://www.redfin.com/NV/Las-Vegas/7448-Desert-Flame-Ct-89149/home/29099277',
'https://www.redfin.com/NV/Las-Vegas/6812-Bottle-Sage-Ave-89130/home/29099692',
'https://www.redfin.com/NV/Las-Vegas/6909-Squaw-Mountain-Dr-89130/unit-202/home/29099788',
'https://www.redfin.com/NV/Las-Vegas/6928-White-Lakes-Ave-89130/home/29099821',
'https://www.redfin.com/NV/Las-Vegas/6717-Snake-River-Ave-89130/home/29099890',
'https://www.redfin.com/NV/Las-Vegas/4920-Black-Bear-Rd-89149/unit-202/home/29100384',
'https://www.redfin.com/NV/Las-Vegas/4830-Grey-Wolf-Ln-89149/unit-203/home/29100732',
'https://www.redfin.com/NV/Las-Vegas/4816-Storm-Mountain-St-89130/home/29101019',
'https://www.redfin.com/NV/Las-Vegas/5520-Jasper-Butte-St-89130/home/29101382',
'https://www.redfin.com/NV/Las-Vegas/6031-Ruffian-Rd-89149/home/29104232',
'https://www.redfin.com/NV/Las-Vegas/6421-Faith-Peak-Dr-89108/home/29117156',
'https://www.redfin.com/NV/Las-Vegas/6616-Ouida-Way-89108/home/29117824',
'https://www.redfin.com/NV/Las-Vegas/4101-Browndeer-Cir-89129/home/29119347',
'https://www.redfin.com/NV/Las-Vegas/4028-Compass-Rose-Way-89108/home/29119619',
'https://www.redfin.com/NV/Las-Vegas/7704-Wedlock-Ln-89129/home/29120301',
'https://www.redfin.com/NV/Las-Vegas/7717-Wedlock-Ln-89129/home/29120310',
'https://www.redfin.com/NV/Las-Vegas/4355-Kevin-Way-89129/home/29120783',
'https://www.redfin.com/NV/Las-Vegas/4001-Antique-Sterling-Ct-89129/home/29120810',
'https://www.redfin.com/NV/Las-Vegas/4785-N-Grand-Canyon-Dr-89129/home/29121009',
'https://www.redfin.com/NV/Las-Vegas/8105-Defiance-Ave-89129/home/29126336',
'https://www.redfin.com/NV/Las-Vegas/3888-Quadrel-St-89129/home/29127580',
'https://www.redfin.com/NV/Las-Vegas/3912-Crema-Ct-89129/home/29128157',
'https://www.redfin.com/NV/Las-Vegas/3804-Bessemer-Ct-89129/home/29128647',
'https://www.redfin.com/NV/Las-Vegas/6557-Canyon-Cove-Way-89108/home/29130773',
'https://www.redfin.com/NV/Las-Vegas/6317-W-Alexander-Rd-89108/home/29131260',
'https://www.redfin.com/NV/Las-Vegas/8250-N-Grand-Canyon-Dr-89166/unit-2184/home/29179937',
'https://www.redfin.com/NV/Las-Vegas/6451-Iguassu-Falls-Rd-89149/home/29180079',
'https://www.redfin.com/NV/Las-Vegas/8318-Alameda-Falls-Ave-89131/home/29271892',
'https://www.redfin.com/NV/Las-Vegas/9336-Cranesbill-Ct-89149/home/29273782',
'https://www.redfin.com/NV/Las-Vegas/9252-Lopseed-Ct-89149/home/29273839',
'https://www.redfin.com/NV/Las-Vegas/6805-Mansion-Oaks-St-89149/home/29274068',
'https://www.redfin.com/NV/Las-Vegas/8448-Lucero-Ave-89149/home/29275197',
'https://www.redfin.com/NV/Las-Vegas/6868-Sky-Pointe-Dr-89131/unit-2083/home/29275396',
'https://www.redfin.com/NV/Las-Vegas/10769-Tuckermans-Ave-89129/home/29276541',
'https://www.redfin.com/NV/Las-Vegas/10438-Gold-Shadow-Ave-89129/home/29277178',
'https://www.redfin.com/NV/Las-Vegas/7971-Lookout-Rock-Cir-89129/home/29479245',
'https://www.redfin.com/NV/Las-Vegas/9535-New-Harbor-Ave-89149/home/29598062',
'https://www.redfin.com/NV/Las-Vegas/8562-Golden-Idol-Ct-89131/home/29598146',
'https://www.redfin.com/NV/Las-Vegas/10715-Iona-Island-Ave-89166/home/29759373',
'https://www.redfin.com/NV/Las-Vegas/7627-Anatolian-St-89166/home/29759399',
'https://www.redfin.com/NV/Las-Vegas/10148-Ocicat-Ave-89166/home/29759419',
'https://www.redfin.com/NV/Las-Vegas/10516-Nantucket-Ridge-Ave-89166/home/29759684',
'https://www.redfin.com/NV/Las-Vegas/6631-Brooklyn-Heights-St-89166/home/29760650',
'https://www.redfin.com/NV/Las-Vegas/10408-Bays-Mountain-Ave-89166/home/29774223',
'https://www.redfin.com/NV/Las-Vegas/8921-Teetering-Rock-Ave-89143/home/29843595',
'https://www.redfin.com/NV/Las-Vegas/7804-Glowing-Peak-St-89131/home/29844676',
'https://www.redfin.com/NV/Las-Vegas/8304-Spanish-Meadows-Ave-89131/home/29892038',
'https://www.redfin.com/NV/Las-Vegas/4435-Peaceful-Heights-Ln-89129/home/29893836',
'https://www.redfin.com/NV/Las-Vegas/5049-Indigo-Gorge-Ave-89131/home/29916708',
'https://www.redfin.com/NV/Las-Vegas/7805-Brent-Leaf-Ave-89131/home/29927599',
'https://www.redfin.com/NV/Las-Vegas/3855-Wiggins-Bay-St-89129/unit-104/home/29930047',
'https://www.redfin.com/NV/Las-Vegas/10125-Cedar-Key-Ave-89129/unit-101/home/29930421',
'https://www.redfin.com/NV/Las-Vegas/9329-Euphoria-Rose-Ave-89166/home/30040711',
'https://www.redfin.com/NV/Las-Vegas/6892-Deer-Isle-St-89149/home/30377787',
'https://www.redfin.com/NV/Las-Vegas/10132-Ocicat-Ave-89166/home/30469451',
'https://www.redfin.com/NV/Las-Vegas/7324-Arlington-Garden-St-89166/home/30469481',
'https://www.redfin.com/NV/Las-Vegas/10529-Bandera-Mountain-Ln-89166/home/40297544',
'https://www.redfin.com/NV/Las-Vegas/7149-Flagstaff-Ranch-St-89166/home/40348542',
'https://www.redfin.com/NV/Las-Vegas/7506-Derby-Wharf-Ct-89166/home/49871406',
'https://www.redfin.com/NV/Las-Vegas/6434-Affermato-St-89131/home/51677901',
'https://www.redfin.com/NV/Las-Vegas/3948-Deluge-Dr-89129/home/52624661',
'https://www.redfin.com/NV/Las-Vegas/7718-Albright-Peak-Dr-89166/home/52625313',
'https://www.redfin.com/NV/Las-Vegas/7327-Brisbane-Hills-St-89166/home/52625334',
'https://www.redfin.com/NV/Las-Vegas/10313-Jolly-Hill-Ave-89166/home/52625380',
'https://www.redfin.com/NV/Las-Vegas/10300-Gibson-Isle-Dr-89166/home/52629297',
'https://www.redfin.com/NV/Las-Vegas/7055-Thistle-Ridge-St-89166/home/56390362',
'https://www.redfin.com/NV/Las-Vegas/5961-Golden-Arowana-Way-89149/home/58810308',
'https://www.redfin.com/NV/Las-Vegas/10744-Merrimack-Ave-89166/home/65555835',
'https://www.redfin.com/NV/Las-Vegas/10725-Crosley-Field-Ave-89166/home/65557121',
'https://www.redfin.com/NV/Las-Vegas/9563-W-Lone-Mountain-Rd-89129/home/65557470',
'https://www.redfin.com/NV/Las-Vegas/8153-Smoking-Jacket-Pl-89166/home/92906248',
'https://www.redfin.com/NV/Las-Vegas/9762-Champagne-Bliss-Ave-89149/home/103972196',
'https://www.redfin.com/NV/Las-Vegas/10804-Leatherstocking-Ave-89166/home/109621118',
'https://www.redfin.com/NV/Las-Vegas/10131-Mayflower-Bay-Ave-89166/home/109621865',
'https://www.redfin.com/NV/Las-Vegas/10439-Prairie-Mountain-Ave-89166/home/109860997',
'https://www.redfin.com/NV/Las-Vegas/7564-Chimney-Point-Ct-89166/home/109862484',
'https://www.redfin.com/NV/Las-Vegas/10757-Wrigley-Field-Ave-89166/home/109864889',
'https://www.redfin.com/NV/Las-Vegas/8391-Aspen-Skye-St-89166/home/109872265',
'https://www.redfin.com/NV/Las-Vegas/7852-Blue-Lake-Peak-St-89166/home/113514096',
'https://www.redfin.com/NV/Las-Vegas/5570-Moore-Cove-Ave-89131/home/145872365',
'https://www.redfin.com/NV/Las-Vegas/7839-Castle-Rock-Peak-St-89166/home/147381249',
'https://www.redfin.com/NV/Las-Vegas/7339-Winesburg-St-89166/home/147874048',
'https://www.redfin.com/NV/Las-Vegas/8302-Skye-Territory-St-89166/home/165776376',
'https://www.redfin.com/NV/Las-Vegas/10146-Skye-Run-Ave-89166/home/165776590',
'https://www.redfin.com/NV/Las-Vegas/6120-N-Monte-Cristo-Way-89130/home/166086307',
'https://www.redfin.com/NV/Las-Vegas/8064-Big-Skye-Ranch-St-89166/home/166088788',
'https://www.redfin.com/NV/Las-Vegas/4377-Velvet-Ridge-Ct-89129/home/166531094',
'https://www.redfin.com/NV/Las-Vegas/10267-Timberline-Valley-Ave-89166/home/166576290',
'https://www.redfin.com/NV/Las-Vegas/10673-Scorched-Skye-Ave-89166/home/167628890',
'https://www.redfin.com/NV/Las-Vegas/8439-Carbon-Canyon-Ct-89129/home/167639222',
'https://www.redfin.com/NV/Las-Vegas/10697-Sariah-Skye-Ave-89166/home/169227559',
'https://www.redfin.com/NV/Las-Vegas/8090-Skye-Dragon-St-89166/home/169227672',
'https://www.redfin.com/NV/Las-Vegas/8048-Skye-Dragon-St-89166/home/169227679',
'https://www.redfin.com/NV/Las-Vegas/10484-Skye-Knoll-Ave-89166/home/169228004',
'https://www.redfin.com/NV/Las-Vegas/8323-Lagunilla-Ave-89149/home/170969787',
'https://www.redfin.com/NV/Las-Vegas/9975-Skye-Gazer-Ave-89166/home/171301529',
'https://www.redfin.com/NV/Las-Vegas/Ruffian-Rd-89149/home/174980410',
'https://www.redfin.com/NV/Las-Vegas/10125-Tree-Blossom-Ave-89166/home/176105333',
'https://www.redfin.com/NV/Las-Vegas/8657-Hartmont-St-89166/home/176105384',
'https://www.redfin.com/NV/Las-Vegas/9601-Jadeite-Ln-89143/home/176643630',
'https://www.redfin.com/NV/Las-Vegas/9827-Mojave-Mountain-St-89166/home/176643796',
'https://www.redfin.com/NV/Las-Vegas/9006-Laurel-Creek-St-89166/home/177117836',
'https://www.redfin.com/NV/Las-Vegas/9001-Pinyon-Bluff-St-89166/home/177117840',
'https://www.redfin.com/NV/Las-Vegas/8989-Pinyon-Bluff-St-89166/home/177117842',
'https://www.redfin.com/NV/Las-Vegas/8935-Windgate-Ranch-St-89166/home/177117950',
'https://www.redfin.com/NV/Las-Vegas/7358-Hadnot-St-89166/home/177118470',
'https://www.redfin.com/NV/Las-Vegas/6546-Castalnova-Bend-Dr-89166/home/177118928',
'https://www.redfin.com/NV/Las-Vegas/6556-Sandero-Springs-St-89166/home/177118988',
'https://www.redfin.com/NV/Las-Vegas/6485-Castalnova-Bend-Dr-89166/home/177119199',
'https://www.redfin.com/NV/Las-Vegas/6515-Corsari-Ridge-St-89166/home/177119246',
'https://www.redfin.com/NV/Las-Vegas/9392-Lunar-Ray-St-89143/home/177326635',
'https://www.redfin.com/NV/Las-Vegas/5120-Olive-Leaf-Ct-89130/home/179963904',
'https://www.redfin.com/NV/Las-Vegas/10596-Bowmore-Ave-89166/home/179964464',
'https://www.redfin.com/NV/Las-Vegas/10375-Sadie-Blake-Ave-89166/home/179980888',
'https://www.redfin.com/NV/Las-Vegas/10391-Thurso-Ave-89166/home/179980934',
'https://www.redfin.com/NV/Las-Vegas/10399-Thurso-Ave-89166/home/179980935',
'https://www.redfin.com/NV/Las-Vegas/10416-Andele-Ave-89166/home/179980938',
'https://www.redfin.com/NV/Las-Vegas/10948-Dixie-Johnson-Ave-89166/home/179981077',
'https://www.redfin.com/NV/Las-Vegas/10854-Trail-Mark-Ave-89166/home/179981328',
'https://www.redfin.com/NV/Las-Vegas/10812-Gentle-Crest-Ave-89166/home/179981381',
'https://www.redfin.com/NV/Las-Vegas/6521-W-Azure-Dr-89130/home/181106130',
'https://www.redfin.com/NV/Las-Vegas/10336-Ansley-Bay-Ave-89166/home/181146167',
'https://www.redfin.com/NV/Las-Vegas/10344-Ansley-Bay-Ave-89166/home/181146168',
'https://www.redfin.com/NV/Las-Vegas/10343-Ansley-Bay-Ave-89166/home/181146169',
'https://www.redfin.com/NV/Las-Vegas/10335-Ansley-Bay-Ave-89166/home/181146170',
'https://www.redfin.com/NV/Las-Vegas/10327-Ansley-Bay-Ave-89166/home/181146171',
'https://www.redfin.com/NV/Las-Vegas/5473-Hickory-Tree-St-89149/unit-14/home/185224192',
'https://www.redfin.com/NV/Las-Vegas/5425-Hickory-Tree-St-89149/unit-16/home/185268980',
'https://www.redfin.com/NV/Las-Vegas/10965-Parsley-AVE-89166/unit-75/home/185735772',
'https://www.redfin.com/NV/Las-Vegas/10974-Parsley-AVE-89166/unit-82/home/185764041',
'https://www.redfin.com/NV/Las-Vegas/10760-Hickam-Ave-89129/unit-68/home/185839508',
'https://www.redfin.com/NV/Las-Vegas/10952-Kingston-Fields-AVE-Unknown/unit-237/home/185972384',
'https://www.redfin.com/NV/Las-Vegas/10960-Kingston-Fields-AVE-89166/unit-239/home/186067031',
'https://www.redfin.com/NV/Las-Vegas/9568-Jadeite-Ln-89143/unit-159/home/186218858',
'https://www.redfin.com/NV/Las-Vegas/10071-Chrysolite-Pl-89143/unit-218/home/186218860',
'https://www.redfin.com/NV/Las-Vegas/10039-Chrysolite-Pl-89143/unit-214/home/186218883',
'https://www.redfin.com/NV/Las-Vegas/4249-Lily-Troter-St-89084/unit-34/home/186430476',
'https://www.redfin.com/NV/Las-Vegas/9040-Proud-Ct-89149/home/29022244',
'https://www.redfin.com/NV/Las-Vegas/9124-Goose-Lake-Way-89149/unit-101/home/29023883',
'https://www.redfin.com/NV/Las-Vegas/7685-N-Rainbow-Blvd-89131/home/29088978',
'https://www.redfin.com/NV/Las-Vegas/8305-Alder-Meadows-Ave-89131/home/29089916',
'https://www.redfin.com/NV/Las-Vegas/4005-Compass-Rose-Way-89108/home/29119665',
'https://www.redfin.com/NV/Las-Vegas/10722-Monaco-Beach-Ave-89166/home/29759630',
'https://www.redfin.com/NV/Las-Vegas/5021-River-Splash-Ave-89131/home/29908637',
'https://www.redfin.com/NV/Las-Vegas/Address-Redacted-89166/home/40548880',
'https://www.redfin.com/NV/Las-Vegas/6885-W-Red-Coach-Ave-89108/home/165797110',
'https://www.redfin.com/NV/Las-Vegas/9568-Jadeite-Ln-89143/home/176643637',
'https://www.redfin.com/NV/Las-Vegas/9552-Jadeite-Ln-89143/home/176643639',
'https://www.redfin.com/NV/Las-Vegas/10947-Cheshire-Woods-Ave-89166/home/177118769',
'https://www.redfin.com/NV/Las-Vegas/10027-Carmine-Clay-Dr-89143/home/179962331',
'https://www.redfin.com/NV/Las-Vegas/10100-Carmine-Clay-Dr-89143/home/179962355',
'https://www.redfin.com/NV/Las-Vegas/10076-Carmine-Clay-Dr-89143/home/179962359',
'https://www.redfin.com/NV/Las-Vegas/10072-Carmine-Clay-Dr-89143/home/179962360',
'https://www.redfin.com/NV/Las-Vegas/10071-Chrysolite-Pl-89143/home/179962373',
'https://www.redfin.com/NV/Las-Vegas/11730-Stonewall-Springs-Ave-89138/home/28985995',
'https://www.redfin.com/NV/Las-Vegas/3504-Desert-Cliff-St-89129/unit-103/home/29006488',
'https://www.redfin.com/NV/Las-Vegas/10524-Danielson-Ave-89129/home/29024389',
'https://www.redfin.com/NV/Las-Vegas/11963-Prada-Verde-Dr-89138/home/29025419',
'https://www.redfin.com/NV/Las-Vegas/620-Apricot-Rose-Pl-89138/home/29025667',
'https://www.redfin.com/NV/Las-Vegas/605-Tandoori-Ln-89138/home/29051442',
'https://www.redfin.com/NV/Las-Vegas/825-Canterra-St-89138/unit-2059/home/29051569',
'https://www.redfin.com/NV/Las-Vegas/2451-N-Rainbow-Blvd-89108/unit-2128/home/29062127',
'https://www.redfin.com/NV/Las-Vegas/10604-Paradise-Point-Dr-89134/home/29105002',
'https://www.redfin.com/NV/Las-Vegas/10212-Pastel-Sky-Ave-89129/home/29105063',
'https://www.redfin.com/NV/Las-Vegas/10304-Back-Plains-Dr-89134/home/29105983',
'https://www.redfin.com/NV/Las-Vegas/10421-Villa-Ridge-Dr-89134/home/29106077',
'https://www.redfin.com/NV/Las-Vegas/2729-Herons-Creek-Dr-89134/home/29106205',
'https://www.redfin.com/NV/Las-Vegas/10809-Clarion-Ln-89134/home/29107551',
'https://www.redfin.com/NV/Las-Vegas/9404-Warm-Waters-Ave-89129/home/29121858',
'https://www.redfin.com/NV/Las-Vegas/3705-Silver-Brook-St-89129/home/29125232',
'https://www.redfin.com/NV/Las-Vegas/3620-Beacon-Point-St-89129/home/29125875',
'https://www.redfin.com/NV/Las-Vegas/8705-Texas-Ranger-Ave-89129/home/29126208',
'https://www.redfin.com/NV/Las-Vegas/6840-Elm-Creek-Dr-89108/unit-204/home/29130460',
'https://www.redfin.com/NV/Las-Vegas/3240-Serene-Dr-89108/home/29131284',
'https://www.redfin.com/NV/Las-Vegas/3637-Solitude-Rd-89108/home/29131469',
'https://www.redfin.com/NV/Las-Vegas/8112-Lake-Hills-Dr-89128/home/29142220',
'https://www.redfin.com/NV/Las-Vegas/3125-N-Buffalo-Dr-89128/unit-2083/home/29145136',
'https://www.redfin.com/NV/Las-Vegas/9013-Faircrest-Dr-89134/home/29146967',
'https://www.redfin.com/NV/Las-Vegas/8913-Desert-Mound-Dr-89134/home/29148240',
'https://www.redfin.com/NV/Las-Vegas/3101-Goodnews-Ct-89134/home/29148821',
'https://www.redfin.com/NV/Las-Vegas/9413-Villa-Ridge-Dr-89134/home/29152349',
'https://www.redfin.com/NV/Las-Vegas/1908-Tierra-Vista-Dr-89128/unit-206/home/29165343',
'https://www.redfin.com/NV/Las-Vegas/6249-Rassler-Ave-89107/home/29196305',
'https://www.redfin.com/NV/Las-Vegas/3151-Soaring-Gulls-Dr-89128/unit-2040/home/29242595',
'https://www.redfin.com/NV/Las-Vegas/3151-Soaring-Gulls-Dr-89128/unit-1194/home/29315121',
'https://www.redfin.com/NV/Las-Vegas/3151-Soaring-Gulls-Dr-89128/unit-1201/home/29315162',
'https://www.redfin.com/NV/Las-Vegas/11441-Allerton-Park-Dr-89135/unit-413/home/29488322',
'https://www.redfin.com/NV/Las-Vegas/1800-Hill-Canyon-Ln-89144/home/29846765',
'https://www.redfin.com/NV/Las-Vegas/3031-Raywood-Ash-Dr-89138/home/165784182',
'https://www.redfin.com/NV/Las-Vegas/748-Banff-Grounds-St-89138/home/170137802',
'https://www.redfin.com/NV/Las-Vegas/11553-Cedar-Shore-Ave-89138/home/173620671',
'https://www.redfin.com/NV/Las-Vegas/424-Cascade-Heights-Ave-89138/home/176106472',
'https://www.redfin.com/NV/Las-Vegas/11489-Vibrant-Heights-DR-89135/unit-33/home/178973247',
'https://www.redfin.com/NV/Las-Vegas/11461-Vibrant-Heights-DR-89135/unit-26/home/179105277',
'https://www.redfin.com/NV/Las-Vegas/11477-Vibrant-Heights-DR-89135/unit-30/home/179105536',
'https://www.redfin.com/NV/Las-Vegas/11620-Desert-Hollow-Ave-89138/home/179981632',
'https://www.redfin.com/NV/Las-Vegas/11493-Vibrant-Heights-DR-89135/unit-34/home/178973290',
'https://www.redfin.com/NV/Las-Vegas/11457-Vibrant-Heights-DR-89135/unit-25/home/179105251',
'https://www.redfin.com/NV/Las-Vegas/11604-Desert-Hollow-Ave-89138/home/179981627',
'https://www.redfin.com/NV/Las-Vegas/11615-Desert-Hollow-Ave-89138/home/179981730',
'https://www.redfin.com/NV/Las-Vegas/2267-High-Perch-St-89138/home/181147097',
'https://www.redfin.com/NV/Las-Vegas/11731-Hatchling-Ave-89138/home/181147154',
'https://www.redfin.com/NV/Las-Vegas/11832-Barcinas-Ln-89138/home/29025442',
'https://www.redfin.com/NV/Las-Vegas/11328-Asilo-Bianco-Ave-89138/home/29009926',
'https://www.redfin.com/NV/Las-Vegas/2130-Sealion-Dr-89128/unit-207/home/29026310',
'https://www.redfin.com/NV/Las-Vegas/3141-Ravenshoe-Dr-89134/home/29105138',
'https://www.redfin.com/NV/Las-Vegas/2125-Hallston-St-89134/home/29108322',
'https://www.redfin.com/NV/Las-Vegas/10317-Bent-Brook-Pl-89134/home/29109289',
'https://www.redfin.com/NV/Las-Vegas/1600-Rappeling-Ct-89144/home/29110085',
'https://www.redfin.com/NV/Las-Vegas/2549-Monarch-Bay-Dr-89128/home/29147165',
'https://www.redfin.com/NV/Las-Vegas/9036-Litchfield-Ave-89134/home/29147931',
'https://www.redfin.com/NV/Las-Vegas/2016-Waterbury-Ln-89134/home/29159654',
'https://www.redfin.com/NV/Las-Vegas/7929-Sierra-Largo-Dr-89128/home/29165299',
'https://www.redfin.com/NV/Las-Vegas/6609-Tampa-Ct-89108/home/29169727',
'https://www.redfin.com/NV/Las-Vegas/5817-Apple-Valley-Ln-89108/home/29186432',
'https://www.redfin.com/NV/Las-Vegas/7960-Terrace-Rock-Way-89128/unit-202/home/29205489',
'https://www.redfin.com/NV/Las-Vegas/133-Lisett-St-89138/home/176106680',
'https://www.redfin.com/NV/Las-Vegas/145-Lisett-St-89138/home/176106681',
'https://www.redfin.com/NV/Las-Vegas/506-Mahogany-Ridge-St-89138/home/173620651',
'https://www.redfin.com/NV/Las-Vegas/11537-Cedar-Shore-Ave-89138/home/173620669',
'https://www.redfin.com/NV/Las-Vegas/493-Highview-Ridge-Ave-89138/home/176105618',
'https://www.redfin.com/NV/Las-Vegas/12172-Harris-Summit-Ave-89138/home/176105984',
'https://www.redfin.com/NV/Las-Vegas/11688-Emerald-Lake-Ave-89138/home/177301808',

    ]

    factors_list = []
    for url in urls:
        factors = parse_factors(url)
        factors['URL'] = url  # Add the URL to the factors dictionary
        factors_list.append(factors)

    save_to_csv(factors_list)

if __name__ == '__main__':
    main()
